In [1]:
import torch
import torch.nn as nn

class ToyModel(nn.Module):
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.fc1 = nn.Linear(in_features, 10, bias=False)
        self.ln = nn.LayerNorm(10)
        self.fc2 = nn.Linear(10, out_features, bias=False)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.ln(x)
        x = self.fc2(x)
        return x

In [2]:
with torch.autocast(device_type="mps", dtype=torch.float16):
    md = ToyModel(5, 2)
    for param in md.parameters():
        original_dtype = param.dtype
        # Use param.data.clone() to create a copy of the tensor data without modifying its dtype
        clone = param.data.clone()
        print(clone.dtype)  # Should show torch.float16 here
        
    x = torch.tensor([1.0, 2,3,4,5])
    md.forward(x)
    print(x.dtype)

torch.float32
torch.float32
torch.float32
torch.float32
torch.float32


In [3]:
from torch.cuda import amp

# Initialize the AMP scaler
scaler = amp.GradScaler()

with amp.autocast(enabled=True):
    # Perform your model computations here
    
    for param in md.parameters():
        original_dtype = param.dtype
        # Use param.data.clone() to create a copy of the tensor data without modifying its dtype
        clone = param.data.clone()
        print(clone.dtype)  # Should show torch.float16 here
    logits = md(x)


torch.float32
torch.float32
torch.float32
torch.float32


/tmp/ipykernel_4151/666721466.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
/tmp/ipykernel_4151/666721466.py:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=True):


In [4]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Case 1: float32 tensor + float32 tensor
s = torch.tensor(0, dtype=torch.float32, device=device)
for i in range(1000):
    s += torch.tensor(0.01, dtype=torch.float32, device=device)
print(s)

# Case 2: float16 tensor + float16 tensor
s = torch.tensor(0, dtype=torch.float16, device=device)
for i in range(1000):
    s += torch.tensor(0.01, dtype=torch.float16, device=device)
print(s)

# Case 3: float32 tensor + float16 tensor
s = torch.tensor(0, dtype=torch.float32, device=device)
for i in range(1000):
    s += torch.tensor(0.01, dtype=torch.float16, device=device)
print(s)

# Case 4: float32 tensor + casted float16 tensor to float32
s = torch.tensor(0, dtype=torch.float32, device=device)
for i in range(1000):
    x = torch.tensor(0.01, dtype=torch.float16, device=device)
    s += x.type(torch.float32)
print(s)


Using device: cuda
tensor(10.0001, device='cuda:0')
tensor(9.9531, device='cuda:0', dtype=torch.float16)
tensor(10.0021, device='cuda:0')
tensor(10.0021, device='cuda:0')


In [5]:
NUM = [f"{i}" for i in range(10)]
SYMB = ["+", "-"]
PAREN = ["(", ")"]



def find_close_paren(i:int, s:str):
    """
    Find the corresponding closing paren of the open paren at s[i].
    """
    parent_count = 0
    i = i+1
    while i < len(s):
        if s[i] == "(":
            parent_count += 1
        if s[i] == ")":
            if parent_count <= 0:
                return i
            else:
                parent_count -= 1
        i += 1

    print("Not found")
    return 


def operate(temp_result:int, num:int, operator_str:str):
    if operator_str == "+":
        temp_result += num
    else:
        temp_result -= num

    return temp_result


def _calculate(s:str, l, r):
    """
    Comptue the value in the range s[l:r+1]
    """
    result = 0
    cached_operator = "+"
    i = l
    while i < r+1:
        # If Number
        if s[i] in NUM:
            # accumulate number
            temp_num = 0
            while i < len(s) and s[i] in NUM:
                print(f"{i}: {s[i]}")
                temp_num = temp_num * 10 + int(s[i])
                i += 1
            # Compute
            result = operate(result, temp_num, cached_operator)

        # If is symbol
        if i < len(s) and s[i] in SYMB:
            if cached_operator == "+":
                cached_operator = s[i]
            else: # cached_operator == "-":
                if s[i] == "-":
                    cached_operator = "+"
        
        # If is opening parenthesis
        if i < len(s) and s[i] == "(":
            close_idx = find_close_paren(i, s)
            inner_result = _calculate(s, i+1, close_idx)
            result = operate(result, inner_result, cached_operator)
            # Set i to the closing paren location
            i = close_idx

        i += 1
    
    return result

s = "(((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(-(94 - 52 + 76 + 90 - 88 + 47) - 48 - 52 - 19) + 73 + 50 - 77 - 88 - 99 - 26 + 65 + 98 + 46 + 4 - 71 + 86 + 81 - 62 - 79 - 45 + 34 + 21 + 67 + 86 + 75 + 19 - 100 - 59 - 10 - 47 - 52 + 1 - 84 - 16 - 91 - 9 - 78 + 0 + 60 - 10 - 5 - 35 + 60) - 76 + 37 + 62 - 71 + 47 - 67 - 96 - 95 + 62 - 69) - 10 - 20 - 72 + 26 - 15 - 14 + 14 - 100 - 79 + 99 - 98 - 73 - 84 + 46 - 19 - 63 - 71 + 16 - 99 + 52 + 64 + 51 + 14 - 97 - 42 + 47 + 6 + 51) - 57) - 55) - 56 + 74 - 91 + 45 - 86) - 36) - 32) + 41 - 61) + 65 - 4 + 81 - 47 + 94 + 84) + 55 - 15 - 88) - 75 + 52 - 0) - 42 - 72 - 8 + 26 + 21 + 9 - 86) - 16 + 21 + 35) - 87 + 48 - 93 - 71 - 62 + 100 - 92 + 3 + 3 + 74) - 57 + 80 + 36 + 68 + 23 + 91 + 41 + 3) - 96 - 14) + 40 - 9 - 79 - 95) - 51 + 33 - 91) + 88 + 38 - 2 + 44 - 75 + 73 - 34) - 85 - 94 - 73 + 61 + 76 + 82 + 48 - 35 + 20 + 1 + 30 + 59 + 60 - 48 - 74 - 75 - 55 + 11 - 43 + 22 - 54 + 58 - 14 - 53 - 45 - 88 - 42 + 99 + 40 - 35 - 98 - 94 + 94 + 4 + 95 + 7 - 81 + 55 - 49 + 44 - 59 + 3 - 17 - 23 - 68 + 9 + 95 + 70 - 92 - 77 + 59 + 37 + 96 + 63 - 82 - 52 + 43 - 70 + 73 + 79 - 55 + 12 - 64 + 34 + 40 - 45 - 63 - 48 + 37 - 43 + 7) - 2 - 25 + 66 - 43) - 34 + 24 + 84 - 28 + 73 - 93 + 89 - 24 + 46 - 75 + 67) + 75) - 31 - 14 + 17 + 100) + 37 + 13 + 25) - 74) + 86 - 90 + 26 - 39 - 15 + 91 - 94 + 48 + 18 - 96 - 98 + 50 - 42 - 25 - 51 - 50 + 39 + 10 + 54 - 37 - 28 + 2 - 83 - 3) - 16 - 29 - 84 - 22 + 19 + 43 + 58) - 89 - 71 - 97 - 96) - 56 - 9) - 52 + 76 - 5 - 11) - 59 + 66 - 41 + 85 + 26 - 22 + 89 + 86) + 32 - 28 - 45 - 51 + 9 - 32 - 76) - 83 + 61 + 26 + 72 - 8 - 31 - 50 - 62 + 37 + 12 + 29 - 11 + 62 + 6 + 95 - 96) + 49 - 36 + 7 - 70 + 76 + 44 - 71 + 92) + 15 - 30 + 37 - 89 + 98 - 23 - 66 + 9 - 7) - 23 + 53 - 16) + 88 + 99 + 28 + 17 + 34 - 68 + 30 - 97 + 19 + 72 - 58 - 42 + 34) + 23 + 7 + 92 - 4 + 69 - 35 - 2 - 24 + 95 + 59 + 37 - 63 - 9 + 33 + 32 - 90 - 41 + 91) - 20 - 45 - 1 - 45 + 10 - 39 + 72 - 55 - 97 - 63 - 95 + 86 - 6 - 23 + 23 - 28 + 40 - 32) + 96 + 48 - 19 + 52 - 7 - 59 + 19 + 17) - 59 - 40 + 50 + 64 - 38 + 9 - 21 - 57 - 77 + 12 - 18 + 45 + 46 - 34 + 3 + 12 + 12 - 10 + 76 - 96) - 47 - 96 - 3 - 32 - 62 - 16 - 51 - 3 + 13 + 66 + 50 - 42 - 65 + 59) - 46 - 46 + 9 - 38) - 35 - 63 - 91 - 39 - 74 + 68 - 27 - 17 - 82 + 43 + 94 + 96) - 73 + 76 + 92 + 10 + 52 + 46 + 61 - 58 - 76 + 76 + 14) + 87 + 71 + 7 - 93 + 3 + 86 - 78 + 12) + 96 - 74 + 15 - 78 + 44 + 60 + 6 - 82 - 55) - 53 + 1 + 13 + 37 - 85 - 72 + 64) - 82 - 32 - 4 - 2 - 40) + 73) + 22 - 86 - 55 + 16 - 89 - 78 + 58) - 89 + 84 - 83 - 92 + 45 + 95) + 57 - 80 + 60 + 30 - 29 + 35 + 23) + 2 - 28 + 74 - 17 + 12 - 89 + 100 - 87 + 61 - 4 - 35 + 59 + 98 - 75 + 12 - 18 + 75 + 82 - 74 + 82 - 98 - 92 + 39) - 36 + 53) - 65 + 43 - 83 + 55 + 91 - 80) - 39) + 66 - 64 - 15) + 36 - 88 + 28 + 6 - 41 + 81 + 5 + 21 + 100 + 84 + 16 + 5 - 40) + 73 + 54 - 76 + 1 - 26 - 66 - 21 + 36 - 16 + 33 + 57) + 59 - 72 - 81 + 57 - 46 - 51 - 71 - 64 - 40 + 78 - 77 - 93 - 70 + 6 - 27 - 41 + 57 + 52 + 65 - 65 + 90 + 30 + 19 + 24 + 21 + 46 - 50 - 47 - 57 + 12 + 96 - 62 - 65 + 91 + 22 + 34 - 88 + 44 - 99 - 76 - 32 - 5 - 74 + 28 - 75) - 86 + 5 + 1 - 80) + 32 - 36 - 43) - 93 + 57) - 74 - 65 - 80 - 56 + 12) - 48 + 99 - 61 - 32 - 49 - 17 + 88 + 69 - 2 - 65 - 0 - 69 + 12 + 76 + 81) + 84 + 90) + 55 + 7 + 34 - 55) - 35 - 12 - 75 + 40 - 67 - 85 + 58 - 70 + 66 - 51) - 93 - 36 - 55 - 86 + 25 + 19) + 11) + 57 - 12 - 66 - 83 + 34 + 38 - 98) + 12) - 4 + 18 - 8 - 57 + 99 - 8 - 93 + 28 - 37 - 67 - 80 - 25 + 23 + 31 + 38 - 87 - 81 - 16 + 15 - 57) + 56 + 43 + 48 + 17 + 2 + 94 - 80 - 61 + 28 - 67 + 86 + 9 + 76 + 77 - 28 + 38 + 86 - 55 + 38 + 14 - 13 + 60 + 95) - 21 + 57 + 89 - 54 + 74 + 62 - 17 + 79 + 82 + 64 + 91 - 22 + 33 - 32 - 19 + 4) + 59 - 57 - 87 + 93 + 41 - 78 - 91 - 15 + 83 - 58) - 6 - 14 - 81 - 66 - 96) - 70 + 67 - 93 - 77) - 80 + 87 + 88 - 32 - 55 + 23 - 99 + 83 - 35 + 17 - 67 - 16 - 16 - 23 - 20 - 93 - 18 + 81 - 35) + 4 - 22 - 54 - 77) - 20 + 59 + 64 - 83) + 55 + 82 - 97 - 32) + 99 - 92 - 32 + 96 - 21 + 93 - 79 - 33 + 57 - 47 - 62 + 46 - 31) - 2 + 58 - 9 - 18 + 34 + 98 - 33 - 89 + 9 + 55 + 35 + 10 + 52 - 43) - 82 + 62 + 22 - 99 - 60) - 27 + 64 - 19 - 79 - 10) + 24 - 41 + 47 + 34 - 77 - 90 - 8 - 57 - 2 + 43 + 30) - 41 + 74 - 96 - 41) + 2 - 95 + 18 + 100 + 2 + 90 - 65 + 77 - 75 + 19 + 87) + 83 + 41 + 10 - 21 + 13 + 43 - 68 - 54) + 83 + 40 - 60) - 58 + 34 + 92 - 44 - 90 - 100 - 9 - 42 + 6 + 29 - 47) - 70 - 88 + 12 - 19 - 49) - 97 - 68 + 54 + 27 + 88 + 32 + 19 - 16 - 50 - 36) + 87) + 49 - 13 + 17 - 7 + 57 + 5 - 48 + 65 + 30 - 37 - 92) + 81 + 60) - 64 - 18 + 82 - 62 - 31 + 65) - 5 + 80 - 49 - 39 - 86 - 48) - 71 + 32 + 66 - 17 - 46 - 77 + 21 - 19 + 44 + 11 + 84 - 54 + 55 + 12 + 90) - 26 + 53 - 83 - 67 - 67 - 11 - 22 - 8 + 16 + 6 + 88 + 17 + 59 + 17 + 43 - 7 + 25 + 24 + 92 + 22 - 83 - 78) - 4) + 9) - 7 - 51 + 50 + 87 + 67 - 52) + 46 - 50 - 27 + 23 - 80 - 37 + 72 + 82 - 56 - 42 - 79 + 58 - 7 + 47 - 76 - 80) + 56 + 49 + 9 + 81 + 36 - 7 - 9 + 66 - 94 + 30 + 42 - 40 - 19 + 58) + 89 + 5 - 45 + 60 - 18 + 79 + 22 - 40 + 14 - 86 - 11 + 6 + 31 + 52 + 1 - 17 + 85) - 50 + 23) + 36 + 79) - 24) - 96 + 71 - 5 - 60 + 91) + 15 - 54 - 33 + 36) - 50 + 98 + 91 - 57 + 93 + 70 + 58) - 2 - 77 + 74 - 42 - 33 + 32 - 8) + 65 - 21 + 29 - 10 + 13 + 33 - 52 + 7 - 9 - 7 - 54 + 50 + 8 + 6 + 16 - 0 + 89) - 87 + 88 + 90 + 18 - 91 + 6 - 34 + 51 - 34 + 6 + 90 - 86 + 59 - 83 - 74 + 7 - 65 + 74 + 17 - 57 + 22 - 57 - 52 - 88 + 78 + 91 + 1 + 53 - 60 - 13 - 36 + 79 + 25 + 40 - 23 + 52) - 71 + 94 - 57 + 83 + 36 - 64 - 15 + 25) + 70 - 0 - 56) - 5 - 15) + 40) - 16 + 4 - 53 - 46 + 83 + 41) - 73 + 22 + 80) + 78 + 34 - 44) + 63 - 54 + 77 + 4 - 76 - 64 - 30 - 40 - 14) - 42 + 40 + 71 - 71 + 8 - 83 + 88 + 78) - 92 - 56) - 47 + 36 + 59 + 55 + 100 - 95 + 51 + 1 + 18 + 36 + 47 + 47 - 96 - 66) + 35 + 29 + 7 + 51 - 53 - 74 - 96 - 28 - 71) - 21 - 17 - 87 + 99 + 92 - 78 + 51 + 34 - 42 - 38 + 58 + 68 + 36 + 69 + 47) + 45 + 67 + 96 - 18 - 99 + 15 - 74 + 2 + 50 + 47 + 68 + 43 - 34 - 63 + 14 - 83 + 91 + 48 - 89) - 97 + 81 + 8) + 12 + 50 - 86 + 99 - 14 - 12 + 25 + 25 + 93) - 43 + 72 + 12) - 73 - 37 + 40 + 81 - 41 + 27 + 53 - 26 + 18 + 91 + 7 - 59 + 99 + 3 + 14 + 6) + 69 + 66 + 78 - 54 + 23 - 66 + 32 - 51) + 22) + 66 + 95 - 92 + 59 - 35 - 21 + 59 - 29 - 99 + 68 - 10 + 16) + 25 + 85 + 61 - 88 - 11 + 38 + 79 + 27 + 24 + 23 + 73 - 79 + 24 + 92) - 39 + 86 - 12 + 47) - 47 - 77 + 96 - 100 + 78 + 47 - 14 + 25 - 93 - 15 - 16 - 12 - 46 + 55 + 24 - 96 + 21 - 22 + 53 - 11 + 76 - 79 - 25 + 54) - 9 - 65 + 3) - 34 + 49 - 9 - 30 + 7 - 95 - 71 + 98) + 79) - 24 - 32 + 1 + 5 + 1 + 44 - 57 - 15 - 13 - 40 + 11 - 37 - 93 - 80) + 74 + 90 + 33 + 87 + 76 + 30 + 94 + 15 + 68 - 65 + 2 - 34 + 85 + 31 - 85 + 95 + 17 - 63 + 87 + 52 + 51) + 48 - 66 + 46 + 18 - 45 - 55 + 97 - 29 - 70) - 32 + 20 - 4 + 6 + 60) - 49 - 5 + 28) - 73 + 81 + 22 - 29 - 4 + 12) + 73 - 19) - 17 - 68 + 73 - 68 - 76 - 91 + 84) + 5 - 7 + 58 - 72 + 3 + 37 + 18 - 40 - 9 + 89 + 87 - 9 - 21 - 51 + 39 + 44 + 42) - 15 + 83) - 12) + 23 + 11 + 30 - 92 + 19 - 33 + 30 - 33 - 47 + 56 - 73 - 68 - 98) - 94 + 72 - 71 - 100 - 69 + 66 - 44 + 20 - 2 - 76 - 23 + 54 - 3 + 7 - 49 - 56 + 49 - 9 - 54 - 98) - 8 + 21 - 71) - 56 - 90 + 16 - 24 - 30 + 24 + 100 + 73 - 94 - 69 - 61 + 81 - 29 - 66) - 70 - 33 - 74 - 9 + 66 - 100 + 91 - 59 + 8 + 7) + 73 + 81 - 12 - 6 + 72 + 97 - 68 - 20 - 91 + 15 - 48 - 61 + 14 + 53 + 31 - 71) + 38 + 82 + 28 + 69 + 0 + 54 - 42 - 31 + 42 + 94 - 94 + 64 + 47 + 62 + 39 - 5 + 42) + 16 - 9 - 100 - 53 + 68 + 50 + 96 - 45 - 48) + 29 - 39 + 73 - 0 - 92 - 6 + 76 + 92 - 12 + 12) - 45 + 48 - 30 + 16 + 100 + 68 + 75 + 12 - 13 - 28 - 100 + 86 - 63 - 64) - 93) + 88 + 61 - 89) - 23 + 86 - 0 + 8 + 94 + 72 + 25 + 33 - 85 + 59 + 58 - 19 + 49 - 35 - 86 + 44 + 51 - 89 + 17 + 84 + 56 + 56 + 55) - 85 + 100 + 29 - 100 + 62 + 54 - 43 - 84) - 20 + 8 + 51 - 58 - 66 + 37 + 37 + 21 - 47) - 5 + 6 + 66 - 80 + 82 + 91 + 96 + 92 + 69 - 36 + 42 + 65 - 66 - 17 - 11 + 79 - 17) - 49 - 20 - 6 + 94 - 25 + 61 - 92 + 41 + 79 - 93 - 6 + 90 + 28 + 6 + 65 - 12 + 61 - 22 - 26) + 70 - 91) + 52 + 51 + 72 + 46 - 21) + 4 + 40 + 62 + 85 + 64 + 91 - 55 - 85) - 37 - 85) + 82) + 91 - 55 - 71 + 40 - 3 - 31 + 85 + 51 - 34 + 27 + 54 - 60 + 51 + 67 + 90 + 58 - 0 + 87 - 72 + 88 - 55 + 39 - 48 + 71) + 19 + 45 + 7 - 40 - 31 - 24 - 30 + 3 - 38 - 25 + 90 + 86 - 94 + 9 - 85 - 69 - 89 + 95 + 68 + 42 - 56 - 15 - 80) - 25 - 57 - 37 + 95 - 77 - 28 - 2 + 56 - 64 + 70 - 62 - 79 + 72 - 70 + 34 + 95 - 30 - 17 + 50 + 97 + 82 - 37 + 66 - 36 - 41 - 6) - 2 - 58 + 72 + 71 - 44 - 67 + 47 - 45 + 54) - 91 - 90 + 77 + 3 + 95 - 81 - 44 - 6 + 90 + 49 - 77 + 59 + 27 + 94) + 37 - 100 - 25 - 29 + 55 - 22 + 27 + 84 - 68 + 47 - 54) + 49 + 61 - 92 - 12 - 43 - 32 + 7 + 33 - 50 + 14 + 70) + 23 - 15 + 59 + 20 + 36 - 38 + 44 - 51 - 2 - 15 - 77 + 10 - 65) - 45 - 26) + 72 - 37 - 88 + 61 + 36 - 79 - 68 + 100 - 49 - 18 - 78 + 49) - 7 - 3 + 47) - 43 - 76 - 49 - 44 + 80 + 94 - 94 + 96 + 29 - 38 - 42 - 42 - 12 + 76 - 13 + 10 - 8 - 37 - 41 + 31 + 29 + 52 + 51 + 97 + 55) + 26 - 3 - 57 + 63 - 47 + 60 - 38 - 24 + 16 + 55 + 15 + 89 - 62 + 56 + 71 + 93 - 58 + 88 - 53 + 53 - 34 - 88 - 8 + 50) - 93 - 95 - 61 - 29 + 82 - 91 + 67 - 91 - 56 + 85 + 46 - 10) + 7 + 67 - 22 - 65 + 1) - 54 + 47 + 58) - 66 - 32 - 72 + 73 - 19 - 40 - 92 - 63 + 82 + 78 - 1 - 36 - 66 + 76 - 45 - 30 - 3 - 30) + 41 - 22 - 96 + 13 + 19 - 58 + 71 - 24 - 60 + 64 - 27 - 25 + 45 - 78 + 95 + 94 - 21 - 43 - 50 - 38 + 2 - 88 - 58 - 2 + 41 - 99 + 47 + 67 - 80 + 64 + 83 + 81 + 21) + 21 - 55 - 48) - 1 - 40 - 65 - 26 - 64 + 57) - 59 - 47) + 33 + 81 - 68) - 82) - 7 + 19 - 8 - 96) + 93 - 68 + 14 - 85 + 93 - 15 - 79 + 0 + 12 - 65 + 98 + 56 + 93 + 62 - 75 + 9 - 10 - 93 - 29 + 75 - 15 - 23 + 20 - 46 + 11 + 0) - 41 + 7 + 32 - 63 + 19 + 45 + 84 - 38 - 82 - 22 - 96) - 63 + 11 - 47 - 37 + 24 + 22 + 34 + 29 + 96 + 41 + 13 + 0 + 0 - 0 + 72 + 45) - 32 - 39 - 34 - 41 + 54 + 21 - 7 - 22 - 91 + 55 - 74 + 91 + 21 - 45 - 2 + 19) + 4 + 57 + 54 + 32 - 59 - 21 + 37 - 28) + 85 - 85) + 55 - 10 - 18 + 49 - 60 - 50 - 91 + 18 + 49 + 29 - 44 - 26 + 24 + 5 + 34 - 58 - 88 - 37 - 41 + 99 + 9 + 6 - 42 + 13 - 97 - 46 - 30) + 47 + 85 + 58 + 27 - 95 - 43 + 70 - 44 + 26 + 20 - 5 - 11 - 46 + 11 + 80 - 62 + 86 + 63 + 66 - 72 + 21 - 2 + 7 - 8 - 13 - 86 - 9 + 6 - 88) + 51 - 57 - 31 + 61 - 10 - 76 - 45 + 61 + 35) + 17 - 61 - 51 - 50 - 14 - 29 + 35 + 67 - 49 + 14 + 8 + 3 + 90 - 24 - 46 + 62 + 39 - 72 - 75) - 93 + 89 + 91 + 35) + 4 - 30 + 45 - 78) + 43 - 58 - 45 - 74 + 87 + 49 - 56 + 27 - 81 + 18 - 43 - 73 + 46 + 37 + 21 + 63 - 37 - 47) + 63 + 50 - 39 + 14 - 35 + 37 - 65 + 46 - 73 + 8 + 40 - 92 - 83 + 7 + 2 - 44 - 20 + 51 + 43 + 3 + 54) - 17 - 39 - 74 - 8 - 46 - 6 - 39 + 52 + 30) + 97) + 82 - 100 + 83 + 50 - 0 - 87 + 20 + 18 + 48 + 79 + 4 + 23 - 77) - 6) + 46 - 6 + 54 + 57 + 21 - 80 - 68 + 2 - 85 + 50) - 90 + 43) - 65 - 88 - 6 + 74 + 15 - 80 - 26) + 38 - 93 + 98 + 70 - 39 - 56 - 3 - 44 + 90 + 58 - 2 + 77 + 39 - 68 - 50) - 71 - 60 - 43 + 51 - 3 + 79 + 35 + 44 - 27 + 87 + 66 - 96 + 61) - 28 + 42 - 44 + 51 - 84 - 51 + 52 + 78 - 17 - 67 + 36 + 97 + 24) - 90 - 23 - 2) - 35 - 67 + 88 + 6 + 37 + 17 - 52 + 3 + 84 - 94 + 52 + 1 + 84 + 71 - 68 - 38 - 78 - 99 - 81 + 96 + 54 + 64) - 44) + 98 + 30 - 93) + 61 + 67) - 15 - 28 + 62 + 76 + 76 + 8 + 90 - 17 - 57 - 92 + 18 + 47 + 68 + 10 + 9 - 6 + 70) + 58 + 9 + 65 + 92 + 78 - 75 - 37 + 36 + 73 + 28 - 18 + 15 - 34 + 17 + 18) + 38) + 29 + 83 - 5 + 10 - 28 - 45 - 4 + 14 + 31) - 94 + 12 - 56 - 84 + 43 - 74) - 12 - 44 + 37 + 95 - 36 - 50 - 36 - 0 - 3 + 42 - 74 - 17 - 51 - 40 - 68) + 3 + 66 - 98 + 45 + 46 + 77 - 63 + 17 + 93 - 42 + 4 - 33 - 60 - 71) + 78 - 36 + 0 + 23 - 17 + 79) - 74 + 50 + 32 - 94 - 57 - 93 - 74) - 53 + 58 - 5 - 99 - 29) + 41 + 57 + 46 + 50 + 87 - 72 - 77 - 95 - 34 - 74 - 29 + 33 - 61) + 38 - 24 + 44 + 70 - 16) + 12 - 37 - 33 + 84 + 100 - 76 - 47 - 21) - 89 - 10 + 68) - 38 + 56 + 4 + 2 + 49 + 76 + 5 - 87 + 10 - 72 + 60 - 64 + 56 + 77 + 94 + 53 + 54 + 93 - 55 + 92 - 3 + 80 - 69 - 62 + 78 + 31 + 86 + 40 - 58) + 15 + 18 - 25 - 6) - 75 - 15 + 92) - 40 + 1 + 98 + 27 - 62 - 54 + 64 - 75 + 65 + 24 - 50 + 88 + 70 - 31 + 45) + 96 - 51) - 95 + 24) + 5 + 71 - 81 + 78 + 83 - 65) + 28 + 64 - 19 + 22 + 79 + 48 - 48 + 0 + 11 - 97 + 57 - 97 + 15 + 13 - 18 - 58 - 95 - 18 - 32 - 66 - 25 + 70 + 16 + 56 - 36 + 92 - 87 - 37 - 32 - 37 + 63 - 75 + 0 + 99 - 31 + 54 - 22 + 41 + 48 + 68 + 19 - 75 + 44 - 76 + 32 - 64 + 15 - 23 - 39 - 31 - 82 + 39 - 55 + 19 - 6 - 69 + 89 - 71 - 10 - 31 + 99 - 13 - 66 - 31 - 46 - 19 - 21 + 68) - 33 + 11) + 17 - 25 - 80 + 63 - 53) + 1 + 63 + 64 - 20) + 37 - 93 - 21 - 60 + 28 + 62 + 25 - 64 - 13 - 41 + 34 + 49 + 87) + 42 - 7 - 50 + 50 + 59 - 33) + 83 + 24 + 18 + 99 - 62 + 84) - 53 - 59 - 68 - 57 + 21 - 8 + 63 + 98 + 8 + 49 + 97 + 57 + 2 - 34 - 24) - 35 + 83 - 73) + 49 + 70 + 1 + 81 - 19) + 60 - 55 - 19 + 85 - 40) - 68 + 76 - 87 - 49 - 95 + 73 - 8 + 77 + 3 + 33 + 33 + 99) + 58 - 11 - 1 + 69 - 15 - 44 + 55 + 37) - 14 - 11 - 12 + 74 + 29 + 8 + 14 + 44 + 68 - 63 - 49) + 72) + 38 + 38 - 88 - 8 - 48 + 20 - 18 - 88 - 60 - 21 + 97 + 2 + 88 + 91) + 82 - 60 - 45 + 67 - 29 - 58 + 37 + 87 - 14 + 33) + 12 - 29) - 55 - 22 + 78 - 8 - 29 + 74 - 27 + 16 + 11 + 20 - 55 + 98 - 66 - 77 + 34 - 45 + 87 - 53 - 45) - 34 + 7 - 72) - 67 + 50 + 19 - 96) + 91 + 19 - 35 + 84 + 90 - 15 - 96) - 84 + 5 + 86 - 70 - 68 - 31 - 42 - 18) + 32 + 45 + 43 - 45 + 51 + 100 - 13 - 64 + 63 - 12) + 12 + 69 - 30) + 90 - 47 - 22 + 60 + 13 - 13 - 53 + 81 + 40 - 76 - 37 - 51 + 91 + 2 + 62 - 7 - 3) - 93 - 51 - 14 - 0 + 95) - 91 - 47 + 17 - 51 - 49 - 61 + 45 - 21) + 70) - 26 - 98 - 96 + 17 + 44 + 10 + 16 - 27 + 48 + 39 + 56 + 78) - 97 - 79 + 60 - 80) - 49 + 27 + 14 + 19 - 85 + 69 + 0 + 56 - 2 - 66 + 44 - 92) + 44 + 31) - 53 - 84 - 2 - 17 - 61 + 40 - 99 + 77 - 44 + 79 - 92 + 4 - 15) + 95 + 0 - 77) - 79 + 44) + 86 - 9 + 52) + 16 - 88 - 19 + 77 - 16 - 49 + 37 + 43 + 18 - 43 + 6 + 44 - 26 + 78 + 96 + 59 + 84 - 68 + 67) - 59 + 21 + 18 + 92 + 18 - 75 + 97 - 61) + 42 - 92 - 89 - 69 + 44 - 69 - 94 - 86 + 64 + 93 + 5 - 94 - 100 + 47 - 50 - 45 - 21 - 40 + 84 + 35 + 93 - 7 + 64 + 71) - 26 + 25 + 37 - 34 - 1 - 90 - 91 + 77 + 56 + 23 - 65) - 45 + 69 + 37 + 27 + 9 - 29 - 22 + 4) - 7 + 17 - 31 - 81 + 69 + 93 - 97 - 87 + 10 + 40 - 94) + 23 - 87 + 16) - 77 - 90 + 43 - 79 - 21 + 46) + 1 - 68 + 6 + 78 - 92 - 22 + 82 + 20 + 21 - 58 - 6 + 74 - 82 + 7 - 67 + 1 - 60 + 18 + 0 + 12 - 56 - 60 + 7 + 39 - 61 + 13 + 90) + 54 - 93 - 3 - 46) + 34) - 91 + 11 + 3 + 68 - 33 + 20 - 98 + 79 + 53 + 20 - 88 - 1 - 68 + 5 + 50 - 4) + 37 - 25 - 2) + 96 - 56 + 69 + 62 - 27 + 12 - 3 + 33 - 59 - 96 + 41) - 67 + 83 + 97 - 86 - 96 - 46 + 19 - 46) - 48 + 41 + 52 - 97) + 69 + 83 - 18 + 61 - 51 + 31) + 98 - 26 + 31 - 42 - 38 + 60 + 98 + 66 + 71 + 90 + 10) - 18 + 40 + 19 + 20 + 75 - 72 - 13 - 36 + 64 + 29 + 31 - 25 + 27 + 87 - 46 + 52 + 78 + 50 + 27 + 41) + 75 - 20 - 95 - 72 - 83 + 12 - 42 + 69 + 56 - 74 + 53 - 25 - 94 - 4 - 53) + 22 - 32 + 80 - 31 + 52 + 59 + 11) + 21 - 16 + 85 + 23) + 64 + 92 + 7 + 71 + 42 + 20 + 45 - 54 - 35 + 48 - 59 - 6 - 27 - 0 - 82 + 65 - 40 + 38 + 22 + 22 - 18 + 57 - 32 - 49) - 51 + 41 + 66 + 29 + 17) + 68 - 95 - 84) + 21 + 28 - 7 + 98 + 91 + 87 - 28 + 50) + 49 - 89 - 57 - 39 - 52 - 89 + 14 + 13 - 96 - 24 + 54 - 85 - 0) - 22 - 90 - 87 + 74 - 12 - 80 - 99 - 24 + 34 + 28 - 22 + 62 + 87 - 46 + 92 - 28 + 39 + 34 - 40 + 58 + 87 - 100 + 24 + 50 - 77 + 82 - 89 - 14 + 6) + 76 - 1 + 64 + 89 + 16 - 32 - 45) + 9 + 71 - 28 + 41 - 24 + 33 + 59 - 73 + 51 - 30 - 77 + 26 - 37 - 56 - 50) + 24) - 80 + 80 + 54 - 12 + 67 - 1 + 2 + 28 + 16 + 1 + 6 - 96 - 18) - 5 + 69 - 81 + 14 - 96 - 59 + 6 + 92 - 70 + 41 - 25 + 13 - 41 - 71 + 92 + 68 + 16 + 39 - 97 - 2 - 44 + 6 - 97 + 29 + 10) - 75 + 45 + 76 + 48 - 88 + 74 + 1 - 13 - 54 - 15 - 30 + 65 - 84 - 89 + 95 + 13 + 46 - 86 + 82 - 12 + 5 + 71) + 15 + 4 - 43 + 26) + 54 + 93 + 58 + 99 - 43 - 77 - 50 + 26 - 44 - 23 - 48 - 11 - 37 - 54 + 19 - 92 - 78 - 9 + 83 + 43 - 72 - 31 - 3 - 52 - 26 - 40 - 17 - 75 - 46 - 99 + 40 + 4 + 17 - 20 - 81 + 45 - 8 + 65 - 67 - 13 - 58 + 35 - 81 + 100 - 13 - 53 - 64) + 69 + 92 + 12 - 39 - 83 + 1 + 48 - 81 + 37 + 51 + 24 + 76 - 93 + 91 - 48) + 59 + 87 - 57 + 37 - 52 + 8 + 45 - 43 + 2 - 3 - 56 + 56 + 40 - 27 - 20 + 93 + 98 - 52 - 87 - 44 + 78 + 72 - 40 - 47 + 98 - 54 + 15 + 7 - 72 + 19 - 78 + 65 + 56 + 53 + 7 + 14 - 65 - 58 + 30) + 73 - 42 - 92 + 25 - 84 - 63 - 25 - 14) - 20) + 67 + 50 + 12 - 8 - 62 + 13 - 41 - 26 + 89 - 53 + 71 - 18 - 82 - 39 - 54) - 13 + 88 - 30 + 79 - 89 - 15 + 50 - 74 - 3 + 28 + 12 - 97 - 52 + 69 - 47 - 45 - 96) - 42 + 86 - 14 - 12) + 5 + 33 - 45 + 56 + 26 - 29 + 3 + 76) - 74 - 25 - 36) + 47 - 16 - 87 + 81 + 34 + 95 + 79) - 91) - 20 + 60 - 73 + 93 - 9 - 49) + 13) + 98 - 49 + 98 + 79 + 41 - 66 + 52 + 67 + 64 - 40 + 8 + 48 - 93 - 61 + 57 + 83 - 11 + 83 + 73 - 33) + 51 - 54) + 36 - 42 + 90 - 39 + 95 - 20) - 6 - 70 - 52 + 93 + 92 - 86 + 92 + 51 + 34 + 61 - 26 - 36 - 3 - 13) + 47) + 52 + 25) + 51 + 56 + 56 - 94 + 72 + 98) - 31 - 98 - 32 + 14 + 3 + 55) + 34) + 14 - 37 - 99 + 13 - 93 - 17 - 7 - 98 + 40 - 98 + 86 + 21) + 6 - 44 - 38 + 40 + 95 + 54 - 40 - 62 + 38 - 69 + 92 + 29) - 45 + 36 - 87 + 57 - 56 + 23 + 41 + 56 - 63 + 56 + 94 + 12 + 69) - 14 + 50 - 97 - 61 - 63 + 8 + 66 - 30 - 21) + 35 - 98 - 55 + 83 - 31 + 45 - 36) - 0 + 65 + 60 + 84 + 53 + 10 + 51 + 21 - 37 + 40 - 86 + 37 + 3 + 51 + 56 + 43 + 8 - 44 - 34 - 59 - 14 - 49 + 64) + 41 - 21) + 91 + 49 - 100 + 86 - 84 - 55 - 57 - 73 + 88 - 48 + 77) + 70 - 89 - 17 - 31 - 40 - 79 - 34 + 17 + 99 - 95 - 80 - 60 - 40 - 16) - 75 + 79 + 77) - 0 + 68 - 6 + 12 + 26 - 59) + 6 + 63 - 31 - 5 + 7 + 25) - 3 - 39 + 33 + 19 + 32 + 82 - 88 - 20 + 28 - 51 - 84 - 27 + 1 + 100 + 38) + 16 + 7 - 88 - 56 - 82 - 66 - 20 + 90 + 3 - 91 + 18 - 46 + 92) + 35 - 10 + 65 + 23 - 65 + 63 - 83 - 84 - 40 + 30 + 63 + 37 - 82 + 44) + 91) - 99 + 88) - 7 - 70 - 19 + 29 - 57 + 14 + 52 + 75) + 63 - 23 + 11 + 87 + 10 + 86 + 14 - 87 - 54 + 27 + 41 - 25 + 77 + 33 + 17) + 36 - 35 - 45 + 55 + 98 + 20 - 59 + 56 - 50 + 47 + 96 - 65 + 99 + 27 - 89 - 6 + 43) + 31 + 90 + 84 - 11 + 42 + 71 - 81 + 78 + 35 - 40 - 77 - 42 + 44 + 74 + 95 - 86 + 44 - 93 + 82 + 2 - 55 - 21 + 83 - 77 + 93 + 56 - 56 - 56 - 29 - 26 - 77 + 8 + 60 + 78 - 1 - 65 - 39 + 13 - 23 + 11 + 9 + 43 - 52 + 12 - 61 - 56 - 67 - 57 + 20 + 61 - 1 + 70 + 62 - 24 + 12 - 98 + 22 + 41 - 74 - 37 + 4 + 56 + 32) - 40 - 52 + 44) - 58 - 45) - 0) + 100 - 98 - 2 + 43 + 93 + 76) - 25 - 8 - 98 + 29) + 90 - 69 - 10 - 33 + 68 + 58 + 46 - 17 + 4 + 36 + 68 + 60 + 24 + 7 - 92 - 55 + 12 + 16 + 59 - 67 - 59 + 65 + 37 + 58 + 85 + 28 + 49 - 28) + 56 - 68 - 43 + 75 + 57 + 18 - 32 + 20 - 10 + 12 + 94 - 78 - 6) + 93 - 55 - 66 - 46 + 1) - 82 - 16 + 87 - 75 - 39 + 86 + 47) + 29 - 4 + 80 + 95 - 12 + 62 - 12 + 19 + 28 - 3 - 24 - 63 + 44 - 93 - 94) + 32 + 53 - 51) - 64 + 1 - 2 + 84 + 37 - 86 - 18 + 66 + 12 - 22 - 59 + 78 + 74 + 5 - 12 - 27 - 28 + 60 - 62) - 90 - 98 - 98 + 6 - 44 + 87 - 22 + 66 + 71 + 89) + 85 - 70 + 27 + 94) + 55 + 52 + 13 + 85 - 14 - 19 - 28 - 53 + 67 + 18 + 1 - 49 + 19 - 72 - 43 - 37 - 45 - 43 - 69 - 78) + 89 + 2 - 60 + 5 - 0 - 49 - 3 + 1 + 34 - 16 + 14 + 40 - 44 + 80) + 76 + 75 - 61 - 81 - 2 + 83 - 37 - 13 - 37 + 70 - 6 + 27 - 56 - 47 - 53 - 54 - 5 - 29 - 79 + 22 - 32) - 9 + 63 - 42 + 100 + 62 + 39 - 57 + 44 + 51 + 13) - 21 + 47 - 86 - 72 - 73 + 74 + 19 + 11) - 32 + 14 + 38 + 23 + 41 - 29 + 43 - 77 - 86 + 28 + 76 - 55 + 35 - 42 - 2 + 51 + 8 + 34) - 96 - 68 - 23 + 26 - 82 - 37 - 73 - 63 + 16 - 80 + 45 - 78 - 38 + 31 - 95 - 92 + 42 - 57 - 16 + 21 + 49 + 35 - 18) - 60 + 67 + 86 + 64 - 91 + 3 + 43 - 90) - 18 - 63 + 99 + 37 - 41 - 85 + 14 - 58 + 57) - 67) + 91) + 47 - 45 + 62 + 38 + 67 + 97 - 67 + 92 + 28 - 72 + 74 - 26 - 99 + 12 + 54 - 65 + 11 - 70 - 3 + 13 - 67 + 40 + 8 - 91 - 82 - 80 + 30 - 38 - 26) + 35 + 9 + 60 + 81) - 75) - 25) + 18 + 5 - 86 - 68 + 60 + 68 + 79 + 75 - 95 - 33 - 6) - 59 + 36 + 65 + 30 - 6 + 61 + 19 + 98 + 36 - 67 + 80 + 12 + 48) - 91 + 18 - 17 - 24 + 75) + 51 - 1 + 5 - 36 - 93 - 98 - 25 + 0 - 15 - 72 - 60) - 100) - 52 + 93 + 47 + 79 + 22 + 83 - 87 - 63 - 59 - 44 - 74 - 99 - 1 + 80 - 97 - 69 + 53 + 95 - 29 - 12) + 58 - 41 + 79 + 22 + 78 + 35 + 47 - 61 + 33 + 35 + 93 + 67 + 95 - 32 + 80 - 72 + 18 + 85 + 72 - 3 - 84 - 89 - 6 - 23) + 31 - 57 - 22 + 44 + 76 - 44 - 57 + 84 + 93 - 66 + 80) - 9 + 1 + 51 - 95 - 2 - 93 - 86 - 66 - 18) + 96) - 67 + 63 - 35 - 49 - 76 + 34 + 85 + 59) + 16 - 82 - 91 - 74 - 73) - 21 - 42 + 35 + 22 + 22 + 64 - 10 - 61 + 44) + 65 - 72 - 54 + 57 - 13 - 95 + 38 + 96 - 10 - 18 - 73 + 61 - 58 - 3 - 58 - 81 + 34 - 80 - 24 - 22 - 36 - 13 + 51 + 29) + 64 - 25 - 33 - 66 + 97 - 50 - 68 + 42 - 26 - 48 + 94 + 24 - 8) - 95 - 59 - 4 + 0 + 95 - 15 - 64 - 75 - 37 + 83 - 18 - 40 - 30 - 93 - 42 + 1 - 30 - 22 - 31 - 13 + 83 - 20 - 0 + 32 + 58 + 27 + 30 + 22 + 13 - 56 - 69 + 58) - 78 - 28 + 97 - 86) - 70) + 14) - 33 + 78 - 92 + 44 - 17 + 50) + 93 - 76 - 100 + 66 + 23 + 78 + 68 + 75 - 99 - 45) + 76 - 60 - 89 - 16 + 82 - 74 - 27 + 22 + 62 + 86 + 34 - 26 - 18 + 46 - 76 - 96 + 91 - 40 + 22 - 79 - 99 + 75 - 35 + 50 - 18 - 22 - 10 + 22 + 36 - 90 + 55 - 97 - 62) - 71 + 90 - 54 - 8 + 95) + 52 + 55 + 56 - 67 + 86 - 77) - 66) - 2 - 70 - 37 - 82) - 26 - 25 - 77 + 97 + 21) - 43 + 19 - 91 + 87 - 55 + 71) + 24 - 19 - 92 + 93 + 95 + 96 - 57 - 74 - 83 + 44 - 45 - 6 + 99 + 27) - 14 + 9 + 5 - 97 - 67 + 71 - 19 + 85 - 6 + 18) - 47 - 78 + 0 - 50 - 98 - 5) - 6 - 29 + 40 + 5 + 26 + 33 + 3 - 76) - 92 - 30 - 33 + 15 - 8 - 6 - 43 - 17 - 26 - 62 + 91 - 22 - 14 - 93 + 52 - 85 - 0 - 25 + 56 - 40 - 66 + 97) - 49 + 52) - 57 + 92 + 76) + 25 - 59 - 31 + 86 - 65 + 64) - 29) + 49 + 30 + 42 + 80 + 69 - 41 - 14 + 40 + 13 - 48) + 4) + 50 + 74 - 9 - 16 + 6 - 63 - 80 - 60 - 25 + 2 - 71 + 36 + 1 + 79 + 2 - 26 - 57 + 34 + 71 + 7 + 82 - 35 - 41 - 54 + 10 + 14 + 18 + 71 - 38 + 1 - 26 + 14 + 24 - 65 + 96 + 52 + 3 - 31) - 72 - 27 + 34) + 2 + 40 - 0 - 7 + 20 - 96) - 14 - 66 - 97 - 47 + 23 + 12 - 75 - 47 + 20 - 25 - 13 - 79 + 18 - 63 + 17 + 69 + 75 - 64 - 100 - 3 - 91 - 63 - 50 + 68 + 76 + 23 - 80 - 20 + 28 + 91 - 83 - 27 + 62 + 7 + 6 + 1 - 37 + 56 - 74 - 36 + 53 - 97 - 48 + 77 - 70 - 72 + 94 + 34 + 78 - 49 - 88 - 57 + 58 + 62 + 42 - 5 + 4 + 95) + 20 - 87 - 28 + 37) + 6 + 73 + 1 + 100 + 92 - 24 - 62 + 26 - 97 - 95 + 11 - 67 - 19 - 18 + 25 - 36 - 25 + 94) - 83 + 45 + 97 - 60 - 16 + 55 - 17 - 35 - 59) + 83 - 85) - 6 + 88 + 81) - 93 + 22 - 61 - 25 + 10 - 61 + 40 + 19 + 10) + 1 + 49 - 52 - 77 + 39 + 86 - 95 - 40) - 5 + 49 + 92 + 5 + 100 + 76 + 21 - 27 - 100 + 30 + 19 + 10 + 9 - 3 + 15 - 64 + 83 + 32 + 70 + 23 + 73 + 75 - 75 + 52 - 11 - 85 - 77 - 76 - 22) - 79 + 60 + 32 + 91 - 43 + 40) + 79 - 91 - 36 + 78) - 0 + 26 + 54 + 8 + 69 - 99 + 53) + 34 - 57 - 15 + 71 + 97 + 33 - 72 - 67 + 48 + 54 + 77 + 36 - 80 + 35 - 50 + 75 - 4 + 64) - 28 + 24) + 57 - 67 - 68 - 83 - 47 - 93 + 21 + 26 - 98 + 26 + 54) - 7) + 59 + 94 - 21 + 97 + 63 + 20 + 92 - 83 - 63 - 75 + 79) + 52 + 47 + 7 - 97 - 26 - 38) - 70 - 41) - 92 - 15 + 36 + 73 + 0 + 88 - 14 + 99 - 38) + 77 + 20 - 38 - 2 + 26) - 37 + 14 + 54 - 94 - 10 + 63) - 63 - 2 + 33 + 81 - 71 - 27 - 98 - 34 - 43 + 99 - 18 + 56 + 85) + 3 + 94 + 55 + 55 + 33 - 83 - 47 + 7 + 71 - 36) - 83 + 89 + 87 + 34 + 26 - 47 + 61 - 2 + 59) - 6 - 35 - 73 - 83 - 8 + 8) + 18 - 65 - 64 + 99 + 31 + 35 - 30 - 28 + 61 - 61 - 10 - 75 + 39 + 21 + 95) + 29 + 68 - 59 - 54 - 11 - 15 + 100 + 13 + 13 - 25 - 19 + 78 + 96) + 46 - 95 - 86 - 30 + 72 + 19 + 84 + 35 - 64 + 16 + 37 - 78 + 35 + 83 - 5 + 57 - 9 + 72 - 82 - 26 + 84 + 43 - 82 + 84 + 62 - 81 - 91 - 43 - 20 + 89 + 98 + 93 - 0 + 88 - 94 - 22 + 47 - 80 + 95 - 15 + 80 - 60 + 23 + 24) + 6 - 96 - 45 - 2 + 95 + 92 + 0 - 7 + 94 - 83 - 69 - 97 - 65 - 11 + 6 + 74 - 25 + 44 - 30 + 60 + 96 - 79 - 77 + 81 - 6 + 90 + 18 - 8 - 22 + 58 + 68) - 16 - 36 + 83 + 4 - 24 + 87 + 21 - 15 - 15 + 26 + 36 + 93 + 33 - 53 - 70 + 40 + 18 - 58 + 22 - 61 + 6 + 89 - 94) + 52 + 35) - 2 - 97 + 6 - 60) + 87 + 33 + 94 + 29 - 18 + 51 + 42 - 31 - 69 - 71 + 93 + 7 + 80 - 0) + 17) - 1 - 0 - 87 + 70 - 57 + 90 + 29 - 24 - 7 - 12 + 46 - 77 - 28 + 67 - 57 + 58 - 23 + 94 - 21 + 83 - 73 + 49) - 26 + 100 + 64 + 58 + 20) - 71 - 59) - 52 - 13 + 91 + 40 - 80 - 79 + 29 - 67 - 47 + 68 + 39 + 78 - 87 - 81 - 61 + 14 + 71 - 54 - 11 + 93 - 21 - 2 - 12 + 20 - 94 - 26 - 100 + 26) - 47 - 65 - 29 + 97 + 4 - 52 - 30 - 50 + 20 + 63 + 11 - 19 + 0 - 78 - 8) - 79 - 50 + 99 + 70 - 86 + 5 - 14 - 98 + 62 + 68 - 74 - 41 + 94 - 87 - 18 - 86 + 89 + 30 - 2 + 89 + 39 + 24 - 8) - 55 + 26 + 78 - 31 + 43 + 30) + 52 - 60 + 2 + 3 + 20 + 63 - 44 - 24 - 63 - 98 - 47 - 14 - 79 - 24 + 13 - 90 + 26 - 76) + 95 + 23 + 91 + 56 - 24 + 10) + 99 + 0 - 27 + 25 - 68 + 22 + 87 + 86 + 57 - 0 - 57 + 83 - 41 + 10 - 42 + 98 - 68 + 83 + 99 + 62 - 94 + 36 - 36 - 59 - 23 - 59) + 27 - 29 - 87 - 26 - 68 - 74 + 60 - 59 - 88 - 48) + 22 - 99 + 96) - 62) - 50 - 17 + 16 - 100 - 91 + 100 - 27 + 82) - 21 + 42 - 24) - 28 - 16) + 85 + 89 + 85 - 66) + 67 + 11 - 20 - 31 - 11 - 29 + 70 - 1 + 22 + 91 + 26 + 46 + 74 - 65 + 14 + 31 + 0 + 35 - 24 - 23 - 79 - 79 - 59) - 66 + 84 + 16) + 25) + 96 - 98 - 33 + 74 - 35 - 25 + 15 - 8 + 70 + 61) + 72 - 77 - 47 + 34 + 84) + 31 - 96 + 87 - 6 - 53 + 14 - 82 + 33) + 30 + 74 - 14 - 90 - 82 - 40 - 87 - 84 + 47 + 100 - 1 - 1 + 12 + 18 - 73 + 9 - 76 + 13 + 39 + 68 + 66 - 88 - 35 + 26 - 82 + 57 + 36 - 10 - 21 - 48 - 17 - 87 + 23 - 40 - 4 + 24 - 4 - 9) + 68 - 97 + 19 - 40 + 49 - 60 - 98 + 32 + 58 + 25 + 82 - 15 + 57 - 84 + 66 + 84 - 62 - 20 + 87) + 11 + 86 - 45 - 3) + 65 - 97 - 91 - 34) - 55 + 80 - 46 + 39 + 32 - 99 - 44 + 88 + 53 + 29) - 8 + 28 + 95 - 31) + 45 + 39) + 14) - 52 - 70 - 21 - 21 - 63 + 60 - 7 + 75 + 77 - 20 + 89 - 46 + 19 - 64) - 77 - 24 - 67 + 75 - 63 - 35 - 17 + 96 + 10 + 77) + 74 + 27 + 86 + 100 + 78 + 58 - 66 + 43 + 76 + 93 - 5) + 62 - 44 - 73 + 84 - 63 - 83 - 11 - 12 + 63 - 90 + 30 - 57) - 52 - 34 - 2 - 89 + 63) - 32 - 60 - 27 - 56 - 7 - 88 - 44 + 26 + 91 - 97 + 18 + 93 - 25 + 48) - 26 + 3 - 62 + 3 - 18 - 88 + 47 + 50 - 47) - 89) - 63 + 50 + 32 + 62 + 100 - 17 + 50 + 85 + 9 + 26 + 92 - 9 + 65 + 41 + 37 - 52) + 83 + 3 + 50 + 29 + 40 + 53 - 27 + 8 + 92 + 36 + 0 - 33 - 22 + 1 + 59 + 62 - 21 - 86 + 5 + 77 - 83 - 92 + 55 + 35 + 91 - 69 + 20 - 73 + 17 + 68 + 28 - 37 + 8 + 64 - 15 + 84 + 22 - 24 - 60 - 98 - 68 + 86 + 10 + 17 + 46 - 44 - 59) - 77 - 72 - 98 - 67) + 20 + 82) - 39 + 19 - 22 - 1 - 60 - 63 + 77 + 68 + 58 + 26 + 33 + 48 - 50 - 0 + 67 + 71 - 79 + 4 + 15 + 97 - 85 + 71 + 6 + 20 - 82 + 22 - 91 + 83 - 82 + 68 + 29 + 50 - 14) + 60 - 84) + 73 - 28 + 71 - 47 + 37 + 38 + 72) - 4 - 93 - 70 + 42 + 80) + 41 - 34 + 38 - 91 - 14 + 0) - 87 - 26 + 68 + 7 + 83 - 20 + 86 + 67 + 14 + 19 + 19 - 47 + 55 + 19 - 53 + 100 - 64 - 0 + 8 + 59) - 9 - 16 - 100 - 50 + 11 + 0 - 56 + 2 - 77 + 3) - 87 + 35 - 91 + 100 - 70 + 52 - 20 - 82 - 27 + 37 + 14 + 22 - 23 + 97 - 28 - 52 - 2 + 87 + 35 + 83 - 6 - 48 + 76 - 36 - 92 - 45 - 66 - 30 + 30 - 70 + 79 + 9 + 4 - 98) + 83 - 94 - 51 + 61 + 77 + 48) + 96 + 8 - 81 + 63 + 69 - 49 + 1 - 2 - 48 + 31 - 8 - 26) - 91 + 27 - 98 + 32 + 48 + 37 + 65 - 80 - 0 - 23 + 36 + 65 - 57 - 13 - 60 - 2 - 18) + 87 + 66 + 0 - 45 - 64 + 74 - 82 + 66 + 71 - 55 - 51 + 43) + 56 + 26 + 85 + 88 - 35 - 53 + 21 - 8 - 45 + 16 + 22) - 9 - 54 - 45 + 84 + 89 - 11 + 100 + 84 - 83 + 62 - 9) + 49 + 85) - 38 - 11 - 88 - 30 + 10) + 34 - 99 - 93 + 99 + 47 - 59 - 46 - 15 + 54 - 67 - 45 + 64 - 36 + 64 - 62) - 81 + 18 - 67 + 68 - 25 + 52 - 35 + 61) + 18 + 49 + 9) - 41 + 86 + 5 + 1 - 57 - 76 + 52 + 52 + 30 + 82 + 3 - 5 - 27 - 45 - 17 - 16 - 55 - 16 - 82 - 58 - 37 - 76 + 9) + 7 + 14 - 87 - 52 - 41 - 43 + 16 - 4 + 48 + 58 + 3 - 20 + 8 - 42 + 80 - 78) + 95 + 62 - 92 + 80 - 37 - 41 + 31 + 32 - 88 + 32 + 54 + 48 + 87 + 58 + 86 + 7 + 46 + 51 - 77 - 85 - 40) - 78 - 78) - 92 + 83 + 90 - 94 - 95 - 1 + 0 - 13 + 72) - 79 + 76 - 88 + 41 - 40 + 14 + 75 - 34 - 51 + 98) + 83 - 91 + 90 + 100 - 88 - 7) + 67 - 96 - 84 + 24 + 40 - 19 - 54 + 20 - 63 + 76 + 29 + 33) - 55) + 98 + 17 + 21 - 38 + 12) - 95 + 85 - 37 - 98 + 36 - 52 - 74 + 11 - 44 - 14) - 59 - 25 + 18 + 81 + 68 + 61 + 100 - 0 - 46 + 36 - 87 - 36 + 99 - 93 + 31 - 44) - 50 - 10 + 73 - 78 - 58 - 89 - 92 - 3 - 95 + 75) + 16 - 38) + 32 - 32 + 71 + 83 - 15 + 18 - 94 + 4) + 2 - 50 - 68) - 29) - 17 - 23 - 7 + 96 + 64 + 39 + 38 + 62 - 43 - 16 - 15 - 54 + 7 - 54 + 3 + 81 - 43 - 20 + 20 + 66 - 45 - 3 + 6 + 70 - 98 - 100 - 56 - 66 + 46 + 21 - 71 - 18 - 12 - 23 + 23 + 20 + 6 + 89 - 12 - 12 + 45 - 28 + 61 + 54) + 92 + 59 + 24) - 46 + 84) + 21 + 65 + 87 + 81 + 1 - 6 + 79 - 25 - 32 + 1 + 30 - 82 - 57 + 9 + 9 + 60 + 92 + 34 - 29 - 29 - 62 - 1 - 97 - 78 - 32 - 11) + 18 + 91 - 34 + 4 - 6) - 5 - 66 + 81 - 98 - 21) + 70) + 33) + 15 - 100 + 47 + 63 + 88 + 41 - 34 - 27 + 88 + 13 + 84 - 5 + 45 + 58 + 89 + 88 - 59 + 41 - 57) + 34 - 92 + 52 - 93 - 4 + 21 - 23 + 13 - 72 + 87) + 8 - 2 - 67 + 42 - 81 + 92 + 88 + 56) - 49 + 37 - 37 + 40 - 14 + 17 + 10 - 61 - 64) + 22 - 96 + 6 + 33 - 14 + 48 - 71 + 65 - 53 + 35 - 95 - 98 - 69 - 54 - 43 - 96 + 51 + 49 + 11 - 57) - 7 - 60 + 18 + 8 + 62 + 9 - 40 - 32 - 89 + 74 - 27 + 95 + 77 + 40 + 20 + 35 - 93 - 99 + 64 + 1 + 10 - 100 + 81 - 5 - 52 - 37 + 73 - 38 + 79 - 82 + 76 - 30 - 8 + 24 + 31 + 59 - 69 - 18 + 93 - 52 - 14 + 17 - 61 + 67 - 23 + 42 - 93 - 4 + 51 + 24 - 53 - 13 + 58) - 56 + 20 - 79 + 51 - 6 + 75 - 83 - 34) + 8 - 98 - 87 + 54 + 5 + 86 + 39 - 15 + 62) + 3 + 90 + 11 - 92) - 72 + 67 - 40) + 0 + 89 + 90 - 43 - 11 - 65 - 54 - 93 - 11 + 4 + 2 + 60) - 22 - 96 + 98 - 89 - 93 + 35 + 47 + 28) + 37) + 24 + 64 - 33 + 91 + 3 + 22 + 84 + 21 + 89 - 83 + 62 - 29 + 98 - 18 + 77 + 69 + 20 - 67 + 96) - 90 + 21 + 44 - 12 - 58) + 0 - 83 - 93 - 30 - 62 + 90 + 47 - 4) - 76 - 99 + 24 - 12 - 42 - 44 + 66 + 0 - 80 - 83 + 90 + 88) - 21 + 6 - 41 - 47 + 24 - 62 + 86 - 2 + 68 - 42 + 99 - 22 + 38 - 33 + 57 + 20 + 87 - 99 + 52 - 82 - 73 + 23 + 1 - 92 - 80 + 48 - 62) + 93) - 0 - 4 + 21 - 85 - 64 + 58 + 95) - 83 - 34 + 53 - 86 + 8 - 37 + 41 + 25 - 83 - 55 + 42 + 74 - 74 + 99 - 61) - 59 + 39 + 96 + 53 + 45 + 68 + 89 + 23 + 85 + 70 - 61 + 64 - 7 - 33) + 26) - 57 + 33 - 41 + 54 - 38 - 82 + 64 - 88 + 53) + 41 + 76 + 97 - 93 + 8 - 76 + 76 + 71 + 68 - 1 + 58 + 80 - 36) + 5 + 66 - 50 - 90 - 95 + 39 + 13 + 87) + 46 + 46 - 44 + 82) + 93 + 98 - 96 - 6 - 67 + 7) + 29 + 61 - 37 + 37 + 15 + 62 - 90 - 49 + 49 - 9 - 66 - 35 - 17 + 24 + 91 + 27 + 32 + 99 - 37 + 78 + 76 + 100 - 7 - 56 - 57 + 60 + 61 - 28 + 20 + 82 - 72 + 41 + 82 + 26 - 76 - 21 - 84 + 22 - 58) + 55 - 15 + 64 - 96 - 0 - 51) - 4 - 80 - 63 + 10 + 81 - 64 + 0 - 85 - 2 + 5 + 18 - 80) - 3 + 12 + 19 + 93) - 96 - 15 + 34 + 14 + 20 - 85 - 17 - 59 + 66 - 34 - 63 + 53 + 45 + 0 - 100 - 33 - 13 - 85 + 24) + 40) + 66 + 87 - 98 - 22 + 10 - 81 - 62 + 77 + 31 + 30 + 52 - 77 - 81 + 66 + 90 + 78 - 94 - 51 + 7 + 63 - 73 + 29 - 57 - 13 - 15 + 52 + 41 - 57 + 0 - 14 - 91 - 52 + 37 - 12 + 68) - 49 - 97 + 97 - 20 - 24 + 67 + 87 + 26 - 13 + 21 - 53 + 43) + 39 - 77 + 25) + 99 + 45 + 96 - 74 + 33 + 29 + 10 - 71 + 65 - 67) - 37 - 39 + 50 - 53 - 54 + 50 - 35 - 32) + 13 + 24 - 88 + 72 + 11 - 19 - 39 - 39 + 33 + 10 + 33 + 20) - 34) - 49 + 2 - 53 - 83 + 32 - 40 + 47 - 37 + 91 + 85 + 24 + 36 + 88 - 15 + 8 - 63 - 58 + 76 + 19 + 35 + 72 - 40) + 37 + 4 - 9 - 7 + 55 + 47 - 81 + 41 + 92 + 10 + 78 - 9 - 57 + 64 - 89 + 31 - 20 - 58 - 34 - 51) - 75 - 92 + 16 + 47 + 72 - 90 + 52 + 18 + 60) - 41 - 78) - 86 - 91) + 33 + 2) - 28 - 91) + 82 - 42 - 86 + 26 - 30 + 89 + 84 + 72 - 77 - 98 + 82 + 5 - 60 - 70 + 63 + 55 + 91 + 62 - 76 + 25 - 100 + 91 + 31 + 17 + 56 + 60 + 31 - 72 - 48 + 79) + 98 + 35 + 39 - 19 + 54 + 41 - 97 + 61 + 44 + 95 + 19 - 59 + 14) - 21 - 28 + 6) + 60 + 4) + 0 + 45 - 82 - 80 + 92 + 74 - 20 - 20 + 94 - 60 - 54) - 65 - 35 - 78 - 47 + 37 + 13 + 18 - 81 - 4 + 6 + 38 - 0 - 47 + 2 + 36 + 86 + 42 + 48) + 29 - 25 + 26 + 56 + 40 - 61 + 4 + 92 + 77 - 37 + 74 + 60) - 43 - 43 - 63 - 6 + 47 + 67 + 95 + 15 + 53 + 15 - 84 + 37 + 28 + 52 - 45 - 12 + 77 - 54) + 19 - 56 - 28 + 44 - 30 + 75 - 36 + 2 + 62 + 53 + 84 - 50 + 13 + 66 + 12 - 39 + 61 - 5 + 47 + 64 - 0) - 12 + 16 + 95 - 48 + 36 + 75 + 19) + 72 + 14 - 72 - 68 - 38 - 66 - 79 + 34 + 54 - 22 + 93 + 8 - 55 - 98 + 99 + 21 + 78 + 36) - 75 + 75 - 16 - 93 - 85) - 84 + 62 + 24 - 59) + 55 + 31 - 10) + 93 + 66 - 42 - 1 + 57 + 87 - 39 - 90 - 17 + 60 - 0 - 44 + 46 + 11 + 33 + 24 + 49 + 25 - 40 - 72 + 78 + 17 - 31 - 40 + 54 + 4 + 54 - 58 - 47 + 10) + 21 - 7 + 92 - 90 - 36 + 53 - 21 + 12 + 51 + 94 - 89 - 57 - 3 + 17) + 42 - 74 - 73 - 69 - 80 + 36 + 96 + 50 - 17 + 44) + 44 - 87) - 74 - 46 + 71 - 42 + 29 - 57 + 66 - 53 + 85 + 15 - 35) - 47 - 79 + 83 - 41 + 16 - 23 + 3 + 48 - 94 - 17 + 20 + 33 + 15 - 3 + 55 - 18 + 47 - 82 + 83 + 51) - 45 + 67 - 75 - 94 + 24 - 83 - 41 - 37 + 11 - 10 - 86 - 0 + 18 - 55 + 57 - 71 + 97 + 76 - 7 - 58 - 73 + 100 + 30 - 37 - 24 + 47 + 53 + 40 + 29 + 56 - 33 + 33 + 50 + 99 - 21 - 81 + 65 + 74 + 16 + 92 - 99 + 19 + 65 - 2 + 19 + 2 + 75) + 2 - 39 - 80 - 26 - 99 - 84 + 12 + 8 - 48) + 42 - 97 + 54 - 65) - 53 - 94 + 30 - 73 - 47 + 63 + 99 + 56 + 63 - 95 + 40 + 51 + 48 + 89 - 59 + 62 + 63) + 13 - 7 + 1 - 94 + 38 - 63) + 21 - 29 + 80 - 17) - 34 - 36 + 8 + 96 - 63 - 98 - 73 - 98 - 100 - 43 + 82 - 56 - 47) + 71 + 20 + 99 - 83 + 58 + 36 - 91 + 80) + 91 + 54 + 51 + 25 + 19 + 0 - 74 + 24 - 72 - 19 + 0 - 79 + 12 + 86 + 35 - 0 + 60 - 86 - 84 - 34 + 41 - 97 - 62 - 43 + 26 + 13 + 77 - 80 + 55 + 97 + 30 - 25 - 52 - 23 - 14 - 85 - 65 + 66 - 53 - 71 + 87 - 18 + 56 - 98 - 45 + 24 - 97 + 87 - 34 - 47 + 39 + 45 + 28 - 75 + 44 - 95 - 48 + 69 + 23 + 24 - 63 + 45 + 86 + 99 + 91 - 37 + 4 - 64 - 42) + 75 - 87 - 36 + 44 - 87 + 51 - 19) - 34 + 87 + 99 - 11 + 87 + 9 - 75 + 79 + 36 + 8 + 19 - 94 + 29 - 88 + 46) - 8) + 30 + 31 + 97 + 100) - 86 - 64 - 18 - 69 - 97 + 8 - 64 - 67 + 35 - 0) + 87 - 58 - 68 + 79 - 60 - 13 - 35 + 15 - 66 - 7 - 72 + 95 + 3 + 46 - 24 - 64 + 20 + 5 + 99 + 55 + 8) - 20 + 75 + 32 + 72 - 21) - 2 + 6 - 57) + 94 + 88 - 70 + 6 - 42 - 70 - 55 + 90 + 34 - 14 - 96 + 48 + 73) + 64 + 64 + 75 + 70 - 12 + 59 + 78) - 46 - 56 + 2 - 67 + 79 - 62 - 55 - 32 + 61 - 27 + 5 - 98 + 24 + 91 + 35 - 92 + 84 - 89 - 85 - 67 + 12 - 22 + 86 - 42) + 35 - 31 + 2 + 7 + 80 + 16 + 78) + 98 - 62 - 76 - 70 + 29 - 98 + 63 - 35 - 57 - 14) + 98 + 82 - 7 - 78 + 12 + 74 - 73 + 51 - 0 - 32 + 97 + 28 - 82 - 30 + 41 + 54 + 82 - 2 + 55 + 12 - 18) + 7 - 95) - 87 - 92 - 51 + 21 - 94 + 16 + 98 + 22 - 37 + 56 - 48 + 35 - 100 + 97 - 66 + 19 + 4 + 45 + 13) + 62 + 83 - 53 - 73 + 92 + 77 + 70 - 21 + 89) + 27 + 94 + 55) + 34 - 48 + 70 - 32 + 14 - 72 + 85 - 33 + 38 + 66 + 45) - 56 + 39 + 36 + 36 + 0) - 51 + 64 + 20 - 55 - 75 - 34 + 6 - 88 + 31 - 3 + 14 - 11 + 78 - 28 - 31 - 63 + 87 + 54 - 82 + 3 + 42 - 66 - 9 + 35 - 34 - 18 - 78 - 78 + 21 - 35 + 27 + 25 + 43 + 18 + 49 - 62 + 58 + 69 - 67 + 46 - 67 + 62 - 88 + 23 - 89 + 61 + 21 + 93 + 79 + 37 + 81 + 65 - 80) + 11 + 93 + 55 + 0 - 91 - 73 - 9 + 100 - 79 + 57 - 75 + 37 + 90 + 36 + 88 + 85 - 97 - 44 - 0 - 52 + 2 - 75 + 34 + 21 + 1) - 67 + 21) - 35 + 15 - 46 - 89) + 83 - 0 - 86 - 29 + 86 + 10 - 66 - 52 - 22 - 27 - 48 + 8 - 37 - 2 - 97 - 65) - 52 + 41 + 13 + 81 + 84 - 46 - 38 + 90 - 39 - 44 + 19 - 58 - 92 - 82 + 9 + 97 + 77 + 32 - 43 + 48 + 61 - 52 - 8) + 4 - 8 + 97 - 9 - 52 + 93) - 79 - 77 - 61 - 2 - 2 - 95 + 53 + 42 - 42 + 60 - 88 + 72 + 58) - 60 - 43 - 58 + 33 + 56) + 75 + 64 + 90 - 26 - 52 - 31 - 77 + 10 - 43 - 88) - 10 - 36 + 49 - 82 + 97 - 26 - 9) + 55 - 45 - 35 - 60 + 4) - 91 - 51 - 96 + 31 + 63 - 91 - 56 - 70 + 6) - 68 - 73 + 43 - 21 - 64 + 20 + 61 - 26 - 88 - 99 - 58 - 90) + 71) + 38) + 33 + 91 + 50 + 7 + 72 + 42 + 24 + 37 - 76 - 24 + 22) + 74 + 27 + 45 - 39 + 66 - 37 + 91 - 67 - 6 - 64 + 68 - 93 + 81 - 92 - 65 + 47 - 33 + 35 + 8 - 52 + 86 + 7 - 82 - 93 - 20 + 8 + 14) + 3 + 84 + 62 + 94 + 78 - 59 + 26 - 9 + 8 + 26 - 42 - 34) - 47 + 38 - 85 - 52 + 20 - 7 + 41 + 31 - 77) - 95 + 81 + 100 - 50 + 70 + 97 - 68 - 63 - 27 + 60) + 63 + 25 - 65 + 5 - 67 + 23 - 88) + 62 + 0) - 97 - 98 + 76 + 51 - 28 - 6 - 69 + 24 - 98 - 99 - 46 + 12 - 7 + 62 - 94 - 94 + 53 - 81 + 60 - 91 + 38 + 9 + 86 + 49 + 99) - 12 - 11) - 99 - 47 - 38 + 71 - 88 - 79) + 83 + 2 - 22 + 44 - 3 + 66 - 7) - 99 + 38 - 78 - 13 - 97 - 71 - 87 - 19 + 98 + 78 - 6 - 47 - 17 + 64 + 77 - 81 + 47 + 98 + 41 - 51 + 58 - 16 - 4 + 77 + 96) + 46 - 81 - 56) - 38 - 95 + 96 + 95) + 42 - 32) - 97 + 15 + 54 + 32 - 10 - 14 + 44) - 59 + 2 - 35 + 64 - 69 + 59 + 9 + 69 + 0 - 55 - 75 + 85 - 31 + 22 + 97 + 6 - 39 + 32 - 41 + 89 - 81 - 29 - 12 - 40 + 25) - 21 - 56 + 24 + 42 - 21) + 47 + 32 + 42 + 28 - 14 + 1 - 4 + 31 + 41) + 59) - 72 - 94 - 79 - 63 - 19 - 0 + 72 - 37) + 15) + 88 - 5 - 16) - 6 + 5) + 73 + 51 + 95 - 72 - 59 - 55 + 67 + 59 - 32 + 48 - 41 - 12 - 87 + 50 + 66 + 56 - 5 - 8) - 37 + 26 + 86 - 16 - 96 - 98 + 55 + 28 - 15) - 53 + 82) - 28 + 25 - 67 + 96 + 2 - 31 + 12 + 99 - 94 + 89 - 24 - 85 + 59 + 39 - 51 + 97 + 4 + 65 - 12) + 44 - 71 - 95 - 7 - 20) + 99 - 61 - 15 - 7 - 98 - 54 + 3 + 96 + 70 - 53 - 2 + 84 - 36 + 83 - 17 - 39 - 23) - 66 + 2 + 52 + 2 + 37 + 24 - 22 - 83 - 86) - 41 + 26 - 88 + 33) - 94 + 27 - 81 - 96 - 79 + 82 + 2 - 25 - 97 - 45 - 100 - 26 + 32 - 52 - 4 + 80 + 35 - 4 + 31 - 5 - 43 + 91 + 5) + 73 - 75 + 9 - 34 - 5) + 57 + 28 - 80 - 10 + 9 + 56) + 79 - 15 - 30 + 64 - 87 - 94 + 69 - 23 + 10 - 7 + 2 - 61 + 48 + 91 + 48 - 90 - 17 + 70 + 63 - 90 + 21 + 30) - 57 - 13 + 72 - 5 - 47 - 32) + 19 + 27 - 13 + 87 - 6 + 50) - 79 + 37 - 74 - 33) + 54 + 18 + 33 + 53 - 66 - 83 - 48 + 21 - 40) - 94 - 83 - 45 - 59 - 89 + 18 + 41 - 10) + 18 - 14 + 23 + 85 - 18) + 55 - 66) - 20 + 40 + 96) + 34 + 56 - 49 - 93 - 43 + 27 + 54) + 7 + 41 + 52 + 16 - 35 + 19 + 49) + 32 + 83 + 10 + 93 + 99 - 18 + 52 - 78 - 76) - 70 - 100 + 9 - 47 - 44 - 27 - 60 - 2 + 39 - 85 - 37 + 3 + 75 + 17 - 15 - 22 - 34 + 88 - 29) + 6 + 67 + 25 + 36 + 19 + 76 - 56 - 19 + 98 + 3 - 27 + 46 - 59 + 98 + 45 + 25 + 32 + 40 - 59 + 9 + 38 - 84 - 90 - 71 + 58 - 62) + 56 - 100 + 56 + 18 + 49 - 44 - 55 + 1 - 74 - 66 - 25) - 68 - 90 + 28 - 70 + 12 + 49) + 33 - 13 - 6 - 54 - 56 - 0 + 79) + 43 + 62 + 18 + 8 - 85 - 40 - 32 + 96 - 4 - 75 + 88 - 39 + 14) + 87 + 44 + 5 + 87 + 97 - 98) + 85 + 74 + 20) + 44 + 62 - 92 - 57 + 25) + 67 + 75 + 88 - 97 + 61 + 84 - 24 - 18 - 39 + 48 + 5 + 27 - 12 + 57 - 24 + 6 + 5 - 12 + 52 - 14 - 64 - 58 + 46 + 70 + 54) + 72 + 13 + 46 - 15) + 33 - 75 + 93 + 83 - 64 - 14 - 47 - 32 - 15 - 29 + 84 - 35) + 7) + 57 + 92 - 27 - 63 + 5 - 6 + 9 - 50 + 98 - 13 + 38) - 34 - 49 + 12 + 40 - 46 - 14 + 7 + 52 - 0 + 52 - 66 + 24 - 87 + 12 - 92 + 29 + 47 + 50 + 67 + 31 + 10 - 1) + 48 - 52 + 3 - 23 + 100 + 33 + 5 + 74 - 67 + 3 + 27) - 60 + 19 + 63 + 91 - 45 - 19 + 34) + 14 - 6 - 41 + 3 - 82 + 97 + 36 + 95 - 71) - 2 - 51 + 97 + 4 + 74 - 80 - 13 - 28 + 36 + 86 - 25 - 89 - 100 + 28 + 14 - 67 + 66 - 92 + 60 - 23 - 63 - 76 + 39 - 99 - 34 - 37 - 28 + 30 - 37 - 23 + 34 - 0 - 94 + 81) + 74 - 39 + 46 + 72 + 53) - 11) - 50 - 9 + 44 + 90 - 8 - 50) + 4 + 61 + 65 - 77 - 37 + 60 + 81 - 99 - 8 + 72 - 12) + 18 - 32 + 38 + 32 - 86 + 23 - 100 + 99 - 99 - 18) + 3 + 34 - 66) + 1 - 16 - 38 + 5 - 62) - 88 - 39 - 87 + 67 - 45 + 20 - 60 - 54 - 95 + 80 - 48 - 76 + 38) - 31 + 47 - 83 - 70 + 55 - 51 + 81 + 6) - 60 + 19 - 56 - 19 - 83 + 81 + 14 - 87 - 38 - 57 - 97 + 36 + 37 - 30 - 9 - 2 - 58 + 20 + 30 - 81 - 20 - 81 - 58 - 59 + 96 + 89) + 85 + 41 - 55 + 70 - 93 + 52 - 34 - 99 - 74 + 14 + 69) + 81) - 30 - 77 - 28 - 29 - 14 - 17 - 40 + 59) - 88 + 90 + 51) - 94 - 87 + 6 + 52 + 13 + 49 + 87 - 31 - 33) - 80 - 73 - 22 - 65 - 90) - 74 - 41 - 0 - 60 + 50 - 63 + 2 - 0 - 79 + 56 + 1 - 24) - 74 - 6) + 95 + 99 - 24 - 24 - 40 + 5 - 87 + 32 + 89 + 35 - 41 + 41 - 96 - 31 + 17 + 80 + 48 + 86) - 8) + 42 + 70 + 12 - 67 - 25) + 19 + 30 + 0 + 3 + 32 - 74 + 91 - 43) - 39 + 64 + 73) - 18 - 100 + 60 - 45 + 93 + 24 + 33 + 82 + 70) + 61 - 78 + 78 + 89 + 0 + 60 - 56 + 63) - 94 + 94 - 99 - 40 - 69 + 19 - 97 + 70 + 74 - 88 + 5 + 86 - 15 - 68 - 86 - 60 - 72 + 85 + 23 + 28 - 62 - 44 - 58 + 82 + 34) - 74 + 42 - 32 - 62 - 98 - 49 + 61 - 96 + 65 - 95) - 72 + 85 - 8 - 52 + 61 + 15 + 19 + 5 + 78 + 71 - 34 + 69 + 53 - 60 - 9 - 30 - 23 + 54 - 69 + 29 - 83 - 74 + 8 - 13 - 20 - 84 + 81 + 28 + 95 + 23 - 6 - 96 + 72 - 87 - 39 - 8 + 70 + 80 + 22 - 54) - 100 - 90) - 45 + 76 + 23 + 69 - 3 - 38 - 8 + 38 + 33 - 38 + 49 - 59 + 90) + 96 - 10 + 11 + 97 + 10 - 86 + 63 - 79) + 28 + 21 + 56 + 25 - 47 + 70 - 54 - 91 - 69 + 66 + 79 - 48 + 28 + 53 - 48 - 98 - 7 - 79 - 11 - 27 - 6) - 34 + 14 - 73 - 22 - 53 - 20 - 40 - 8 - 38 + 25 + 10 + 38 - 75 + 36 - 11 + 49 - 13 - 4 - 87 - 63 + 86 - 43 - 58) + 94 + 29 - 59 - 88 + 84 + 81 - 5 - 25 - 47 + 71 + 50 - 87 - 22) - 24 + 11 + 12 + 24 + 73) + 83) + 74 + 15 + 91) - 77 + 94 + 19 + 1 + 20 - 74 + 58 - 23 - 66 - 21 - 5 + 53 - 81 - 49 + 83 - 42 - 85 + 32 + 95 - 34 + 77 + 50 - 89 + 0 + 92 - 73) + 100 - 72 + 94 - 95 - 25 + 14 + 4 - 80 - 26 - 57 + 57) - 48 - 4 - 22 + 57 + 55 - 56 + 39 - 76 - 40) + 90 - 53 - 0) + 18 + 52 + 2 - 51 + 45 + 88 + 32 + 28) + 91 - 97 - 25 + 69 + 19 + 20 + 55 - 92 - 18 - 53 + 18 - 44 - 21 - 81) - 15 + 13 - 51 + 100 - 18 - 83 + 73 - 71 + 43 + 96 - 93) + 51) - 82 + 1 - 94 + 27 - 91 + 78 - 12 + 21 - 42 + 13 - 15 - 6 + 25 + 4) + 41 + 47 - 15 + 12 - 9 + 84 - 56 - 94 + 19 + 16) + 6 + 1 - 64 - 89) - 6 - 82 + 43 - 12 + 26 + 87 + 54 - 60 + 65) - 19 + 1 + 18 - 86 - 82 - 40 + 55 - 57 - 28) - 47 - 1 + 99 + 72 + 91 - 78 - 65 - 86) + 51 + 89 + 54) - 85) - 44 + 54 - 13) + 13 - 53 + 98 + 31 - 64 + 15 - 2 + 37 - 86 + 5 - 4 - 37) + 37 + 83 + 21 + 56 + 17 + 88 - 10 + 52 - 11 + 26 - 20 - 56 + 22 - 25) - 61 - 39 - 58 - 5 - 33 + 75 - 36 + 37 + 7 - 2 + 67 + 18 - 2 + 26 + 0 + 68) - 32 - 23 - 91 + 26) + 81 - 23 - 73 + 13 - 39 - 75 + 5 + 26 - 87 - 20 + 54 + 61 - 50 - 19) - 39 - 40 - 0 + 75 - 53 + 81 - 5 - 44) - 60 - 94 - 15 + 27 - 87 - 63) - 10 + 52 - 66) - 74 - 7 + 48 - 63 - 27 - 58 + 81 + 20) + 33 - 0 + 11 + 7 + 35 + 0 + 94 - 29) + 27 - 14 - 90) + 13 + 43 - 30 + 98) + 69) - 88 - 77) + 99 - 1 - 9 - 81 - 79 - 17 + 83 + 98 + 49 - 82 - 51 - 39 - 93 - 4 - 28 + 31 - 50 - 72 + 31 + 19 - 76 + 86 - 80 - 10 - 46 - 44 - 51 + 15) + 51 - 46 + 39 + 34 - 52 - 74 + 88 - 81 + 57 + 12 - 44 - 92 - 54 + 97 + 96 - 4 + 26 + 70 + 54 - 62) + 92 + 96 - 22 - 63 + 49) + 49 - 40) + 39 + 93 - 1 - 55 + 100 - 61 + 98 - 79 + 14 - 94 + 56 + 54 + 88 + 8 - 19) + 39 + 65 - 76 - 27 - 2) + 3) - 94 + 50 - 86 - 18 + 92 + 17) + 29 + 24 - 40 - 54) + 50 - 7 - 64) - 25 + 21 + 59 - 57 + 59) - 99 + 11 - 21 - 73 + 82 - 88 - 64 + 32 + 78) + 98 - 5 + 12 + 34 + 89 + 20 + 45 + 72 - 20) + 79 - 36 + 24 - 49 + 60 - 2) - 72 - 2 + 41 - 28 + 8) - 16 - 74 + 94 - 5 + 46 + 40 + 22 - 32 + 86) - 54 + 47 + 55) + 80 + 93 - 1 + 51 - 12) - 93 - 2 - 69) + 50 + 55 + 7 - 88 + 75 + 84 + 64 + 11 + 23 + 43 + 26 - 41 + 34 + 28 - 28 + 93 - 46 + 100 + 44) + 56 + 6 + 56 - 72 - 36 - 57 - 29 - 9 - 39 + 32 - 22 - 15 - 94 - 12 + 54 + 27 - 88 + 52 + 41 - 53) + 52 - 79 + 19 + 7 + 48 - 15 + 56 - 70) + 30 - 82 - 24) + 5 - 68 + 56 - 73 - 89 - 25 + 96 - 96 + 78 - 23 - 85 + 56 + 12 + 9 + 88 - 54 + 34 + 13 - 13 + 95 + 20 - 60) + 94 + 32) + 2 + 93 + 87 - 80 - 73 - 95 + 21 - 54 - 36 - 69 + 25 + 58 + 30 - 34 + 89 - 75 + 45 - 11 - 14 - 92 + 13 - 62 - 59 - 56 + 26) + 25 - 39 + 93 - 24 + 43) - 30) - 93 + 20 + 20 + 32 - 95 + 47 - 44 + 59 - 95 + 69 - 63 - 90 - 0 - 42 - 33 + 29 + 25 + 50 - 38 + 39 + 86 - 10 - 62 - 59 + 63 - 51 + 44 - 86 - 30 - 6 - 18 - 56 - 23) + 80 + 43 + 11 + 59 + 62 - 68 + 68 + 36 - 11) + 73 - 52 + 83 + 1 + 4) + 5 - 74 - 72 + 27 + 81 + 71) - 80 - 24 + 69 - 21) + 35 - 70 + 35 + 95 - 47) - 11 + 63 - 34 - 64 - 26 + 91 - 39 - 45 + 18 - 52 + 87 - 47 + 75 + 6 - 5 + 51 - 89 - 12 - 64 + 81 - 76) - 92) + 59) - 88 + 3 - 71 + 1 - 89 + 63 + 37 + 30 + 0 + 90 + 15 + 90) + 53 - 48 + 81 - 9 + 41) + 84 + 77 + 69) - 56 - 96 + 45 - 29 + 15 + 66 - 24 + 86) - 88 - 29 - 50 + 29 + 19 - 68 + 98 + 8 - 47 - 75 - 22 - 86) - 25 - 58 + 72 - 21) - 85 - 42 + 88 + 42 - 74 + 4 + 94 + 34 - 91 + 54 - 62 + 23 + 56 - 44 + 93) + 1 + 36 - 78 + 56 - 12 + 61 + 43) - 46 + 2 - 55 - 97 - 54 + 43 + 95 - 36 + 92 - 75 + 29 - 55 + 24 - 71 + 78) - 81 - 67 - 26 - 53 + 32 - 76 + 42 - 8 + 68 + 77 + 92 - 100 - 7 + 33 - 58 + 82 + 19 - 0 - 87 + 14 - 46 - 35 + 25 - 20 - 98 - 62 + 39 + 2 + 76 + 8 - 53 - 0 + 54 - 43 - 33 + 74) + 9 + 22 + 7 - 74 + 82 - 75 + 68 - 53 - 82) - 52 - 7 + 67 + 10 + 55 + 13 + 41 - 14 + 50 - 28) + 58 - 33 - 49 - 69 - 72 - 56 + 65) + 34) + 81 - 47 + 37 + 29 - 9 - 17 + 64 - 98 - 75 + 23 - 40 - 92 + 90 + 4 - 80 + 12 + 99 - 65 - 44 + 63 - 24 - 22 - 15) - 10 - 57 - 74 - 20 - 17) - 65 + 89) - 62 - 48 - 68 + 32 + 20 + 94 - 70 + 52 + 10 - 30 - 61 + 26 - 40 + 89 - 46 + 27 - 32 - 98 - 90) - 35 + 88 - 12 - 99 + 7) - 35 + 80) + 26) - 58) + 38 + 21 + 49) + 4 + 81 + 49 + 75 + 40 + 39 + 85 + 4 - 99 + 86 - 45 + 0) + 79 + 33) + 42 + 30 + 86 - 80) + 16 - 86 + 92) - 4 + 79 + 68 + 0) + 66 - 92) + 16 + 12 + 30 - 33 - 49 - 13 + 63) + 47 + 88 - 15 + 68 + 48 - 75) - 45) - 87) - 88 - 55 + 83 - 49 - 79 - 59 + 28 - 68 + 22 + 29 + 5 + 8 - 55 + 8 + 100 - 59 + 5 - 8 - 14 + 59 + 6) + 14 + 55 - 33 + 39 - 0 - 78 - 86 + 21 + 76 + 64 + 2 - 72 + 75 + 55 + 27 + 76 - 37 - 79) - 15) - 3 + 80 + 56 + 5) - 18 + 58 + 75 - 93 - 2 + 17 + 52 + 16 + 28 + 20 - 95 - 6 + 19 - 32 - 97 + 58) + 1 - 25 - 5 + 91 - 67 + 18 + 71 - 4 - 0 + 72 + 66 + 31 + 47 - 50 - 16 + 86 + 37 + 99 - 8 - 71) - 52 + 85 + 11 - 30 + 93 - 93 + 99 - 29 - 0 + 95) + 76 - 14 - 3 - 89) + 65 - 41 - 21 - 27 + 20 + 30 + 83 - 2) - 68 + 86 - 42 + 12) - 45 - 47 + 43 + 19 - 49 - 41 + 97 + 64 - 97) - 93) + 13) - 62 + 54 + 16) + 35 - 60) - 56 + 81 + 16 - 94 + 95 + 44 - 2 + 97 + 81 - 24) - 10 + 16 + 50 - 85 - 21 + 13 + 85 - 88 - 20) + 40 + 29) - 19 - 77 - 57 - 51 + 82) + 54 - 10 - 3 - 73 - 23) - 97 - 8 + 51 + 91 + 96 - 48 + 82 + 75 - 32 - 10 - 22 + 15 + 80 - 24 + 84 - 89 - 37 + 31) - 71 - 60 - 1 + 71 - 65 + 8 - 74 - 92 + 21 - 29 - 38 + 57 - 43 - 28 + 48 + 60 - 23 - 7 + 87 - 74 - 89 - 59 - 27 + 80 - 45 - 11 + 69 + 81 - 22 - 38 - 3 + 60 + 22 - 85 + 30 - 53 + 47 - 95) - 22 - 18 - 67 - 61 - 54 + 41) + 2 + 90 - 81 + 99 - 51 - 39 + 3 + 4 - 59 - 67 + 1 + 74 + 27 - 9 - 34 - 69 + 20 - 0 + 64 + 79 - 9 + 69 - 4 - 83 - 76 - 29 - 48 + 72 + 1 + 27) + 95 + 33 - 33 + 4 - 10 + 43 - 51 + 19 - 13 - 12 + 55 + 99 + 28 + 92 - 28) + 20 + 2 + 96 - 68 - 41 - 23 - 44 + 52 - 24 - 77 - 88) + 93 - 37 - 97) + 70 - 26 - 17 + 37 - 100 - 98 + 55 + 36 + 65 - 90 - 10 - 90) - 93 - 49 + 54 + 95 + 53 - 23 - 78 + 19 + 38 + 80 + 4 + 44 - 19 - 51 + 85) + 3 + 91 + 42 - 93 - 64 + 50 + 39 - 0 - 17 - 74) - 17 - 40 - 43 - 64 - 32 - 61 - 55 - 53) - 48 - 3 - 68 + 61 + 25 + 28) - 32) - 95 - 5 + 13 - 82 - 57 - 77 - 97 + 93 - 48 + 30 - 48 + 48 - 51) - 54 + 21 - 80 + 75 - 81 + 100 - 82 + 37 - 66 + 35 + 15 + 84 - 36 - 32) + 31 + 85 + 23 - 97) + 97 - 23) - 36 + 14 - 5 + 83 - 65 + 91 - 34) + 12 + 9 - 94 - 33 - 31 - 85 + 100 + 57 + 55 + 22 + 28 + 77 - 8 + 71 + 4 + 32 + 37 + 60 - 79 + 75 - 72 + 91 - 67 + 14 + 16 + 68) + 88 + 11 - 19 - 65) + 89 - 33 + 12) + 78 + 53 - 88 - 82) + 32 + 36 - 25 + 44 + 27 - 12 - 55 + 43 + 32 - 65 + 77 - 47 + 91 - 5 - 94 - 49 + 41 - 89 + 76 + 46 - 55 + 96 + 41 - 78) - 8 + 61 - 99 - 5 + 90 + 40) + 77 + 43 + 78 - 34 + 100 + 6 + 5 + 23 + 50 - 35 - 25 - 24 + 21 - 21 + 22 + 32 - 81 - 74 + 97 - 70 - 59 + 12 - 89 + 62 - 66 + 14 - 46 + 26 - 28 - 90 - 68 - 20 + 35 + 97 - 51 + 4 - 70 + 39 - 24) - 86 + 13) - 27 - 53 - 83) - 61 - 22 - 63 + 73 + 76 + 0 - 24 + 69 - 35) - 74 + 19 - 57 + 73 + 19 - 94 - 13 + 65 - 23) - 15 - 95 + 97 - 18 + 14 + 83 - 2 - 20 - 47 + 12 - 5 + 63 + 73 + 0 - 68 + 73 - 30 - 56 + 49 - 70 - 64 + 8 + 92 - 86 + 21 - 29 - 83 + 91 + 22 + 81 - 22 + 28 + 82 - 91 - 46) - 71 + 64 + 82 - 27 + 45 + 69 - 83 - 39 - 32 + 56 + 38 - 30 + 4 + 44 - 17 - 54 + 31 - 76 - 80) + 1 + 1 + 74 - 28 + 75 + 75 - 29 - 14 + 93 - 93 - 37 + 91 - 89 + 38 + 79 + 84 - 63 - 53 + 80) - 95 + 28 - 64 + 91 + 36 + 58 + 67) - 93 - 6 - 66) - 36 + 11) + 30 - 62 - 90 + 79 - 50 + 98 + 34 + 2 + 80 - 48 + 13 + 66 + 25 + 55 - 47 + 68 - 26 + 79 + 18 - 87 - 51 - 31 + 40 - 0 - 8 - 50 + 65 - 71 + 10 + 14 - 31) - 36 - 54 + 95 - 14 + 31 + 10 + 6 + 86) - 26 + 75 + 74 + 81 + 72 + 24 + 51 - 45 - 99 - 65 + 100 + 15 + 47 + 36 - 33 - 67 + 4 - 71 + 14 - 13) + 20 - 56 + 62 + 71 - 2 - 63 + 92 - 23 - 87 - 82 - 48 + 42 + 18 + 51 - 50 - 72 + 74 + 60 - 91 + 68 - 29 - 93 + 94 - 81 + 20 + 37 + 54) - 76 + 97 - 49) + 29 + 53 - 78 - 13 - 25) + 32 - 41 + 55 - 93 - 100 - 97 + 43 - 15 + 29 - 94) + 20 - 51 - 55 + 100 + 54 - 28 - 93 - 80 + 100 - 65 + 84 + 37 + 63 - 89 + 49 - 24 + 59) + 87 - 67 - 64 + 71 - 81 - 100 - 87 + 26 + 2 - 50 + 74) + 86 - 60 - 14 - 39 - 21 - 68 - 36 + 12 - 53 - 17 + 97 + 45 + 16 + 13 - 68 + 90) + 78) + 97) - 14 + 64 + 48 + 15 - 37 + 45 + 71 - 60 + 31) - 36 - 98 + 22 - 58 + 59 + 6 + 84 - 34 + 8 - 31 + 47 - 3 - 86 + 44 - 11 + 20 + 61 - 79 - 54 - 24 - 41 + 31 + 43 - 71 - 59 - 31 - 100 + 81 + 29 + 99 - 46 + 8 + 65 - 12 - 66 - 37 - 76 - 73 - 37 - 90 - 46 - 71 - 5 + 37 + 45 + 67 - 18) + 69 + 34 + 24) + 18 + 71 - 73 - 13 - 19 - 38) + 96 - 54 - 80 - 62 + 31 + 23) - 28 + 5 - 21 - 20) + 18 + 72 - 33 + 3 - 100 + 22 + 51 + 84 + 15 + 98 + 19 + 96 + 89 - 48 - 19 + 75 + 51 - 5 - 41 + 74 + 75 - 78 - 66 + 91 + 95 + 69 - 72 - 33 + 56 + 75 + 65 - 29 - 22 + 64 - 4 - 40) - 11 + 9 - 95 + 58 + 16 + 9 + 44 - 63 + 26 + 82 + 21) + 36 - 88 + 98 - 9 - 13 + 72) + 46 + 41 - 67 + 16 - 35 + 70 + 19 - 53 - 42 - 54 - 84 - 18 + 92 + 88 + 60 + 84 + 10 + 48 - 25 - 98 + 31 - 20 + 39 + 90 + 86 - 26 + 70 - 34 + 73 - 79 - 61 - 39 - 61 + 72 + 45 + 99 - 77 - 21 + 95 + 85 - 48 + 67 - 40 + 95 - 53 - 74 - 50) + 82 - 77 - 37 - 64 + 22 + 11 - 61 + 94 - 99 - 91 + 67 - 36 - 92 + 87 + 95 + 97 - 51 - 63 - 82) - 9) - 17 + 66 - 100 + 13 - 4 + 91 + 68 - 51 + 47 - 12 - 76 + 40 + 66 + 12 - 67 - 40 - 4) - 80 - 84 + 89) + 1 - 7 - 19 - 88 - 79 + 39 - 100 + 71 - 57 + 89 - 1 + 38 - 52 - 44) + 55 + 28 - 58 - 47) + 100 - 84 + 55 - 40 - 86 - 20 - 38 + 83 - 97 + 9 + 83) + 46 - 34 - 42 + 68 + 15 - 64 + 38 - 47 - 24 + 28 - 81 - 30 - 41 + 26 - 90) - 79 + 90 - 59 + 5) + 0 - 23 + 59 + 98 - 40 + 7) + 88 + 13 - 14 - 95 + 53 + 4 + 21 - 14 + 91 - 35 - 14 - 6 - 33 - 72 - 32 + 92 + 46 + 48 + 63 + 81 + 78 + 58 - 82 - 76 - 6 + 52 - 8 + 72 + 68 + 47 + 98 + 46 + 15 + 80 + 9 - 48) + 1 + 97 - 90 + 12 - 55) - 70 - 81 + 14 - 36 + 20 + 15 + 90 - 36) + 92 + 53 + 92 + 78 - 75) - 62 - 3 + 96 - 61 - 76 + 61 - 48) + 56 + 91 - 11 + 16 - 31 - 25 + 98 + 46) + 40 + 30 - 93 + 26 + 78 - 66 - 77 + 40 + 60 + 4) - 59 - 73 - 61 - 69 - 38 - 15 - 93 + 72 - 92 + 39 + 4 + 86 - 74 - 54 + 100 - 31 + 11 - 3 + 18 + 7 - 6 + 65 + 58 + 98 + 9 + 63 - 34 + 25 + 30 + 26) - 23) - 91 + 0 + 49 + 8 + 98 - 66 - 42 - 20 - 10 - 8 + 96 + 46 + 64 - 48 + 19 + 92 - 93 - 26) + 31 - 6 + 12 - 0 - 18 + 72 + 30 + 2) - 75 - 100 - 81 - 38 - 14 + 40) + 17 - 6 + 40 + 44) - 34 + 79 + 73 + 73 + 3 + 56 + 1 + 20 - 66 + 66 - 75) - 32 - 89 - 86 + 70) - 34 - 34 + 89 - 52) + 21) - 52 + 76 - 40 - 66 + 95 + 86 - 97 + 3 + 69 + 99 + 0 + 92 - 66 - 22 + 75 - 10 + 38 + 89 - 9 - 38 - 64) + 11) - 54 - 32 + 83) - 28 + 98 - 67 + 1 - 61 + 85 + 15 + 84) + 50 + 82 - 26 - 77 + 2 + 39) - 11 - 51 + 86 - 51 + 10 + 49 + 99 - 26 + 84 - 92 - 86 + 28 - 10 - 23 - 48 - 90 + 65 - 81 - 47) + 95 - 83 - 90 + 41) - 31 + 44 - 99 + 94 + 42 - 1 + 22 + 16 + 32 - 97) - 13) - 0 + 38 - 26 - 24 - 52 - 50 + 75 + 60 - 62 + 0 + 47 + 28 + 80 + 64 + 45 + 100 - 61 + 86) + 4 + 98) - 25) - 38 - 96 + 57 - 19 - 57 + 55 - 5 - 49 - 84 - 0 + 16 + 88 - 35 - 39 - 3 + 94) - 13) + 82 + 29) + 29 + 71 - 58 + 77 + 35 - 30 + 36 + 84 + 47 + 96 + 71 - 17 + 19 + 83 - 95 - 36 - 95 + 51 + 80) - 41 - 11) + 18 + 40 - 80) + 81 - 26 + 97 + 45 - 41 + 11 - 27 + 38 - 33 - 6 - 13 + 30 + 13 - 58 - 10 + 2 + 20 + 90 - 47 - 67 - 37 - 20 - 26 - 45 + 33 - 67 + 82 + 68 + 14 + 83 + 29 - 89 - 23 + 41 + 72 - 22 + 98 + 23 - 62 - 61) - 45 + 70 + 64 + 58 - 53 + 54 - 47) - 71 + 30 + 95) - 83 + 27) + 36 - 57 + 53 - 98 - 51 + 30) + 24 - 58 + 52 - 52 - 54 + 43 + 90 + 69 - 66 + 90 + 20) + 88 - 99) + 6 - 69 + 52 + 1 + 52 + 13 + 71 + 12 - 48 - 53 - 69 - 2 + 68 + 66 - 45 + 23 - 76 - 4 - 19 - 63 + 67 - 0 - 7 + 72 + 24 + 38) - 22 + 33 + 51 + 99 - 67 - 29 + 86) + 66 + 36 - 96 + 22 - 84 + 34 + 84 - 27 - 68 + 5 + 62 - 24 + 54 - 95 + 23 - 75 + 73 - 3 - 49 - 43 + 93 - 53 + 55 + 37 + 58 - 40 + 85) + 8 + 54 - 88 + 15 + 70 + 30 - 18 + 83 + 77 - 86 + 24 - 82 - 85 - 46 - 31 - 1 - 35 + 53 - 86 + 73 + 27 - 60 + 18) + 20 - 72 + 53 + 16 + 5 - 72 - 45 + 68 - 5 + 24 + 98 - 4 + 72 - 34 - 25 - 42 - 32 - 87 + 26 + 100 + 14) - 95 + 30 - 69 + 73 + 10 - 80 + 84 - 89 + 41 - 5 - 27 - 38) + 44) - 44 - 60 - 75) + 74 + 36 - 94 + 38 + 41 + 2 - 34 - 78 - 60 + 44 + 56 - 59 + 15 + 76 - 89 + 54 + 90 + 90 + 31 - 32 - 87 + 87 - 31 - 76 - 47 + 86 - 66 - 20 - 9 + 10 + 24 + 11 - 57 - 97 - 34 + 51 - 33) + 96 - 66 + 82 - 40 + 98 + 32 + 24 - 51 + 20 + 44 - 3 + 49 - 47 - 43) + 59 + 52 + 73 + 30 + 7 - 3 - 7 + 93) + 81 + 82 + 37 - 58 + 5 + 9 - 79 + 4 + 31 - 94 - 97 - 39) + 87 - 42 + 4 - 5 - 5 + 55 + 50 + 14) - 2 - 35 - 55 - 2 + 10 + 55 + 57 + 32 + 6) - 91 + 56 - 69 + 26 - 61 - 8 + 32 - 12 - 8 + 28 - 41 + 65 - 30) - 74 - 91 + 10 - 6 + 19 + 53 - 76 + 88 + 86 + 24 + 7 - 43 - 11 + 39 + 81) - 2 + 8 + 47 + 9 - 62 + 58 + 43 + 47 + 23 - 77 - 88 + 93 + 74 + 56 + 25 - 43) - 31 + 87 - 61 + 5 + 79 + 92 - 43 - 79) - 64) + 28) + 25 - 31 - 11 + 30 - 49) + 9 + 90 + 68 - 14 - 7 + 70 + 42 + 34 - 89) + 52 - 21 - 19 + 34 + 64 + 84 - 45 + 60 + 49 + 49 - 56 - 56 - 0 + 71 - 60 + 78 - 89 - 69 - 6 + 26 - 54 - 75 - 51 - 84 - 48 - 40 - 35 - 18 - 69 - 75 - 91 - 61 + 21 + 98) - 17 + 69) - 10 + 74 - 55 + 79 - 68 + 71 - 29 + 56 - 47 + 82 - 35 - 93 - 49 + 66 + 99) - 5 - 77 - 43 + 8 - 97 - 51 - 80 - 87) - 65 - 41 - 35) - 73 + 68 + 99 + 87 - 67 - 86 + 57 + 0 + 39 - 15 - 40 + 80 + 3 + 19 + 54 - 91 + 8 + 81 - 32 + 83 - 13 + 95 + 74 + 83 + 58) - 18 - 75 + 26 + 32 - 37 + 47 - 13 + 81 + 48 + 69 - 5 - 79 + 45 + 59 - 100) + 55 - 62 - 100 + 6) - 76 + 65 + 99) + 38) + 80 + 3 + 10 + 40 + 66 + 63 - 26) + 57 - 100 - 90 + 5) + 96 + 72 - 22 - 100 - 67 - 64 + 99 + 20 + 52 + 84 + 97 + 24 + 53 + 46 + 62 + 3 + 10 + 9 + 82 + 32 - 36 - 45 - 42 - 49 - 8) - 23 + 91 + 82 - 51 + 19 + 35 - 98 - 23 - 10 - 45 + 16 - 24 - 92 + 43 + 27 + 53 + 36 - 34 - 60 - 27 + 53 - 36 + 21 - 79 - 7 - 44 + 56 - 59 - 65 - 96 - 49 - 27 + 22 + 79 - 13 - 73 + 84 - 68 - 55) - 17 + 28 - 58 + 82 - 48 + 44 - 98 + 51 - 93 + 98 + 24) - 67) - 34 - 64 - 52 + 60 - 25 - 65 - 59 - 49 - 9 - 45 - 5 - 100 + 90 - 87 + 28 - 10 - 33 - 65 - 78 - 9 - 50 + 71 + 86 + 21 + 35 + 39 + 56 - 29 - 91 + 83 - 3 + 59 + 20 + 36 + 16) + 52 - 73 + 20 + 2 + 26 - 45 + 86 - 78 + 91 - 45 - 51 - 64 + 22 - 18 - 16 - 54 + 49 - 45 + 28 - 2 - 3 - 38 + 65 - 100 + 11 - 96 + 36 - 81 + 33 - 5 - 94) - 48 - 69 - 4 - 21) - 98 + 83) - 55 - 6 + 28 - 28 + 37 - 29 - 79 - 74 - 6 - 50 + 58 - 28 + 23 - 29 + 11 + 47 - 97) - 28 + 68 + 78 + 81 + 3 - 45 + 74 + 88 + 40 + 2 + 74 + 81 + 63 - 60 - 93 + 5 - 96) + 68 - 51 - 9 + 66) - 10 + 26) - 25 - 79 - 33 - 17 - 32 - 16) + 59 - 34 + 40 - 26 - 28 - 91 + 40 - 56 + 53 - 2 + 96 - 5) + 12 - 45 - 56 + 21) + 25 - 9 - 91 - 51 + 100 + 10 - 20 + 15 + 0 - 74 - 99 + 38 + 24 + 58) - 72 + 70 + 37 + 26 + 34 - 10 - 64 - 82 + 70 - 49 + 77 + 17 - 98 - 51 - 96 + 68 + 29) + 18 + 95 + 76 - 68 - 54 + 87 + 14 - 53 + 61 - 8 + 33 + 60 - 28 - 18 + 14 - 29 + 52 + 70 + 71 + 60 + 69) - 98 + 6 - 19 + 77 + 59 - 71 + 92 - 26 - 89) - 33 + 32 + 87 + 32 + 89 + 92 - 46 + 14 + 73 - 95 + 54) + 3) - 63 - 34 + 80 + 78 + 92 + 39 + 84 + 42 - 25 + 80 - 50 + 98 - 38 + 93) - 81 - 91 + 26 + 42 + 40 - 59 + 6 - 78 - 86 - 46 + 83 - 50 - 38 + 85 + 59 - 37 - 13 + 79 - 47 - 64 + 60 + 93 + 13 + 63 + 22 - 79 + 2 - 44 + 5) - 99 + 42) + 86) - 7 - 18 + 79 + 56 - 72 + 82 - 34 - 36) + 73 + 41 - 91 - 46 + 65 - 23 - 34 + 2 + 17 + 57 - 11 + 55 - 26 - 48 + 18 - 36 - 19 - 53) + 62 + 72 + 42 + 8 - 51 - 41 + 32 - 88 + 86 + 72 - 64 + 4 + 53) + 5 - 76 - 1 - 57 - 43 - 41 + 16 + 0 - 36 + 29 - 27 + 51 - 70 - 88 - 85 - 21 - 8 - 53 - 59) + 51 - 84 + 51 - 99 - 24 - 6 - 93 + 3 + 24) + 47 + 20 + 35) + 25 + 59 - 14 + 56) - 84 + 95 - 100 + 68) + 20 - 33 + 29 + 58 + 58 - 100 + 8 - 15) + 99 - 18 + 41 - 15 - 18 - 33 - 42 - 64 - 46 + 88 + 37 + 85 - 81 + 16 - 91 + 14 + 30 - 79) + 42 - 8 - 79 + 46) - 53 - 2 + 72 + 12 + 52 - 49 + 98 - 22 - 5 - 27) - 62 + 27) + 28 + 13 - 9 - 53 - 96) + 14 + 93 - 22 + 64 - 85 + 28 + 44 + 1 + 70 + 81 - 66 - 4 - 20 + 2 + 96) + 15 - 87 + 78 - 20 - 15 + 76 - 15 - 38 - 9 + 48 - 20 - 78) - 82 - 52 + 87 - 94) - 87 - 90 - 32 - 81 - 2 + 28) - 29 + 40 + 35 - 88) + 4 - 91 - 49 - 38 + 43 + 79 - 67 + 47 + 41 + 77 - 74 + 99 + 39) - 34 + 56 + 83 - 26 + 31 - 5 - 26 - 32 + 88 + 84 - 10) + 58 + 93) - 46 - 28 - 29 + 40) + 59 - 94) + 63 - 9 - 65 - 78 - 21 - 26 - 61 + 51 - 1 + 47 + 38 - 83 + 52 + 36 - 6 + 86 - 28) - 16 - 68 - 4 + 27 + 37 - 63 + 11 + 77 + 14 - 16) + 83 - 15) + 60 + 40 - 11 - 60 - 94 + 49 + 54 - 58 + 68 - 5 - 61) - 44) + 8 + 58 + 32 - 75 + 75 + 10 - 59 - 89 + 61 - 55) - 72 - 34 - 3) - 34) + 69 + 36 + 74) - 22 + 60 - 88 - 35 - 62) - 70 + 88 + 41 - 94 - 61 - 76 + 19 - 45) + 6 - 58 + 67 + 68 + 20 - 88 - 19 - 14) - 74 + 15) + 93 + 70 + 76 - 87 + 3 - 18 - 66 - 40 - 60 + 96) - 27 - 6 + 81 + 83 - 54 - 32 + 71 + 96 - 86 + 29 - 77 - 7 - 77 + 22 + 87 - 33 + 4) - 48 - 59 - 24 + 30 + 23 - 12 + 80 - 22 - 11 - 37 + 6 - 18) - 65 - 31 - 48 - 28 - 40 - 11 - 35 + 31 + 91 - 11 + 62 + 82 - 79 - 8 + 60 + 84 + 57 - 1 + 61 - 56 + 20 - 73 - 32 + 100 + 75) + 8 + 31 + 86 + 43 + 71 - 63) - 75 + 45 - 51 + 90 + 28) + 96 + 72 + 14 + 80 + 95 + 83 - 86) - 23 + 3 - 63 - 79 + 48 + 99) - 27 + 6) + 1 + 8 + 19 - 69 + 8 + 9 - 40 - 86) - 23 + 43) + 77 - 9 + 0 + 30 + 29 - 40) + 8 + 27 + 94 + 45 + 18 - 74 - 10 + 30 + 5 - 82) + 77 - 21 + 30 - 76 - 72) + 48 + 8 - 77 - 53 - 90 + 50 + 86) + 30) - 22 + 99 - 36 + 58) - 53 + 90 - 27 - 65 - 48 + 95) + 0) - 44 - 84 - 22 + 77) + 12 + 34) + 96 - 85 - 93 - 44 + 6 - 10) - 85 - 89 - 31 + 45 + 81 + 3 - 35 + 47 + 81 + 88 - 25 - 80 - 37 - 11 - 49 - 24 - 55 + 2 - 48 + 59 - 68 - 64) - 49 + 59 - 100 + 45 + 20 + 59 - 49 - 25 - 23 - 9 - 22 + 57 - 49 - 13 - 90 - 2 + 26 + 32 - 56) + 82 + 28 + 78 - 34 - 77 - 76) - 58 - 5) - 8 - 60 - 10 + 52 - 14 - 53 - 94 + 65 - 71 + 41 - 52 + 26 + 4 + 4 + 32 + 53 - 93 - 75 - 49 - 29 - 87 + 20 + 65 + 4 + 27 + 88 + 9 + 8 - 23 - 84 + 27) - 68 + 52 - 78 - 62 + 68 + 90 - 80 + 28 + 32) - 56 - 66 - 6 - 90 + 84 + 12 + 85 + 69 + 52) - 73 - 24 + 57 - 48 - 58 - 43) - 43 - 100 - 52 + 37 + 31 + 44 - 17 + 86 + 20 - 61 + 4 + 87 - 50) + 51 + 78 - 13 + 58 + 80 - 16 + 25 - 67) - 94) - 99 + 93 - 4 - 96 - 88) + 97 + 91 + 16 - 7 + 66 + 93 + 30 - 98 - 69 + 30 + 21 - 33 + 20) - 92) + 23 - 26 - 21 + 46 + 85 - 47) - 81 - 71) + 59 - 93 - 47 - 1 - 83 - 8) + 48 - 73 + 6 - 62 - 91 - 63 - 6 - 55 - 82 - 38 - 54) + 27 + 23 + 8 - 66 - 3 + 57 - 75 - 98 - 45 + 83 - 44 + 73 + 15) + 63 - 99) - 20 - 53 - 78 + 80 - 54 - 18 + 65 - 60 + 72 + 1 - 20 + 42) - 64 - 57 - 80 - 69 + 62 - 66 + 42 - 76 + 35) + 48 - 86 - 18 + 5 - 5 + 57 + 53 + 1 - 97 - 44 - 70 + 50 + 58 + 2 - 7 - 20 - 88 + 92 - 69 + 97 - 65 + 58 - 20 - 78 + 20 + 83 - 93 + 24 - 91 + 88 + 53 + 90 - 45 + 21 + 2 + 93 - 87 - 53 - 45 - 4 - 62 - 71 + 24 + 39 - 74 - 87 - 10 - 94 - 78 - 31 + 51 + 40 + 27 + 34 - 23 + 52 + 43) - 69 - 30 + 77 + 80 + 43 + 35 + 5 - 10 - 63 - 91 - 95 + 80 + 21 - 43 + 31 - 44 - 53 - 41) - 60 + 27) - 76 - 36 + 89 + 86 + 96 - 52 + 91 + 36 + 43 + 51 + 84 + 70 + 49 - 87 - 31 + 30 - 95 - 23 + 61 - 12 + 18 - 28 + 46 - 80 - 95 + 27 + 75 + 49 + 34 + 83 + 65 - 33 - 93 + 18 + 89 + 96 + 91 - 78) + 74 + 84 + 14 + 73 - 3 - 32 + 29 + 25 - 45 + 94 - 43 - 90 + 57 - 70 - 27 + 44 + 90 - 25 - 7 + 79 - 9 - 31 - 49 - 47 + 14 - 67 - 4 + 45 - 81 - 20 + 85 + 36 + 81 - 70) + 81 - 39 - 15 + 100 + 8 + 5 + 16 - 79 + 7 - 52 + 46 + 12 + 95 + 3 + 38) - 35 - 26) - 85 - 47) - 28 - 51 + 67 - 25 - 47 - 84 + 100 - 66 - 37 - 54 - 39 + 55 - 95 + 8 + 51 - 76 + 21 + 91 + 84 + 95 + 60 + 72 - 60 - 22 + 13 + 3 + 78 - 21) - 39 + 62 - 10 - 5 + 51 + 1 + 25) - 4 - 33 - 84 + 95 - 56 - 34) - 70) + 66 - 96 + 34 + 9 + 71 - 68 - 59 + 30 - 41 + 10 - 56 + 69) + 88 - 85 + 40 + 43 - 42 - 41 + 12 - 27) - 94 - 78 - 96 - 44 - 6 - 35 + 1 - 40 - 50 - 88 + 37 + 73 + 31 + 1 - 67 + 17 + 26 - 49 - 56 - 43 - 39 - 92 - 67 + 46 - 16 + 39 + 31 + 18 - 24 + 94 - 10 - 70 + 79 - 97 - 98 - 37 - 29 + 88 + 3 - 52 + 69 - 65 - 26 + 76 + 42 - 14 - 1 - 16 - 99 - 73 + 0 - 70) - 61 + 99 - 70 - 23 - 33 + 86 - 85) + 3 - 25 + 84 + 75 + 76 - 80 + 29) + 24 + 22 + 14 + 13 + 48 - 1 - 72) + 10 - 87) - 16 - 3 + 87 + 25) - 91 - 63 + 1 - 36 - 77 - 37 - 32 + 38 + 68 + 97 + 78) - 90 - 83 - 75 + 33) + 24 - 62 - 7 + 6 + 24 - 71 + 50 + 45 + 0) + 3 - 61 - 78) + 56 + 99 + 58) + 40) - 21 + 50 - 17) + 91 - 73 + 35 + 88 + 45 + 39) - 44 - 42) + 71 + 9 - 58 - 70 - 7) + 72 + 7 - 90 + 82 - 0 - 99 - 6 + 0 + 58) + 17 - 43 + 20 + 11 - 98 - 13) + 4 + 48) + 58 - 88 + 95 + 1 - 18 + 57 - 20 + 47 + 11 - 88 - 31 + 3 + 76 + 50 - 30 + 59 - 86 - 42 - 21 - 40 - 58 - 84 - 59 + 12 - 49 - 52 - 2 - 60) + 1 - 95 + 37 - 62 - 25 + 16) + 95 - 83 - 58 - 18 + 41 - 81 + 34 + 69 - 86 - 33 - 32 + 56 - 99 - 17 - 79 - 25 - 94 + 90 - 58) - 40 - 98) + 54 + 11 + 62 + 52 - 54 - 49 - 7 - 0 + 67 + 20 - 52 - 76 + 51 - 51 + 3 + 42 - 77 + 46 + 40 - 57) - 42 + 59 - 50 - 20 + 27 - 7 + 59 - 93 - 81 + 51 + 64 + 35 + 84 - 17 - 7) - 32 - 86 + 56 + 94 + 92 + 1 + 92 + 80 + 59) + 37 - 13 + 9 - 27 - 90) + 20 - 16 - 27) + 93 + 39 - 24 + 53) + 3 - 50 - 52 - 82 + 89 - 59 - 96 + 14) - 46 + 30 + 8 - 22 - 91) + 33 - 64 - 99 + 84 + 31 - 78 + 75 + 79 + 90 + 98 + 73 - 15 + 0 + 64 + 56) - 75 - 98 - 67 + 70 + 72 - 0 + 78 - 15 - 83 + 57 - 8 + 43 - 71 - 66 + 31 + 37 + 39) + 73 + 49 + 42 + 67 - 27) + 38 - 1 - 63 - 84) - 37 - 63 + 11 + 58) - 99 - 65 + 20 - 23) - 73) + 86 + 73 - 44 + 78 + 57 - 27 - 89) + 31 - 84 + 2 - 18 + 68 - 98 - 1) + 89 + 35 - 31 - 13 - 31) - 97 + 60 - 74 - 94 + 51 - 14 - 37 + 10 - 49 - 6 - 84 + 73 + 98 + 37 - 49 + 68 + 61 - 6 + 77 - 24 + 55 - 46 + 65 - 82 - 50 - 25 + 61 - 95 - 17) + 87 + 90 - 92 - 49 - 27 - 81 - 0 + 65 - 61 - 13) - 54 - 52 - 34 - 91 - 23 - 5) - 21 - 10 + 3) - 40 - 20 - 72 - 0 + 34 - 43 - 34 + 71 - 80) - 32 + 53 - 97 + 1 + 8 - 2 - 15 + 41 - 56) - 20) + 29 - 8 + 5 - 40 - 99 + 37) - 95 - 98 + 38 - 23 - 15 - 62 - 67) + 24 - 10 + 35 - 25 - 95 + 48) - 60 + 99 + 78 - 69 + 73 - 43 + 51 + 97 - 69 + 51 - 29 - 79 + 26 + 78 - 33) + 55 + 95 - 65 - 48 - 10 - 23 + 44 + 45 + 47 + 52 + 53 + 79 + 4 - 73 + 79 + 78 - 100 + 0) - 97 + 5 + 48 - 61 + 99 - 97 + 10 - 43) - 56 - 59 + 85 + 51 + 9 - 86 - 4 - 92 - 1) + 65 - 71 + 33) - 31 - 13 - 62 + 61 - 32 - 95 - 74 + 96 - 66 - 29) + 11 + 89 + 15 + 23 + 52 + 24 - 31 - 59 - 88 + 0 + 95 - 51 + 84 - 14 - 34 + 82 - 33 + 72 - 48 + 67 + 72 + 15 + 13 + 10 + 14) + 77) + 9 - 1) - 10 - 96 - 43 - 69) - 99 + 43 - 62 - 89 + 82 - 17 + 35 + 28 + 37 + 10 - 14 + 26 + 88 + 22 + 67 - 93 + 10 + 61 - 62 - 31 - 55 + 7 - 75 + 40 + 50 - 99 + 49 + 47 + 30 - 93 + 44 + 57) + 84 + 98 - 34 - 82 - 65 + 100 - 80 - 13 - 69 + 86 - 49 + 19 + 49 - 49 - 28 - 8 - 15) - 94 - 17) + 15 + 88 - 44 + 18 + 65 - 8) + 87 + 63 - 69 - 83) + 22 + 9 - 25 + 90 + 51 + 16 - 80 - 77) - 31) + 95 - 59 + 46 - 70 + 95 + 33 - 0) + 90 + 39 - 60 - 25 - 18 + 72 + 91 - 68 - 62 - 19 + 33 - 61 + 26 + 15 - 28 + 26 + 98 - 32 + 15 - 28 + 73 + 80 - 69 - 1 + 43 + 23 + 43 + 79 - 73 - 49 + 30 + 52 + 12 + 82 + 5 + 52 - 16 - 24) - 65 - 11 - 38 + 61 - 66 + 50 + 71 + 69) + 63 - 68 + 49 + 13 + 47 - 29 + 21 - 83 + 42 - 46 - 54 - 63) + 28 - 62) + 5 + 53 + 61 + 32 - 68 + 70 - 65 + 53 + 41) + 3 + 97 - 21 + 75) - 22 - 44 + 77 - 80 - 65) + 8 + 72 + 52 - 71 + 62) + 35 + 26 - 15 - 3 + 10 - 52 - 88 - 83 - 14 + 94 - 82 + 59 - 80 + 84 - 12 - 62 + 72 + 53 - 17) - 9 - 96) + 94 + 65) + 39 + 74 - 6 - 8 - 30 + 80 + 43 + 95 + 58 + 95 - 69 + 87 + 90 + 76 + 98 - 62 + 58 - 93 + 69 - 98 - 14 + 9 - 98 + 65) + 12 + 81 - 46 + 79 - 51 + 45 - 88 - 31 - 26 - 62 + 44 - 49 - 40 + 75 - 21 - 70) + 96) + 58 + 32 + 44 + 26 + 68 - 87) + 66 + 17 + 19 + 8 + 55 - 67 - 100 + 3 - 68 + 68 + 8 + 86 + 78 - 7 + 80 - 31) - 60 + 10 - 93 - 77 - 11 - 91 - 55 - 66 - 70 + 62 + 52 - 90 - 36 + 57 + 59 + 37 - 74 + 93 - 61 - 85 - 100 + 6 - 82 - 83 + 93 - 59 - 65 + 23 - 98 + 94 - 86 + 21 + 20 - 62 - 20 + 94 - 88 + 62 + 62 - 22 - 4 - 75 - 87 - 72 - 71 - 33 - 42 + 16 - 28 - 70) - 33 - 3 + 78 - 93 - 60) - 98 - 63 - 28 - 24 + 86 + 100 + 79 + 81 + 72 - 33 + 89 - 31 + 72 + 88 - 1 - 46 + 11 - 51 - 6 - 92 + 49 - 60 + 79 - 59 - 51 - 8 + 41 + 23 - 16 - 69 + 47) - 15 + 47 + 85 + 89 - 64 + 90 - 30 - 84 + 51 + 55 - 62 - 42 + 76 - 69 - 95) - 76 - 25 - 35 - 28) - 2 - 79 - 47 - 51 + 39 + 74) - 61 + 68 - 8 - 14 - 31 - 3 + 98 + 87 + 44 + 45 - 38 - 71 + 62 + 38 - 93) - 8 + 15 - 84 - 99) - 75 + 91 + 94 - 64 + 77 - 59 + 29) - 32 - 99 - 83 - 27 - 71 - 20 + 23) - 96 + 43 + 26 - 95 - 89 - 10 + 76 + 1 + 83 + 45 + 56 + 66 - 60 + 100 + 73 - 95 - 15 + 57 + 6 + 71 + 13) + 1 + 58 + 21) - 83 - 74 - 62 + 70) - 92 + 72 + 55 + 45 - 32 - 58 + 1 - 45 + 35 + 23) - 94) - 21 + 32 + 23 + 21 - 64 + 81 + 64) + 45 + 72 - 70) + 27 + 65 - 4 - 67 - 49 + 49) + 25 + 54 - 99 + 66 - 17 - 62 + 33 - 54 - 27 + 14 + 95 + 2 + 15) + 26 + 6 + 38 + 16 - 9) + 3 - 78 + 30 + 19 - 53 - 10 + 34 - 24 + 91 - 86 + 15 - 17 + 96 + 15 - 73 + 41 - 52 + 14 - 14 + 56 - 89 - 59 - 54 - 97 - 89 - 15 - 47) - 18 + 1 + 64 + 59 + 13 + 8 + 46 + 67) + 77 - 65 + 21 + 21 + 38 + 90 + 62) + 49 + 68 - 34 + 93 - 85 - 87 - 16 + 39 + 39 - 33 + 21) + 64 + 62 + 19 + 54 - 62 - 10 - 48) - 30 - 13 + 71) - 86 + 44) - 3 - 27 - 61 + 76) - 84 - 79 - 20 + 47 + 63 - 71 - 5 + 92) - 65) + 23 + 75) + 32 - 77) - 39 - 91 - 75 + 17 + 42 - 0 + 99 - 78 - 17) + 92 + 80 + 35 + 87 + 64 + 18 - 84) + 64 - 31 + 73 + 66 - 78 + 37 - 44 - 44 - 79 + 73 + 15 - 22 - 45 + 60 + 34 - 14 - 38 - 78 - 10 - 98 + 27 - 89 - 55 - 25 - 23 + 59 + 32 - 54) - 82 - 28) + 44 + 35 + 1 - 98 + 33 - 22 + 78 - 21 + 7 - 97 - 93 - 0 - 51 - 20 + 57 - 26 + 99 + 47 - 25 + 82 - 64 + 48 - 45) + 41 + 41 - 51 - 28) - 74 + 90 + 29 + 85 + 95 + 15 - 32 - 10 + 42 - 22 + 100 + 78 + 95 + 77) + 96 + 78 - 40 - 77 - 49 - 24 + 19 - 50 - 54) + 35 + 78 - 92 - 49 - 29) + 72 - 100 + 94 - 62 - 2 + 46 + 82 + 6 - 22 + 81 + 28 - 13 - 55 + 82 - 39 - 20 + 53 - 13 + 95 + 97 + 90 - 56 + 68 + 39) - 91 + 48 - 75 - 63 - 44 - 76 + 91 + 10 + 1 - 10) + 25 + 42 - 87 + 9) - 37) - 68 - 72 + 29) + 64) - 25 + 79 + 42 - 24 + 39 + 47) - 61 - 11 - 30 - 44 + 51 - 87 - 0 + 61 + 56 - 11 - 5 + 45 + 96 + 32 - 81 + 63 - 57 - 25 - 23 + 36 - 87 - 48 + 73 + 14 - 36 + 7 + 81 - 27 - 45) + 61 + 63 + 30 - 26 + 41) - 56 + 17 - 70 - 93) - 43 - 37 - 76 + 65 + 33 - 25 - 99 + 4 - 29 + 85 + 21 + 16 + 83 + 70 + 57 - 80 - 65 - 83 - 62 + 36 + 1 - 18 + 21 + 3 - 60 + 66 + 61 - 89 - 14) - 58 - 10 - 41 - 34 + 79 - 26 - 12 + 6 - 77 - 63) - 33 + 95 - 54 - 94) + 19 - 100) + 8 - 21 - 83 + 18 + 64 + 8 - 0 + 29 - 49 + 36 + 73 - 96 + 76 - 75 + 94 - 16 - 3 - 79 + 71 + 7 - 10 + 69 + 60 + 29 + 3 + 89 - 14 + 24 - 97 - 80 - 90) + 44 + 2 + 40 + 38) + 53 + 67) + 11 - 81) - 80 - 82 - 49 - 49 + 67 + 9 + 94 - 19 - 24 - 27 + 17 + 58) - 97 - 17 + 30 - 78 + 100 - 77 + 28 - 52 + 35 + 1 + 90 + 64 - 33) - 93 + 40 + 57 - 40 + 84 + 64 - 15 + 29) + 81 - 78 + 69 + 63 - 20 + 35 + 89 - 86 - 36 - 32 - 13 + 46 + 96 + 49) + 38 + 75 - 54 + 86 - 43 + 69 - 17 + 97 + 61 - 51 - 34 - 95 - 5 + 97 + 42 + 1 - 58 - 1 - 54 - 84 + 72 - 40 - 64 + 12 + 66 - 75 + 91 - 68 + 50 - 84 + 31 - 32 - 89 - 42) - 87 + 31 + 91 - 17) + 82 + 0) + 7 - 42 - 64 + 78 - 49 + 98) - 54) + 59 + 26 + 47 + 43 + 18 - 32 + 83 - 9 - 99 + 95 - 18 + 89 + 24 - 20 + 42 + 44 - 87 + 56 - 27 - 76 - 73 + 73 - 37 - 89 - 58 - 25 - 85 + 77 + 12 - 67 + 71 - 63 + 50 - 51 + 49) + 96 - 94 - 94 + 16 + 37) - 12 - 63 + 43) + 19 + 22 - 42) + 40 - 18 - 29 - 23 + 40 + 75 - 49 + 54 - 74 - 11 - 34 - 23 + 32 + 57 - 25) + 60 - 80 + 43 - 36) - 70 - 33 + 93) + 54 + 79 - 9 + 57 - 0 - 49 + 58 + 62 - 94 - 21 - 33) - 3 + 81 - 10 - 90 + 29 + 68) - 63 + 43 - 49 - 80 + 37) + 88 - 44 - 79 - 35 - 49 - 17 - 60 - 88 + 90 + 85 + 63 + 92 + 24) - 48 + 21 - 69 - 49 - 9 - 10 - 32 - 74 + 61) + 51 + 71 - 62 - 50 + 87 - 43 - 49 - 49 - 67 + 24) - 28 + 69) - 1 - 41 + 27 - 31 + 80 + 21 + 56 + 44 + 60 + 22 + 2 - 98) - 17 - 9 + 13 - 52 - 76 + 90 - 15 - 17 + 81 + 27 + 3) - 68 - 87 + 77) + 43 - 95 + 71 + 96 - 81 + 44 - 2 + 66 - 99 - 29 - 4 - 58 - 16) - 99 + 24 - 34 + 62 + 72) - 64 + 2 - 74 - 79 - 94 - 74 - 12 + 87 + 30 - 23 + 54 + 21 + 80 - 37 - 53) - 13 - 68 - 97 + 41 - 29) - 1 + 25 + 24 + 29) - 13 + 37 - 47 - 61 - 16 - 10 + 88 + 27 - 41 - 26 + 36 - 53) + 21 + 71 + 28 + 18 + 74 - 12 - 75 + 35 + 96 + 20 - 41 + 25 + 38 + 5 - 50 + 59 + 68 + 76 + 18 - 4 + 22 + 34 + 55) + 33 + 66 - 87) - 18) - 29 + 15 - 44 - 23 - 69 - 96 + 43 + 32 + 28 + 25) - 96) + 76 + 35 + 27 - 8 - 70 - 11 + 65 - 8 + 45 + 86 - 79 + 76 + 21 + 58 + 18 - 75) + 76 - 24 - 12 + 85 - 1 - 45) - 44 - 91 - 37 + 76 - 67) - 14 - 20 + 76 + 39 - 35 - 52 - 7 - 11 + 37 - 92 + 63 - 18) - 53 - 96 - 75 - 93 - 55 + 99 + 30 - 29 - 26 + 83 + 4 + 28 - 77 - 89) - 81 - 84 - 1 - 13 + 89 + 72 + 45 - 31 + 19 + 26 + 98 + 99 - 74 + 36 - 32 + 19) - 38 + 64 + 47 + 4 + 90 - 94 - 45 - 2 + 0 - 10 - 47 + 97 + 86 + 88 - 64 + 41 + 41 - 23 + 100 + 9 - 58 + 33 + 28 - 6 + 10 - 100 + 35) + 40 - 37 + 48 - 86 + 17 + 11 + 71) - 55 - 74) - 86 - 34 - 36 + 10) - 60 + 57 - 89 + 23 - 93 - 7) + 89 + 4 - 4 - 73 - 76 + 45 - 31 - 93 + 23 + 93 + 3 - 65 - 91 - 67) + 47 + 25 + 65 - 73 + 37 - 16 + 20 - 97) + 74 - 35 + 20 + 68 - 18) - 17 + 6 - 38) + 73 + 40 + 80) + 30) + 98 + 100) - 85 - 79 + 67 - 37 + 1 + 29 + 64 + 96 + 69 + 41 + 21 + 1 - 73 - 93 - 84 + 1 - 17) - 18) + 26 - 89 - 32) + 25 - 39 + 33 - 72 - 16 + 99 - 59 + 92 + 26 - 63 - 91 + 28 + 82 - 67 + 1) - 88 + 38 + 34 + 57 + 79 - 9 - 39 + 47 - 59 + 63 + 49 - 97 + 75 - 58 + 37 + 87 + 18 + 36 - 27 - 74 + 54 - 18) - 93) + 34 + 69 - 96 - 7 + 58 + 54 - 5 + 16 + 92 - 6) + 39 + 68 + 81 - 38 - 2 - 48 - 83 + 55 - 1 - 55) + 18 - 2 + 69 - 83 + 78 + 33 + 42 + 79 - 0 - 54 + 19 - 43 + 94) - 18 + 12 - 62) - 29 + 15 + 66 + 3 + 13 - 43 - 25 + 6 - 27 - 26 - 39 + 55 + 59 + 79 + 33 - 18 - 6 + 81 - 53) - 40 - 38) + 91 - 73 + 86 + 89 + 73 + 52 - 100 + 38) - 16 - 51 - 69 + 97 - 12 + 12 + 97 - 58 - 32) + 52 - 73 + 95 + 25 - 37 + 6 + 47 + 76 + 8 + 83 + 28 - 82 + 40 - 33) - 100 - 54 + 7 + 64) - 70 - 60) - 67 + 28 + 98 + 39 + 50 - 73 + 88) + 74 - 90 - 78 - 70 + 8 + 54 - 62 + 82 - 5 - 30 - 27 + 18 - 28) + 21) - 26 + 69) - 97 - 0 - 78 - 26 - 13 - 46) - 49 - 91 - 19) + 3 - 68) + 82 - 7 + 8 + 69 + 25 - 41 - 66 - 8) + 12 + 27 - 66 + 99 + 42 + 72 + 96) - 92) - 66 - 14 - 92 - 31 + 23 + 75 - 62 + 67 - 58 + 62 - 83 - 27 + 62 + 100) - 86 - 66) - 17 - 67 + 93 - 84 - 11 - 59 - 65 + 93 + 4 - 19 - 85 - 95 - 10 - 35 - 39 + 10 - 27 + 52 - 100 + 10 + 26 - 66 - 72 - 88 - 83 - 31 - 55 - 3 + 53 - 40 - 60 - 97 - 51 - 48 + 68 - 27 - 2 + 15 + 11 - 41 - 87 + 63 - 22 + 91 - 96 + 32 + 62 - 100 + 62 - 6 + 30 - 85 + 22 + 48) - 78 + 66 + 15 - 34 - 26 - 42 - 30 + 62 + 62) + 60 - 61 - 60 + 4 - 79 + 79 - 77 - 55 + 62) + 55 + 90 + 49 - 51 - 86 - 55) - 56 - 93) + 12 - 38 + 61 + 77) + 83 + 96 + 1) + 83 + 58 + 61 - 43 + 76 - 49 - 61 + 78) + 62 - 48 - 30 - 63 + 24 - 12 + 50 + 80 - 74 + 47 + 65 - 70 + 41 + 37 - 11 + 8) - 19 + 97 - 33 + 50 - 35 - 26 - 11 + 79) + 3) - 25) + 72 + 34 + 42 + 30 + 92 - 40 + 46 + 87 - 71 + 33 + 98 - 94 + 43 - 60 + 23 - 78 + 5 + 35 - 84 - 71 - 26 - 14 - 40 - 40 + 86 - 24 + 82 + 70 + 3 + 57 - 73 - 4 + 71 - 78 - 54 + 62) + 84 + 41 - 25 - 53 + 91 + 53 - 34 + 71) + 1) - 55 - 44 + 57) + 34 + 88 - 36 + 95 + 60 - 61 - 39 + 0) - 1) - 86 - 83) - 13 + 1 + 0 + 31 + 89 + 0 - 94 - 63 - 61) + 37 + 54 - 5 - 6 + 65) - 100 + 57 + 0 - 78 + 80 - 8 - 57 - 79 + 99 + 69 - 81 - 66 - 72 - 81 - 90 - 0) - 57 + 71 + 95 - 42 - 66 - 66 + 82) - 65) - 61 - 86 - 93 - 1 - 96 + 31 - 79 + 69 - 86 - 95 - 82 + 48 + 57 - 48 - 49 - 50 + 71 - 26 - 28 - 79 - 17 + 46 + 80 + 68 + 87 - 75) - 13 + 92 + 63 - 32 - 42 + 24 + 1 + 67 + 22 - 3 - 88 - 28 - 97 + 29 - 62 - 84 + 7 + 33 + 57 + 51 - 67 + 96 + 59 + 69 - 77 - 54 + 0 + 83 - 33 - 70 - 56) + 69 - 68 + 8) - 7) + 20) + 70) + 84 + 51) + 27 + 71 + 22 - 43 + 39 + 24 + 99 - 86 - 88) - 5 - 47 - 25 - 43 + 18 + 22 - 4) + 3) + 7 + 23 + 0 + 25 - 92 - 95 - 90 + 6 - 70 + 31 - 60 - 86 - 74 + 47 + 71 - 65 + 19 - 14 - 88 + 23 + 1 - 87 + 26 + 50 + 32 - 39 + 5 + 36 + 68 - 89 + 46 - 74 + 86 + 64) + 38 - 36 + 25 - 89 - 59 + 39 + 77 - 61 + 14 + 12 - 40 + 71 + 0) + 65 + 2 - 96 + 98 - 42 - 61 - 61 + 63 - 57 - 14 - 57 + 75) - 0 + 63 - 62 + 74 + 98 - 16 - 28 - 47 - 27 - 47 - 15 - 50 - 8 - 26 - 78 + 91 - 49 + 64 + 5 - 45 - 34) + 7 - 71 - 19 - 59 + 98 + 71) + 24 + 32 - 59 - 90) + 47 + 95 - 34 + 11 + 99 - 49 + 20 + 63) + 1) + 87 - 9 + 68 + 51 + 46 + 87 - 79 + 41 + 7 + 86 - 90 - 44) + 14 + 6 - 10 - 93 + 40 - 76 - 47 + 74 - 3 - 99 - 56 + 97 + 38 + 85 + 81 + 83 + 69 + 83) - 2 - 44 + 74 - 84 + 69 + 48 + 64 - 64 - 52 + 1 + 73 + 52 - 68) + 44 + 91 + 45 + 81) - 5 - 1 + 36 - 83 + 69 - 88 - 96 - 70 + 94 - 72 + 51 - 38 + 87 + 18 - 59 + 56 - 23 + 72 - 69) - 93 - 90 + 49 - 91 - 35 + 42 + 43) + 25 - 12 + 23 - 73 + 91) + 77 + 34 - 26 + 34 + 82 - 69 + 66 + 57 + 51 + 3 + 8) + 18 + 97) + 7 + 33 - 18 + 71 - 18 - 92 + 17 + 36) + 55 - 17) - 17 + 40 - 21 - 47 - 52 + 2 + 22 + 91 + 92 + 65 - 54 + 88) - 16) + 23 + 86 - 76) + 72 - 19 + 10 - 63 + 10 + 53 - 22 - 15 - 98 + 92 - 48 - 92 + 60 - 71 + 12 - 59 - 44) - 32 + 29 + 58 - 20 - 4 + 10 - 88 + 64 - 79 + 31 + 91) - 94 - 78 - 99 - 3 - 87 - 97 + 80 - 96 + 44 - 81 - 16) + 84 + 90 - 22) - 37 + 40 + 89 - 45 - 96 - 38 + 62 - 50 + 93 - 43 + 50 + 14 + 49 + 24 + 81 + 27 + 68) + 65 + 49 + 71 + 0 - 80 + 88 + 62) + 32 + 15 - 21 - 41 + 4 - 78 + 10 - 16 - 89 - 73 + 3 - 89 + 3 + 88 - 0 - 51 - 73 - 70 + 99 - 21 - 14 - 9 + 24) + 43 - 89 - 11 - 37 - 90 - 34 + 12 + 9 - 14) + 22 + 70 + 41 + 48 + 47 - 37 - 92 + 32 - 9 - 57 + 60 + 43 + 85 + 72 - 66 - 60 - 72 + 82 - 13 + 47 - 38 - 71 - 86 - 46 - 99 + 6 - 98 - 45 + 53 - 38) - 89 + 100 - 13 + 45 - 14 - 54 + 11 + 81 - 12 + 6 - 41 + 93 + 75) + 72) + 24 - 73 + 4 - 20 + 3 - 94 + 92 - 64 - 84 - 30 - 100 - 41 + 23 + 98 + 30 + 100 + 97 - 9) + 62 + 20 - 85 - 28 - 54 - 100 - 46 - 99 + 63 + 0 + 21 + 98 - 76 - 76 - 88 + 75) + 22) + 50 - 23 - 39 - 38 - 77 - 80 + 100 + 2 - 32 + 79 - 57 + 23 - 51 - 19 + 33 + 49) - 98 - 47 - 49 - 14 + 45 - 44 - 32) - 99 + 45) + 49) - 10 + 12 - 28 - 22 + 87 - 58 + 58 - 37 + 65 + 61 + 86 - 32 - 16 - 6 + 75 - 52 - 67 + 16) - 11 + 94 - 50 - 37 - 56 + 88 + 0 - 8 + 93 + 14 + 23 - 69 + 33 - 20 - 10 + 87 - 0 - 93 + 80 + 71 - 13 - 20 + 97 - 34 + 43 - 40 + 58 + 51 - 29 - 9 - 81 - 58 - 73 - 18 + 61 - 77 - 6 - 48) - 42 + 36 - 66 - 2 + 78 + 70 - 68) + 68 + 10 + 95 + 52 - 48 - 77 + 21 - 99 - 28 - 97 + 17 + 71 - 14) - 62 + 100 - 60) + 45 - 92 + 50) + 70 + 67 + 11 + 37 + 35 - 1 - 32 - 56 + 83) + 37 + 48 + 48 - 70 - 62 - 65 + 30) + 15 + 96) + 25 - 68 - 31 + 87 - 14 - 14 + 55 - 34 + 11 + 17 + 73 + 79 - 42 + 95 + 91 - 99 + 47 + 75 + 25 + 71 - 36 - 45 + 59 - 84 + 7 + 1 + 3 + 91 - 90 + 3 + 100 + 100 + 92) + 25 - 14 - 80 + 41 + 17 - 98 - 84 + 49 - 0 - 60 + 68 + 76 + 87 + 60 - 0 + 52 + 93 + 78) - 13 + 27 - 59 - 81 + 76 - 45) - 34 + 87 - 41 - 10 + 42 - 28) - 5 + 19 + 86 - 40 - 50 + 6 + 97 - 32 - 51 + 65 + 67 + 64 - 1 + 5 - 43 - 14) - 64 + 82 - 34 - 65 - 31 + 49 + 49 - 14 - 29 + 48 + 42 + 34 + 86 + 16) + 4 - 43 - 84 + 50 - 76 + 70 + 85 + 61 - 6 - 81 + 64 + 81) + 97 + 25 - 28 + 40 + 4 + 10 + 10 + 45 + 12 - 67 - 78 - 17 - 52 + 98 - 69 - 67 - 81) - 18 - 82 + 43 - 72 - 42 - 53 + 87 - 56 + 17 + 29 - 1 + 87 - 13 - 26 + 85 - 38 + 10 - 46 + 35) + 94 - 94 - 28 - 71 - 80 + 0 - 41 - 33 + 85 + 49 - 62 + 69 - 61 - 48 + 65 - 49 + 63) - 32 + 85 - 67 + 92 - 97 + 48 + 71 - 47 + 41 - 36 - 1 - 62 + 55 + 99 - 23 + 15 + 46 + 47 - 59) - 65 + 38 - 27 - 66 - 28 - 47 - 17 - 7 + 77 - 100 + 92 - 28 + 45 + 31 + 50 - 28 - 83 - 29 - 6 - 21 + 63 + 54 - 1 + 23) + 94 - 4 - 75 + 49 + 5 - 98 + 70) - 35 + 53 - 5 + 47 - 60 - 90) + 42) + 57 - 15 - 41 + 8 - 86 + 72 + 19 - 13) + 75 - 12 + 27 - 81) - 23 + 98 + 32 + 42 - 44 - 35 - 59 + 70 + 66 + 37 + 0 - 71 + 63 + 67 - 21 + 18 + 55 - 20 + 31) + 52 + 6 - 9 - 92 - 16 - 35 - 36 - 60 - 91 + 80) + 49) - 86 + 92 + 33 + 72) - 32 - 40 + 40 - 35 - 9 + 26 - 95 + 5 - 78) + 26 + 47 + 59 + 41) - 58 - 7 - 26) + 13 - 6 - 24 - 44 + 14) - 21 - 98 + 31 + 27 + 74 - 14 + 97) - 78 - 72 - 51 + 44 - 22 - 62 + 56 + 70 + 24 + 73 + 93 - 60 + 44 + 37 - 25) + 60 - 87) + 23 - 18 + 59 - 8 + 62 + 17 + 66 - 37 - 49 + 29 - 75 - 62) - 91 - 90 + 100 - 17 + 24 + 43) - 4 - 48 + 91 + 36) + 48 - 87 + 6 + 53) - 44 - 82 + 47 + 14 - 38 - 10 + 7 - 99 + 1 + 93 - 41 - 55 - 49 - 96 - 92 + 80 - 84 + 36 + 24 - 54 - 26) - 0 - 62 + 62 - 40) + 92) - 39 - 51 - 44 + 90 - 20 + 31 - 0 + 88 + 39) - 84 + 83 - 30 - 22 - 63 + 73 + 92 - 26 - 73 + 9 - 42 + 2) + 68 + 52 + 66 - 14 + 75 + 52 - 77 + 83 + 32 - 36 - 16 + 52 + 92 + 34 + 92 + 79 + 19 + 96 + 79 + 74 - 34 + 46) + 12 - 17 + 33 + 10 + 98 - 85) + 69 + 38) - 86 + 60 - 54 - 74 - 91 - 70 - 95 - 28 + 85) - 18 + 12) + 15 + 12 + 57) - 37 - 11 + 38 + 72 - 46 + 17 - 9 - 12 - 38 - 75 + 44 - 50 - 96 - 36 - 12 + 30 - 55 - 83 - 4 + 76 + 56 + 12 + 43 - 66) + 88) + 69 + 49 - 56 + 66 - 31 + 87 - 40 - 19 - 11 + 74 + 23 + 17 + 61 - 50) - 52 - 21 + 21) + 4 - 0 + 75 - 15 + 42 + 23 + 30 + 52 + 71) - 5 - 86 - 47 + 51 + 25 + 18 - 71 + 92 - 94 + 47 + 98 + 63 - 61) + 86 - 31 - 54 + 81) - 34 + 67 + 73) - 17 + 28 - 28 - 45 - 92 + 82) + 98 - 83 + 72 + 65 + 53 - 95) + 59 - 53 + 82 - 14 + 66 - 56 - 3) - 41 + 31 - 34 - 11 + 27 - 37 + 58 + 61 - 47 + 60 + 54 + 79 - 1 + 9 - 97) + 55) - 73 - 67 + 55 + 7 + 88 - 26 + 89 - 89 + 54 + 94 - 6 + 60 - 35 + 44 - 25) + 24 + 56 + 42 + 6 - 43) - 15 + 80 - 85 + 97 - 21 + 36 - 13 - 48 + 73 + 53 - 68 - 83 + 96) - 79 + 64 + 50) - 72 + 44 + 66) + 55 - 32 + 39 + 15 + 25 - 42 - 58 + 8 - 5 + 38 + 37 + 90) - 7 - 10 + 5 - 27 + 62 + 90 + 17 - 55 + 65 - 97 + 48 + 88 + 90 - 25 + 55 + 72 + 55) + 11 + 56 - 12 - 56 - 31 + 54 - 7 - 48 - 65) - 8 + 26 - 0 - 78 + 69 + 20 - 49 + 62 + 64 - 25 - 42 + 14 + 61 - 95 + 42 - 71 - 72 + 36 - 8 + 79 + 76 - 95 - 15) - 8 + 86 - 70 + 92) + 44 + 82 - 45 + 29 + 22 - 72) - 7 - 37 + 26 - 100 - 84 + 22) + 30 - 94 - 64 + 37 + 49 - 85 + 32 + 67 + 59 + 56 - 81 + 82 - 56 - 59 - 22) + 8 + 37) + 82 - 62 - 10 - 19 + 39 + 22 + 7 - 59 - 41 - 98) - 12 + 85 + 67 - 93 - 38 + 70 + 71 + 93 + 47 - 94) + 42 - 2 - 90 - 75 - 77 + 39 + 5 + 24 - 89 + 72 + 21 - 65 - 50 + 76 + 82) + 13 + 73 - 44 + 26 + 61) - 67 + 86 + 29 + 22 + 86 - 64 + 77 + 61 - 86 + 93 - 11 - 72 - 78 - 70 - 1 - 3) - 26 - 45 + 13 - 50 - 43 - 56 + 45 + 11 - 75 + 97) - 58 + 35 + 49) - 50 + 86 - 2 + 37 - 28 + 18 - 71 - 56 - 55) + 81 - 78 - 66 + 28 - 20 + 74) - 98 - 97 - 37 - 0 - 5 - 91 + 80 - 62 - 96 - 16 + 65 - 22 - 36 + 12 - 11 + 77 + 55 - 26 + 81 - 99 - 19 - 4) + 55 - 55 - 42 + 26 + 84 + 69) - 37 - 47 + 72 - 1 - 90 - 94 + 1 - 92 - 58 + 40 - 74 - 20 + 56 - 3 + 64 - 54 - 44 + 65 + 13 + 23 - 85) - 78 - 71 + 90) + 2 - 17 + 36 - 32 + 16) - 30) + 36 + 5 + 12 - 64 - 48 + 76 + 58) - 81 - 34 - 31 - 22) - 36 - 61 + 52 - 93 - 59 - 52 - 39 - 30 + 52 + 90 + 17 + 1 + 73 + 25 - 51 + 77 - 80 + 71 + 32 + 62 + 64 - 5 + 9 - 53 + 54 - 74 - 92 + 52 + 1 + 2 - 44 - 2 - 94 + 88 - 87 + 62 - 34) + 54 + 7) - 11 - 46) + 30 + 31 + 15 + 90 - 98 + 53 - 5 + 24 - 69) + 80 + 83) + 73 + 25 + 73 - 24 - 24 + 100 - 42 - 88 - 11 - 91) + 92 + 51 + 49 + 35 - 15 + 98 - 69 - 72 - 89 - 49 + 35 + 58 + 15 - 90 - 88 + 14 - 87) + 74) + 82 + 41 - 78 - 100 + 93 + 24 + 87 - 82 - 64) - 21 + 30 - 67 - 64 + 1 - 61 - 51 - 28 + 39 - 77 + 64 - 39 + 24 - 33 - 52 + 71 + 68 - 28 - 89 - 41 + 92 + 94 + 72 + 34 + 61 - 58 - 68 - 43 - 43) + 26 - 27 + 49 - 96 + 20 - 61 - 14 + 32 - 0 - 100 - 18) + 47) + 9 - 44 + 14 - 41) - 57 - 58 + 47 - 66 + 82 - 88 - 90 - 94 + 39 + 89 - 85 + 38 + 34 - 6 - 74 + 23 - 75 + 80 - 12 - 8 - 98 + 63 - 99 - 81 + 81 - 50 + 34 + 62 + 51) + 12 + 86 + 71 - 71 - 43 - 39 - 94 + 15 + 48 - 69 + 74 - 29) + 45) + 5 + 41 - 75 - 5 - 59 + 44 - 61 + 60 - 66 + 100 + 57 + 49 + 17 + 10 - 6 - 69 - 43 + 68) + 34 - 7 - 41 - 84 + 36 + 84 - 39 - 61 + 86 + 29 + 83 - 13 + 81 + 21 - 61 - 60) + 26 + 71 + 98) + 71 - 36 - 25 + 5 - 8) + 17 - 45 + 56 - 85 - 41 + 61 + 25 + 21 - 46 + 24 - 21 - 94 + 73 + 25 - 71 + 40 - 21 - 13 - 90 - 79 + 82 + 23 + 34 - 64 + 6 + 74 - 98 + 27 - 16 - 53 + 100 - 25 + 86 - 83 - 61 - 99 - 27 - 85 - 62) - 41 + 63 - 17 + 15 - 27 - 27) - 61 + 40 + 10 + 47 - 73 + 8 - 87 + 46 + 76 - 72 - 51 + 91 + 90 - 39 + 86 - 94) - 100 - 87) + 6 + 44 + 19 - 74 - 84 + 75 - 11 - 17 + 26 + 12) + 82 + 51 + 10 - 46 + 61 + 24 - 53 + 82 + 26 - 86 + 12 + 86) + 83) - 77 - 56 - 66 + 49 - 71) - 27 + 76 - 70 - 28 + 73 - 33) - 43 - 66 - 41 + 9 + 6 + 35 + 9 - 45 + 6 + 85 + 38 - 79) - 3) + 23 + 81 - 71 + 38 - 25 - 43 - 97 + 21 - 60 - 21 + 24 + 68 - 45 + 13 - 21 - 81 + 70 + 50 + 5 - 13 + 82 - 28 - 86 - 97 - 25 + 19 - 78 - 6 + 27 - 94 + 36 - 94 - 10 + 52 - 1 + 35 + 81) - 10) - 54 - 53 - 66 - 30 + 75) - 12 + 49 + 3 + 49 + 30 + 75 - 63 - 56 + 84 - 59 + 91 - 78 + 50) - 84 - 64 + 76 - 9 + 4 - 47 - 72 + 58 - 22 + 75 + 12) - 77 + 33 - 98 + 35 + 39 + 82 + 92 - 61 - 87) + 31 + 53 - 97 + 100 - 86 - 88 - 28 + 5 + 97 + 20 + 24 + 93) - 86 + 4 + 4 + 7 - 14) - 82 - 36) + 44 - 79) + 71 + 53 + 13 + 34 + 41 - 28 - 77 - 99 + 83 - 17 + 51) + 50 + 62 - 48 - 50 - 49 - 53) + 31 + 23 + 50) + 21 - 70 - 57 + 76 + 28 + 34 - 91 + 2 - 49 + 90 - 30 - 86 - 69 + 44 + 72 + 33 + 47 - 70 - 51 - 31 + 51) - 88 - 19 - 22 - 99 + 4) + 93 - 4 - 13 - 34) + 37 - 85) + 85 - 4 - 46) + 25 + 68 - 17 - 8 + 46 + 92 - 90) - 12 - 35 - 11 + 36 - 15 - 88) - 68 - 33 - 1 + 27 - 0 - 41) + 8 + 48) + 6 + 37 - 47 + 69 + 16 + 87 + 99) + 62 - 46 + 81 + 93 + 67) - 74 - 99) - 43) - 92 + 37 + 2 + 34 - 2 + 6 + 100 + 1 - 84 - 93 + 37) + 98 - 41 - 23 - 17 - 16 + 20 + 77 + 68 + 6) + 55 - 94 + 76 + 19 + 35) - 76 + 22 - 72 + 79 - 70 + 77 - 14 + 12 + 99 + 28 - 81 - 45 + 89 + 86 + 93 - 14 + 1 - 84 + 83 - 58 + 82) - 74 + 58 + 70 + 17 + 100 - 58) + 91 - 54 + 33 + 11) - 71 + 93 + 47 - 24 + 62 + 20 - 17 + 5 + 26 + 38 - 63 - 57 + 74 - 51 - 61 - 23 + 52 + 63 + 69) - 6 + 54 - 18 + 18 + 6 - 24 - 75 - 0 - 21 - 14 + 86 + 90 - 63 + 30 + 51 - 86) + 56 + 4 + 44 - 34 - 58 - 49 - 57 - 38 + 20 - 53 - 27 - 76 - 40 - 93 - 71 + 56 - 85 - 74 + 21 + 32 + 23 + 19) + 1 + 95 - 63 + 21 + 82 - 7) - 81 + 5 + 17 + 81 + 60 - 20 - 13 - 4 + 88 - 45 - 10 + 23 - 26) + 24 + 92 + 94 + 30 - 61 - 8 + 76 + 74 - 32 - 9 - 15 + 8 - 23 + 57 + 36 - 64 - 69 + 21 - 97 + 83 + 46 + 14) - 71 + 94 - 64 + 15) + 43 + 74 + 62 + 12 + 93 + 86 + 8 - 6 + 70) - 73 - 60 + 100) - 18) + 86 + 69 - 10 + 0 - 71 - 51 - 88 - 30 - 3 + 80 + 6 - 65 + 14 + 67) + 78 + 51 - 38 + 38 + 49 + 51 - 23) + 22 - 83 - 56) + 100 + 96 - 54 - 5 - 85 + 43 - 63) - 67 + 56 + 61 + 22 - 75 + 87 - 84 - 38 + 5 + 12 + 43 + 37 - 34 + 92 - 0 + 74 + 80) - 93) - 38 - 52 - 90 + 21 + 57 - 90 + 88 - 31 + 51 + 94 + 53 - 85 + 49 + 20 + 13 - 79 - 88 - 48 - 29 - 92 + 63 + 95 + 10 - 65 + 70 + 82 + 39 - 84 - 32 + 57 + 77 + 95 - 8 - 68) - 94 + 40 - 19 + 88 - 91) + 60 + 80 - 80 - 17 - 56) + 99 - 68 + 89 + 58) - 78 + 18 + 26 - 33 + 76 + 6 - 94 - 20 - 69 - 8 + 21 + 46 + 29 + 14 - 44 - 96 + 14 + 95 - 17 + 4) + 13 - 23 + 62 + 27 + 42 + 73 - 37 + 58 - 48 - 90 + 1 + 100) - 27 - 15 + 45 - 46 + 76 + 18 + 46 - 75 - 54 + 21) - 92 - 77 + 87 - 56 - 45) + 77 + 12 - 36 - 99 - 94 + 85 - 93 - 11 - 57 + 94 - 84 - 43 + 0 + 65) - 79 + 90 + 78 + 53 - 39 + 66 - 33 - 54 + 38 - 23 + 47 + 69 + 32 - 80) - 23 + 50 + 40 + 6 - 59 - 40 + 31 + 74 + 4 - 72 - 59 + 26 - 71 + 43) + 82 - 37 - 92 - 58 - 56 + 98 + 57 - 49 + 83 - 74 + 7 - 60 - 69) + 100 - 41 + 77 - 14 + 66 - 97 - 29 - 89 + 44 + 6 + 60 + 29 - 55 + 73 + 79 - 80 - 73 - 49 - 77 + 30 + 97 + 49) + 85 + 22 + 75) - 96 + 26) + 92 + 78 + 24 + 84 + 3 - 80 - 49 + 23 + 53 + 34 + 82) + 10 + 71 + 69 - 0 + 76 - 47 + 62 - 11 - 8 + 56 + 30 + 1 + 75 + 25) + 37) + 5 + 53 + 45 + 9 - 9 - 60 + 90 + 72) + 21 + 40 + 100 + 68 + 17 + 63 - 96 - 1) - 95 - 5) - 68 - 33 - 16 + 12 + 74 - 43 + 93 + 23) - 12 + 44 - 20 - 36 - 39 - 7 + 93) - 8 - 53 - 24) - 47 - 71) - 20 + 65 - 73 - 87) - 92 + 33 - 50 - 53 - 93 - 20 - 31 + 86) - 17 + 29 + 57 + 2 + 91 - 88 + 3 - 31 + 96 - 13 - 93 - 26 + 72 + 33 - 59 + 59 - 7 - 69 - 13) - 91) + 95 - 15) - 70 + 14 + 24 - 12) + 5 + 93 + 41 - 15 + 52 - 93 + 60 + 76 + 89 - 38 - 96 - 53) + 43 - 19 + 2 + 51 - 85 + 34) - 40 - 86 + 84 - 5 + 64 - 2 + 1 - 46 - 58 - 99 - 2) + 28 + 83 - 14 + 33 - 2 - 58 - 20 - 51 + 98 + 43 + 91) + 5 - 5) - 65) - 75 + 53 + 52 - 22 + 96 + 98 + 43 - 45 + 50 - 32 + 86 + 10 + 87 - 3 - 69 - 94 + 87 - 72 + 74 + 95 - 58 - 89 - 28 + 21 + 86 - 64 + 68 + 56 - 0 - 13 - 82 + 0 - 24 - 20 - 20 + 61 - 36) - 78 - 59 + 16) + 92 - 87 - 13 + 87) + 41) - 2) + 55) + 52 + 34 + 23 - 14) + 11 + 27) + 68 + 88) - 15 + 42 - 27 + 50 - 1 + 71 - 18 + 68 - 60) + 42 - 36 - 48 - 22 + 9 - 34 - 20) + 76 + 28 + 20 - 51 - 19 + 23 - 48 + 98 - 78 + 65) + 92) - 14 - 9 - 100 + 72 + 37 + 58 + 35 - 19 - 38 - 53 + 55 - 70 - 77 - 52 - 93 - 25 - 16 - 65 + 94 - 56 + 81 + 13 - 73 + 40 + 4 + 43 + 34 + 93 + 36 + 12 + 64 + 2 - 93 - 17) + 13 + 63 + 3 - 97 + 24 - 18 - 53 - 18 - 69 - 98 + 96 + 28 - 38 + 77 - 84 + 85 + 72 - 75 + 13) + 66 + 54 + 60 - 50 + 51 - 25 + 49 - 68 - 16) - 9 - 90 + 96 + 0 - 26) + 18 - 38 + 48 + 82 + 42 - 43 - 84) + 7 - 14 + 7 - 45) + 14 + 80 - 65 - 85 + 68 - 26 + 17) - 49 + 41 + 49 + 80) - 94) - 2 + 5 - 36 - 58 - 69 + 14 - 16 + 99 + 22 + 35 + 74 + 14 + 5 - 46) + 25 + 63 - 12 + 97) - 75 + 67 - 48 - 7 + 7 - 3 - 4 - 99 - 54 - 44 + 7 + 21) + 17 + 70 - 39 - 79 + 71 + 47 + 74 + 92 + 90 - 23 - 90 + 90 - 51 + 45 - 5 - 45 + 69 + 85 - 13 - 73 + 42 - 100 + 57) + 36 + 93 + 15) + 13 - 98 - 88 + 14 - 31 + 9 + 88 - 48 + 30 - 28 + 0 - 80 - 77 + 53 - 92 + 2 + 88 + 2 - 97 + 64 - 33) - 97 - 79 + 59) + 36 - 62 + 32 + 41 + 53 + 98 + 51) + 33 - 58 + 9 + 44 + 81 - 93) - 93 - 56 - 9 + 89 + 61 - 49 + 34 + 82 + 30 - 96) + 79 + 81 - 9 - 12 - 25) - 54 - 24) - 23 - 31 - 98 + 75 + 35 - 12 + 71 - 19 - 23 + 71) + 67 + 76 - 28 - 55 - 8 + 70) + 6 + 84 + 6 + 0 - 9 + 84 + 95 + 36) + 82 - 34 - 0 - 84 - 40 - 96 - 67 + 50 - 40 + 61 + 80 - 10 - 15 + 95 - 33 + 13 + 19 - 65 - 28 - 18 + 71 + 71 - 27 + 20) + 99 - 54 - 79 - 65 + 76 + 38 + 57 - 56) + 73 + 72 + 69 - 57 + 18 - 4 - 40 + 67 - 23 + 50 + 60 + 72 - 49 - 47 + 24 - 48 - 55 + 88 + 22 + 73 - 60 - 5 + 87 - 68 + 12 + 51 + 66 - 20 + 61 - 22 - 12 + 36 - 31 + 51 - 86 + 26 - 93 + 61 - 44 - 64 - 14 + 28 - 17 + 12 + 78) - 58) + 47 + 23 + 11 + 55 - 62 + 42 - 66 + 86 - 46 - 9 + 100) + 68 + 79 + 42 - 50 + 25) - 82 - 14 + 88) - 17 - 65 - 22 - 29 - 64 + 7 - 22 + 1 + 7 + 82 + 9 + 14 - 6 + 52 + 37 - 60 - 88 - 96 + 2) - 44 - 32 - 14 - 23 - 68 - 87 - 33 + 78 - 19 - 18) - 81 - 47 + 70) + 53 + 12 + 86 + 45 - 16 + 27) - 44 + 91) - 18 - 75 - 41 - 36 - 71 + 41 + 67 + 26 - 19) + 78 + 6 - 35 + 59 - 56 - 98 + 40 + 13) - 78 + 58 + 69) + 64 + 65 + 37 + 52 + 36 + 82 - 50 + 10 + 18 - 76 - 2 + 15 + 51 - 78 + 81 - 2 + 27 - 38 - 0 - 57) - 93 - 21 + 81 + 64 + 47 + 95 + 40 - 100 + 85 + 96 - 25 - 8 + 43 - 71 + 51 - 95 + 56 - 74 - 62 - 79) - 55 - 0 + 36 + 58 - 6 - 72 - 2 + 2 + 49 - 29 - 100 - 59 - 2 + 45 + 31 - 13 + 95 + 70 + 22 - 61 + 31 - 29 - 60) + 57 - 62 + 10 - 39 - 27 - 20) + 5 - 53 - 28 - 21 - 5 - 49) - 89) + 53 + 90 + 48 - 64 - 57 + 98 + 91 + 15 + 85 + 22 + 69) - 25 - 60 + 49 + 21 - 76 + 31 - 35) - 70 - 73 + 83 - 27) - 19 + 87 + 89 + 15 - 82 - 75 - 22 + 88 + 42 - 51) - 93 - 6 + 60 - 0 + 18) + 60 + 6) + 14 + 2 - 46) - 61 + 29 + 52 + 73 - 74 + 81 - 8 - 23 + 81 - 62 + 18 - 98 + 23 + 84 - 55 + 75) + 44 + 96 + 88 - 70 + 98 + 46 - 93 + 23 + 54 + 7 - 15 - 86 - 46 + 57 + 65 + 47 + 25 + 3) + 56 + 37 - 83 - 45) + 77 + 71 - 66 + 59 - 1 - 78 - 71 + 2) + 18) + 22) - 82 + 78 - 73 + 77 + 16 + 77 + 90 - 10 - 7 + 59 + 32 - 3) - 13 - 45 + 91 - 82 + 18 + 39 + 40 + 61 - 6 - 73 + 12 - 42 + 22) - 51 - 55 + 70 + 58) + 68 + 7 - 29 + 53 + 44 + 82 + 20 - 24) - 65 - 52) + 60 - 26 - 53 + 63 + 12 + 95 - 51 - 1 - 6 - 78 - 28 + 60 + 48 + 50 + 85) - 16 + 18 + 22 - 49 - 98 + 64 - 62 + 76 - 41 - 100 + 42 + 48 + 80 + 36 + 87 - 4) - 26 + 78 + 81 + 45 - 72 + 55 + 17 - 94 - 71 + 91 - 51 + 5 + 98 - 6 - 47) + 5 - 12 + 59 - 59 - 94 + 36 + 2 + 4 + 18 - 7 - 54 - 22 + 30 - 37) + 13 + 9 + 24 - 58 - 18 - 59 + 54 + 4) - 59 + 4 + 51 - 21 - 27 - 100 + 13 - 8 - 86 - 43 - 11 - 89 - 78 - 67 - 27 + 50 + 92 - 60 + 88 + 65 + 0 - 36 - 5 - 3 - 52 - 84 + 59 + 59 + 80 + 68 + 88 + 57) - 8 + 21 + 3 + 24 + 86 + 100 + 54 - 29 + 57 - 65) + 96 + 99 - 26 + 74 - 27 - 100 - 17 - 6 + 77 + 31 + 39 - 70 + 19 - 9 - 31 - 2) + 44 - 13 + 40 - 93 + 83 + 75 + 65 + 88 - 5 - 93 + 40 + 26 - 57 - 78 - 87 - 1 + 58 + 3 - 69 - 4 - 0 - 68 + 88 - 11 - 67 - 2 + 55 - 66 + 70 - 65 + 68) + 9 + 58 + 39 - 90 - 88 - 32 - 10 - 87 + 59 - 6 + 88 + 93 - 16 - 89 + 62) + 98) + 36 - 94 + 82 - 35 + 82 - 56 + 35 - 75 + 54) + 4 - 52 - 19 + 46 + 99 - 8 + 11 - 30 - 95 - 86 + 22 - 39) + 34 - 91 + 56 + 95) - 86 - 45 + 24 + 33) + 28 - 20 - 75) - 96 + 75 + 70 - 34 - 15 + 17 + 86) + 95 + 74 + 100) - 83 + 75 + 29 + 53 + 79 + 48 - 9 - 83) - 90 - 34 - 34 - 21 + 82 - 17 + 87 - 85 + 6 + 84 + 8 - 14 + 27 - 27 + 48 - 72 + 94 + 41 - 5 + 59 + 22 + 20 + 18) + 51 - 93 - 58 - 62 - 44 - 12 + 48 - 65 + 91 + 41 - 82 + 36 - 66 - 99 - 5 - 55 - 12) + 90 + 72 + 34) + 48 + 85 + 98 + 99) + 82 - 75) + 41 + 35 - 1 - 16 + 75 + 45 - 19 - 53 - 46 - 82 - 49 + 21 - 14 - 48) + 87 - 95 - 27 - 30 + 8 + 71 - 36) - 87 + 57 + 77 - 57 - 8) - 81 - 88 + 72 - 100 - 49 - 40) + 46 - 73 - 31 - 10 - 28 + 47 + 90 + 83 - 94 + 30 - 90 - 61 + 71 + 88 + 56 + 14 - 65 + 79 - 89 - 25 + 61 + 81) + 18 + 28 - 58 + 38 - 68 - 65 - 56 - 16 + 82 + 89 + 74 - 56 + 82) + 94 - 83 + 8 + 22 + 54 + 29 - 47 - 12 + 37) + 83) + 57 - 6) + 95 - 33 + 63 - 87 + 72 - 21 - 27 + 22) + 32 - 59 + 68 - 49 + 100 + 59 + 59 - 97 - 35 - 34 - 94 + 81 + 13 - 7 - 33 + 49 + 40 + 89 + 99 + 73 + 67 + 38 - 87 + 43 - 75 - 66 - 38 + 15 + 64 + 10) - 95 + 21 + 8) + 91 - 86 + 27 - 96 + 65 - 35 + 20 - 48 - 51 + 90 - 23 - 45 - 89 - 59 - 66 + 4 + 93 - 100 - 100) + 73 - 52 + 10 + 90 - 98 + 58 - 21 + 2 + 35 + 69 + 97 + 3 + 92 + 52 - 17 - 40 - 44 - 76) - 84 + 88 + 84) + 4 + 1 + 43 - 15 - 69 + 61 + 22 + 7 + 83 - 76 + 42 - 85 - 86 + 20 + 56 - 84 - 92 - 48 - 92 + 40 - 55 - 92 + 55) - 68 + 82 - 17 - 26 - 33 - 27 + 33 - 49 + 96 - 6 + 6) - 1 - 41 - 14 + 64 - 84 - 4 - 51 - 94 + 24 + 14 + 32 + 100 - 66 + 89 - 25 - 83 + 42 - 82 - 59 - 47 + 79 + 44) + 57 + 74 + 91) - 1 + 94 - 76 + 98 - 79 + 99 - 27 - 59 - 98 + 11 - 29 - 20 - 36 - 83 - 8 + 40 + 44) - 78 - 9) - 82 + 90 - 35 - 42 + 71 - 53 + 32 - 38 - 47 - 76 - 60 - 34 + 85 + 17 - 47 + 27 - 75 - 67 + 2 + 3 - 87) - 57 - 36 - 0 - 21 + 3 + 2 - 75 + 93 - 39 - 76 + 30 + 1 - 73 + 1 + 35 - 80 - 39 + 73 + 2 + 98 - 7 - 72 + 1 - 96 - 3 + 50 - 81 + 74 - 48 + 3 + 74) - 100 + 28 - 18 - 14 + 100 - 5 + 21 - 14 - 26 - 76 + 84 + 40 - 67 + 40 - 24 - 26 + 77 + 97) + 57 + 57) + 28 - 32 - 59 + 69 - 57 + 82 - 89 - 19 + 23 - 81) - 76 + 16 + 1 - 40 + 67 + 81 + 3 - 31 + 87) + 34 - 69 - 83 + 25 + 74 - 81 + 92 + 90 - 46 + 19 + 15 - 37 + 17 - 80 + 81 - 22 - 42 - 2 + 7 + 96) + 60 + 98) + 98 - 54 + 23) - 72 - 41 - 50 - 95 - 78 - 48 + 62 + 16 + 26 - 77 - 37 + 46 + 80 - 36 - 14) - 4 - 3 + 6 + 25 + 85 - 64 - 99 + 68 - 40 + 1) + 1) - 43 + 92 - 47 + 79 + 22 - 70 + 20 - 2 - 64 + 8 + 24 + 57 - 42 - 88) + 78 + 26 - 43 - 97 + 4) - 22 - 100 - 76 + 62 - 84) - 11 - 46 - 49 - 1) + 74 + 56 - 80 - 47 + 49 + 52 + 10 - 52 + 90 - 34 - 35 + 31 + 56 + 48 + 40 - 79 + 76 + 66 - 81 + 8) - 48 + 92 + 60 - 22 - 77 + 39) - 28 + 12 + 60) - 53 - 5 + 27 - 41 - 15 - 97 - 26 + 17 - 11 - 100 - 97) + 54 - 72 - 17 + 16) + 24 + 66) - 48 - 36 - 1 - 41 - 19 - 3 + 41 - 69 + 16 + 86 + 34 + 57 - 93) - 37 + 20 + 70 - 77) - 41 + 60 + 58 + 89 + 28 + 65 + 73 + 63 - 52 + 74 + 57 + 85 - 7 + 88 - 91 + 74 + 11 - 87 - 6 - 57 + 90 + 97 + 94 + 5 + 90 - 42 + 60 + 22 - 23 + 29 + 27 - 98 - 49 - 54 + 48 - 9 + 56 + 93 - 38 + 96 - 90 + 64 + 48 + 44 + 8) - 36 - 6 - 12 - 59 + 90 + 97 + 24 + 85 + 57 + 92 + 72 - 87 - 70 - 90 - 32 - 76 + 73 + 74) - 45 - 65 - 6 + 0 - 85 - 3 + 52) + 31 + 47) + 30 + 28) - 24 + 73 + 40 - 92 + 72 - 28 + 86 - 61 - 99 - 87 + 50 + 40 - 22 + 92 + 85 + 67) - 50 - 5 - 98 - 48 + 62) - 84 + 55) - 31 + 41 + 8 - 94 + 95 + 31 + 48 - 21 - 56 + 14 - 41 - 48 + 2 + 18 + 69 + 11 - 87) + 58 - 14 + 23 + 55 + 40 - 30 + 60 - 23 - 24 - 82 + 65 + 0) + 20 - 88 + 42 - 22 + 7 - 90 + 73 + 40 + 97 + 87 - 97) - 17 - 15 + 10 + 12 - 75 - 9 + 25 - 18 + 55 + 12 - 54) - 100 + 90 - 14 - 57 + 55 + 28) + 31 - 31 + 91 + 23 - 38 + 100 + 51 + 42 - 4) + 31 - 61 - 37 + 58 - 71 + 55 - 50) - 67) + 57 - 18) - 92 + 85 - 31 + 0 + 27 - 8 - 22 - 52 - 57 - 70) + 58 + 51 - 35) + 57 - 32 - 37 - 56 - 74 + 17 + 40 - 23 - 81) - 72 + 94) - 46 - 64) + 54 + 50 + 87) + 69 - 68 - 68 - 33 + 82 + 29) + 10) - 48 + 67 - 42 + 53 + 52 - 76 + 52 - 23 + 74) - 73 + 64 - 91 - 8 - 93 + 66 - 89 + 40 - 88 + 58 - 9 - 63 + 65 - 91 - 1 + 47 + 45 - 0) + 77 - 69) + 2 - 63 + 61 - 64 - 26 + 100 - 31 - 63 - 57 - 75 + 81 + 26 + 68 - 6 - 98 + 12 + 79 + 52 + 4 + 30 + 15 + 98) - 14 - 5 + 70 - 9 + 23 + 20 + 12 - 9) - 36) - 44 + 37) - 79 + 28) - 75 - 84 + 93 + 90 - 34) - 97 + 56 + 90 + 11 + 76 - 10 + 76 + 27 - 65 + 8 + 75 + 99 + 61 - 99 - 61 + 40 - 59 - 14 - 78 + 64) - 21 + 85) + 25 + 51 + 35) - 91 - 44) + 2 + 25 + 87 + 87 + 100) - 18 + 40 - 49 - 87 - 18 + 9 + 26 - 4 + 27 - 51 - 91 + 94 + 65 + 4 - 56 - 16 + 23 - 11 + 6 - 85) - 4 - 35 - 92 - 53 + 61 - 6 + 63 - 43 + 9 - 84 - 9 + 87 + 24 - 0 + 58 + 62 - 52 - 11 + 33 - 96) + 83 + 79 - 24 - 24 - 97 - 6 + 79 - 72) + 88 + 88 - 69 + 6 - 64 + 1 - 70 + 40 + 31 - 15 - 8 - 98) - 53 - 3 + 17 - 83 + 70 - 11 - 23 + 78 - 56) - 61 + 89 + 92 - 20 + 56) - 72 + 76 - 59 + 62 - 66 + 39 - 1 + 18 + 71 - 27 + 8 + 22 - 63 - 21 + 25 - 100 + 35 - 81 - 68 + 22 - 77 + 18 - 95 - 8) - 76 + 78 + 24 + 5 - 46 - 73 + 29 + 43) - 50 - 82 + 30 + 33 - 51 - 31 - 57 + 73 - 98 - 8) - 50 + 52 + 14 + 32 - 21 - 3 + 76 + 74) + 57 + 73 - 9 - 42 - 37 + 30 - 37) - 24 - 60 + 39 - 95) - 83 - 26 + 93 + 79 + 47 + 15 - 54 + 79 - 50) + 58 - 71 + 47 - 54 + 96 + 50 + 58 + 38 + 84 + 19 + 19 - 42 - 42 + 85 + 77 + 13 + 4 - 15) + 94) + 94 + 11 - 88 + 17 + 46 - 64 + 57 + 67 + 51 - 69 + 18 + 72 - 1 + 28 + 92 + 74 + 37 - 75 + 54 + 35 + 83 + 12 + 18 - 60 - 59 + 62 + 51) - 47 + 94 + 73) + 9 + 23 + 82 - 55 - 12 - 18 - 13 - 75 - 57 - 41 - 20 + 67 + 87 - 9 - 95 - 18 - 44 + 59 - 77 + 21 + 68 - 59 - 20 - 58) + 90 + 1 - 59 - 66 - 84 + 97 + 12 + 18 + 80) - 47 + 84 + 96 - 18 + 0 + 4 + 96 + 55 + 24 - 83 + 5 - 39 + 80 - 92 - 27 + 9 + 13 + 61 - 36 - 59 - 28 + 43 + 69 + 86 - 27 - 1 + 64 + 51 - 0 - 36 - 9 - 9 + 27 + 28) - 35) + 83 - 26 + 76 + 50 + 40) - 99 + 54 - 77 + 19 - 90 + 45 - 92 + 92 - 59 - 18 - 39 + 7) + 88 + 20 - 89 - 52 - 82 - 39 - 6) + 14 + 30 - 8 - 93 - 88 - 4 + 47 + 71 + 27 + 75 + 74) - 59 + 45 - 31 - 60 + 17 - 61 - 98 - 10 + 31 + 23 - 35 + 4 + 61 - 51 - 99 - 8 - 42 - 76) - 44 - 87 + 89 - 37 - 62 - 77 + 44 + 58 - 36 + 17 + 85 - 98 + 5 + 28) + 27 - 30) + 56 - 58 + 29 + 98 - 35 - 69 + 23 + 51) + 85 - 28 - 93 + 14 - 44) + 98 - 28 - 45 + 76 + 3 - 40 - 79 - 97 - 8 - 21 - 42 + 40 + 28) - 88) + 8 + 24 + 97 + 15 + 16 + 6 + 20 - 25 - 75 + 83 - 74 + 90 - 90 + 44 - 99 + 24 - 60 - 23 - 61 + 99 - 60 - 83 - 81 - 69 + 55 + 91 + 89) + 21 - 89 + 13) + 96 - 15 + 27 - 45 + 59) - 47 + 14 + 80 - 87 - 2 - 99) - 8 + 91 + 22 + 75 + 66 - 83 + 19 - 50 + 39 - 73 - 38 + 67 - 83 - 51 - 51 + 3 + 38) - 71 - 82 + 55 - 89 + 56 + 83 + 30) + 57 - 46) - 41 - 37 - 82 + 78 - 54 - 50 + 85 - 66 + 84 - 95 + 16 + 55 - 75 - 51 + 80) - 78) + 79 - 62 - 21 + 42 - 46) + 25 + 17 - 27 - 69 - 13 + 37 + 26 + 71 - 98 - 25 - 79 - 66 + 92 - 10 - 29 - 91 + 25 - 29 - 95 + 39 - 29 - 87 - 7 - 38 + 95 + 53 - 54 - 10 + 12 + 72 - 8 - 36) + 23 - 72 - 37 + 12 - 1 - 74) - 87 + 55) + 79 + 86 - 47) + 60 + 57 - 52 - 59 + 34 - 8 + 72 - 100 + 55 + 85 + 96 - 85 + 15) - 83 - 41 - 65 - 46 - 85 - 48 - 1 + 53 + 28 + 71 + 29 + 72 + 52 + 18 + 12 - 85 + 84 + 37 + 80 - 24 - 65 + 82) + 18 - 63 - 97 + 83 + 65 - 70 - 45 + 21 + 93 - 39 + 93 - 14 - 12 + 95) + 46 - 85 + 30 - 1 - 28 - 9 + 52 + 55 + 63 - 97 - 59 - 64 - 31 + 22 - 28 + 6 - 39 - 28 + 76 + 0 - 14 + 11) - 96 + 34 + 82 - 44 + 80 + 43 - 17 + 91 - 48 + 33 - 8 - 1) + 84 - 89 - 26 + 77 - 69 + 57 + 46 + 51 - 38 + 66) + 8 - 55 - 8 + 97 - 78 - 94 + 56 - 10 - 81 - 45) + 3 + 25 + 6 + 60 - 85) + 39 - 94 - 30 + 78 + 81 - 50 + 83 + 82 + 52) + 44 - 32 - 16 + 49 + 52 - 67 - 31 - 0 + 25 + 31) + 12 - 39 - 21 + 57) + 49 - 32 + 27 - 75 + 7 - 83 + 30 + 64 + 49) + 0 + 73 - 61 - 30 + 58 + 5 - 60) - 59) + 64 - 12 - 12 - 70 + 52 - 59 + 60 - 31) + 55 + 25 + 62 - 60 - 95 + 49) + 8 - 81 - 85 + 30) - 31) - 22 + 98 - 63 + 34 - 16 + 23 + 65 - 66 + 99) + 94 - 91 + 57 - 39 + 16 - 100 - 16 + 33 - 40 + 98 + 16 - 55 + 45 + 40 + 32 + 85 + 21 - 6 + 51 - 48 - 26 + 62 - 76 + 93 - 33 + 44 + 26 - 10) - 53 - 81 + 59 - 85 + 45 + 14 - 64 + 90 - 69 - 44 - 78) + 35 + 80 + 62 - 78 + 27 + 63 - 14 + 92 - 53 + 43 + 96 - 18 + 6 - 74 + 71 + 64 - 13) - 95 - 99 - 9 - 5) - 88 - 83 + 33 - 61 - 35 - 50) - 76 + 22 + 89 + 77) - 48 - 95 + 15) - 1 - 50 + 63 - 78 + 29 + 61 - 12 + 41 - 78 + 31 + 33 + 51 + 79 + 18 - 23 - 8) - 35) + 51 + 76 + 13 - 86 - 27 - 89 + 9 + 54 - 32 - 81 + 69 - 4 + 60 - 9 + 7) - 100 - 34 - 49 + 37 - 49 - 91 - 11) - 20 + 99 - 78 + 6 - 82 + 62 - 10 + 22 - 80 - 2 + 26 + 95 + 91) + 3 - 7 - 58 + 71 + 66 + 37 + 56 - 61 - 90) - 57 + 68 + 66 - 22 - 69 - 62 - 87 - 63 + 77 - 50) + 28) - 58 + 92 + 22 + 96 + 46 - 48 + 59 - 88 + 42 - 80 - 84 + 25) + 57 - 26 - 88 + 28) - 86 + 9 - 74 - 40 + 25) - 39 - 37 - 33 - 65 - 60 + 96 - 88 + 81 + 26 + 24 - 9 + 62 - 34 - 69) - 10 - 47 - 31 - 96) + 38 - 19 - 6 - 65 + 58 + 74 + 54) + 79 - 8 - 41 + 32 + 76 - 15 - 92 + 32 - 35 + 6 - 5 - 71 - 34 - 74) + 45) + 19 + 76 - 54 - 91 - 54 + 47 - 92 + 74 + 78 - 69 - 68 - 37 + 66 + 21 + 17 + 48) + 57 + 35 - 37 + 91 - 64 + 73 - 90 - 64 - 84 + 84 - 12 + 91 + 71 - 67) - 4) - 90 + 65 - 19 - 12 + 84 - 26 - 91 + 20 + 25 - 64 - 11 - 0 - 38 - 92 + 80 - 74 - 23 - 11) - 24 - 68 + 86 - 82 - 22 - 18 + 67 - 91 - 48 - 56) - 93 - 65 - 43 - 30 - 29 - 52 + 96 - 21 - 46 + 29 - 36 + 87 + 37 - 45 + 16 + 74) - 75 - 90 - 97 - 49 + 25 - 57 - 64 + 76 - 3 + 98 + 3 + 17 - 0 - 12 + 75 - 9 - 27 + 17 + 55 - 54 + 33 - 16 - 72 - 35 + 8 + 74 - 88 + 42 + 40 + 62 + 98 - 66 + 83 + 40 - 40 + 68 + 66 - 25) - 88 + 35 + 22 - 78 + 17 - 56 + 93 + 31 - 90) + 79 - 82 - 58 + 67 + 14 - 69 + 70 - 78 + 10 - 11 - 42 - 53) - 45 + 35 + 100 - 4) + 18 - 38 - 24 + 5 - 55 - 64 - 56 + 21) + 15 - 9 - 82 + 63 + 76) - 51 + 1 - 5 - 67 + 0 + 71 - 9 + 78 - 85 - 88 + 11 - 71 + 81 - 64 + 73 - 35) + 3 + 78) - 59 - 45 - 3 + 97 + 76 - 84 + 46 + 20 + 15 - 74 - 53 - 27 - 86) - 89 + 30 - 36 + 27 - 100 + 8 + 53 - 26 - 6 + 35 + 13 + 1 + 6 - 89 - 47 + 17 - 29 + 95 + 59 + 79 - 40 + 0 - 10 - 2 + 73 + 74 + 38 - 76 + 60 + 23 + 10 + 56 + 62 + 92 - 13 - 39 - 52 + 87 + 94 + 5 - 44 - 34 + 71 + 83 + 64 - 93 - 81 - 52 - 10 - 41 - 29 - 36 - 86 + 2 + 30 + 25 - 25) - 59 + 45 - 42 - 48 - 99 - 81 + 31 + 62 - 43 + 93 + 40 + 53 - 74 - 15) - 36 - 89) + 14 - 57 - 0 + 36 + 35 + 63) + 6 + 78 - 48 + 57 + 77 + 31 + 74 - 15 + 65 - 10 - 60 - 49 + 73 - 2 + 95 - 5 + 90 + 88 - 84 + 28 + 25 + 100 + 43 + 68 - 89) + 21 + 56) - 80 - 69 - 81 - 21 - 34 - 0 - 44 + 24 - 17 - 21) + 93 + 98 - 6 - 30 + 54 - 70 - 17 + 0 + 99) + 19 - 12 + 32 - 0 - 80 - 95 + 2 - 28 - 24 - 64 - 40 + 73 + 68 - 60) + 27 + 40 + 40 + 82 + 64 - 69 - 48 - 31 - 98 + 58 + 61 + 29 - 97 + 87 - 69 - 26 - 77 - 75 + 44 + 4 - 5) + 49 - 37 - 27 + 18 + 92 + 13 - 1 + 20) - 77 + 32 + 49 - 86 - 44 + 21 - 1 - 76 - 86 - 78 - 52 - 69 - 78 - 2 + 63 + 91 - 50) + 1 + 47 + 73 - 20 + 58 - 32 + 42 - 60 - 64 - 4 + 56 - 82 - 74 + 29 - 40 - 21 - 96 + 84 + 4 + 0 - 50 - 2 - 96 + 8 - 47) - 14 - 33 + 88 + 9 - 34) + 80 + 97 - 3 - 45 - 84 + 30 - 62 - 30 + 37 - 17 + 43 - 65 - 43) - 15) + 42 + 92) + 15 - 31 + 44) - 50 - 42 + 41 - 47 + 21 - 8 - 98) + 47 - 47 - 100 + 11 - 24 - 17 + 44 - 32 + 61 - 22 + 73 - 39 - 8 + 74 - 69 - 3 + 62 - 91 - 34 + 86 - 78 - 19) - 96 - 93 + 52 + 71 + 80 + 4 - 69 - 17) - 69 + 81 + 98 + 73 + 51 - 20 + 58) - 55 + 76 + 74 + 2 + 7 + 35 - 67 - 29 - 74 - 93 - 93 + 8 - 13 + 30 + 52 - 99 + 11 + 74 + 11 + 76 + 100 + 81 - 77 - 98 - 37 + 3) - 13) + 100 + 83 - 69 + 87 + 100 - 14) - 14 - 57 - 5 + 41 - 0 - 49 - 11 - 27 - 0 + 36 - 17 + 39 - 1 - 5) - 99 - 11 - 74 - 21 + 79) + 62) + 36 - 89 - 80 + 11 - 17 + 59 + 87 + 43 - 77 - 52) + 36 + 43 - 34 - 68 + 73 + 49 - 79 - 73 - 0 + 6 + 32 - 89 + 29 + 20 + 86 - 56 + 92 - 57 - 73 + 35 + 77 - 57 + 2) - 2 + 65) - 99 - 16 - 35 - 33 + 1 + 71 - 35 - 17 + 1 + 21 + 54 - 45 + 63 + 28 - 52 + 56 + 22 + 12 - 24 + 21 + 74 - 36 + 67 + 42) - 24 - 32 - 2 - 5 - 88 - 48) - 6 + 32 - 100) - 29 - 56 + 24 - 58) - 83 - 30 + 11 + 83 + 18 - 74 - 63 - 8 + 73 - 80 + 56 + 55 - 65 - 61 + 93 - 32 + 72 + 37 + 25 - 16 + 4 + 38 + 33 + 30 - 25 - 16 - 57 - 86 + 7 - 54 - 77 + 87 + 17) + 27 - 66 - 14 - 15 + 18 + 6 + 97 + 2 - 26 - 89 - 21 - 32 + 67) + 27 + 61 - 96 + 23 + 94 + 27 + 39 - 96 - 1 - 24 - 45 + 11 - 48 + 85 - 62 - 17 + 75 + 62 + 93 + 92 + 88 - 27 - 12) + 75 - 51 - 72 + 17 + 29 + 94) - 83 - 55) + 55 - 86) - 14 - 47 - 4 + 23 - 5 + 94 + 19 + 32 + 34 - 19 - 26 - 77 + 2 + 2) - 5 - 93 - 14 - 5 - 97 + 98 + 87 + 98 + 5 + 81 + 73) + 12) - 80 + 21 - 43 + 44) + 61 + 25 - 46 + 70 + 93 + 8 + 83 - 89 - 37 - 93 - 52 - 91 + 23 + 77 + 66 + 98 + 43 - 95 - 93 + 45 + 18 - 96 - 55 + 31 - 4 - 16 - 73 + 42 - 10 - 68 - 61 + 100 - 7 - 44 - 13 + 9 - 44 + 83 - 7 + 99 + 91 - 25 - 13 - 33 - 39 + 24 + 90 + 22 - 23 + 3 + 45) - 54 - 25 - 33) + 0 + 19 + 24 - 1 - 90 - 8 - 95 - 61 - 14 - 72 + 75 - 35 - 45 - 90 - 56 - 14 + 0 - 87) - 69 + 1 + 0 - 29 - 52 - 7 + 52 + 36 - 77 - 41 - 78 - 76) - 70 - 7 - 82 + 28 + 58 - 14 - 47 + 90 - 77) + 68 - 2 - 30 + 76) - 7 + 27 + 41) + 83 - 58 + 24 + 70 + 47 - 16 + 61 + 40 + 99 + 12 + 5 - 75 + 13 + 79 + 31 - 80 - 43 - 21 + 10 + 12 + 7 + 23 + 60 - 73 + 28 - 0 + 21 + 30 + 17 - 90 + 72 + 63 - 22 - 37 + 74 - 75 - 8 + 78 - 4 - 96) - 98 + 30 - 3 - 31 - 42 - 86 + 2 - 53 + 73) + 79 + 79 + 31 + 75 - 32 - 33 - 51 + 85 - 32 - 91 + 5 - 2 - 81 - 100) + 86) + 34 - 74 + 27 - 33 + 77 + 60 - 82 + 92 + 32 - 34) - 83 + 20 - 78 + 63 + 66 + 21 - 44 - 70 + 97 + 9 - 76 - 23 - 78 - 68 + 93 + 76 - 93 + 64 + 7 + 78 - 24 + 71 - 15 + 38 + 32 - 56 - 56) - 68 - 65 - 69 - 69 + 28 + 72 + 77 - 78 + 44) - 21 - 3 + 10 - 35 - 35 - 70 + 8 - 73 + 6 + 31 + 47 + 39 - 29 + 82) - 63 - 63 + 66 + 37 + 83 - 25) - 1 - 97 - 56 + 100 + 16 + 17 - 3 - 30 - 6 - 78 + 37) - 75 + 7 - 97) + 22 + 44 - 62) - 65 - 90 - 82 - 69 - 18 - 62 + 37 - 62 - 40 - 33 - 73 + 20 + 3) - 38 - 67 - 74 - 26 + 46 + 81 - 97 - 17 - 38 - 0 + 8 - 56 + 53 - 34 - 27 + 76 - 96 + 3 + 79 + 3 - 43 + 9 + 60) + 35 - 31 - 77 - 49 + 24 + 82 + 76 + 66 + 26) + 92 + 74 - 20 + 3 - 19 - 56 - 97) - 4 - 97 + 5) - 74) + 17 - 26 - 14 - 78 - 9) - 86 - 1 - 33 - 21 - 54 - 70 - 21 - 96 + 51 - 18 - 88 + 79 - 41 + 3 + 79 + 17 - 91 + 8 - 0 - 12 + 60 - 65 + 91 + 15 - 98 + 38 - 71 - 0 + 82 - 44 - 58 + 62 + 59 - 9 + 78 + 17 - 42 - 96 - 29 - 33 + 25 - 64 + 29 + 20 - 26) - 79 - 13 - 28 + 87 + 71 + 55 + 53 - 16 + 46 - 19 + 9 - 15) + 98 - 30 + 16 + 41 + 29 + 34 - 31 - 64 - 15 + 100 - 67 + 63 - 22 + 49 - 2 + 6 - 59 + 62 + 77 - 51 + 91 - 81 + 8 - 95 + 24 - 76 + 82) - 33) - 98 + 40 - 10 - 8 + 54) - 21 - 64 + 32 + 15 + 10 + 58 + 93 + 55 - 41 - 26 + 11 + 86 + 60 + 66) - 49 - 25 - 45 - 13 - 1 - 15 - 75 - 38 + 44 - 14 - 60 + 45 - 17 - 59 - 72 + 71 + 37 - 61) + 43 + 29 - 75 + 63 - 22 - 65 + 48 - 96) - 23 + 99 - 30 + 98 + 26 - 24 - 9 - 47 + 56) - 80 + 70 - 40) - 93 - 12 + 1 + 41 + 78 + 38 - 30 + 10 + 96 + 77 - 19) + 14 + 94 - 21 + 51 + 99) - 10 + 33 - 100 + 71 - 4 - 58 + 25 - 66 + 66 - 100 + 31 - 11) + 56 - 75 + 12 - 0 + 43 - 98 + 21) + 54 + 90 - 96 + 88 + 7 + 90 - 88 + 21 - 27 - 23 + 82 - 95 - 12) - 35 + 7 + 45 - 73 - 81 + 84 - 29 + 1 + 70 + 12 + 98 - 48 - 33 + 60) - 10 + 88 - 8 + 63 - 4 - 16 + 64) - 0 - 13 + 54 + 99 - 42 - 42 + 8 - 95 - 27 - 61 - 83 - 12 + 9) + 42 + 81) + 14 - 36 + 14 - 62 - 80 + 80 + 27 + 7 + 58 + 62) + 4) + 73) + 1 + 97 + 50 + 9 - 82 - 98 - 94 + 78 + 23 + 69 + 80 - 27 - 14 - 81 - 78 - 23 + 49 - 31 - 7 + 99 + 44 - 39 + 64 - 19 - 80 - 32 - 16 + 90 - 68) - 20 - 64 + 63 - 75 - 64 - 57 - 86 + 33 - 51 + 22 + 58 - 59 - 36 - 80 + 35 + 81 - 52 - 88 - 63 - 87 - 56 - 67 + 76 + 71 + 47 - 24) + 32 - 15 + 73 - 89 + 41 + 2 + 28 - 81 + 53 - 32 + 63) + 31 - 81) - 72 - 94 + 45 + 76) + 44 + 14 + 89 - 70 + 58 - 88 - 81 + 8 - 39 + 49 - 62 - 19 + 89 - 57 - 7 + 51 - 79 + 47 - 82 + 18 - 12 + 16 + 90) + 5 + 20 - 69 - 51 + 49) + 90 + 39) + 34 - 88 + 30 + 21 - 6 - 37 - 99 - 38 - 14 + 32 + 47 + 87 - 88 + 43 - 92 - 65 + 82 - 46 - 64 + 43 + 84 - 50) - 74 - 6 - 2 - 43 - 83 - 80 - 58 - 88) - 88 - 7 + 93) + 32 + 51 + 11) + 96 - 99 - 12 - 15 + 58 - 29 - 63 + 50 + 34 + 43 - 14 - 45 - 50 + 22 + 78 + 22 - 60 + 0 - 27 - 47 - 52 + 31 + 19 + 74) - 38 + 27 + 25 + 66) - 74 + 97 + 75 - 54) - 94 + 60 + 84 - 41) + 71 + 40 - 55 - 79 + 23 - 97 - 18 + 21 + 14) - 94 + 27 + 13 - 88 + 31 + 90 + 79 - 79 - 85 - 59 + 55 + 59 + 40 - 1) + 6 + 94) + 39 - 29 - 81 - 49 + 84 - 12) - 31) + 72 + 91 - 69 + 44 - 32 - 14 - 44 - 25 - 47 - 52 - 12) - 93 - 2 + 82 + 81 + 16 + 0 + 78 - 75 + 56 - 59 - 52 - 40 - 70 + 29 + 45 + 66 + 35 + 52 - 36 + 35 - 34 - 63 - 88 - 41 + 3 + 60 - 88 - 82 + 8 - 33 + 28 + 20 - 63) - 60 - 71 + 85 + 0 - 40 - 83 + 16 + 71) - 6 - 5 + 8 - 21 - 4 + 56 + 15 - 28 - 16 - 48 + 7 - 6 - 47 + 55 - 32) + 52 + 9 - 62 - 2) + 25) - 5 - 9 + 11 + 46 - 59) + 84) + 94 - 50 + 29 - 63 + 3 + 99 - 85 - 30 - 54 - 9 + 47 - 7 + 21 - 40 + 11 + 11) - 27 + 78 - 40 + 94 - 45 - 14 - 56 - 90 + 31 - 78 - 54 - 8) + 63 - 98 - 94 + 98 + 58 + 41) + 78 - 88 - 17 + 6 - 72) - 84 - 53 - 11 + 72 + 56 + 85 + 46 - 27 + 44 - 11 + 45 - 1 + 78) - 19 + 48 - 2 + 48 + 3 - 14) - 73) + 87 + 85 + 84 - 37) - 77 + 71 + 35 + 52) - 74 - 50 - 22 + 54 + 64 + 69 - 1 + 96 - 68 + 56 + 25 + 30 - 36 + 86 + 59 + 1 - 21 + 8 - 70 - 47) + 91 - 10 - 46 - 89 + 100 - 79 - 16 - 79 - 65 + 26) + 77 - 88 - 28) + 100 - 93 - 98 + 16 + 76 - 84 - 43 + 17 + 68) - 7 - 88 + 18 - 64 + 92 - 55 - 82 + 63 - 52 + 38 - 11 + 51 + 62 - 87 - 71 + 82 - 98 - 36 - 77 + 3 + 89 + 30) + 20 - 15 - 48 + 37 + 33 - 20 + 62) - 63) - 55 + 56 + 75) - 72 + 23 - 27 + 42) + 41 - 14 + 43 + 87) - 93) - 60 + 77 - 26 - 37 + 76 - 15 - 92 + 60 - 69 + 78 - 41 + 50 + 3 - 93 - 83 + 32 + 44 + 19 - 4 - 35 - 36 + 86 + 37) + 0 + 20 + 62 - 69 - 53 - 46 - 1 + 31 + 6 + 48 - 81) + 37 - 26 + 9 + 39) - 52 + 27) + 90) - 23 + 46 + 60 + 1 - 77 - 75) - 28 - 36) + 96 + 62) - 52 - 62) + 38 - 41 + 90) - 79 - 55 + 77 + 71 + 57 - 37 + 52) + 50 - 41 + 63 - 95 + 76 - 55 - 23 + 86 - 55 - 80 + 73 - 44 + 44) - 50 + 46) - 56 + 97) - 40) + 48 - 28 - 20 + 98) + 24 + 73 + 24 + 76 + 14 + 33 - 75 - 77) + 80 + 99 + 85) - 4 + 4 + 72 + 22) - 59 - 83 - 38 + 85 - 3 + 28 + 62 + 5 + 9 + 11 - 82 - 15 + 26 + 44 + 1 - 31 - 22 - 4 + 84 - 60 - 80 + 69 + 86 + 1 - 39 - 64 - 4 + 50 - 8 + 64 - 2 - 87 - 41 + 16 - 3 + 7 - 64 - 71 + 40) - 41 - 75 + 14 - 67 + 56 + 66 - 44 + 14 - 5 - 13 + 20 + 18 + 28 - 48 + 33 + 78 + 71 + 83 + 20 - 64 + 59 - 48 - 24 - 19) + 45 - 20 - 8 + 96 + 41 + 45 + 96 - 80 + 80 - 50 - 49) - 87 + 54 - 7) + 73 + 51 + 29 + 41 + 49 - 88 + 35 - 92 + 39 - 18 + 8 - 36 + 83 + 69 - 71 + 47 + 19 - 94) + 72 - 55 + 15 + 22 - 50 + 95 + 49 + 22 - 90 - 22 - 12 + 88 + 96 + 45 - 62) + 42 + 38 - 46 + 28 - 22 + 1) - 54 - 30 + 30 + 99 + 56 + 6 - 84 - 50 - 52 + 2 + 86 + 10) + 84 - 61 + 43 + 83 - 55 - 66) + 87 + 55 - 73 + 42 - 90 + 41 + 75) - 4 + 73 + 5 + 24 - 38 + 18 + 94 + 2 + 10) + 82 - 1 - 11 + 58 - 63) - 92 - 59 + 33 - 81 + 37 - 33) - 63 + 50 + 95 - 6 + 95 + 27) - 29) + 5 + 99 + 11 - 15 - 39 - 14 + 48 + 96 - 1 - 94 - 15 + 58 + 8 + 12 - 21) + 41 - 17 - 65 - 64 - 30 + 53 + 90 - 78 - 37 - 67 + 38) - 37 + 48 - 20 - 98 + 24 + 85 + 21) - 58 + 52 - 67 + 6 + 1 + 77 - 48 + 13 + 73 - 40 - 66 - 100 - 4 - 100 - 20 + 45 + 32 - 83 + 87 + 69 + 70 + 9 - 100 + 39 + 39 + 36 - 4 + 90 - 48 - 73 + 99 - 60 - 46 - 55 - 34 + 68 - 49 + 18 - 82 - 44 - 7) + 18 + 66 + 58 + 83) - 38 + 54 + 77 - 42) - 13 + 53 + 88) - 40 - 61 - 44 - 65 + 57 - 65 + 4) - 92 - 3 + 16 + 97 - 24) + 79 - 21 + 27 - 9 - 57 + 81 + 42 - 17 - 54 - 15) - 68 + 26) - 69 - 34 - 75 - 83 - 3) - 90 + 82 - 81 + 2 + 73 + 32 - 57 - 86 - 98 + 0) - 48 + 44 + 71 - 15 - 7 - 19 + 8 + 94 + 26 + 91 + 44 - 84 + 40) - 14 - 97 - 20) + 37 + 85 - 26 - 3 + 75 + 22 - 61 - 29 + 13 + 13 - 6 - 86 - 31 - 31 + 72 + 96) + 19 + 24 - 0 + 61 - 94 - 60 - 81 - 67 - 66 + 29 - 82 + 30 - 68) - 92 + 65 - 82 + 54 - 37 + 77 - 43 + 12 + 84 + 32) - 2 + 39 + 35 + 56 - 66 + 84 - 42 + 4 - 87 + 48) + 13 - 64 + 34 - 93 - 70 + 34 - 84 + 23 + 75 + 66 + 82 - 56 + 52 - 4) + 61 - 16) - 7 + 74 + 21 + 95 + 26 - 35 - 0 + 59) + 81 - 41 - 75 + 66 + 66) + 72 - 79 + 57 - 39 - 88 - 31 - 85 + 91 - 29 + 17 - 69 - 60 + 52 + 18 + 58 - 56 + 68 + 82 + 48) + 60 - 85 + 20 + 27 + 18 + 47) + 88 - 35 + 49 - 61 - 6 - 58 + 30 + 70 + 26 - 30 - 10 - 52 + 22 + 32 + 58 - 58) - 50 + 20 - 7) + 11 - 55 + 24 - 67 - 46 - 30) + 49 - 7 + 47 + 15 + 99 + 63 + 33 + 22 - 24 + 96) + 80 - 45 + 65 - 92 - 33 + 26) - 48 + 68 - 47 - 79 - 74) + 87 + 77 - 33 + 56 - 74 - 22 - 71 + 41 + 95) - 20 + 22 - 93) - 80 - 54 + 0 + 29 - 72 + 8 + 47 - 22 - 7 - 55 + 77) + 34 - 14) - 7 + 81) + 29 - 1) - 12 - 78) + 58 + 97 + 56 + 10 + 77 + 10 + 29 - 50 - 92 + 32 - 31 + 8) + 79 - 62 - 58 - 97 - 1 - 70 + 81 - 95 + 55 - 76 - 79 + 71 + 65 + 16 - 5 - 84 - 73 - 5 + 57 + 43 + 21 - 22 + 76 - 5 - 57 + 36) - 38) - 85 + 92 - 40 - 80 - 8 - 75 + 59 - 79 - 48 + 88 - 48 + 97) + 91 + 94 - 93 + 63 + 82) + 2) - 12 + 64 - 21 - 74 - 17 + 57) - 50 + 13) - 79) + 55 - 80 + 79 + 59 - 61 - 24 - 30) + 91 - 69 + 12 - 25 + 32 - 65 + 26 + 75 - 73 - 52 + 7 + 59 - 48 - 95 + 77 + 49 + 51 - 69 + 23 - 32 + 21) + 54 + 81) + 8 + 64 - 52) + 78 + 23 + 60 - 28) + 94 + 7 - 91 - 15 - 6 - 60 - 83 + 34 - 25 + 46 - 89 - 14) + 100 + 72 - 97 - 40 - 5 + 39 + 87 - 53 + 57 - 100 + 70 + 87 - 8 - 67 + 6 + 50 + 81 - 43 - 77 + 70 + 91 + 30 + 62) - 57 + 34 - 65 - 46 - 96 + 89 + 41 + 50 - 7 + 70 + 30 + 45 + 62 + 56 - 43) + 16 - 40 + 6 + 31 + 0 + 57 - 13 - 31 - 74 - 46 - 36 - 74 + 74 - 84 - 93) + 7 + 75 + 25 - 95 + 86 - 36) - 35 - 57 - 97 - 0) - 77 - 64 + 80 - 8 - 17 - 96 - 91 - 87 - 1 + 91 - 86 - 33 + 24 - 85 - 57 - 58) + 10 + 35 - 45 - 97 - 23) - 83 - 11 - 73 - 26 + 28 - 51 + 36 + 95 - 28 + 49 - 74 - 98 - 36 + 33 - 99 + 62 + 24) - 94 - 72 - 24 - 89 - 54 + 0 - 90 + 82 + 0 + 86 + 74 - 97 - 73 + 32 - 56 - 75 - 10 - 21 - 39 + 20 - 78 - 38 - 7 - 93 + 12 + 40 + 9 - 72) - 58 + 75) + 33 - 51) + 90 - 15) + 19 - 69 + 39) - 96 + 40 - 62 + 80 - 63 - 19 - 51 + 23 + 6 - 1 + 9 - 54 + 16 + 22 + 97 + 66 + 10 - 85 - 63 + 21 - 11 + 82 + 50) + 63 + 3 + 8 + 7 + 55 - 4 + 3 + 52 + 93 - 75 - 67 + 75 + 93 - 80 + 54 + 48) + 60 - 67 - 92 - 2 + 54 - 89 - 30 + 61 + 76 - 35) + 54 + 33 - 67 - 52 + 15 + 88 + 42 - 20 - 34 + 23 + 63 + 8 + 13 - 25 - 100 + 90 + 35 + 42 - 0) + 96) + 15 + 49 + 81) + 20) + 81 - 3 + 95 - 16 - 24 + 77 + 11 + 20) + 37 + 57 + 87 + 18) + 16 + 16) + 12 - 5 - 94) + 0 + 94) + 33 + 79 + 48 - 3 + 85 + 12 - 40 - 49 + 88 + 61 - 7 - 68 - 7 - 21 - 63 - 72 + 93 + 83 + 75 - 51 - 84 - 27 - 95 - 83 + 34 - 77 + 27 + 98 - 0 - 55 - 44 + 39 + 100 + 89 - 12 + 3 + 0 + 28 - 96 + 60 + 60) - 43 + 30 + 81) + 40 - 11 + 63 + 16 + 18 + 94) - 75 + 8 + 58 + 88 - 0 - 11 - 35 - 77 - 0 + 71 - 63) + 29 + 51 - 28 - 42 + 73 + 70 - 39 - 57 + 98 - 3 + 75 - 33 - 44 - 74 - 15 + 96 - 10 - 10 - 76 - 28 - 50 + 88 + 78 + 44 + 55 - 29 - 71 + 32 + 95 + 64 + 61 - 30 + 75 + 3 - 14 + 90 - 91 - 22 + 13) + 53 + 36 - 2 - 28 - 49 - 54 + 23 - 47 + 85 + 91) + 79 - 83 + 56 + 52 + 28 - 89 + 32 + 92 + 39 - 92 - 46 + 31 - 91 + 65 - 96 + 29 - 5 - 9 + 28 - 6 - 7 + 48 + 69 + 53) + 74 - 100 + 87 + 70 + 77 - 92 - 96 + 13 - 83 + 88 - 30 + 49 - 79) + 2 + 77 + 32 - 44 + 63 - 29 - 84 - 97 - 15 + 41 + 6 - 22 - 65) - 30 + 16 - 86 + 20 + 1) - 79 + 93 - 85 + 22 - 62 - 2 + 94 - 19 + 38 + 34) + 50 + 9 - 48 + 58 + 29 + 55 - 100 + 9 - 24 - 65 + 28 + 39 + 1 - 10 - 42 - 0 + 32 + 27 - 78 - 49 + 0 - 97 + 34 + 36 - 13 + 83) - 0 - 66 + 93 - 64 - 21 + 73 - 8 + 46 + 20 - 59 - 61 - 32 - 32 + 76 + 37 + 42 - 24 - 8 + 86 + 48 + 29 - 95 + 98 + 63 + 2 - 29 - 3 + 89) - 72) + 43 - 28 - 96 + 74 + 33 - 35 + 91) + 23 - 47 + 67 - 86 - 7 - 38) + 16 + 15 - 56 + 13 + 62 + 63) - 97 + 66 - 37 - 93 - 54 + 9 + 91 + 15 + 0 - 7 - 37 + 69 - 21 + 42 - 71 + 18 - 73 + 28 - 21 - 0) + 93 + 65) - 27 + 27 + 88) - 80 + 77 + 27 + 91 - 64 + 35 - 17 - 13 - 68 - 96 - 25 - 90 - 27 + 86 + 63 - 95) + 82 - 95 + 50 - 2 + 67 + 58 + 84 + 4 - 56 + 82 - 16 + 2 + 5) - 62 - 39 + 87 + 26 - 95 - 86 + 69 - 42 - 80 - 37 - 76 + 77 + 68 + 81) + 17 + 48 - 35) + 99 - 67 - 50 + 58 - 70 + 25 - 16 - 24 + 34 - 83 - 5 - 79 - 33 + 6 - 70 - 76 + 96 - 9 + 27 - 83 - 15) - 70 - 72 - 50 + 0 + 28 - 14 - 89 + 95 + 71 - 21 - 39 - 99 - 72 + 83) + 94 + 33 + 5 + 3) + 54 + 74 + 79 - 0 + 30) - 33 - 73 + 29 - 34) - 52 - 44 - 60 - 62 - 49 - 31 - 10 - 93 - 72 - 54 - 82 + 85 - 100 - 66 - 60) - 90 - 68 + 92 - 15 - 72) - 9) - 99 - 21 + 9 + 58 - 3 - 72 - 40 - 26 + 89 - 54) - 73 + 21 + 68 - 59 - 49 + 89 - 66 - 34 + 23 - 97) - 18 + 95 + 6) - 77 + 16 - 25 - 52 - 20 + 18 - 72) + 30) + 63 - 59) + 53 + 54 - 62 - 54 - 13) + 60 - 43 + 54 - 27) + 69 + 85) - 34 + 17 - 78 + 85 - 4 + 94 - 88) + 38 + 72) + 53 + 25 - 20 + 54 - 29 - 81 - 61 + 53 + 5 - 6 + 59 - 56 - 39 + 91 + 24 - 50 - 68 + 5 + 68 + 55 + 75 + 58 - 46 + 98 + 84) - 73 - 40 - 50 + 80 + 70 - 34 + 67 - 91 - 95 + 97 + 32 - 95 + 88 + 49 + 36 - 19 - 13 + 7 - 46 - 93) - 43 - 34) - 75 + 51 - 37 + 62 - 67 - 97 + 88 + 90 + 14 - 47 - 35 - 31 + 69 - 9 - 81 - 87 + 16 - 65 - 0 - 77 + 54 - 16 + 2 - 16 + 48) + 56 + 1 + 5 + 30) - 97 - 77 + 91) - 29 - 14 + 88 - 25 - 70 + 55 - 73 - 55 - 1 + 15 + 37 + 0 - 74 - 48 - 78 + 0 + 49 + 58 - 12 + 51 - 31) + 92 - 13) + 84 + 74) + 33 + 69 - 96 + 45) - 38 - 46 - 0 + 55) - 32 + 31 + 36) - 64 + 70 - 38) + 59 - 18 + 5 + 9 - 12 - 55 - 33 - 36 + 71 - 27 + 29 - 65 + 78 + 32) + 65 + 84) + 37 + 90 - 29) - 36 + 0 - 74 + 58 + 90 - 43 - 67 - 64 - 95 - 39 - 1 + 45 - 97 + 82 - 100 + 76 + 9 - 53 - 21) - 16 + 5 + 54 + 97 - 61 - 54 - 72 - 39 - 33 + 4 - 32 - 85 - 43 + 5 - 94 + 21 - 66 - 93 + 52) - 71 + 18 + 42 - 93 + 91 + 76 + 83 - 21 + 77 + 92) + 64 + 90) + 82 + 100 + 61 - 19 + 36 + 29 - 80 + 69 + 18 + 48 + 15) - 99 - 72 - 76 + 86 - 54 + 13 - 37) + 69 - 37 - 67 - 79 + 9 + 36 + 17 - 52 + 3 + 37 - 72 - 0 + 65 + 48 - 88 - 68 - 59 - 54 + 43 + 94 + 18 + 65 + 22 + 9 + 35 - 70 + 75 + 87 - 72 + 17 + 12 + 71 + 69 - 20 - 73 + 73 - 20 + 21 - 46 - 17 - 39 - 13 + 41 + 11 + 100 + 75 - 94 + 9 - 43 + 0 - 88 - 28 - 55) - 71 + 17 + 39 + 54 - 7 - 92 + 62 + 37) + 6 - 47 - 12 - 4 - 78 - 11 + 57 - 15 + 83 - 59 + 8 + 51 + 7 + 33 + 42 + 32 - 34 - 90 + 34 - 77 - 33 + 73 + 60 - 42 - 13 - 51 - 76 - 24 - 3 - 28 - 78 - 0 + 33 + 4 - 36 - 65 + 64 - 51 - 88 - 35 - 92 + 7 - 30 + 92) - 28 - 80 + 41 - 32 - 70 - 54 + 73 - 81 + 63 + 19 - 44 + 58 - 33 + 44 - 18 + 74 - 59 - 25 - 17 - 51 - 3 + 63 + 62 - 9 - 49 + 12 - 44 + 44 - 17 - 50 + 12) + 18 + 87 - 90 + 8 - 71 + 12 - 74 - 49 - 70 + 21 + 100 - 17 - 90 - 97 - 45 + 12 + 32 + 37 + 59) - 55 + 83) + 86) + 16 + 45 - 3 - 86 + 51 + 1 + 52 + 21 + 21 + 20 + 69) + 12 - 99 + 55 + 59 - 24 + 7 + 34) + 68 + 59 + 98 + 17 + 68 + 64 + 46 - 73 - 55 + 7 - 10 - 45 + 39 - 0) + 25 - 85) + 68 + 54 + 86 - 64 + 37 - 59 + 35 + 77 - 90 + 96 - 68 + 54 - 67 + 13 + 33) - 48 + 9 - 47 - 70 - 79 + 48 - 30 + 82 - 41 + 34 - 57 - 17 - 84 - 64 + 99 + 36 - 22 + 35 + 37 - 54 + 87 - 53 + 27 + 56 + 62 + 59) - 85 - 39) + 84 + 43 - 29 - 24 + 72 - 100 - 95 + 99 + 49 + 77 + 49 + 65 - 63 + 27 - 65 + 34 - 38 + 4 + 75 - 86 - 96 + 60 - 79 - 80 - 83 - 39) - 83 - 24 - 65 - 51) - 66 - 6 + 1) - 4 + 95 - 91 + 81) + 60 + 76 + 16 + 71 + 72 + 72 - 99 - 9 - 41 - 66 + 98 + 46 + 29 - 13 + 73 - 84 - 63 - 90 + 41 + 66 - 42 - 83 + 36 + 3 + 100 - 85 - 50 - 2 - 32 - 55 + 95 + 14) + 61 - 64 + 64 + 87 + 90 - 17 - 42 - 28 - 77 - 26 - 50 + 24 - 46 - 90 + 18 + 17 - 23 - 65 - 66 - 42 - 65 + 57 - 68 + 18 - 65 - 94 - 88 - 90) + 60 - 44 - 85 + 22) - 27 + 14 + 77 - 9 + 69 + 87 + 51 + 42 - 53 + 25 - 65 - 91 - 46 - 94 + 98 - 9 - 99 - 83 + 98 + 52 + 72 + 65 + 41 - 2 - 67 - 27 + 59 + 2 - 81 + 94) + 44 + 39 + 93 - 73 + 94 - 50 + 55) + 89 - 48 - 66 - 67 + 63 + 4 + 48 + 90 + 85) - 70 - 54 - 3) + 4 - 97 - 34 - 72 + 52 - 48 - 37 - 5) + 96 - 99 + 31 + 65 + 69) + 8 + 59) + 34 + 49 + 8 + 57 - 98 + 97 - 38 - 76 + 14 + 43 - 98 + 26 + 98 + 0) - 89 - 50 + 94 + 24 - 13 - 79 - 46 + 70 + 65 - 14 + 85 - 84 - 62 + 54 - 57 + 5 + 58 - 52 - 41 - 54 - 45 - 96 + 15 + 6 - 2 - 26) + 97 + 84 + 56 - 66 - 69 - 73 - 37 + 22 - 86 + 25 + 18 - 66 - 45 - 17 + 50 + 53 - 68) + 93 - 84 + 67 + 44) - 25 - 84 - 85 - 81 + 94 - 14 + 89 + 43 - 48 + 50 + 1 - 9 - 67 - 4 - 0 + 55 - 68 - 4 - 100 - 44 + 14 + 21 - 57 + 42 + 59 - 58 + 76 + 44 - 2 - 29 - 36 - 6 - 75 - 10 - 93 - 70 - 39) + 1 + 76 - 46 - 56 - 38 + 27 - 48 + 96 - 67 + 53 - 92 + 0 - 84 - 76 + 9 + 55 + 96 - 27) + 82 - 75) + 26 - 99 + 73 - 15 + 27 - 48 - 42 + 76 - 26 - 90 - 28 - 7 + 97 - 94 - 72 + 72 + 37) + 60 + 25 + 16 - 35 - 77 - 25 + 66 - 18 + 81 + 45 + 35 + 10 - 59 - 71 - 50 - 53 + 100 + 18 + 87 + 81 - 78 + 89 + 72 + 76 - 34) + 28 + 1 - 89 - 78 + 88 - 53 - 33 + 25 - 51) - 39 + 11 + 98 - 57 + 7 + 95 - 52 - 12 - 10 - 37 - 46 + 95 + 93 - 25 + 10 + 49 - 98 + 88 - 45 - 75 - 83 - 63 + 45) + 44) + 46 - 24 - 44 + 98 - 64 - 64 + 87 + 72 - 53 - 5 + 97 + 40 + 64 + 56 - 45 - 89 + 79 - 49 - 76 + 11 - 38 - 58 - 34 + 89 - 17 + 76 + 81 - 47 - 64 + 21 + 37 + 31 - 73) - 74 - 54 + 54 + 2) - 20 + 65 + 32 - 85 - 41 - 19 + 25 + 86 + 90) + 71 + 18 - 79 + 43 + 94 + 71 + 42 - 83) + 55 - 80 - 66) - 17 + 92 + 0 + 64 - 30 - 6 - 76 - 15 + 53 - 28 + 35 - 23 + 2 - 7 - 76 + 1 - 7 + 20 - 23) + 1 - 66 + 69 + 69 - 91 - 98 - 47 + 30 + 90 - 97 - 77) - 21 + 75) - 38 - 19) - 76 - 20 + 24 - 76 - 37 - 51 + 75 - 45 - 35 + 37 + 6 + 28 + 100 + 54 - 36 - 75 + 90 + 29 - 68 - 50 + 16 - 36 - 8 + 40 - 75 - 11) - 26 + 57 + 97) - 95) - 15 + 26 - 62 + 75 + 13 + 35 - 21 + 49 - 38 - 95 + 99 - 64 + 42 - 95) + 39 + 60 - 69 - 36 - 96 - 14 - 50 + 59 + 85 - 69 - 100 - 91 - 75 + 87 + 5 - 4 - 37 + 99 - 17) + 20 - 32 + 63) - 66 + 99 - 91 + 44 - 2 - 22 + 84 + 32 - 54 + 26 - 51 - 33 - 59) + 64 + 67 + 2 - 27 - 67 + 87 + 93) + 38 + 40 - 73 + 9 - 67 + 26 - 38 - 58 - 23 + 85 - 95 - 50 + 64 - 32 - 2 + 45 + 33 + 0 + 44 + 11 - 58 + 9 - 81 - 35 - 17 - 29) + 51 - 98 + 88) - 46 - 6 - 18) - 20 + 84 - 50 + 78 + 87 + 26 + 60) + 99 + 33 + 48 - 4 + 59 + 10 - 7 + 99 + 22 + 16 - 32 + 89 - 94 + 61 + 21 + 76 + 65 + 82 + 36 + 62 + 68 - 71 + 10 + 6 - 64 - 52) + 62 + 21 - 90 - 34 + 59 - 22) - 45) + 42 - 40 + 73 + 15 - 27 + 60 + 60 - 59 - 81 - 26 - 19 - 99 - 98 - 80 + 23 - 57 + 27 + 60) + 38 - 79 - 8 + 80 - 13 + 52 + 56 + 43 + 85 - 1 - 94 - 58) - 73) - 56 - 76 + 2 - 60 - 31 - 18 + 96) + 91 + 94 + 29) + 43 - 85 - 45 - 85 + 2 + 80 + 21 - 85) - 89 - 99 + 65 - 8 - 68 - 41) - 68 + 15 + 60 - 47 - 20 - 7 - 41) - 60 + 21 + 1 - 50) + 70 + 5 - 14) + 12 + 45 - 29 - 67 + 40 - 72 - 5 + 93 + 88 + 34 - 31 - 63 + 2 + 69 + 87 + 75 - 11 - 76 + 74 - 47) + 3 - 33) + 3 + 4 + 84 - 95 - 55 + 14 + 39 + 50 + 15 - 8) - 34 + 88) - 3 + 69 - 96 - 68 - 81 - 60 - 20) - 54 - 90) - 55 + 42 - 49 - 85 + 25 - 65 + 11 - 77) - 45 + 18 + 3 - 71 - 63 - 49 + 1 - 5 - 55) + 22 + 100 - 0 + 82) - 56 + 33) + 98 - 15 + 95 - 46 + 97 + 40 - 41 - 1 - 73 + 39 + 22 - 71 + 93 + 76 - 96 + 69 + 75 + 77 + 79 + 4 + 15 - 27 - 95 + 100 + 63 + 35 + 46 + 0 + 24 + 19) + 67) - 7 + 61 - 2 - 77 + 83 + 14 + 28 + 77 + 70 + 9) - 1 - 8 - 44 + 59 + 12 - 2 - 30 + 72 - 89 + 65 - 87) + 47 - 38 + 40 + 77 - 98 - 40) - 54 - 5) - 72 + 52 - 33 + 79 + 79 - 23 + 52 - 63 + 1 - 50) - 53 - 96 + 76 - 36 - 47 - 87 + 79) - 71 + 43 - 2 + 54 - 29 - 46 - 0) + 69 + 57 + 2) + 92 + 48 - 85 + 28 + 23) - 63 + 33 + 26) + 58) - 65 + 89 + 92 - 25) - 98 + 9 - 35 + 5 - 17 - 79 + 28 - 90 - 39) + 43 - 13 + 96 + 36) - 12 + 66 + 26 - 91 - 20 - 41 - 76 + 19 - 54 + 24 + 2 - 98 - 49 - 95 - 94 - 46 - 29 - 45 + 99 + 2 + 63 + 18 - 100 + 9 + 94 + 42 + 56 + 69 + 83 - 66 + 31) + 31 - 32 - 36 - 15 - 95 + 56) - 30 - 32) - 40 + 20 + 26 - 33 - 0 + 52 + 54 - 57) + 42 - 4 - 58 - 67 - 35 + 92 + 90 - 5 + 28 + 36 + 39 - 98 + 86 - 24 + 14 - 21 - 50 + 92 - 67 - 72 + 54 + 10 - 65 - 59) - 61 - 23 - 78 + 15 - 12 + 38 - 81 - 26 + 8 - 86 + 42) - 46) + 29 + 34 + 16) + 78 - 83 - 34 + 94 - 78 - 17 - 0) - 9 + 34 - 3 - 16 - 9 - 28 + 78 - 69 + 50) - 44 - 22 - 99 + 96 + 36 - 51) - 17 - 54) + 64 + 73 + 31 + 78) - 25 - 11 - 24 + 38 + 34 + 41 - 97 + 17 + 8 - 21 - 67 + 15 - 65 + 89) - 32 + 23 - 0 + 29 + 83 - 88 - 70 + 28 - 38 + 35 + 90 - 96 + 18 + 86 + 96 + 46 + 45 - 0) + 10 - 15 + 90 - 23 + 30 + 97 + 54 + 9 + 37 + 85 - 13 - 39 + 87 + 51 + 66 + 54 + 84 - 19) - 31 + 65 + 27) - 53 + 3 - 43 + 29 + 25) + 79 + 36 - 87 + 1) + 14 + 41 + 48) + 24 - 9) - 91 - 7 + 82 - 69 - 62 + 43) - 4) - 93 + 72 + 78 + 57 - 74 - 81 + 9 - 24 - 80 + 80 - 3 + 79 - 68 - 38 + 82 + 57 - 98 + 29 + 21 + 96 - 78 - 8 + 79 + 20 - 83 + 91 - 5 + 14 - 49 + 97 - 48 + 47 + 84 - 47) - 2 + 99 + 0 - 3 + 20 + 54) - 36 - 99 + 56 + 70 + 79 + 9 + 31) + 100 + 10 + 47 - 37 - 50 + 94 + 18 - 1 - 28 - 27 + 70 - 89 - 26) - 94 + 1 - 31 - 63 - 18 + 24 - 97) - 17 - 73 + 49 - 64 + 57 - 51 + 0 + 15 - 89 - 9 + 48 - 60 + 77 + 25 + 93 + 15) + 29 + 69 + 28 - 26 - 8 + 75 - 25 - 14 - 11 - 86 + 25 - 40) - 90 - 80 - 22 + 79 - 69 + 11 + 76 + 25 + 78 + 65 + 64 + 27 - 62 + 20 - 68 + 11 - 85 - 49 + 97 - 73 - 10 - 19 - 15 - 63 + 39 - 89) - 18 - 41 - 52 + 31) - 18 + 84) + 22 - 58) - 88 + 45 + 57) - 71 + 5) + 60 + 88 + 78 - 97 - 98 + 68 + 29) + 75 + 47 - 56 + 17 + 28 + 70 + 81 - 99 + 64 + 1 - 58 - 87 + 96 - 36 - 1 + 99 + 25 + 74 + 37 + 23 + 80 - 57 - 72 + 34 - 31 - 85 + 52 + 7 + 91 + 7 - 80 + 29 + 98 - 98 + 54 + 51 + 67 - 74 - 14 - 84 + 18 - 18) + 91 + 30 - 63 - 74 - 57 - 16 + 60 + 83 + 62) - 75 + 93 + 30 - 57 + 0 + 88 - 31 + 69 - 100) + 96 + 37 - 27 + 37 - 50 + 8 + 31 + 58 - 30 - 44 - 59 - 87 + 59 + 53 + 14 + 79) + 63 - 23 - 79 - 86 + 55 - 75 - 10 + 80 - 56 + 89 - 39 - 20 + 7 - 49 + 47 + 98 - 1 - 43 + 49 + 21 + 18 + 33 + 51 - 96 - 78 + 91) + 26 - 66 - 6 + 71 + 21 - 61 - 36 - 18) + 48 + 53 - 87 - 26 - 94 - 47 - 42 - 9 - 92 - 64 + 88 - 37 - 34 - 97 - 13 + 35 + 96 + 67 + 40 + 26 + 89 + 14 + 65 + 52 + 71 - 92 - 27 + 8 - 90 - 68 - 15 - 21) - 16) - 73 - 23 - 45 - 30) - 82 - 45 + 45 + 0) + 85 + 40 - 0 - 95 + 17 - 18 - 40 + 32 - 19 - 24 + 35 - 25 - 3 + 44 + 37 + 39 - 83 + 50 + 57) - 0 + 33 - 13 + 4 + 98 + 40 - 28 + 97 + 68) - 71 + 96 - 8 + 3 + 3 + 29 - 24 - 67) + 24 - 2) + 99) + 3) - 75 + 75 + 44 + 97 - 53 - 17 - 82 + 4 - 43 + 96 - 33) + 70) + 51 - 95 + 54 - 2 + 51 - 54 - 58 - 96 - 50) + 56 + 0 + 50 + 90 - 53 + 57) - 8 - 9 - 41 - 41 - 55 - 34 + 99 - 93) + 86 + 53 - 54 + 69 - 72 + 99 + 74 + 79 - 19) + 42 - 57 + 90 - 84 - 93 + 95 + 53 + 17 + 33 - 4 + 29 - 20 - 78 + 53 - 25 - 62 + 27 + 76 + 23 + 97 - 37) + 88 + 15 + 71) - 56 + 99 - 72 - 16) - 94 - 83 - 81 + 15 + 70 + 91 - 84 - 9 - 19 + 26 + 62 - 35 + 93 + 54 - 61 - 47 + 56 - 55) + 96 - 77 + 12 + 4 - 81 + 17 + 11 - 19 - 39 - 17 - 67 + 81 + 67 + 24 + 41 - 45 + 47 + 48 + 68 - 65 - 53 - 61 - 41 + 74 - 77) + 33) + 75 + 6 + 48 - 25 - 11 - 82 + 92 - 15 - 29 - 35 + 75 - 1 + 76 - 50 - 37 - 67 + 94 + 79 + 1 + 4 + 13 - 84 + 79 - 83 - 32 + 7 + 13 - 36 + 92 - 72 + 49 + 52 - 86 - 13 + 38 - 65 - 41) - 47 - 84 + 67 + 93 - 41 + 81 + 65 - 62 - 59 - 97 + 89 + 59 - 21 + 83 - 78 - 20 + 86 + 97 - 41 + 59 - 68) - 92 - 44) + 64 - 17 + 55 + 83 - 91 + 16 + 28) + 99 - 85 - 93 - 84 - 0 + 8 - 35 + 68 + 15 + 98 - 92 - 25 - 84) - 85 - 29 + 45 + 11 - 44 + 0 - 35 + 73 + 45) - 63 + 99 - 26 - 2 + 32 - 56 - 70 + 70 - 49 - 48 - 29 + 47 - 21 - 100) + 26 + 74 - 43 - 12 + 25 - 38 - 22 - 73 + 55 - 72 + 0 - 55 + 82 + 69) - 26) + 46 - 31 + 78 - 72 + 30 - 2 + 54 - 69 + 56 + 21) + 79 - 49 - 33 - 20 + 59 + 66 - 10 + 1 + 49 - 75 + 74 - 83 - 5 - 28 - 80 + 55 - 48 + 12 + 9) - 88) + 2 + 40 + 75) - 26 + 30 + 73) + 95 - 90 - 40 + 57 + 71 + 34 + 67 + 18 - 32 - 54 + 69 - 24 + 6 - 40 - 71 + 98 - 18 + 48 + 77 + 32 + 72 - 44 - 55 + 41 + 33 + 22 - 14 + 85 + 21 + 38 + 67 - 81 - 14 + 73 - 7 + 79 - 14 - 34 + 34 + 28 - 54 - 98 - 29 - 91 - 90 - 35 + 69 - 89 - 97 - 94 - 9) + 10 + 56 + 86 - 27 + 31 - 3 - 63 + 12 - 91 + 55 + 35 - 63 + 80 + 7 + 86 + 3 - 13 + 63 - 27 + 88 - 54 - 0) - 61 + 86 - 23 + 9 + 75 + 7 + 25 - 96 - 96) + 64 - 75 - 95 - 1 - 68 + 60 - 69 - 17) + 94 - 44 - 72 + 93 - 51 + 64 - 98 - 75 - 65 - 84 - 99) - 58 - 34 + 16 + 76 - 63 - 15 - 90 - 36 - 60 - 27 - 58 + 94 - 25 + 33 - 94 - 21 + 61 - 28 + 64 - 40 - 96 - 5 - 76 - 3 + 2 - 23 + 64 - 30 - 100 + 33 - 0 + 68 - 93 + 28 + 0 - 66 - 93 - 84 - 64 + 49 - 78 + 100 + 65 + 12) - 49 + 87 + 76 + 1 + 8 - 22 + 61 + 95 - 33) - 96 + 85 - 85 + 35 - 95 + 38) + 35 - 2 + 64 - 76 - 35 + 8 - 78 + 77 + 69 + 18 + 70 + 87) - 46 - 54 + 21 - 87 + 23 + 45 - 69 - 36 + 81 + 53 + 21 - 86 - 75 + 95 + 82 - 70) + 90 + 27 - 39 - 42 + 42 + 33 + 21 + 39 + 73 - 4 - 55 - 72 + 51 + 45 - 89 + 70 + 56 - 49 + 58 + 65 + 16 - 53 + 34 - 74 + 65 + 54 - 21 + 84) - 89 + 35 + 84 + 43 - 59 + 55 - 48 + 64 - 94 - 26) + 92 - 44 + 49 + 89 + 20 - 11 - 38 + 75 - 8 - 24 - 24 - 21 - 93 - 15 - 69 - 18 + 9 + 80 - 100 + 34 - 25 - 15 + 93 + 29 - 59 - 72 - 68 - 21 + 21 - 25 + 21 - 57) - 50 + 5 - 53 - 74 - 72 - 17 - 81 - 28 + 60) - 94 + 9 + 30 - 11 - 76) - 56 + 4 - 61 + 48 + 97 - 88 + 76 + 70 - 99 - 92 - 30 - 20 - 34 + 77 + 40 - 15 + 15 - 81 + 70 - 82 - 85 + 47 + 3 + 84 - 56 + 88 + 55 + 48 + 49 - 83 - 1 + 39 - 52 - 96 + 1 + 48 - 11 + 66 - 13 + 45 + 51) - 16 + 65 - 17 + 18 + 69 + 67 + 12 + 63 + 45 + 77 - 99 - 91 + 21 + 5 - 88 + 27 + 29 - 89 + 86 + 49 - 26 + 64 - 23 + 85 - 91) - 46 + 8 - 18 - 35 + 57 + 100 - 80 - 26 + 55 - 45 - 61 + 74 + 14 + 30) + 95 - 66) + 42 + 42 + 46 - 38 + 8 - 32 - 88 + 93 + 20 - 98 + 58 - 96 - 90 + 39 + 58 + 30 + 33) + 74 + 53 - 53 - 40) - 5 + 88 + 93) + 0 + 41 + 7 - 53 - 37 - 0 - 21 + 82 + 4 + 41 + 42 - 70 - 20 - 97 - 7 + 86 - 58 - 77 + 81 - 92 + 39 + 67 - 83 + 11 + 17 + 21 + 6 - 85 + 18) - 41) - 82 - 92 - 12 + 19) + 81 - 39) - 20 + 87 - 36 - 69 + 62 - 19 - 59 + 90) - 94 + 82 + 69 + 42 + 51) + 19 - 16 + 96 + 5 + 69 + 97 + 23 - 87) - 29 + 53 - 53 + 51 + 48 - 18 - 86 - 23 - 11 + 40 + 54 - 100) - 76 + 38 - 67 - 0 + 48 + 51) - 66 + 10 - 35 - 17 - 72 - 11 + 65 + 62 - 66 - 62 + 20 + 72 + 84 + 39 + 3 - 76 - 84) + 27 + 99 - 61 - 61 + 33 + 89 - 12 - 10 - 30 - 26 - 68 - 28 + 7 - 85 + 35 - 78 - 50 + 55 + 54 + 12 - 51 - 79) + 93) - 33) + 7 - 85 + 20 - 53 + 75 + 65 + 48 + 79 - 22) - 13 - 66 - 5 - 33) - 59 - 68 - 97 + 13 + 54 - 5 + 51) - 62 - 21 + 75 + 59 - 82 - 17 + 41) - 31 + 68 - 75 + 100 + 92 - 72 + 18 + 71 + 47 + 87 + 74) + 6) + 73 + 91 + 80) - 5) - 11) + 29 - 48 - 50 - 41 + 64 - 75 + 78 + 67 - 30 - 44 - 55 - 4 - 59 - 53 + 70 - 2 + 66 + 20 + 51 - 99 + 65 + 76 - 27 - 35 + 88 + 6 + 12 + 76) - 37 + 1 + 12 - 14 + 19) + 53 + 30 - 41 + 7 + 69 + 98 + 45 - 72 + 52) - 1 + 20 - 46 - 77 + 80 + 19 - 89 - 15 + 35 + 86 + 67 + 90 + 14 + 59 - 73 + 65 + 28 + 62 - 85 - 91 - 50) + 92 + 7 - 73 + 37 + 98 + 70 - 23 + 12 + 93 - 81 - 35 + 94 - 27) + 47 - 84 + 48) + 87 + 28 + 52 + 50 + 1 - 21 + 17 - 83 + 75 - 55 + 41 - 65 + 86 - 63 + 97 + 72 + 100 - 62 - 48 - 40 - 0 + 71) + 37 + 2 + 23 + 42) - 39 + 26 - 80 - 9 - 25 - 66) - 74 - 77 - 54 - 94 + 24 + 37 + 37 - 51 + 59 + 85 + 9 - 0) + 1 - 69) - 55 + 81 + 78 + 75 - 33 + 66 - 31) - 16 - 5 - 100 + 2 + 20) + 43 - 86 - 41 - 27) + 96 + 68 + 71 + 73 + 57) - 54) + 77 + 91 + 6 - 45 + 97 - 96) + 67 + 21 - 34) - 66 - 59 - 73 - 77 + 62 + 51 - 20 - 36 - 36 - 90 + 37 - 61) + 57 + 13 + 30) - 71 - 15 - 69 - 58 + 94 + 49 + 67 + 32 + 86 - 100 + 15 - 4 - 66) + 40 - 37 - 18 + 53 + 13) - 47 - 77 + 62 - 18 + 98 - 18 + 41 + 100 + 40 - 26 + 57 + 18 - 48 - 79 - 9 - 83 - 1 + 70 + 43 - 67 + 67 + 72 - 27) - 32) - 4 + 60 - 33 + 57) + 52 - 18 + 56 + 93 + 37 - 70 - 98 - 94 - 97 + 63 + 3 + 100 - 45) + 39 + 91 - 31 + 3 + 51 + 37 - 55 + 38 + 17 + 84 - 69 + 72 + 16 - 60 - 95 + 42 + 23) + 4 - 11) + 30 + 78 + 62 + 14 - 39 + 95 + 78 - 17 + 35 + 77 + 17 + 46 - 60 + 19 + 69) - 33 - 39) - 18 + 37 - 52 + 77 - 3 - 5 - 83 + 15 - 94 + 37 - 55 - 0 - 23 + 31 + 73 + 83 - 1 + 7 - 58 + 55) - 5 - 59 + 5 + 5 + 32 + 59 - 79) + 66) - 53 + 19 - 11 + 53 - 13 - 10) - 60 - 45 + 14 + 18 + 65 + 81 - 34 - 23 - 78 - 27 - 14 + 64) + 37) + 85 + 92 - 57) + 92 + 33 + 7 + 98 - 9 - 73 + 63 + 42 - 88 + 5 - 55 - 46) + 66 - 100) - 36 - 67 + 55 - 26 + 65 + 96 + 70) - 55 + 90 - 98 - 96 - 28 - 34) - 11 - 63 - 79) - 74 + 80 + 49) - 95 + 16 - 3 - 94 + 51 - 51 + 43 + 87 + 28 + 83 - 19 - 93 + 1 + 76) - 2 + 62) + 75 - 96) + 77 + 61 - 39 - 35 - 39 + 61 - 83 - 13 - 29 + 78 + 69 - 38 + 5 + 84 + 100 + 30 + 69 + 93 + 49 - 16 - 21 - 83 + 10 + 76 - 17 + 83 - 78 - 3 + 93 - 62 - 46 + 99) - 60 + 26) - 31 - 19 + 64) - 21 - 22 + 4) - 41 + 68 - 32 + 55) - 58 - 33 + 49 + 82 - 3 + 31 + 76 - 25 - 100 + 100 + 80 - 51 + 94 + 33 + 61 - 84 - 23 - 67 + 6 + 50) + 54 - 36) - 9 + 66 - 0) + 9 - 55 - 86 + 50 - 32 - 21 - 55 + 27) - 86 + 50 - 85 + 52 - 84 + 47 - 59 + 64 - 14 - 56 + 72 + 57 + 33 - 67 - 9 - 36 - 82 - 34 + 51 - 57 + 42 + 65 - 30 - 94 + 10 + 33 + 49 + 97 - 2 + 29 - 18 + 86 + 69 + 39 - 47 + 1 + 80 + 100 + 90 + 0 + 10 - 24 + 63 + 24 - 100 - 59 + 84 + 80 - 64 + 23 - 4 + 93) - 15 - 33 + 35 + 6 + 36) + 89 - 34 + 26 - 65 - 63 - 84 + 37 - 100 + 51 - 84) + 28 + 69 - 82 - 95) - 75 + 27 - 97 + 39 - 79 - 37 + 32 - 83 + 14 - 27 + 76 - 13 - 6 + 16 - 30 - 92 - 35 - 2 - 57 + 53 - 90 - 49 - 88) + 81 + 88 - 22 - 78 - 29 - 1 + 40 - 28 + 66 + 72 - 38 - 49 + 78 + 19 - 62) + 81 - 4 - 6) - 23 + 95 - 13 - 1 + 11 - 24 + 5 + 81) + 33 - 96 - 6 + 93 + 93 - 96 - 26 - 0 + 71) + 34 + 72 + 50 + 38 - 92 + 57 - 77 + 95 + 38 + 76 - 68 + 97 - 82 - 11 - 23 - 80 - 69 - 18 - 44 + 84 + 42 - 76 + 97 - 80 - 72 + 30 + 84 - 59 + 32 - 79 - 24) - 72 - 43 + 34 - 38 - 31 - 2) + 2 + 40) + 25) - 3 + 81 - 91 - 21) - 91 - 73 - 55 + 66 + 14 + 18 - 42 + 43 + 7 + 93 - 64 - 74 - 100 - 54 + 29 - 58 + 0) + 48 + 48 - 37 + 64 - 3 - 42) - 98 - 28) - 77 + 30 + 58 - 7 + 19 + 79) - 77 + 27 - 3 - 91 + 85 - 20 - 50 - 44 - 95) + 83 + 18 - 28 + 36 + 38 - 42 - 65 - 76 - 42 + 82 - 2 + 70 + 74 - 64 + 18 - 11) + 38 + 5 + 13 + 61 + 60 - 73 + 63 - 91) + 97 - 79 - 29 - 69 - 3 + 23 - 54 + 11 + 89 + 73 - 21 - 34 - 85 + 11 + 67 - 1 + 100 + 50 - 47 - 58 - 24 - 32 - 90 - 34 - 65 - 32 + 46 - 86 + 3 + 8 - 7 - 18) + 31 - 13 - 27 + 35) + 90 - 4 + 50 + 91 + 86 + 0 - 69 - 1 - 67 - 25 + 95 - 58 - 78 - 89 - 39 - 3) + 56 - 23 - 95 + 94 - 78 - 89 + 75 + 33 - 72 - 65 - 89 + 83 + 29 - 63 - 68 - 13 - 40 + 63 - 43 + 85 + 14) - 12 - 77 + 62 - 63) - 38 + 23 + 71 + 11 + 59 - 46 - 1 + 48) + 94 - 30 - 57 + 27 + 91 + 38 - 6 - 69 - 55) - 37 + 42 + 37 + 1) - 81 - 9 - 81 + 37 + 33 - 1 + 82 + 13 - 51 - 19 + 28) + 71 + 50 + 67) - 11 + 49 - 95 + 90) + 47 + 2 - 49 - 78 - 44 + 1 + 11) + 62 - 99 + 98) - 4 + 58 + 58 - 19) + 19 - 74 - 62 + 58) + 48 + 17 - 60 - 58 + 43 - 81 - 53 - 22 - 52 - 50 - 53 + 100 - 68 - 68) + 86 + 7 + 76 - 76 + 6 - 23 - 97 + 95 + 71 + 42) - 77 + 96 - 73 - 29) - 75 - 91 - 21) + 68 + 29 + 77 - 75 - 58 - 48 - 66 + 51 + 84 + 15 + 2 - 22) - 84 - 93 + 87 + 78 - 73 + 28 + 94 + 31 + 11 + 27 - 72 + 61 + 58 - 55 + 36 + 95 - 2) + 75) + 54 + 12 + 80 - 57 - 54 + 7 + 89 + 60 + 85 + 24 - 90 - 44 + 9 + 35 - 77 + 68 - 54 + 45 + 41 + 39 - 93 - 68 + 88 + 68 + 42 + 46 - 86 + 3 + 96 + 45 + 77 - 60 - 45 + 3 + 60) + 77 - 82 + 93 + 13 - 79 - 88 + 74 + 64 - 36 + 71 + 59 - 88 + 55 + 52 + 72 - 59 + 39 + 82 - 32 + 82 - 32 - 16 + 13 + 23 - 74 + 43) + 49 - 29 - 4 - 41 - 100 + 50 + 19 - 78 - 51 - 32 + 43 + 53 + 47 - 15 + 66 - 53 - 61 + 20 + 55 + 68 + 89 + 35 - 62 - 39 + 72 - 19 + 99 - 85 - 5 + 7 + 46 + 100 - 21 - 63 + 95 - 90 + 28) - 16 - 14) + 71 + 35 + 27) + 20 + 97 - 91 - 86 + 72 - 91 - 93) + 63 + 28 - 47 - 35 + 37) - 82 + 37 - 24 + 13 + 68 + 79 + 73 - 58) + 67 - 12 - 1 - 28 - 36 + 58 + 62 + 8 + 53 + 90 + 27 + 31 - 49 + 84 + 82 + 61 + 36 - 45 - 72 - 61 - 34 - 13 + 96 + 11) - 73) - 57 + 87 + 98 - 69 + 49 - 46 + 49 - 50 - 81 - 72 + 40 - 92 - 23 + 54 + 82 + 90 - 27 + 67 + 46 - 48 - 56) + 67 - 91 + 18 - 100 - 28 + 98 - 11 - 25 - 48) - 57 + 90 - 45 - 38 + 67 - 57 - 17 - 76 - 68 - 9 - 92 + 70 - 35 + 75 - 100 - 82 - 31 - 60 - 81) + 51 + 70 - 97 - 68 - 2) + 70 - 10 - 14 - 28 + 2 + 5 + 81 - 26 - 60) + 87 - 2 + 85 - 22 + 34 - 64 - 51 + 64 - 8 + 12 - 57) - 9 + 24 - 98 + 83 - 53 - 8 - 80 - 80 + 3 + 25 - 34 - 32) - 32 + 18 - 3 - 64 + 100 - 39 - 18 - 9 + 81) - 21 + 44) + 12 + 87 + 29 - 88 - 44 - 34 + 65 + 46 + 51 - 35 - 85 + 34 + 98 + 11 - 79 + 84 + 77 - 96 - 24 - 27 - 8 + 66) - 81 + 56 - 99 + 98 - 66 + 61 + 37) - 79 + 66 - 36 - 34 - 55 - 28 + 26 - 52 - 54 - 21 - 25 + 93 - 1 + 90 + 55 - 88 - 80 + 62 - 7 - 57 - 93 - 94 + 17 - 68 + 58 + 64 + 21 - 9 - 42 + 53 - 89) - 74 + 80 + 68 - 48 - 93 + 68 - 13 + 68) - 94 + 23 - 12 + 93 + 66 + 59) + 46 - 80) - 56 + 83 + 72 - 44 - 80 - 38 - 64 + 43 - 72 + 71 + 71 + 28 + 4 + 81 - 74 + 47) + 50 - 51) + 19 + 20 - 24 + 39 + 49 - 69 - 16 - 29 - 78 - 35 + 4 - 65 - 45 - 24 + 65 + 84 + 38 + 64 + 0) - 72 - 39 + 59 - 7 - 94 + 100) + 55 - 53 + 44) - 46 - 36 + 13 + 5 + 93 + 61 - 28 + 24 + 95 + 65 + 36 - 29 - 13 - 87 + 65 - 92) - 51 - 53 - 66 - 8 - 77 - 95 - 74 - 17 - 7 + 11 + 27 + 7 - 88 + 45 + 86 + 86 - 19 - 73 + 6) + 40 + 2 + 32 - 12 - 42 - 91 + 83) + 77 + 85 + 39 - 46 + 57 - 13 + 76 + 64 + 0 + 3 - 21 + 8 + 29 - 18 + 26 + 24 + 19 + 95 - 42 + 20 + 48 - 34 - 44 + 12 + 60 - 91 - 4 + 82 - 5 + 64 - 88 - 62 + 52 - 100) - 18 + 42 + 60 - 42 - 21) - 6 + 71 + 81 - 49 - 78 - 79) + 22) + 45 + 37 - 46 + 19 + 7 + 31 - 80 + 79 - 72 - 28 - 40 + 96 - 75 - 52 - 45 + 78 - 84) - 71 + 33 - 86 + 17 + 43 + 89 + 0) + 78 - 20 + 10 + 51 - 21 - 12) - 16 + 68 + 64 + 60 + 98 - 80 + 45 + 57 - 99 + 61 + 100 - 82 - 68) - 75 - 53 - 61 + 0 + 89 + 6 - 70 - 62 - 40 - 2 - 15 + 28 - 60 + 58) - 59 + 34 - 74 - 2 - 58 + 77) + 11 - 26 - 55) - 100 + 76 - 3 + 62 - 84 - 7 - 19) + 97 + 20 - 46 - 62 - 82 + 58 + 97 + 65 + 40 + 28 - 64 + 8 - 49 - 15) + 10 - 56 - 20 + 52 - 64 - 0 - 69 - 0 + 31) - 96 - 5 + 97 + 97) + 4 - 16 - 71 - 13 + 17 - 71 + 59 + 20 - 46 - 40 + 6) - 6) - 10 + 15 - 79 - 90 + 80 - 53 - 66 - 51 - 10 + 59 + 12 + 38 - 26 + 61) + 33) + 68 - 92 - 9 - 69 + 32) - 19 + 9) + 37 - 74 - 93 - 14 + 0 - 10 - 46) + 34) - 0 + 95 - 18 - 65 - 63 - 82 - 1 + 81 + 89 - 70 - 95 - 20 - 89 - 98 + 80 - 64 - 91 + 45 + 96 - 28 - 91 + 73 - 90 + 32 + 70 + 55 + 66) - 88 - 1 + 26 + 50 + 22 + 78 + 91 + 77 + 28 - 91 - 39 + 85 - 43 + 21 - 5 - 36 - 43 + 69 - 34 + 35) + 86 + 62 + 20 + 69) + 63 - 22 + 40 + 22 - 18 - 42 - 2) - 74 - 5 + 57 - 57 - 56 + 17 + 84 + 64 + 77 - 6 + 32 - 42 + 72 + 23 + 84) + 41 - 88 - 41 - 34 + 41 + 5 - 16 + 16 + 85) - 84 - 4 - 49 - 60 + 92 - 64 + 56 + 30) - 19 - 6 - 87 + 24 + 53 + 43 + 63 + 23 - 40 + 80 - 36 + 71 - 67 - 10 + 13 - 77 - 50 - 65 + 99 + 94 - 72 + 92 - 7) - 48 - 44 - 66 + 71 - 84 + 1 - 5 + 56 - 100 + 13 + 4 + 93 + 9 + 66 - 49 - 8 + 100 + 77 - 99 + 60 + 60) + 69 - 18) - 93 + 55 + 42 + 11 - 14 + 49 - 39 + 3 + 49 + 42 + 41) + 53 + 35 - 59 - 29 - 42 + 68 - 57 - 53 - 25 + 11 + 32 + 53 - 35 + 36 - 61) - 20 + 25) - 14 + 10 + 65 + 31) - 40 + 63 - 58 - 37 + 61 + 73 - 89 - 35 + 83 + 18 - 73 + 54 - 65) - 62 - 21 + 97 + 24 + 77 + 31 - 40 - 95 - 76 - 4 - 85 + 53) + 99 - 22 + 91 - 29 - 21 - 21 + 74 - 53 + 12) - 33 - 47 + 45 - 0 + 31 + 12 - 40) + 40 + 67 - 59 + 6 + 62 + 11 - 51 + 27 + 41) + 90 + 91 - 49 - 93 - 42 - 14 - 13 - 77 - 9 - 70 - 93 - 3 + 87 + 57) + 99 + 29 - 66 - 59 + 5 + 5 - 54 - 85 - 72 + 78) + 92 - 56 + 46 - 27 + 26 + 44 + 40 - 88 + 98 + 57 + 18 + 64 + 44 + 85 + 11 + 93 + 90 - 71 + 24 + 59 + 42 - 11) - 55) - 40 + 22 + 78 - 84 - 72 + 7 + 76 + 93 + 47 + 57 - 20) - 0 - 7 + 37 - 80 + 74 - 71 - 60 + 95 - 10 + 14 - 75 - 22 - 86 + 96 + 13 + 66) + 90 - 77 - 86 + 66 - 71 - 57 + 14 - 46 + 27 - 33 + 40 - 20 - 68) + 11 + 61 - 41 - 43 + 96) + 86 - 13) + 40 - 31) - 49 - 71 - 33) - 33 - 60 + 7 + 89 + 79 - 86 - 18 - 82 + 40) - 6 - 83) - 2 - 81 - 83 - 68 + 83 + 89 - 3 + 72 - 53 - 26 + 67 - 11) + 33 + 76 + 51) + 45 - 77 + 86 - 19 + 89 + 65 + 89 - 75 + 66 - 95 + 42 + 43 + 33 - 28 + 53 + 23 - 38) + 37 - 7 - 16 + 56 + 55 + 68 - 91 - 99 + 42 + 25 - 83 - 24 - 12 - 17 - 59 + 60 - 58 - 22 + 13 - 87 - 84 - 72 - 16 + 80 - 9 - 32 + 69 - 16 - 0 + 60 + 44 - 67 + 47 + 53 - 75) + 85 + 4 - 31 + 62 + 56 + 2 - 77 + 16 + 65 - 29) + 62 + 2) + 95 + 98 + 80 - 69) + 47 + 7 - 39 + 0 + 97 + 18 + 47 + 78 - 9 - 65 - 12 + 10 - 31 - 26 - 4 - 40 + 66 - 29) - 25 + 22) - 37 + 68 - 15 - 86) + 49 + 20 - 8 - 56 - 74) - 24 + 62 - 2 + 33 + 13 + 10) - 89 + 78 + 86 - 11 + 99 + 89 + 48) + 65) + 58 + 47 + 90 + 36 + 92 + 9 + 85 - 50 - 96 + 7 + 28 - 4 - 55 - 21 + 31) + 41 - 54 - 6 - 8) - 2 - 17 + 59) - 16 + 30) + 17 + 4 + 54 - 0 + 61 + 44 - 71 + 22 - 79 - 31 + 53 - 18) - 42 - 42 - 37 + 36 - 55) + 33) + 49 + 86 + 44 + 78 - 89 + 17 - 30) + 95 + 51 + 94 - 76 + 77 + 15 + 35 + 39 + 14 + 83 - 72 - 37 + 78 - 13 - 74 + 33 - 94) - 17) + 56 - 19 - 4 + 17 - 100 - 46 + 2 + 77 - 98 + 35 + 40 + 31) + 28 + 21 - 55 - 29 - 41 + 50 - 21 + 21 + 2 - 28 + 14 + 46 + 52 + 53 + 71 + 12) - 5 - 19 - 57 + 43 + 92 + 8 + 49) - 57 - 42 + 44 - 76 + 44 + 54 + 15 - 78 - 59 - 98 + 41 - 68 - 56 + 46 - 23 + 70 - 46 - 24 - 81 - 67 + 76 - 8) + 12 + 95 - 98 - 0 - 39 + 70) + 56 - 65 - 62 + 87 - 75 + 11 + 88 + 98 - 84 + 5 - 56 - 56 - 49 + 72 + 6 + 10) + 37 - 4 + 15 + 13 + 91 + 73 - 1 + 25 - 54 + 55 - 46 + 32 + 1 + 19 - 27 + 59) + 74 - 53 + 70 - 30 - 24 - 41 - 78 - 13 + 29 - 61) + 86 + 98 - 74 + 40 + 74) - 32 - 43 - 62 + 54 - 9 - 55 + 7 - 93 + 63 - 36 + 85 + 82 + 98 + 2 + 87 + 17 - 92 - 86 - 48 - 32 - 3 + 98) - 1 + 28 + 81 - 3 - 20 + 39 + 24 - 56 + 40 + 66 + 77 + 41 + 29) + 68 + 83 - 37 - 71 - 82 + 26 + 62 + 88 - 18) + 77 + 24) - 68 - 30 + 36 + 7 + 90 + 96 - 92 - 95 + 38 - 93 - 23) + 86 - 93 + 48 + 56 + 96 + 47 + 49) - 4) + 15 + 45 + 29) + 96) + 77 + 65 - 98 + 72 - 63 - 85 + 62 + 21 - 85 + 30 - 55 + 39 - 3 + 66 - 46 + 75 + 19 + 39 + 51 + 61 - 43 + 65) + 75 + 95 - 44 - 43 - 65 + 17 - 21 - 5 - 41) - 78 - 8 - 40 - 57 + 26 + 12 + 29 - 33 + 73 - 44 - 14 + 8 + 79 - 81 - 2 + 26 + 34 + 85 - 0) + 47 - 66 + 50 + 32) - 80 - 41 - 10 - 89 + 29 - 37 - 2) + 97 + 29 - 32 - 97 - 33 + 27 - 31 + 67 + 23 + 41 - 16 - 64 - 6 + 65 - 36 - 74 + 64 + 33 + 81 + 28) + 46 - 92 - 68 + 83 - 29 - 82 + 39 - 16 + 90 + 61 - 73 + 1 + 40 + 11 - 48 + 9) + 92) - 29 + 7 - 42 - 10 - 51 - 83 + 4 - 86 - 57 + 36) + 32 + 44 + 54 + 93 + 44 + 89) + 7 + 83 + 7) + 11 + 6 - 74 - 81 + 10 - 30 - 77 + 73 + 64) + 17) - 56 - 84 + 33 + 93 - 75 + 36 + 33 + 51 - 51 + 28 + 88 + 57 + 9 - 32 + 53 + 51) - 48 - 72 - 4 - 50 + 28 + 69 + 31 + 81 + 64 + 94 + 32 - 33 + 68 + 27 + 90 + 66 + 39 - 78 + 63 - 75 + 22 + 84 + 3 + 72 + 19 - 92 + 0 + 75 + 20) + 45 + 64 + 72 - 96 - 90 + 97 + 80 + 12 - 54) + 66 - 100 - 74 - 71 - 40 - 64 + 48 - 47 - 48 - 6 - 16 + 9 + 60 + 17 - 64 - 29 + 70 + 70 + 90 - 1 + 87 - 23 + 8 + 60 + 39 - 23 - 7 - 74 - 83 - 19 - 45 + 58) + 83 - 56 - 47 - 46 + 7 + 85 + 42 + 61 - 0) + 100 + 13 - 87 - 59 + 80 - 39 - 73 - 79 + 76 + 44 + 30 - 44 - 99 + 60 + 89 + 1 + 35 - 34 + 20 + 31 + 15 + 24) - 81) + 86 + 77 + 47 + 11 - 90 + 55 + 8 + 85 - 46 + 34 - 13 - 86 - 27) + 31 + 18 - 31 + 48 - 17 - 84 - 85 + 27 - 13 - 46 + 31 - 48 + 73 - 64 - 98 + 41 + 27 - 14 - 10 + 33 - 93 + 95) - 9 - 84 + 73 - 74) + 20 - 61 + 22 + 28 + 2 - 50) + 8 - 55 + 93 - 24 - 83 + 78 + 42 + 0 + 33 - 94 - 9 - 71 + 47) - 91 + 37 - 96 + 26 + 69 + 57 - 39) + 28 - 72 - 71 - 14 + 84 + 57 - 68 - 3 + 80 + 1 - 1 - 52 - 53) - 86 - 32 - 29 + 43 + 71 - 44 - 77 + 66 - 14 + 80 + 18 - 42 - 43 - 2 + 90 - 25 + 16 + 99 + 45 - 68 - 20 + 56 - 8 - 57 + 100 - 75 - 83 + 83 + 72 + 54 + 90) - 75 + 97 - 90 + 47 + 32 + 25 + 59 - 68 - 76 + 41) - 45 - 40 + 99 + 28 - 48 + 57 - 30 + 86) - 84 - 86 - 42 + 41 + 57 + 44 - 64 - 40 + 32 + 59 + 67 + 70 - 45 - 19 - 33 - 94 - 43 - 98 + 4 + 35 + 7 + 98 - 86 + 62 - 28 + 83 - 77 + 36) - 19 + 47 + 90 - 64 - 75 - 45 + 66 - 37 + 20 - 46 - 40 - 31 + 97 - 76 + 31 + 71 + 57 + 54 - 15 - 91 - 26) + 62 - 1 - 51 + 37 - 43) - 31 - 94 - 72 - 69 + 27 - 80 - 22 + 100 - 96 - 88 - 41 + 24 - 85 - 33 + 40 + 98 + 86 + 89 - 14 + 44 - 98 - 22 - 84 - 14 - 13 + 34 + 49 + 7 - 8) - 92 - 9 - 52 + 70 - 94 - 1) + 4 + 16 + 27 - 56 - 84 + 67) + 79 + 84 - 44 + 56 - 72 + 56) - 35 - 32 - 92) + 74 + 82 + 13 + 70 - 36 + 16 + 89 - 63 + 4 + 28 + 62 + 28 + 51 - 7 + 59 - 27 - 68 - 25 - 21 - 91 - 23 + 67 + 77 - 94 + 96) - 3 - 7 - 73 - 85 - 41 + 96 - 59 + 16 + 84 - 13 + 27 + 26 - 66 + 93 - 72) - 94) - 74 - 33 - 64 - 38 + 32 - 24 + 70 - 97 - 21 - 68 - 42 + 85) - 6 - 6 + 16 + 94) + 24 - 0) - 96 + 45) - 8 - 65 + 1 + 13 - 22 + 29 - 33 + 73 + 22 + 60 + 79 - 47 - 81 + 44 - 20 + 40 - 9 + 92 + 30 + 19 + 9 - 45 - 38 + 31 - 26 - 40 + 17 - 64 - 49 + 8 - 62 + 39 + 97 - 5 - 24 - 38 + 97 - 68 - 26 - 17 - 61 + 88 - 72) + 55 + 62 - 69 + 87) + 67 - 59 - 61 + 22 - 95 + 69 - 47 - 44 - 23 - 76 + 21 - 45 - 25 - 27 - 87 + 8 + 20 + 20 - 99 - 18 + 20 - 21 + 29 - 1 - 26 + 37 - 74 - 39 + 13 - 13 + 18 + 34) + 16 - 32 + 95) - 53 + 1 - 27 + 88 + 1 - 40 + 64 + 21 + 43 + 37 - 23) + 73 + 95 - 25) - 95 + 20 - 63 - 79) - 18 + 94 + 62) + 36 - 60 - 85) - 43 + 75) - 13 - 13 - 83 + 11) - 88 - 42 - 48 - 30 + 25 + 23 - 1 - 6 - 99 - 52 + 11 - 34 + 25 + 11 - 57 + 85 - 51 + 31 + 4 - 100) - 20 + 18 - 6 - 14 + 3 - 10 + 62 - 27 + 34 + 18 + 16 - 5 - 14 - 94 + 93 + 81 + 67 - 44 + 23 + 87 + 3 + 51 + 30 + 60 + 5 + 93 - 22) - 82 + 40) - 6 - 0 - 11 - 4 - 91 - 51 - 41 - 32 - 80 + 80) - 3 + 78 - 60 + 83) - 100 - 70 - 20) - 24 - 92 - 19 + 1) + 18 + 37 - 8 + 19 + 72 - 93 - 74 + 89 - 96 - 97 + 88 + 9 - 93) - 38 - 16) - 10 - 56 - 1 - 53 + 77 - 21 - 23 + 51 + 46 + 42 + 94 - 87 + 25 - 53 - 4 + 28 + 72 + 12 - 95 + 14 - 13 + 51 + 66 + 19 + 62) - 69) - 16) - 74 + 83 - 71 - 45) - 93 - 12 - 70 + 43 + 63 + 17 + 27 - 81 + 92 - 33 - 16 - 63 + 77 + 5 - 100 - 50 - 98 - 77 + 98 - 15 + 22 + 90 - 51 - 55 - 78 - 86 - 93 + 32 + 83 + 92 - 36 - 41) + 66 + 89 - 42 + 91 - 52 - 11 - 73 + 23 - 8 + 81 + 12 - 68 + 47 - 77 - 29 + 10 - 81 - 95 + 38 - 75 + 62 - 32 + 54 + 42 + 16 - 56 + 75 + 4 + 17 + 52 + 14 - 60 - 41 + 28 + 75 - 50 + 81 + 14 - 86 + 86 + 53 - 63 - 47 + 90 - 14 - 35 - 40 - 49 + 96 - 53 + 71 + 56 - 96 - 67) + 1 - 9 - 76 + 62 + 26) - 89 + 12 - 22 - 22 + 61 - 83 + 92 - 67 - 12) + 86 + 25) - 96 + 7 - 93 + 20) - 16 + 63 - 43 + 62 - 25 - 79 - 72 + 87 - 83) + 27 - 6 - 16 - 25 - 55 + 72 - 4 + 10 + 72 - 1 - 19 - 64 - 59 + 66) - 9 + 46 - 68 + 83 + 34 - 67 + 35 - 100 + 73 + 29 + 97 - 79 - 23 + 55) + 45 - 45 + 27 + 30 - 70 - 94 - 86) - 78 + 49 + 62 - 33 - 7 - 74 + 1 + 36 - 83) - 32 + 57 - 78 + 57 + 91 + 22 + 58) - 94 - 46 + 37 - 81 + 28 - 42 - 80 - 94 + 41 - 60 - 13) + 16 + 7 - 33 + 99 + 40 - 64 + 79 - 94 - 46 + 35 - 7 + 58 + 77 - 53) - 5 + 31 - 96 + 4 + 85 + 24 - 70 - 12 - 79 - 92 + 30 + 65 + 53 + 45 - 44 - 18 + 58) + 32 + 9 + 83 + 76 + 68 - 54 - 74 - 91 + 71 + 16 + 47 - 15 + 21 + 52 + 67 - 3 - 89 - 88 - 11 + 63) - 18 + 96 - 30) - 70 + 23 + 41 + 97 - 57 + 86 + 9 + 17 + 36 + 22 + 64 - 93 + 97 - 65 + 52 + 36 - 57 - 69 - 5 + 35 - 12 + 37 - 9 + 31 - 43) - 0 - 83 + 6 - 48 + 15 + 27 + 61 - 15 + 23 + 79 - 21) - 73 + 23 + 87 + 71 - 6 + 73 + 97 + 29 + 79 + 47 + 55 + 13 - 81 - 62 + 30 - 51 + 61 - 13 - 0 - 28 - 64 + 1 + 64 - 33 - 22 + 66 + 80 - 50 + 49 + 67 - 9 - 77 + 75 - 5 - 14) + 26 - 20 - 3 - 56) + 24 + 15 + 80 + 26 - 31 - 8 - 30 - 81) - 89 + 22 - 94 + 11 + 47 + 85 + 63 + 83) + 67 - 50 + 12 + 97 - 27 + 28 + 10 + 16 + 58 + 13) + 94 - 0 + 32 - 63 - 54 - 97 - 64 + 26 - 70 + 21 + 2 - 10 - 63 + 92 - 5 + 46 + 97 - 92 - 72 + 24 + 30 + 22 - 36 + 2 - 62 - 65 + 69 - 31 - 63) - 85 - 14 - 18 + 28 + 99 + 3 - 46 + 69 + 6 - 62) - 82 - 55 - 71 + 56 - 71 - 90 - 46 + 72 - 30 + 82 - 2 - 81 - 25 + 25 + 65) + 45 + 98 - 60 + 14 - 92 + 4) + 22) - 4 - 62 + 60 - 15 + 32 - 54 - 70 + 32 + 0 - 84 + 91 - 85 + 98 + 34 - 66 - 32 + 93 + 71 - 23 - 81 - 85 + 62 + 41 + 38 - 80 - 0 + 92 + 91 + 72 - 33 - 74 + 53 - 71 - 53 - 92 + 39 + 55 + 27 + 16 - 61 - 84 - 59 + 60 + 24 - 37 + 12) + 43 - 50 - 56) + 88 + 90 + 67 + 92 + 95) - 95 + 91 + 9 + 0 + 31 - 91 + 63 + 13 + 0 + 46) - 3 + 23 + 62 + 87 - 6 - 20 + 55 - 1 - 88 - 87 - 45 - 17 + 29 - 96 + 45 - 34 + 87 - 74 - 43 - 60 - 50 + 58) - 31 + 28 + 34 + 23) - 6 + 1 - 78 + 8 + 70 - 92) + 19 + 95 - 67 + 99 + 95 + 12 - 21 + 27 - 52 - 73 - 13 + 74 + 48 + 55) - 67 + 53 - 8 - 73 + 51 - 38) + 21) - 27 + 53 - 66 - 36 - 58 + 92 + 86 - 26) - 43 - 74 + 84 - 97 - 11) - 22 + 14 + 84 + 32 + 100 - 54 + 71 - 13 - 99 - 65 - 56 - 20 + 88 + 51) - 64) + 79 - 52) + 26 - 4 - 79 + 52 - 91 - 10 + 9 + 48 + 51 - 74 - 5 - 50 + 35) - 6 - 51 + 32) + 97 + 23 + 93 + 9 + 6) - 87 - 97 + 14 - 5 - 32 - 15 - 92 + 8 + 67 + 20 - 10 + 53 + 45 - 88 + 13) - 16 - 27 - 96) + 96 + 21 - 61 - 53 + 75 + 73) - 7 - 46 + 22 + 19) - 49) - 93 + 71 - 64 + 66 + 15 + 43 + 85 - 58 + 99 + 59 + 34 - 7 + 25 + 58) + 21 - 63 - 63 - 4 + 74) - 33 + 85) + 87 - 61 - 24 + 42 + 93 - 88) - 99 - 41 + 24 - 62 - 34 + 94 - 78 - 98 + 29) + 63 + 63 + 52) - 34 - 3 - 26 + 89 - 72 + 31 + 35 - 92 + 49) - 10 + 8 - 46 + 51 - 73 + 46 - 42 + 13 - 90 - 54 + 38) + 27 - 89 + 32 - 2 - 78 - 31 - 34 + 4 + 93 + 76 - 1) + 32 - 76 + 40 - 30 - 18 - 49 + 50 + 87 + 3 + 19 + 41 + 24 - 88) - 46 - 44 - 45 + 98 + 52 + 34 + 9 + 36 - 47 + 91 - 71 + 20 - 17 - 29 + 40 - 33 - 26 + 86 - 71 + 36 + 29 + 33 + 47 + 54 + 48 - 62 + 36 + 4) - 27) + 58 + 99 - 87 + 81 - 26 - 13 + 21 + 53 + 94) - 4 - 41 + 16) - 60 + 6 + 29 + 29) - 41 + 94 - 89 + 54 - 67 + 84 + 82 + 71 + 13 - 3 - 25 - 80) + 21 - 23 + 24 + 3 + 38) + 10 + 56 + 18 - 63 + 3 + 77 - 88 + 19 - 37 + 26 + 42 - 19 - 8 + 100 + 21 + 56 - 1 - 70 + 48 - 93 + 22 - 70 - 4) + 25 - 57 + 38 + 35 - 69 + 12 + 61 + 31 + 95 + 84 + 20 - 90 - 22 + 58) - 32 - 81 + 91 + 25 - 34 + 84 - 58) + 64 - 11 + 70) - 76 + 63 - 87 + 23 - 53 + 11 - 81 + 32 - 89 + 20 + 47 + 35 - 86 + 29 - 54 - 30 + 8 - 60 - 86 + 46 + 22 - 25 + 89 - 53 - 14 + 28 - 69 + 25 + 96 - 92 + 66 + 5 + 34 - 62 - 79 - 69 + 64 - 78 + 50 + 65 - 76 - 47 - 4) + 21 + 47 - 99 - 18 - 87) - 50 + 46 + 69 + 45 - 40 + 22 + 61 - 85 - 82 + 95 + 94 - 12 + 90 - 51 + 51 - 94 + 24 + 7 + 78 - 27 + 30 + 44 + 55 + 6) + 58) + 8 + 71 + 12) + 23 + 36 - 65 - 0 - 17 - 15 - 90 - 93 + 31 + 19 - 72 - 70 - 28 - 25 - 51 - 44 - 67 + 28 + 96 - 66 - 89 - 20 - 42) + 77 - 68 + 73 - 4 + 6 - 46 + 28 - 99 + 46 - 5 + 61 + 5 - 53 + 94 + 98 + 37 - 66) - 94 - 59 - 42 - 68 - 62 + 72 - 63 + 98 - 43 - 82 + 52 - 92 + 72 - 94 + 56 + 21 + 13 + 32 + 12 - 25 + 79 + 30 - 2 - 76 + 25 - 74) - 96 + 86 + 92 + 48 - 51) + 74 - 34 - 26 - 65 + 82 + 50 - 49 + 14 - 1 - 49 - 73 + 32 - 3 + 5 - 28 - 18) - 22 + 73 + 26 - 0 + 97 + 62 + 49 + 30 - 57 - 31 + 17 - 66 + 95 + 25 - 24 - 13 + 52 - 8 + 79 + 56 - 44 - 17 + 97 + 97 - 87 - 8 + 77 - 21 - 80 + 27 - 69 + 64) - 17 + 77) + 24 + 63 + 85 - 95) - 5 - 64 + 12 - 63 - 55 + 24 - 35 + 45 + 75 + 53 - 70 + 9 - 37 - 57) - 89 + 46 + 61 - 80 - 92 + 58 + 92) + 42 - 100 + 81 + 65 + 2) + 14) - 84 + 88 + 58 + 88 + 28 - 82 - 79 + 79) - 16 - 95 + 93 + 11 + 29 + 35 + 73 - 71 + 33 - 73 - 89 + 22 - 73 - 51 - 94 + 93 - 53 - 10 + 64 + 30 + 80 + 50 - 42) + 10 - 48 + 86 + 55 + 7 + 73 - 11) - 76 - 54 - 93 + 4 - 60 - 63 + 52) + 31 + 80 + 21 - 42 - 77 + 26 - 96 - 88 - 42 + 18 - 64 - 47 + 43) + 89 + 72 + 57 + 94) - 33 + 58 - 6 - 43 - 88 - 93 + 47 + 74 - 85) - 53 + 32 - 58 + 46 + 43) + 99 + 78 - 71 - 40 - 21 + 24 - 52 + 90 - 65 - 82 - 48 + 66 - 11 - 60 - 42 - 99 - 55 + 29 - 20 - 23 + 71 - 96) - 59 + 100 + 76 - 23 + 8 + 40 + 47 - 16 - 18 + 70 + 66 - 48 + 73 + 30 - 49 + 87 - 67 + 7 - 60 + 41 - 0 + 68 + 12 + 88 - 13 - 11 + 100 + 73 + 61 - 18 + 42 - 71 - 48) + 67 - 0 + 67) - 25 - 14 - 35 - 0 + 26) + 3 - 10 + 95 - 58) - 71) + 48) - 43 + 0 - 16 - 6 - 47 - 26 - 41 - 14 + 73 - 78 - 91 - 94 + 66) - 96 - 45 + 63 + 17 - 47 + 64 + 55 + 57 + 70 - 86 - 4 + 31 - 93 + 94 + 62 + 37 - 68 - 58 - 32 - 69 + 72 + 75 + 25) + 23 + 47 - 72 - 58 - 95 + 51 - 39 + 27 - 17 + 83 + 9 + 32 + 38 - 73 - 62 + 23 + 99 + 77 + 16 - 87 + 100) + 49 - 95 - 75 - 75 + 7 + 57) + 55 - 70 - 0 + 22 + 88 + 8 + 30 - 47 - 56 - 24) - 36 - 2 + 59 - 35 - 39 - 64 + 52 - 14 + 8 - 27 + 54 - 93 + 0 - 15 + 15 - 17 - 19) + 98 - 3 + 67 - 77 - 99 + 47 - 79 + 37 + 0 + 50) - 56 - 95 + 66 + 16 + 22 - 37 + 88 + 61 - 55 + 40 + 64 + 87 + 78 + 71 + 82 - 15 - 78 + 5 + 86 - 2) - 50 - 14 - 25 + 10 + 15) + 12 + 69 - 4 - 92 + 39 - 95 + 42 + 85 + 2 - 10 - 78 - 51 - 13 + 46 - 24 - 8 - 25 - 35 + 15 - 7 + 80 - 76 - 49 - 40 - 95 + 44 - 9) + 82 - 35 - 20 - 30 - 68 + 5 - 54 + 16 - 3 + 46 - 14 + 18 - 7 + 87 - 70 + 8 + 84 - 50) + 34) - 34 + 98) - 100 + 98 + 60 - 6 + 100 + 75 - 13 + 66 + 9 - 76 + 33 - 99 + 69 + 93 + 90 - 77 + 71 + 63 - 52 + 37 - 85 + 61 - 91 - 51 + 50 - 90 - 94 - 18) - 20 + 39 - 38 + 26) + 51 + 42 + 44 + 61 - 99 + 67 + 58 + 2 + 79 + 54 - 65 + 86 + 6 - 13 + 67 - 18 - 92 + 16 + 99 + 43 + 29 - 78 - 58 - 49 + 14) - 22 + 28 - 69 + 29 - 32 - 46) + 21 + 63 + 16 - 26 - 59 - 2 + 12 - 79 + 13 + 21 - 29 + 1 + 10 + 13) - 25 - 67 + 59 - 68 + 32 + 76 + 69 - 84 + 81 - 57 + 78 + 27 - 8 + 48 + 92 + 69 + 59 - 82 - 39 + 45 + 17 - 5) + 74 + 59 - 6 + 12 + 13 + 26 - 47) + 40 - 35 + 2 - 63 - 62) + 29 + 89 + 54 + 76 - 100 + 17 + 46 - 94) + 78 - 77 - 19 + 30 + 39 - 7 - 33 - 35 + 93 - 74 - 85 - 30 - 76 + 16 + 92) - 98 + 90) - 88) - 23 - 52 + 17 + 78 - 32 - 95 + 38 - 50 - 54 - 29 + 99 - 55 - 59 - 48 + 62 - 57 + 58 + 37 + 54 + 70 + 66 - 66 + 12 + 4 + 11 + 53 + 67) - 70) + 3 + 97 + 31 + 62 - 47 + 100 + 59 + 26 - 34 - 65 - 36) - 0) - 57 - 5 + 7 + 7 + 28 + 34 + 3) + 6 + 88 + 13 - 76 - 2 - 40 + 46 + 97 - 8 - 56 - 36 - 80 + 31 + 59 + 6 + 13 - 39 + 67 + 87 + 33 + 58 - 61 + 17 - 81 + 6 + 23 - 50 + 69 - 72 + 12 - 59 + 95 + 45 + 25 + 44 - 62 - 58) - 66 - 77 - 95 + 33 + 87 + 22 + 33 - 75 - 76 - 96 - 8 - 18 + 59 + 86 + 22 + 62 + 40 - 94 - 18 - 66 - 28 + 89 + 21 - 76 - 44 + 55 - 66 - 50 + 84 - 69 - 38) - 37 - 44 + 80 - 67 + 66 + 6 - 90 + 33) - 71 - 89 + 30 + 74) + 28 - 16 - 32 - 91 + 86 + 41 - 63 - 30 - 63 - 31 - 75 - 54 - 28 - 19 + 9) + 75 - 54 + 81 - 62 - 34 - 33 + 51 - 22 - 7 - 24 + 47 + 66 - 41 + 40 - 29 + 7 - 78 - 58 - 17 + 46 + 89 + 26 - 51 + 95 - 44 - 6) - 59 + 69) - 87 - 62 + 36 + 83 + 33 - 29 + 28) - 18 + 98 - 26 - 30) - 23 - 60) + 96) + 70 + 71 - 75 - 78 + 1 + 74 + 88 + 29 + 51 + 22 - 8 - 31 + 23) - 100 + 76 - 25 - 75 - 84 - 8) - 7 + 87 + 9) + 65 - 90 - 23 + 47 + 22 + 30) - 64 + 100 - 73 - 81 - 53 - 97 + 0 - 14 - 0 + 63 + 35 - 71 + 92 - 11 - 42 + 32 + 66 + 22 - 25) - 26 + 0 - 29 - 91 - 31 - 1 + 56) - 80 - 37 - 63 + 98 - 33 - 37 + 34 + 8 + 23 + 6 + 9) + 26 - 34 - 51 + 24 + 66 + 1) + 25 + 54 - 57) + 53 - 68 + 64) - 89 - 43 - 50 - 94 + 98 + 36 - 93 - 20 + 62 + 16 + 25) - 1 + 58 - 53 + 21 - 62 - 61 + 53 + 33) - 44 - 53 - 93) + 44 - 18) + 67 - 39 - 66 + 43 + 63 - 33 - 10 - 85 - 78 + 60 + 59 + 52 - 57 - 14) - 82) - 22) - 72 - 72) + 17 - 81 + 68 - 8 + 89) - 46 - 24 - 71 - 23) + 97 + 51 + 8 - 32 - 55 - 47 - 70 + 55 + 37 + 39 + 32 + 60 - 47 + 38 - 87 - 24) - 14 - 86 + 81 + 35 - 65 + 15 + 11) + 60) + 69 - 65 + 7 + 78 - 62 - 12 - 50 - 20 + 51 + 73 - 87 - 75) + 24 + 26 - 13 - 32 + 24 - 54 + 38 + 56) - 87 + 96 + 44 - 78 - 22 + 28 - 62 + 65) + 63 - 13 + 89 + 51 - 26 + 39 - 10 + 93 - 32 + 74 + 86 + 12) + 66) + 1 + 74 - 7 - 62 + 92 + 65 + 17) - 67) - 2 + 9 + 57 + 76 - 85 - 74 - 75 - 96 - 78 - 41 + 51 - 50 + 37 - 37 - 86 - 21 + 72 - 37 + 18 - 88) - 51 + 87 - 71 + 21 - 50 - 60 - 0 + 2 + 68 - 62) - 41 - 13 + 69 - 48 - 0 + 77 + 60) - 23 - 24 + 99 - 88 + 55 + 86 - 63 - 14 - 25 - 73 + 44 - 68 - 38) + 90 + 31 - 35 + 28 + 76 - 44) + 39 - 57 - 88 + 31 - 26 - 100 - 66 - 27 - 47 - 56 - 41 + 92 - 80 - 17 + 96 - 61 - 11 - 75 - 82) - 51) - 34) - 88 - 76 - 77 + 25 - 14 - 76 + 98) + 17 - 89 - 9 - 0 + 78 + 76 + 65 + 56 - 80 + 32 + 56 + 45 + 62 - 43 + 26 - 49 - 7 - 85 + 60 - 65 - 16 + 98 + 20 - 58 - 29 + 75 - 86 + 48 - 38 - 56 + 35 - 9 + 26 + 1 - 51 + 59 + 82 + 43 - 51 - 62 - 32 - 10 - 12 + 92 + 24 - 98 + 99 + 95 + 23 - 38 + 100 + 66 - 79 + 51 - 62 - 70 - 99 + 8 + 43) - 15 - 73 - 93 - 52 - 80 - 91 + 80 + 55 - 13 + 50 - 10) - 36 - 53 - 6) + 83) + 41) - 29 - 87 - 63 - 37 + 0 + 85 + 84 + 96 - 23 - 98) - 80 - 50 - 37 + 47 - 50 + 81 - 0 + 29 - 17 - 59) + 83 + 78 + 19 - 18 - 33 + 34 + 91 + 88) - 15 + 2 - 85) + 45 + 59 + 60 - 70 + 68 + 34 - 19 + 67 - 100 + 32 - 39 + 4 + 81 - 72 + 58 + 65 - 5 - 1 - 43 + 56 + 47 - 51 - 50 + 4 + 32 + 78 + 66 + 55) - 73 - 44 - 83) - 29 + 30 + 30 + 94 - 3 + 48 + 36 - 34 + 50 + 73 - 48 + 32 + 31 - 45 + 6 - 78 - 12 + 43 + 7 - 97 - 80 - 78 + 82 - 18 - 0 - 7 - 81 - 84) - 40 + 56 - 80 + 67 - 2 + 58 - 86 + 5 - 79 + 64 + 13 - 10 + 94 - 10 + 4 - 74 - 33 + 18 + 99 - 79 + 60 + 58 - 58 - 76 - 38 + 73 - 17 - 0 + 38 + 79 + 19 - 1 - 22) + 95 + 85 + 87 + 78 - 32 - 59 - 19 - 49 - 14 - 100 + 42 - 6 + 11 - 1 - 7 + 72 - 80 - 66 - 89 - 16 - 36 + 78 + 29 + 31) + 14 - 83 - 5 + 4 - 11 - 23 + 49 - 57 + 40 + 41 + 20 + 7 + 16 + 65 + 36 + 76 + 10 + 51 - 3 - 76 - 6 + 25 + 80 + 62 - 2 - 12) - 19 + 58 + 10 + 59 + 68 - 58 - 13 + 51 + 79 - 91) + 58 + 58) - 100 + 32 + 53 + 22 + 53 - 72 - 27 - 86) + 77) + 65 + 100 - 81 + 40 + 87 + 33 + 57 + 62 + 9 - 45 - 32 - 38 - 32 + 67 + 41 - 3 + 53 - 20 - 4 - 67) - 40 + 74 + 53 - 90 + 39 + 54 + 27 - 30 + 42 + 63 + 11 - 68 + 18 + 15 - 89 + 36 + 52 - 55 + 94 - 66 + 54 - 31) - 33 - 11 - 81 - 73 + 85) + 43 - 37 + 54 - 47) + 82 + 20) - 10 - 65 - 27 - 22 - 35 + 85 + 63 + 60 + 52 + 5 + 36 + 75 + 31 + 20 + 43 + 24) + 70 - 99 + 84 + 53 + 21) + 12 - 50 + 99 + 75 - 44) - 67 - 6 - 27 - 33 - 80 + 88 - 77) + 96 + 98 - 78 + 17 - 65) - 60 - 37 + 82) + 79 + 6 + 40 - 49 + 31 + 64 + 43) + 38 - 27 + 14 + 20 - 54 - 8 - 67 - 19 + 13 - 33 + 67 - 58 - 51 + 9 - 21 + 68 - 55 - 35 - 41 - 9 - 90 + 47 - 66) - 63 - 38 + 37 - 90 - 54 + 28 + 17 + 8) + 14 + 68 - 77) + 55 + 7 - 6) + 28 - 26 - 78 + 42 - 71 + 82 - 12 - 56) + 47 - 64 - 26 - 50) - 56 + 41 + 82 - 7 - 85) - 43 - 69 + 43 + 61 - 11 + 30 + 10 + 57 + 95) - 30 - 64 - 18 - 45 - 74 - 37) - 42 - 45 - 64 + 39 + 25 - 50) - 79 + 69 - 15 + 46 - 12 - 83 - 31 + 9 + 27 + 31) - 48 - 60 - 25 - 67) + 57 + 36 - 50 - 43 + 47 + 92 - 18 - 60 + 50 - 80 - 20) - 84 + 99 + 34 + 41 - 84 - 17 - 13 - 8 - 74 + 90 + 14 + 75 - 92 + 4 + 63 + 53 - 9 - 21 - 17 - 68) + 79 - 35 + 77 + 96 - 36 - 81 - 5 - 95 + 23 + 33 + 28 - 75) + 12 + 50 + 89 - 26 + 38 + 82 + 90 + 93 + 98 - 13 - 45) - 24 + 81 + 97 - 34 + 39 - 64 + 97 + 41 - 87 + 71 + 96 - 42 - 99 - 48) + 22 + 14 + 95 + 76 - 89 - 41 - 5 - 28 + 87 - 3 + 38 + 24 - 66 + 66 + 100 + 55 + 67 - 44 + 9) + 67 - 50 - 11 - 33 - 42 - 40 + 16 - 55 + 80 + 40 + 16 - 82) - 41 + 0 + 77 + 25 + 88 + 91 + 79 + 80 - 85) + 3 + 47 + 58 + 12 - 88 + 4 + 62 - 92 + 15) + 24) + 63) - 60 + 28 + 17 - 21 + 89 - 98 - 41 + 80 + 100 - 68 - 39 + 39 - 28 - 12 + 50) - 40 + 49 - 61 - 35 - 75 - 54 + 50 - 91 - 4 - 49 - 59 + 92 - 86 - 82 + 71 + 91 + 100) + 11 - 11 + 69 + 26 - 1 + 34 + 78 + 91 + 80 + 78 - 71 + 96 + 64 + 41 - 61 + 30) - 49 + 25 - 39 - 65 + 80 + 53 - 8 - 89 + 51) - 5 - 4 + 49 + 9 + 88 + 58 - 99 + 16 + 90 - 4 - 82) - 64 + 79 + 42) + 33 - 35 + 59 + 78 - 39 - 74 + 85 + 8 + 14 + 24 - 39 - 41 - 38 - 63 + 8 - 51 - 30 - 16 + 59 + 18 - 78 - 20 - 17 + 100 + 60 + 95 - 97 + 98 - 100 - 70 - 96) - 57 + 79 - 6 - 51 - 11 + 23 + 36 + 34 - 49) + 73 + 14) + 24 + 89 - 92 + 46 + 21 + 45 + 48 - 52 - 21 - 16 + 12 + 12 - 88 + 12 - 97 - 8) + 57) - 83 + 25 + 12 + 35 - 58 - 80 - 34 - 42) + 65 - 57 - 6 - 19) - 89 - 56 + 25 - 7 - 12 - 17 + 40 + 53 + 64 + 92 - 37 + 83 - 100 - 66 + 48 + 58 + 50 + 69 + 28) - 60 - 17) + 62 - 52) - 87 + 69 - 33 - 11) + 96 - 36) - 51 - 97 + 19) + 5 - 56 - 78 + 65 + 29 + 21 - 15 + 10 + 40 + 63 - 18 + 96) + 71 + 46 - 3 - 23 + 63 + 82) + 37 + 75) + 45 + 16 + 52 + 47 + 51 - 28 - 8 + 53 - 14) - 85 + 19 - 31 + 87 + 95 + 14 - 30) + 49 + 94 - 23 + 34 - 6 + 23 + 61 - 94) - 49 + 3 - 27) - 85 + 17 + 20 + 51) - 8 + 91 + 100 + 42 - 89 - 36 + 30 + 65 + 8 - 43 + 77 - 31 - 91 - 28 + 46 + 68 - 28 - 24 + 94 - 65 + 26 - 85) + 0 + 46 + 2 - 9 + 33 - 0 + 87 - 97 + 36 + 69 + 28) + 95 + 43) + 88 - 59) - 84 - 89 - 59 - 91 - 26 + 31 + 75) - 74) + 85 - 78) - 52 - 15 - 48 + 90 + 18 - 91 - 96 + 26 - 79 + 72) - 32 - 3 - 43 - 66 - 90 + 91 - 68 + 11) - 8) + 13 + 43 + 60 - 47 + 83 + 99 + 5 - 22 - 48 + 17 + 43) - 93 + 6 - 10 - 61 + 90 - 40 + 41 - 99 + 48 + 96 - 62 + 87 - 37 + 9 - 89 - 7 + 44 + 97 + 26 + 52 - 49 + 71 + 96 - 98 - 67 + 80 + 52 + 5 - 14 - 54 + 73 - 60 - 59 + 86 - 36 - 48 + 29 + 93 - 16 - 81 + 35) - 85 - 95 - 51 + 100) - 61 + 43 - 98 + 61) + 84 + 68 - 89) + 85 - 48 - 94 - 24 - 34 - 79 + 4 - 3 + 88 + 33 - 42) + 73) + 10 - 37 + 7 + 68 - 19 - 87 - 31 - 47) + 30 + 45 + 47) - 70 - 63 + 100 - 49 + 31 - 70 + 87) + 10 - 63 - 82 - 52 - 41 - 28 - 11 - 8 + 20 + 32) - 36 + 30 - 15 + 73) + 21 - 2 + 27 - 42 - 11 - 34 + 54 - 88 + 86 + 97 - 29 - 17 + 49 - 39 + 85 - 57 - 7 + 66 + 35 + 10 + 4 + 42 + 36 - 25 - 34) + 59 + 48 - 39 + 15 + 69 + 74) + 94 - 42 + 4 + 66 - 13 - 64 + 53 - 75) - 32 - 82 + 1 + 42) + 59 - 54 - 26 - 8) - 82 + 9 + 87 - 91 - 19 - 45) - 74 - 22 + 79 + 76 + 33 + 29 - 28 + 28 - 33 + 85 - 74 + 59 + 29 + 9 + 74 - 25 - 55 + 12 + 5 - 56 - 6 + 35 + 8 - 76 - 54 - 15 + 78 - 37 - 56 + 76 + 26 - 22 + 10 - 47 - 17 + 65 + 26 + 81 - 55 - 80 - 86 - 15 - 3 + 45 - 14 - 0 + 11 - 84 - 39 - 64 + 50 - 34 + 46 + 68 - 9 - 45 + 87 + 92 - 48 - 43 + 52 - 48 + 19 + 9 - 77 - 98 - 24 - 67 - 90 - 67 - 29 - 90 + 69 - 83) + 28) + 78 + 98 + 85 + 41 + 23 + 47 - 37 - 9 + 85 - 87 + 2 - 59 - 71 + 23 + 94 + 1 - 21 - 51 - 35 + 63 + 93 + 27 + 16 - 16 - 32 + 3 - 21 - 62 + 68 + 57 - 23 + 76 - 41 + 5 + 0 + 42 + 15 - 27 + 12 + 3 - 60 + 100 + 0 - 67 - 57 - 40 - 22 + 89 + 35 - 99 - 63 - 30 - 0 - 62 - 75 - 30) - 49 - 42 - 93 - 81 + 67 + 2 - 100 - 93 + 30 + 14 - 66 - 72 + 20 - 60 - 97 - 83 + 37 + 17 + 79 - 39 - 23) + 78 - 83 - 72 + 50) - 11 + 46 - 53 - 47 + 4 - 93 + 11 + 15) + 94 + 77 - 98 - 36 - 28 - 4 - 58 + 11) + 97 + 11) - 76 + 53 + 16 - 19 - 43 - 57 - 44) + 60 + 72 + 54 - 33) + 85 + 56 - 34 - 21 + 64 + 51 + 1 + 83 - 13 - 20 + 45 + 100 + 7 - 44 + 95 - 36 + 67 + 94 + 47 - 73 - 13) + 37 - 96 + 58 - 52 + 19 - 46 + 30 - 18 - 65 - 43 - 55 - 48 - 42 + 51 - 44 + 62 + 63) - 44) - 76 + 51 - 1 - 57 + 38 + 22 + 9 - 98 + 37 + 79 + 58 - 53 - 36 - 74 + 33 + 25 - 40 - 90 - 92 - 48) + 55 + 48 - 31 + 89 - 28 + 0 + 45 - 16 + 30 + 20 - 61 - 94 + 21 + 26 - 25 + 19 + 21 + 89 + 55 - 50 + 37 + 74 - 74 - 41 + 14 + 15 - 6 + 100 + 20 - 40 - 65 + 93 - 24 - 20 + 36 + 70 - 20 + 95 - 83 + 94) - 78 + 15 - 57 - 48 - 39 - 28) + 96 + 24 + 70 - 0 + 67 + 38 + 19 + 95 + 64 - 36 - 35 + 4 - 92 - 49 - 80 - 33 + 37) - 28 - 88 + 97 + 62 - 37 - 58 - 54 + 26 - 13 - 27 + 99 - 81 - 63 - 61 + 72) + 16) + 86 + 16 - 37 - 43 - 100 + 44 - 42 + 17 + 39 - 52 + 39 + 0 + 45 + 25 - 2 + 4 - 35 - 44 - 16 + 56 + 81) + 97 - 73 - 77 - 24 + 90 - 95 + 15 + 29 + 39 + 48 - 92 - 19 - 23) - 74 - 70) - 45 + 25 - 7 - 70 + 14 - 50 - 44 + 60) + 10 + 8 - 81) + 60 + 67 - 67 + 6 + 91 - 73 + 23 - 65) + 54 - 55 + 43 - 100 + 62 + 33 + 36 + 74 - 62 + 65 - 56 + 37 - 5 + 3 + 53 - 82 - 83 - 93 + 55 + 4) + 58 + 61 - 51 - 13 + 28 + 22 - 57 - 88 + 57 + 96 + 21 + 39 + 95 - 94 + 47 - 76 - 88 + 66 - 24 - 65 + 10 + 23 + 78 + 78 - 50 - 72 - 56 + 44 + 74 + 60 - 17 + 7 + 69) - 24 - 25 + 14 + 73 + 24 - 16 - 44 - 65 + 95 + 85 - 90 + 48 - 3 + 81 - 15) + 99 + 1 + 58 + 72 - 78 - 86 - 24 - 19 + 43 + 31 - 75 - 80 + 11 - 61 + 69 + 78) + 0 + 44 + 72 - 26 + 82 - 77) + 9 - 17 + 50 + 73 - 67) - 31 + 14 - 29 - 59 - 90 + 49 + 22 - 2 + 37 + 96 + 77 - 16 + 64 + 39 + 0 + 21 + 98 + 31 - 47 - 59 - 41 - 94 - 48 - 52 - 57 - 3 - 12 - 42 - 34 - 35 - 63 + 78 + 45) - 26) - 97 + 9 + 10 - 11 - 55 - 25 - 38 + 53 - 11 - 17) + 0 - 68 + 23 - 75 - 97 - 63 - 83 + 54 + 24 + 93 - 3 - 59 - 57 + 87 + 7 + 92 + 80 + 71 + 43 - 78 + 94 + 36 - 30 - 24) - 9 - 1 + 4 - 14 + 89 + 43 - 90 - 46 + 25 + 67 - 27 + 96 - 44 + 46 + 13 + 76) - 17) + 94 - 88 + 71 - 51 - 65 + 19 - 71 - 54 - 90 - 92) - 61 - 68 - 19 + 78 + 80 + 11 + 21 + 73 - 83 - 28 + 15 + 97 - 41 + 3 - 36) + 61 + 100 + 66 + 100 + 94 - 64) - 42 - 75 - 94 + 21 - 68 - 67 + 60 - 74) - 66 - 68 + 84 - 70 - 41 + 24 - 35) - 29 + 99 - 22 - 48 - 44 - 99 + 85 + 6 - 76 - 89) - 56 - 69 + 17 + 15 - 93 - 1 - 79 - 98 + 67 + 54 + 89 + 37 - 6 - 89) - 14 + 7 + 22 + 17) - 66 - 26 - 7 - 52 - 69 - 15 - 34 - 16 + 57 + 99 + 24 + 30) + 60) - 43 + 4 - 58 - 73 + 46 - 46) + 2 - 72 + 77 - 51 - 8 - 30 - 31 - 80 + 40 + 77 - 0) + 34) + 55) - 7 + 76 - 67 - 50 - 13 - 4 - 80 + 6) + 82 + 75 + 2 - 75) - 2 - 82) + 45 + 50 - 94 + 71 + 63 + 28) - 15 - 99 - 20 - 12 - 11) + 22) + 42) - 4 - 93 - 6 + 3 - 37 - 13 - 24 - 18 - 41 + 16 + 40 - 54 - 54 + 28 - 58 - 46 - 70 + 11 - 74) + 17 + 40 - 37) + 97 + 81 - 23 + 80 + 21 + 43 - 20 - 92 + 2 + 70 + 16 - 19 + 46 - 85 + 91 + 73 + 82 - 100 - 64 + 2 - 58 - 12 - 66 + 74 + 79 + 12 - 87 - 76 - 87 - 25 - 20 + 61 - 34 + 48 + 49 + 47 + 40 - 25 - 76) - 84 - 92 + 88 - 43 - 91 + 4 + 89 + 50) - 66 + 36 + 12 - 12 + 48 - 14 - 29 - 4 + 76 + 51 - 46 - 17 + 36 + 82 - 94 + 22 - 59 + 87 - 100 - 56 + 38 + 96 - 99 - 97 - 36 - 64 - 61 - 14 - 45 - 7 + 0 + 72 + 52 + 98 + 34 - 48 - 28 + 56 - 67 - 16 + 25 + 72 + 85 + 29 + 53 - 52 - 66 + 68 + 87 - 87 - 58 + 43) - 33 + 73 - 79 + 84) - 83 + 45 - 9 - 80) - 66 + 49 - 100 + 52 + 55 - 47 + 85 - 95 - 59 - 89 + 81 - 86 + 13 - 42 + 78) - 74 + 86 + 48) - 83) + 7 - 21 + 72 + 76 - 69 - 83 + 32 - 28 + 21 + 95 - 8 - 7 - 21 - 40 - 5 + 79 - 23 - 47 + 41 - 93 - 69 - 83 + 49 - 71 - 83 + 92 + 7 + 62 + 36 + 67 + 29 + 66 - 74) - 66 - 39 - 96 + 30 + 31 + 45 + 58) + 5 + 54) - 78 + 73 + 57 + 65 - 50 + 50 - 21 - 100 - 46 + 62 + 15 + 28 - 79 + 52 - 33 + 0 - 36 + 11 - 36) - 13 + 52 - 66 + 14) - 53) + 23 + 29 - 29 + 69) + 91 + 54 + 53 - 87 - 32 - 11 - 94 + 100) + 62) + 36 + 16 + 30 + 73 + 96 + 26 + 50 + 14) - 14 - 68) + 21 + 56 - 89 + 51 - 46 - 82 - 48) - 6 - 36 + 18 - 86) + 56 - 69 - 70 - 59 - 79 - 26 - 32 - 39 - 99) + 95 + 56 + 5 - 40 - 29 + 67 - 68) - 36) + 71 + 19 - 64 - 54 + 94 + 92 + 6 + 33 + 95 - 94 - 78 - 16 - 82 + 84 + 24 - 68 + 14 - 1 - 82 - 3) + 21 + 84 - 96 - 5) + 26 + 86) + 1 - 5 + 21 - 94 + 67 - 0 + 53 + 83 - 54 + 84) - 98 + 74 - 99 + 22 - 38 + 63 - 98 + 51 + 98 + 1 + 54 + 13 - 79 - 60 - 92 - 3 - 67 - 60 + 65 + 23 + 75 + 94 - 62 + 100 - 90 - 98 - 18) - 39) + 20 + 31 - 49 - 52 - 74 + 67 + 33 - 0 + 33 + 70 - 97 - 74 - 54 + 63 + 69) - 9 + 24 - 49) + 8 + 90 + 57 - 60 - 76 - 2 + 18 + 22 - 62 + 42) - 65 - 9 - 28) + 67 - 85 + 70 - 53 + 72) + 93 + 22 + 38 + 90 - 12 + 53) - 65 - 84 + 97 + 3 - 25 - 13 - 24 - 32 + 7 - 50 - 3 + 30 + 25 - 2 + 24 + 3 + 57 - 83) + 82 - 60 - 99 - 46 - 63 + 46 - 5 - 63 - 36 - 49 - 51 - 5 + 29 + 71) - 2 + 41 + 78 - 51 - 71 - 10 - 31 + 6 - 26 + 14 - 9 - 75 + 99 + 20 - 85) + 70 - 29 - 24 - 87) - 49 + 43 + 37 + 37) + 38 + 91 - 56 + 36 + 46 + 77 - 72 + 26 + 50 - 28 - 30 + 32) - 53 + 92 - 77 - 24 + 10 - 57 + 8 + 49 + 100 - 13 - 36 - 9 - 5 - 84 + 95) + 30 - 43 - 99 - 36 + 49 + 67 + 36 - 94 - 38 - 75 - 70 + 30 - 35 - 47 - 66 + 38 - 18 + 39 - 62 + 33 - 73 - 52 + 70 + 80 - 18 - 31 - 92 + 85 - 12 + 65 - 68 - 32) - 19 - 52 + 41 - 21 - 85 - 92 - 60 + 93) - 23 + 53 - 23 - 93 + 22 + 37 + 100 - 60 + 70 - 6 + 23 + 18 + 88 - 15 - 79 - 90 - 4 + 75 - 31 + 24 - 42 - 89 - 24 - 60 - 9 - 8 + 43 + 39 + 75 + 69 - 15) + 17 + 38) - 42 - 92 + 9 + 9) - 46 + 7 - 13 - 27 + 37 - 51) + 70 + 60 + 86 - 89 - 73) + 35 + 91 + 55 - 90 - 98 - 16) - 85 + 17 - 10 - 4 - 15 - 64 + 63) - 77 + 97 + 21 - 24 - 21 - 36 + 16 - 85 + 70 + 15 + 30 + 56 - 36 + 6 - 50 - 36 - 20 + 49 - 31 - 7 + 58 - 12 - 71) - 13 + 86 + 8 - 21 + 71 - 50 - 70 + 93 + 3 - 32) - 65 - 68 - 37 + 90 + 66) - 1 + 27 - 9 + 68 - 91 + 48 - 67 + 84 + 26 + 54 + 24 - 59 - 50 - 60 - 51 + 9 + 52 + 13 - 55 - 94 - 84 - 56 - 99 - 48 - 24 - 72 - 92 + 91 + 29) + 43 + 44 + 39 + 3 - 10) + 41 + 72 - 6 - 68 - 29 + 47 - 3 + 38 - 8) - 64 + 28 - 100) - 5 - 9 - 35 - 49 + 44) - 89 - 31 - 54 - 22 + 10 + 8 + 42) + 35 + 78 - 16 + 29 + 84 + 6 - 31 - 19 + 85 + 23 + 41 - 73) + 92 + 34 + 62 + 19) - 89 - 27 + 95 + 39) - 24 - 50 - 87 + 61 + 34 - 45 + 44 - 100 + 64 + 87 - 80 - 25 - 58 - 95 + 37 + 84 - 91 - 72 - 29) - 52 + 4 + 94) + 35 + 94 - 27) + 18 - 48 - 81 + 25 - 63 - 78 + 33 - 74 - 70 + 34) + 3) - 67 + 86 + 24 + 9 - 18 - 68 - 81 - 45 - 83 - 36 - 63 + 62 - 34) - 6 + 94 + 70 - 47 + 52 - 18 - 17 - 55 + 86 + 62 - 10 - 85 - 47 + 7 - 56 - 41 - 38 - 82 - 35 - 33) + 67 + 75 + 76 - 87 - 92) - 94 + 85 - 32 + 72 - 72) + 68 - 32 + 62 + 28) - 46 + 65 - 91 + 38 - 37 + 36 + 27 - 50 - 80) + 38 - 49 + 93 + 50 - 10 - 22 + 86 + 89 + 96 - 66 - 93 - 8 - 26 + 4) - 43 - 74) - 48 + 39 + 82 - 25 - 92 - 84) + 72 + 73 + 88 - 97 - 56 - 99 + 18 + 45) + 44 + 40 - 61 + 96 - 38 + 17 - 21 + 23 - 51 - 18 - 89 + 65 + 77 - 86 + 9 + 82 - 62 + 40 - 22 - 59 + 47) - 47 - 4 + 14 - 46 - 51 + 90 - 48 + 28 + 11 - 13 - 66 + 74 + 18 - 74 + 89 + 27 - 10 - 22 + 83 - 46 + 60 + 58 + 13 + 23 - 81 + 31 + 11 + 98 + 46 - 23) + 30 + 62) - 63 + 39 - 94 + 30 - 16 + 41 + 40 - 51 + 12) - 29 - 51 - 16 - 65 + 26 + 99 - 5 - 61 - 55 + 89 + 79 + 32 + 59 - 28 + 19 + 78 - 7) + 98 + 90 - 4 - 38 - 12 + 99 - 47 + 73 - 42 + 79 + 82 - 55 - 45) - 0 - 30 + 60 + 24 - 35 - 32 - 89 - 13 + 60 + 77 - 77 + 89 + 48 + 30 - 26 - 87 - 35 + 67 - 3 - 81 + 42 - 2 - 96 + 90 + 29) + 43 + 2 + 84 + 72 + 99) + 84 + 44 - 9 - 89 - 89 + 78) + 25 + 65 - 32 + 60 + 36 + 20 - 82 + 3 + 15) + 92 + 85 - 55 - 20 + 0) + 32 - 64 - 48 - 25 - 62 - 49 - 86 - 74 - 3 + 57 - 76 + 46 - 94 - 67 + 59 - 80 + 12 - 50) - 37 - 63 - 74 + 22 + 81 + 57 + 80) + 67) + 16 - 21) + 62 - 66 - 43) + 60 - 63 - 16 - 64 + 44 + 66 + 26) - 100 + 27 + 37 + 86 - 47 + 78 + 49 - 42 + 41 - 98 + 82 - 26 + 43 + 92 + 59 + 29 - 78 - 57 - 38 - 64 + 13 - 46 - 89 - 91 - 59 + 68 + 40 + 33 - 61 + 20 - 40 - 41 + 97 - 86 - 13 + 93 - 86 - 61 + 84 + 88 - 1 - 6 + 95 - 72 - 62 - 59 + 29) - 33 - 37 + 65 - 42) - 55 - 26) - 79 - 39 + 19 - 62 - 80 + 53 - 39 + 39 + 23) - 50 + 51 - 33 + 5 - 42 - 53 + 90 - 91 - 18) - 50 - 3 - 60) + 58) + 72 + 36 + 73 + 14 - 47 - 99 + 66 + 89 + 45 + 27 + 51 + 52 + 23 + 39 - 96 + 65 - 73 - 33 - 99 + 97) - 38 - 8 - 3 + 96 - 43 - 38 - 82 + 42 + 8 + 50 + 15 + 10 - 95 + 11 + 67 - 89) - 37) - 98 + 24 - 88 - 89 - 52 + 41 - 77 + 12 - 44 + 68 - 92 + 1 - 6 - 23 - 33 - 63 + 83) + 20 - 33 + 56 - 11 - 100 - 61 - 68 - 46 - 78 + 41 - 65 + 76 - 33 + 8 + 1 + 28 + 41 - 6 + 65 + 54 - 61 - 75 + 51 - 72 + 92 - 8 + 74 + 26 - 56 - 97 - 19) + 68 - 11) + 66 - 24 + 28 + 23) + 94 + 55 + 28 + 91 - 55) - 65 - 67 + 22 + 39 - 66) + 54 - 83 - 60 + 55 + 89) + 57 - 60 + 44 + 62 + 50 - 92 + 37 - 95) - 32 + 18 - 60 + 96 + 65 - 96 - 4 - 14 + 89 - 32 - 50 - 39 + 5 - 62 + 72 + 68 - 96 + 74 + 68 + 55) - 74 + 22 + 6 - 66 - 40 - 76 - 57 + 68) + 25 - 53 + 24 - 74 + 80 + 74 + 58 - 34) + 91 - 77 + 90 + 41 - 83 - 47 + 74 - 89 + 47 + 93 + 27 + 77 - 59 - 77 - 10 - 18 - 87) - 38 - 52 - 55 + 6 + 14 - 25 + 18) - 10 - 50 - 35) - 78 - 85 - 74 + 83 + 2 - 38 - 1 + 22 - 11 - 9 - 82 - 62) + 63 - 2 - 95 + 42 + 81) - 80 - 89 - 2 - 51 + 5 - 12 + 10 - 79 - 4) + 65) - 66 - 70 - 41 + 92 + 57 - 53 - 47 + 64) + 70 + 96 - 26 + 92 - 88 + 36 - 38 - 10 + 22 + 84 - 36 - 26) - 69 + 55 + 51 - 78 + 48 - 37) + 20 - 79 + 55 + 26 + 70 + 3 - 1 + 32 - 43 - 71) - 25 - 93 + 46 + 75 + 56 - 61) - 10 - 19 - 39 - 43 + 74 + 58 + 73 + 39 + 82 - 49 + 75 + 98 + 25) + 37 + 21 - 79 + 3 - 9 - 70 + 30 + 77 - 56 - 45 + 57 - 20 - 54 + 59 + 67 + 77 + 17 - 74) - 25 + 43 - 99 + 17) - 38 - 5 - 91 + 39 + 70 + 14 - 69 + 83 - 47 + 99) - 28 + 5 - 59 - 33) + 87 + 66) + 16 + 44 - 42 + 39 + 6 + 83 - 73 - 84 + 60 + 75 + 19 + 66 + 98 + 6) + 15 + 34 + 61 - 7 + 26 - 88 - 35 - 14 + 46 - 29 - 4 + 63 - 54 - 15) - 86 - 81) + 67) - 68 - 29 + 39 - 19 - 34 + 76 + 1 + 20 + 83 - 60 + 84 - 82 + 74 + 90 + 99) + 73 + 1 - 45) - 10 + 16 - 14 + 93 - 61 + 23 - 9 + 23 - 35) + 89 - 47 - 95 + 64 - 59 - 10) - 6 + 59 + 72 - 85 - 39 - 86 - 71 + 51 + 12 - 9 - 22) + 84 - 35 - 100 - 24 + 39 - 14 + 60 - 85 - 38 + 28) - 15 - 74 - 32 + 81 + 38 - 24 - 52 + 17 - 21 + 51 - 100 + 94 - 67 - 26 - 93 + 21 + 37 + 82 + 80) + 24 - 42 - 10 + 15 - 34 + 70 - 10 + 96 + 19 + 0) - 33 + 84 + 36 + 49 + 96 + 61 + 14 - 67 - 94 + 16 + 74 - 64 - 80 - 49 - 19 - 11 - 9 - 3 - 5 - 41 + 29 + 7 + 94) + 33 + 18 + 22 + 63 - 67 + 55 - 81 + 26) - 31 - 36 + 48 - 4 + 75 - 54 + 88 - 61 + 24 + 69) - 80 + 5 + 72 + 78) - 41 - 56 - 87 - 42) - 22 + 96 + 29 - 62 - 18 - 58 - 95 - 8 - 31 - 94 + 83 + 32 + 65 - 46) + 11 - 13 - 91 + 82) + 50) + 46 + 84 - 27 - 80 + 22 - 90 + 71) + 73 - 13 + 4 + 40 + 69 + 44 - 33 + 2 - 80 + 12 - 69 + 97 + 56 - 77 - 84 - 7 - 61 - 21 + 29 + 85 - 88) - 64 + 63 + 49 + 35 + 14 - 12 + 91 - 45 + 43 - 23 - 88 - 47 - 98 - 65 - 72 - 49 - 91 + 88 + 73 - 29 + 55 - 25 - 5 - 30 - 79 - 5 - 95 - 32 + 73 + 40 - 98 + 60) - 13 + 14 + 65 + 30 + 88 - 58 + 91 + 43 - 40 - 69 - 92 - 69 + 52 + 88 + 18 - 80 + 47) + 81 - 34 + 19 - 12 - 70 + 89 + 6 - 95 + 95 + 87 + 51 - 69 - 4 + 25 - 83 - 30 + 82 + 22) + 77 + 90 - 8 + 47 - 1 - 90 - 49 - 24 + 49 - 76 + 10 - 90 - 57 + 92 + 86 - 37 - 49 + 98 + 89 + 91) - 12 - 76 - 79 - 45 - 6 - 75 - 51 - 56 + 12 + 0 - 3 + 17) + 97 - 85) - 90 - 52 - 55 + 49 - 87 + 98 - 92 + 23 + 26 + 17) - 32 - 26 + 15 + 63 + 12 + 80 - 71 + 9 - 22 + 98 - 8) - 66 - 6 - 54 - 76) - 90 - 70 - 15 - 41 - 71 - 85 - 23 + 96 - 18 + 92 - 35 - 93 - 82) + 21 + 38 + 100 + 2 - 24 - 69 + 84 - 87 - 27 + 30 + 84 + 50 - 84 + 0 - 73 + 77 - 18 - 28) - 6 - 2 - 85 - 68 + 24 - 54 + 14 - 69 - 87 + 40 + 100 - 16 + 48) - 45 - 78 - 77 + 39 - 4) + 34 - 90 - 92) + 59 + 11 + 51 - 23 - 90 + 62 - 49 + 0 - 79 - 43 + 74 + 4 + 68 + 29 + 65 - 36 + 33 - 25 - 91 + 98 - 38 - 45 + 31 + 75 + 1 + 52 - 46 - 20) - 3 + 45 + 53 - 7 + 53 - 95 + 92 + 15 - 72) + 17 - 62 - 52 - 90 + 99 - 94 - 96 + 74 - 9 - 14 - 15 + 78 - 67 - 19 + 79 - 43 + 15 - 12) - 45 - 75 - 18 + 87 - 50 + 79 - 80 + 10 + 99 + 1 - 18 - 100 - 92 - 4 + 87 + 79 - 98 - 32) - 62 - 60 + 7 - 77 + 19) - 59 + 99 - 49 - 59 - 100 + 60 - 75 + 90 + 51 + 39 - 99 - 94 + 85 - 13) - 46 - 94 + 83 - 73) - 89 - 51 - 68 - 24) + 87 - 94) + 92 - 85 + 90 + 26 - 23 + 87 - 18) + 40 + 21 + 93 - 67 + 87 - 89 - 25 + 92 - 97 - 36 - 4 + 32 - 79 - 42 + 59 - 85 - 10 - 35 + 82 + 64 - 10 + 30 - 15 + 34 - 22 - 79 - 41 - 34 + 24 - 11 - 71 + 18 + 55 + 94 - 0 + 13 - 8 + 9 - 81 + 32 + 89 + 9) + 83 + 15) + 42 - 98 + 35 + 41 + 57) - 10) - 30 - 73 - 2 - 20 - 91 + 36 - 44 - 71) - 65 + 86 + 11 + 33 - 9) - 51 + 60 - 55 - 50 + 14 + 38 + 53 + 80 - 34 - 45 - 39 - 92 + 50 - 86 + 40 + 27 + 28 - 77) - 69 + 42 + 64 - 20 + 3 - 79 - 9) + 80) - 72 + 19 + 31 - 35 - 0 + 87 + 74 + 85 - 40 + 19 - 66) + 27 - 9 - 47 - 52 + 85 + 97 + 21 - 61 - 95 - 55 - 94 + 39 + 54 - 97 + 17 + 30 - 47 - 82) - 23 + 56 + 33 + 43 + 59 + 11 + 13 - 71) - 11 + 65 + 26 + 94 - 82 + 41 - 61 + 35 + 88 - 55 + 12 - 72 - 83 + 60 + 80 - 68 - 79 - 10 + 17 + 11 - 65 - 100 + 61 + 48 + 17 - 49 + 40 - 78 + 16 + 22 - 52 - 57 + 34 - 24 - 56 + 68) - 19 + 81 - 2 - 63 + 78 + 53 + 1 + 3 + 87 - 17 - 75 + 63 - 14 + 17 - 61 + 88 - 83 + 80 + 23 + 62 - 81 + 42 - 28 + 99 - 2 + 21 + 40 + 20 - 60 + 31 + 57 + 22 + 87 - 82 - 22 + 24 - 51 + 94 + 62 - 0 + 11) + 24 + 2) - 63 - 88 - 96 + 31 + 85 - 57 + 98 - 59 - 4 - 59 + 61 + 91) + 89 - 62 - 49 - 28 - 77 + 83 - 68 - 75 - 36 - 3 + 52) - 22 + 13 - 53 - 39 + 20 + 36 + 60 - 74 - 17 + 19 - 18 + 47 - 69 - 39 + 8 - 22 + 17 - 42 - 74 + 82 + 24 + 64 - 49 - 33) - 95 + 51 - 19 - 16 + 100 + 31) + 13 - 1 - 59 + 67 - 81 - 75 + 13 - 14 + 36 - 59 - 87 - 51 + 53) + 33 + 89 + 40 + 37 - 33 + 80 - 90 - 30 + 67 - 67 - 9 + 4 - 54 - 78 + 78 - 82) - 54 + 57 - 6 + 1 + 41 - 88 - 46 - 2 + 3 - 12 + 58 - 67 - 85) + 69 - 84) - 41 + 34 + 77 - 7 - 56 + 13 + 27 - 24 - 33 + 10 + 17 + 91 + 10) - 27 + 27 - 46 + 33 + 58 + 71 - 10 - 65) - 72) + 97 - 47 - 93 - 26 + 36 + 17 + 30 + 18 - 32 + 96 - 27 + 21 + 35 - 58) + 14 - 75 + 78 + 15 + 45 - 4 - 37 + 99 + 83 - 84 - 88 + 4 + 28 - 56 - 8 - 24) + 64 - 23 + 76 + 36 - 60) - 65 - 94 + 76 + 42) + 6 - 37 + 30 + 68 + 29) - 48 - 72 - 20 + 17 - 15 + 62 - 91) + 8 - 5 - 1 - 69 + 14 + 69 + 19 + 32 + 64 + 93 - 63) + 56 - 37 - 33) - 72) - 58 - 45 + 20 + 60 + 60 + 26 + 34) + 54 + 37 - 3 - 7) - 73 + 76 + 61 + 2 - 46 - 36 + 14) + 33 - 14 - 59 - 6) - 9 - 10 + 60 - 10 - 84 - 45) - 87 - 84 - 91 - 83 + 86 + 28 - 85 + 77 + 86 + 39 - 58 + 9 + 9 - 65 - 79 + 70 - 66 - 50 - 44 - 71 + 5 + 53 - 24 + 14) + 13 + 58 - 25) + 91) - 69 + 42 + 53 + 54 - 79 - 34 - 73 - 49 - 85 - 10 + 44 + 24 - 37 - 64 + 53 - 33 + 70 + 51) + 37 - 45 + 96 + 71 + 1 + 73 + 5 + 63 - 96 - 49) + 77 + 20 - 48 + 38 - 44 - 87 - 68 - 35 + 52 - 31 + 6) + 13 + 97 - 33 - 33 - 68 - 79 + 9 - 38 - 89 - 56 - 24 + 0 - 42 - 45 - 88 - 55 - 60 - 8 - 54) - 71 + 10 + 6 - 6 - 84 - 65 - 28 - 24 - 1 - 50 - 8 - 30) - 23 + 92) - 81) - 52 - 93 + 0 + 36 - 25) + 0) + 75 + 86 - 0 + 49 - 85 + 22 + 5) + 29 - 45 + 31 - 48 + 10 - 60 - 67 - 21 - 69 + 71 + 45 + 4 + 87 + 1 - 11 - 46 + 46 + 57 - 44) - 36) + 63 - 45 + 1 + 88 + 32 - 12 + 39 - 96 + 36 + 9 + 25 + 67 - 85 - 41 - 4 + 60 - 90 + 49 - 73 + 76 - 81 + 56 - 68 - 100 - 59 + 96 - 40 - 84 + 23 + 95 + 57 + 66 + 64 - 63 - 34 + 15 + 8 + 30 - 3 - 58 - 24 - 72 - 33 + 19 + 57 - 81 + 63 - 34) + 9) + 37 - 37 - 90 + 75 - 50 - 72 - 20 + 56 - 19 - 33 + 36) + 39 - 55 - 93 + 91 - 74 - 1) - 70 + 30 + 50 - 54 + 35 - 75 - 75 - 58 - 19) + 83 - 53 - 66 - 89 - 97 - 20 + 53 - 30 - 68 + 12 + 60) + 75 + 30) - 13 + 60 + 99 - 39 - 88 + 99 + 29 + 70 + 90 - 20 + 23 - 71 + 48 - 20 - 24 - 71 + 33 - 91 + 65 + 93 - 63 - 96 + 78 + 92 + 3 + 73 - 87) - 95 - 82 - 20 - 97 - 45 - 31 + 9 + 72 + 14 - 7 + 79 + 75 + 36 - 82) + 8 + 82 + 26) + 74 - 67 + 96 + 74 - 74 + 85 + 90 + 57 - 8 + 60 - 1 - 86 - 70 + 25 + 47 - 8 + 73 - 81) + 31 + 64 - 98 + 27 - 80 + 49 + 6 - 17 - 35) + 85) - 77 + 77 + 7 - 93 - 76 - 78 - 18 - 24 + 56 + 100 + 86 + 18 + 86 - 76 - 34 - 3 - 70 + 84 + 1 + 44) + 71 + 97 + 75 - 69 + 88 + 24 - 53 - 49) + 87 - 4 + 57 - 89) + 51 - 6 + 29 + 55 + 11 + 20 + 13 + 92 - 97 + 87 + 18 + 0 - 60 - 43 + 50 - 83 + 38 - 82 - 5 + 65 + 7 - 25 - 33 + 21 - 98 - 57 - 2 + 79 - 13 + 53 + 26 + 94) - 82 - 71 + 80 + 44 - 33 - 30 + 25 + 69 - 22 + 8 - 8 - 21 + 30 - 90 + 25 - 39 + 26 - 11 - 12 + 46 - 64 - 68 - 85 + 33 - 49 - 23 + 84 - 82 + 55) + 7 - 74 + 43) - 6 + 2 + 47 - 73 + 55 + 74 + 69) - 76 + 68 - 80 - 37 + 55 + 85 - 57 - 21) + 31 - 23 - 47 - 42 + 87 + 16 + 27 - 69 + 35) + 75 - 80 - 20 + 78 + 83 + 99 + 1 + 4 + 73 + 88 - 100 - 32) + 61 + 4 - 99 + 39 - 78 + 95 - 61 - 53) - 69 - 42 - 23 + 83 - 88 + 24 - 36) - 51 - 21 + 58) + 17 - 78 + 76 - 87 + 38 - 80 - 79 - 28 + 66 - 72 + 10 + 25 - 94 - 17 - 8 + 42) - 20 - 79 - 81 + 1 - 18 - 25 - 36 + 47 - 60 + 68 - 7) - 20 - 53 + 81 - 71 - 78 + 94 - 60 + 22) + 78 - 35 + 50 + 44 - 76 - 66 + 51 - 61 - 19 - 15 - 40 - 76) - 20 + 90 + 7 + 41 - 93 - 15 - 38) + 79 - 22 + 42) + 98 + 42 + 55 + 92 + 98 - 74 - 36 + 55) - 77 + 72 - 37 - 12 - 73 - 12 - 62 + 38 - 0 - 77) - 26 - 43 - 95) + 14 - 31 - 71 + 5 + 17 - 5 - 41) - 45) + 36 - 92 + 21 + 86 + 40 + 34 - 93 + 60 + 24 - 91 + 25 + 77 + 15) + 43) + 82) - 67 + 43 + 36 - 72 + 84 - 17 - 35 + 42 + 43 - 27 + 72 - 70 - 33 - 27 + 47 - 85 + 49 - 72 - 29 - 20 - 88 - 14 + 68 - 80 + 91 + 54 - 98 + 9 + 78) - 89 - 11 - 87 - 74 - 18 + 56 + 97 + 62 + 64 - 19 + 64 + 87 - 95 + 11 + 34 + 2) + 40 - 32 + 44 - 63 + 33 + 98 + 80) - 29 + 86 + 9 - 86 - 2 - 72 + 30 + 75 + 17 - 89 + 81 - 65 - 23 + 26 - 19) - 65) + 1 + 10 - 71 + 96 - 10) - 63 - 51 + 39 - 63 - 45 + 55 + 0 + 16 + 13 + 39 - 66 - 66 + 48 + 5) + 71 + 46 - 6) + 15 - 24 + 88 + 83) - 69 + 84 + 25 - 68 - 45 + 29) - 26 - 60) + 7 - 39 + 99 + 88) - 53 + 49 - 100 + 49 - 19 + 17 + 66 - 34 - 97 + 17 - 42 + 35 - 5 + 63 + 95 - 71 - 74 + 39 - 40) + 16 - 42) - 69 + 22 + 53 - 41 + 24 + 91 + 68) + 57 + 16 - 87 + 68 + 58 + 98 + 22 + 52 - 47) + 98 - 5 - 81 + 7 - 37 + 92 + 9) + 86 + 25 - 10) + 32 + 95 + 11 + 35 - 99 + 65 + 48 - 35 - 22 + 20 - 58 - 12 + 6 + 90 - 39 - 28 - 4 + 49 - 23 - 63 - 12) - 39 + 93) + 73 + 28 + 22 + 29 + 99 + 62 - 37 - 14 - 94 + 100 - 80 - 34 - 40 + 48 + 11 - 63 - 95 + 22 - 86 + 72 - 70 - 38 + 31 - 40 - 38) + 53 + 19 - 15 + 68 + 57 - 11 + 82 - 64 + 21 - 32 - 37 + 1 - 69 + 75 - 16 + 97) - 90 + 7) - 35 - 65 - 94 + 44 - 87 - 68 - 73 - 19 - 94 + 24 + 67 + 83 - 26 - 44 + 10 + 15 - 47 + 28 + 44 - 43) + 81 - 66) + 16) + 93 + 56 - 16 + 11 - 94 + 85 - 86 - 87 + 3 + 84) - 33 + 19) - 48 - 48 + 8 + 67 - 35 - 48 + 73 + 22 - 74 - 19 + 60 - 57 - 92 - 87 - 97 - 35 + 89 + 87 + 24) + 32 + 8 - 75 + 98 - 19 - 97 - 24 + 12 - 81 - 47 + 4 + 47 - 31 + 30 + 33 + 2) - 36 - 92) - 43 + 4 - 75 + 12 - 22 + 41 - 84 + 100 - 63) + 85 + 20 - 11 - 12) + 36 + 52 + 69 + 68 + 87) - 85 + 20 + 45 + 28 - 25 + 92 - 10 - 42 - 94 + 34 + 85) + 30) - 80) + 86 + 16 + 35 - 85 - 7 - 40 + 67 + 75 + 21 + 98 + 36 + 70 - 32 - 0 + 64 - 57) + 10 - 18 - 9 + 53 - 100 - 62 + 68) + 0 + 91 - 62 + 43 + 69 + 1 - 52) + 43 - 75 - 100) + 16 + 93 + 34 + 92 - 81 + 12 - 86 + 3) - 29 - 57 - 86) + 95) - 13 - 89 - 53 - 72 + 96 - 73 + 17 - 67 + 86 - 99 - 79 + 32 - 84 - 32 - 68 + 82 - 23) + 99 + 12) - 54 - 48 - 40 + 2 + 15 + 9 - 2 + 73 - 53 + 82 + 99 - 82 - 54 - 99 + 91 - 100 + 65 - 50 + 71 + 35 - 27 + 34 + 30 - 27 - 65 + 44) - 5 - 33 + 86 + 46 - 31 + 54 + 41 + 68 + 34 - 99 - 99 + 6 + 5 + 73 + 73 + 11 + 54 + 22 - 51 + 22 + 35 - 65 + 8 + 58 + 64 - 27 - 91 - 6 - 50 + 7 + 28 + 100 - 88 - 22 - 86) + 16 - 82 + 82 - 78 - 54 + 17 - 38 + 93 - 15 + 29) + 67) + 8 + 67 + 96 - 57 - 58 - 62 + 94 + 76 - 6) + 99 - 48 - 17 - 32) + 46 + 52) + 37 + 96 - 77 + 66 + 28 - 23 + 78 - 14 + 79 + 79) - 51 + 58 + 25 + 96 + 58 - 25 + 62 + 54 - 26 - 22 - 82 - 64 - 82 + 52 + 0 - 75 - 100 + 0 + 71 - 45 + 55 + 7 + 9 - 41 - 40 - 62 - 31 - 10) + 61 - 68 + 15 - 52 + 46 - 70 + 19 + 83 + 65 - 4 + 15 + 11 + 75 + 1 - 48 + 29 - 24) - 47 + 71) - 96 + 35 + 45 - 67 - 31 + 91 - 31 + 100 + 58 - 77 + 52 - 23 + 32 + 51 - 13 + 47 - 8 + 82 - 27 + 18 - 62 - 84 - 6 - 25 + 16 + 3 - 52) - 22 - 68 - 5 - 27 - 62 + 46 + 64 + 29 + 90 - 39 - 37 - 65 - 92 - 86) + 85) - 97 + 4 + 78 + 55 - 38) - 51 - 54 - 63) + 2 - 99 - 71 - 40 - 75 + 42) + 98 - 95) - 55 + 24 + 31) + 15) - 38 - 7 + 37 + 97 + 72 + 43) - 13 + 74) + 72 + 17 + 82 + 12 + 41 - 72 - 87 - 47 + 43 + 3 - 62) + 15 - 74 + 36 - 28 + 31 + 25 - 21 + 45 - 86 - 35 - 100 - 46 + 58 - 0 + 90 - 99 - 54 - 79 - 54 + 37 + 22 + 27 - 20 + 83 - 77 - 98 - 0) - 29 - 39 - 99 + 7 + 11 - 12 + 44 + 5 - 45 + 55 - 84) - 100 - 72 - 99 + 60 - 26 + 45 - 85 - 19 - 84 - 88 - 85 + 33 - 0 + 7 - 2 + 58 + 37 - 83 + 13 + 36 - 89 - 31) - 96 - 10) - 12 + 81 + 35 + 97 + 17 + 1 + 37 + 8 + 14 + 44 - 74 - 76 - 45 - 49 + 8 + 91 - 92) - 59 + 62 + 92 + 96 + 60 + 81 + 28 - 75 - 78 + 47) - 76 - 75 - 35 + 29 - 80 + 36 - 33 - 80 - 42 + 11 - 53 - 25 - 36 - 22 + 62 - 3 - 30 - 2 - 97 + 48 + 97) + 53 + 89 + 68 + 47 + 45 - 18 - 5 - 64 + 17 + 85 + 48 + 39 + 71 - 52 - 92) + 62 - 47 - 82 - 3 - 43) - 96 - 58 - 19 + 82 - 21 + 3 + 40 - 20 + 7 - 82 - 27 - 16 + 72 - 72 - 40 + 38 + 80) - 2 + 35 + 35 - 74 - 71 - 51 + 84) + 5 + 37 - 38 - 97 - 89 + 48 - 87 - 19 - 33 + 33 + 69 - 27 - 3 + 14 - 67 - 85 - 45 - 57 - 5 - 12 + 18 - 21 - 24 + 93 - 50 - 56) + 76 - 57 + 93) - 21 - 45 + 52 - 46 + 54 + 59 - 98) + 80 - 60 + 70 + 18 - 4 - 59 + 82 + 32) - 41 - 28 + 94 - 21) + 54 - 94 - 7 + 37 - 60 - 80 - 84 - 15 + 56 + 25 + 93 + 22 - 38 - 11 - 15 - 39 - 36 + 41 + 83 - 78 + 88 - 21 - 43 + 3 + 8 + 21 - 81 - 95 + 33 - 14 + 49 + 76 - 21 + 58 - 79 + 9 + 3 + 29 - 94 + 76 + 92 - 25 - 68 + 56 + 20 + 95 + 87) + 57 + 79 + 3 - 42 - 7 + 77) + 92 - 19 + 2 - 96 + 40 + 40 - 20 + 71 - 76 - 27) - 59 + 32 - 44 - 26 - 34 + 90 + 98 - 40 + 100 + 6 - 92 - 63 + 67 - 51 + 8 - 4 + 83 - 35 - 74 - 98 + 85 + 4 - 70 + 42 + 9 + 38 + 50 + 100 + 92 - 6 - 57 + 56 - 3 - 96 - 68) - 99 - 58 - 44 - 69 + 92 + 58 - 19 + 17 - 30 - 50 + 54 - 70 - 76) + 80 + 89 - 8 - 70 - 42 + 34 - 30 + 90 + 63 - 26 + 36 + 31 - 91 - 74 + 32 - 14 + 64 - 40) - 1 - 3 + 74 - 47 + 26 + 81 + 22 + 44 - 69 + 68) + 85) + 33) - 50 - 73 - 62) - 46 + 53 - 10 + 59 - 6 - 67 + 80 - 26 + 79 - 99 - 87 + 89 - 91 + 96 - 85 - 100 - 22 - 15 + 7 - 59 + 36 + 5 - 93 - 23 - 72 - 45 + 5 + 62) - 8 + 72 - 16 + 74 - 40 - 64 + 28 + 36 + 19 + 11 + 36 - 48 + 74 + 73 + 25 - 16) - 15 + 63 + 6 - 41 + 84 + 67 + 97 - 7 - 21 + 64 - 5 + 83 + 72 - 100 - 63 + 47 + 41 + 56 - 68 - 88) + 97 + 87 - 12 + 53 + 4 - 31 - 96 - 44 + 25 - 44 + 79 + 77 + 25 + 20) - 10 - 97 + 63 - 67 - 92 + 45 - 12 + 73 - 53 - 97 - 38 + 29) - 35 - 100 - 19) - 53 + 99 - 48 + 10 - 31 + 52 - 57 + 22 - 19 + 48 - 54 + 71 + 28 - 62 + 51 - 23 + 39 + 11 + 3 - 79 + 23 - 81 + 25 - 100 + 22 + 79 - 83 - 61 + 43 + 90 + 14 + 91 + 40 - 36) + 90 - 92) + 52) - 51 - 83 + 22 + 23 + 93 - 52 + 32 + 73 - 6 + 41 + 2 + 86 - 35) + 12 + 82 - 36 + 32 + 49 - 31 + 66 - 46 + 99 - 37 - 86) - 81 + 65 + 90) - 39 - 59 + 9 + 85 - 96 - 27 - 13 - 59 + 14 - 27 - 67 - 2 - 24 - 9 - 8) + 19 - 39 + 99 + 58 + 87 + 72 - 41) + 26 - 38 + 84) - 60 - 92 + 34 + 39 + 82 - 47) + 92 + 14 + 41 - 6 - 88 + 21 + 12 - 28 + 13) - 36 - 13 + 36 - 35 - 20 - 8) - 69 - 67 + 69 - 59) + 61 + 14 - 31 + 70 + 41 - 24 + 2 + 43 + 40 + 68 + 37) - 71 + 72 + 21 + 80 + 95 + 15 + 54 - 87 - 62 - 47 + 6 + 72 + 58) + 57 - 12 - 75 - 51 + 36 - 49 - 2 + 94 + 6) - 14 + 27 + 95 + 22 - 37 - 65 + 75 - 59 + 13 - 31 - 67) + 17 - 72 + 62 + 50 + 45 - 85) + 28 + 51) - 18 - 35 - 35 - 50 + 96) + 31 + 64 + 29) - 1 - 28) + 10 - 58 + 85) - 22 - 94 + 48 + 61 + 71) + 96 + 59 + 44 + 6 - 97 + 28 - 34 + 5 - 99) + 56 - 100 - 3 + 99 - 23 - 20 - 62 - 100 - 18 + 3 + 26) - 29 + 74) + 76 - 71 + 64 + 46 + 26 + 76) - 35 + 85 - 60 - 69 + 56 - 5 - 0 - 33 + 50 - 64 - 36 - 32 - 66 + 62) - 63 + 95 - 100 + 96 - 44 - 6 + 32 + 69 + 96 + 78 - 12 - 9 - 44 - 85 + 16 + 90) + 85 + 14 + 81 - 69 - 58 - 89 - 67 - 79 + 50 - 25 - 0 - 35 + 64 + 44 - 75 + 53 + 11 + 68) + 94 - 58 - 23 + 88 + 84 - 72 - 83 + 19 - 45) - 58) - 62) - 68 + 84 + 58 - 50 + 88) - 9 + 6 - 93 + 72 + 31) + 94 - 95 - 20 - 24) + 25 - 56 - 74 - 47) + 91 - 9) - 33 + 91 + 43 + 95 - 91 - 31 - 86 + 2 - 23 + 82 - 2 - 74 + 25 - 0 - 24 + 71 + 76 - 51 + 86 - 27 - 54 + 63) + 48 - 88 - 1 + 92) - 96 - 38 - 95 + 92 + 70 - 41) + 88 + 97 + 28 + 92 + 82 + 18 + 68 + 14 - 71 - 3 + 86) - 79 - 86 + 45 - 84) - 12) - 39) + 100 - 64 - 16 + 21) + 68 - 55 + 75 + 55 + 45 + 17 + 66 - 63 - 98 - 5) - 74 + 52 + 85 - 24 + 25 + 58 - 51 - 49 - 69 - 11 + 87 - 45 - 12 - 56 + 71 - 62 - 92 + 96 + 83 - 45 - 8 - 48 + 34 + 73 - 37) - 62 - 92 + 55 + 2 - 37 - 39 - 32 + 52 + 56) + 41 - 23 - 27 + 59 + 2 + 39 - 94) + 60 + 86 - 56 - 25 - 41) - 42 - 26 + 50) + 53) + 90 + 60 - 17 + 13 + 64 + 61 - 87 + 23 - 97 + 38 + 88 + 17 - 12 - 9 + 77 + 20 - 81 - 97 + 81 - 85 + 4 - 48 - 89 + 43 + 36 - 67 + 16 + 4 - 83 + 95 + 65 + 26 + 14 - 57) - 67 + 10 + 59 + 42) + 71) - 49 - 69 + 96 - 54 - 49 + 36 + 32 - 50 + 85 + 1) - 52 + 45 + 59 + 46 + 4 + 53) + 21 - 55 - 82) - 92 - 85 + 26 + 65 - 61) + 66 - 28) - 52 + 74) - 76 + 2 - 86 - 62 + 88) - 5 - 24 + 38) - 20 - 70 - 23 - 43 - 45) - 64 - 90 + 98 + 62 - 61 - 26 + 39) + 75 - 23) - 49 + 48) + 60 - 30 - 52 - 57 - 73 - 57 - 47 - 82 - 23 + 29 + 65 - 19 + 52 + 31 - 47 + 41 - 8) + 48) - 52 + 21 - 19 - 99 + 89 - 96 + 35) + 46 - 76 + 88 + 48 + 35 - 9 + 4 + 48 - 42 + 36 - 1 - 10 + 92 + 48 - 57 - 33 + 74 - 12 - 96 - 53 - 87 - 69 - 12 + 62 + 15 + 33 + 29 + 15 + 96 + 83 + 82 + 93 + 47 + 62 - 22 + 19) - 39 + 20 + 72 + 51 + 98 - 31 + 80) - 97 - 77 - 94 + 54 - 23 - 43 - 71 - 50 - 31 + 1 + 52 - 34 + 93) - 34 - 81 + 82 + 72 + 47 + 91 - 17 - 38) + 47 - 50 - 49 - 50 + 72 - 68 + 3 + 43 - 20 - 31 + 98 + 45 - 35 + 14 + 77 + 73 + 10 + 41 + 41 + 49 - 5 - 58 + 37 + 36 - 94 - 64 - 33 + 36 - 73 - 1 - 70 + 21 - 21 - 86 - 46 + 99 - 73 - 36 + 91 + 28 - 39 + 52 + 32 - 100 - 94 - 66 + 48 + 59 - 47 - 99 - 9 - 70 - 89 - 22 - 82 + 51) + 20 + 21 + 72 + 16 + 7 + 49 + 34 - 61) - 44 + 11 - 17) + 46 - 1 - 91 + 67 - 52 - 63 - 90 + 9 - 95 - 86 - 97 + 22 - 95 + 8 + 95 - 3 - 81 - 58 - 65 + 53 - 24 + 42 - 53 + 26 - 52 - 37 + 14 - 64 - 40 - 69 + 8 - 97 + 47 - 74 + 87 + 82 + 37) - 59 - 2 + 77) + 46 + 23 + 68 + 8 - 30 + 45 - 52 - 36 + 0 + 11 + 45 - 38 + 69 + 98 + 36 - 67 - 63 - 10 - 73 - 32 + 5 - 86 + 35 - 1 - 45 + 21 - 94 + 2 - 83 - 92 - 57 - 11 + 51 + 31) + 98 - 20 - 28 - 61 - 17 - 33 - 60 + 43 + 54 + 97 + 63 + 64 + 85 + 62 + 20 - 83 - 52 - 39 - 54 + 88 + 39 - 50 - 64 + 53) - 15 + 86) - 65 - 4 + 83 - 1 + 24 - 65 + 87 + 64 - 46 + 31 - 82 + 99 + 17 - 26 - 15 - 38 - 61) + 38 + 36 - 23 + 90 - 23 + 26 + 47 + 23 + 72 + 32 - 69 + 29 - 84 - 41) + 83 + 40 - 22 - 77 - 91 + 23 - 100 + 52 - 79 - 43 - 49 - 30 + 62) + 100 - 58 + 21 - 2 - 70 + 3 - 69 - 63 + 16 + 81 - 51 - 97 + 91 + 30 - 79 - 55 + 61 + 75 - 3 + 94 - 81 + 3 - 8 + 74 - 0 + 30 + 32 + 23 - 16 + 66 + 5) - 19 - 15 + 11 + 4 - 16 - 96 - 50 + 57 - 40 - 98 - 50 - 79 - 38 + 24) - 85 + 70 + 39 - 93 + 71 - 48 + 2 + 25 + 63) - 53 - 47 + 60 - 65 - 11 - 59) - 64) + 52 - 19 - 16 - 13 + 1 + 70 - 99 + 88 + 69 - 15 - 35 + 74 + 20 + 32 + 51) + 75 - 83) + 93 - 31 + 98 + 6 - 32 - 99 - 55 + 0 + 6 + 5 + 3 + 44) - 54 - 9 + 44) + 20 - 76 - 15 + 39 - 63 - 40 + 55 - 99 + 90 - 41 - 75 - 96 - 51 - 56 + 64 - 1 + 29 - 30 + 20 + 14 - 6 - 48 + 52 + 85 + 89 - 46) + 66 + 3 + 89 + 50) + 37 - 91 + 86 - 68 + 62 + 91) - 12 - 2 + 4) - 66 + 14 + 67 + 6 + 48 + 12 - 74 - 55 + 39) + 51 + 96 + 9 - 94) - 78 - 43 + 29 + 68 - 16 + 15 + 58 + 26) - 82 + 63 + 0 - 40 + 51 - 70 - 1 - 19 - 8 - 58 + 27 - 99 + 69 + 40) + 19 - 64) + 83 + 10 + 54 - 87) + 31) + 16 + 76) + 54) - 56) + 80 + 66 + 29) + 4 - 45 + 87 + 42 - 84 - 94 + 74 - 76 - 85 - 78) - 73 + 49 - 1 + 4) + 54 + 54 + 48 + 44 - 95 + 64 - 6 - 26 + 100) - 47 - 65 + 10 + 79 - 47 + 36 - 22 + 23 + 85 + 48 + 12 + 99 + 10 - 70 - 49 - 28 - 20 + 52 + 58 - 27 + 19) - 80 + 39 + 79 + 67 - 71 - 81 + 7) - 94 + 27 - 81 - 74 + 62 - 95 - 99 - 8 + 6 - 21 - 75 + 0 - 72 + 93 + 18 - 13 + 99 - 52 + 98 + 76 + 80 - 95 - 7 - 2 + 76 - 58 - 53 + 52 - 57 - 72 + 100 + 40 - 72 + 64 - 20 + 96 + 88 + 63 - 7 + 11 + 56 + 0 - 86 - 22 - 79) + 39 + 22) - 59) + 63 - 79 + 86 - 97 - 55 + 9 - 59 - 10 + 87 + 40 + 68 - 47 + 48 + 24 - 43 + 68 + 53 + 47 - 25) + 64 - 70 + 71 - 98) + 67 + 68 - 97 - 49 - 32 + 96 - 45 - 74) + 60 - 0 + 51) + 60 + 83 - 80 - 8 + 83 + 22 + 55 + 9 - 55 + 96 + 21 + 90 - 75 - 100) - 83 + 56 + 90 + 54) - 79 - 7 + 93) + 21 - 44 + 87 - 48 + 90 - 52 - 4) + 69 - 76 + 34 - 82 + 78 + 60 - 78 - 67 + 18 - 43 - 1 - 66 - 95 + 7 - 76 - 56 + 50 - 73 + 82 - 59 + 87) + 57 - 68 + 28 + 37 - 32 - 22 + 28 + 39 + 64) - 72 + 91 - 37 - 2 + 8 + 36) - 26 + 79 - 94 + 25 - 37) - 54 + 31 - 69 + 11 - 50 + 91 + 90 - 23) + 0 + 44) - 97) - 73 + 35 + 14 - 16) - 5 + 94 - 87 + 42 + 65 + 3 + 37 - 20 + 97 - 79 - 20 + 85 - 34 + 35 - 16 + 48 + 35 + 51 + 61 - 27 + 93 + 11 + 82 - 85 - 54 - 4 - 27 + 94) + 16 + 48 - 38 + 9 - 82 + 68 + 16 - 0 + 24) + 17 - 86 + 37 + 9 + 51) + 21 + 56 - 34 - 19 + 99 + 73 - 70) + 85 - 74 + 38 - 83 - 53) + 79 + 40 - 60 - 70 - 17 + 7 + 80 + 46 + 73 - 14 + 95 - 96 - 64 - 30 + 14 + 58 - 92 + 32 - 91 - 71 + 18 - 75 + 21) + 98 - 42 - 95 - 10 + 28 + 21 + 55 + 11 + 48) - 53) - 59 + 21) + 80 - 40 + 77 - 64 + 14 - 76 - 34 + 32 - 3 - 48 + 69 - 44 - 20 + 69 - 32 + 43 + 79 + 3 + 95 - 70 + 8 - 86 - 19 + 87 - 18 + 98 - 28 + 64 + 6 - 23 + 68 + 62 - 11 - 23) + 0 - 33 + 24 + 34) + 28 + 56 - 59 - 87 - 61 + 51 + 12 - 57 + 0 - 54 + 2 + 79 - 44 + 57 - 39 + 67 + 96 - 54 + 25 - 37 + 79 - 96 - 65 + 50 - 27) - 31 - 64 + 4 + 45 + 64 - 47) - 100 - 89 + 68 + 50 + 29 + 65 + 2 - 50 - 11 - 17 - 37 - 13 + 36 + 27) + 28 + 36 - 73 + 10 - 24 - 93 - 85 + 81 - 15 - 35 - 8 + 79 + 69 - 88 + 54 + 62 - 2 + 62 + 45 - 8 - 72 + 16 + 23) + 62 + 55 - 50 - 24 - 38 + 39 + 19 - 90 + 59 + 18 - 52 + 67 - 35 + 55 - 15) + 85 - 62 - 29) - 86 + 28) + 34 - 17 + 72 - 93 + 75 + 6 - 46 - 14 + 62 + 55 + 26 + 53 + 37 + 8 + 12) - 11 - 17 - 78 - 25 - 97 - 17) - 1 - 22 - 19 - 48) + 86) + 11 + 91 - 97 - 9 + 8 + 65 + 66 + 26 - 11 - 41 - 43 + 90 - 60 + 97 - 5 + 52 - 100 - 40 - 65 + 40 - 11 + 75 - 81 - 78 - 74 - 9 + 71 - 95 + 33) + 94 - 99 + 52 + 75 + 42 + 94 - 27 + 76 - 9) + 34 + 99 + 95 + 79 + 74 - 48 - 11 - 12 - 38 + 92 - 74 - 31 + 60) + 72) - 55) + 71 - 96 - 50) - 72 + 44 + 53 + 90 - 89 + 100) + 21 - 29 + 71 - 11 + 36 + 14 + 37) - 7 - 12 + 43) - 50 - 47 + 7 - 39 + 60 + 60 - 6 + 99 - 21 - 45 - 86) + 40 - 56 + 33 - 9 - 98 - 26 - 66 - 12 - 32 - 21) - 15 - 84) + 69 + 49 + 99 + 86 + 35 + 82 + 88 - 60 + 49 + 19 - 77 + 80 + 31 + 24 - 58 - 73 - 23 + 84 - 53 + 13) + 17 - 99 - 51 - 70 - 85 + 71 + 52 - 53 + 25) - 21 + 66 + 70 - 42 - 9 - 2 + 36 + 91 - 69 + 36) + 38 + 12 + 64 + 30 + 48 + 96 + 67 - 84 + 83 - 17 + 35 + 95 - 74) + 42 + 0 + 92 - 54 + 17 + 84 + 83 + 67 + 50 + 56 - 93 + 93 - 22 + 31 + 89 + 43 + 23 - 76 - 34) + 27 + 96 + 84) + 34 - 96 - 80 - 57 + 97 - 21 - 72 - 52 - 86 + 55 + 85 - 82 - 33 + 22 - 99) + 93) - 42 + 65 - 37) - 58 - 28 - 91 + 91) + 59 + 38 + 32) - 86 + 50 + 57 + 6 - 18 - 51 + 98 + 66 - 51 + 37 + 43 - 59 + 27 + 22 - 100 + 15 - 20 - 55 - 50 + 42 - 9) + 36 + 85) - 46 - 86 + 98 + 61 + 29 - 31 - 28 + 77) + 35 + 66 + 46) + 45) + 84) + 84 + 6 - 4 + 54) - 38 + 58 - 1 - 73 + 35 - 38 - 32 + 46 + 30 + 0 - 0 + 64 - 22 + 14 - 45 + 70 + 21 - 32 - 58 - 38 + 22 - 40) + 78 - 39 - 63 - 73 + 12 + 54 + 80 - 87 - 57 - 69 - 23 + 50) - 35 + 92 + 5 - 37 - 26 + 95 - 96) + 0 + 52 + 35 - 16 + 39 + 96 - 64 - 41 + 18 - 28 + 45 + 31 - 99 + 45 + 24 - 35) - 69 + 47 + 81 + 91 - 30 - 59 - 59) - 26 + 63 - 59 + 77 - 31 - 30 + 20 - 44 - 3 + 21 - 85 + 71) + 80 - 69 + 56 - 11 + 97) - 26 + 24 + 5) - 38 - 71 - 97 + 100 + 50 + 17 - 28 + 85 - 44 + 58 + 46 - 78) + 5 + 16 + 51 - 17 - 54 - 56 - 8 + 42 - 71 - 24 - 21 - 36 + 72) + 24 - 61 - 18 + 16 - 38 + 53 + 33 + 85 - 63 + 80 - 17 - 52 + 5) - 60 - 68 + 67 - 79 - 37) + 78 + 85 + 57 - 41 - 20 + 29 + 28) + 79 - 86 + 74 + 15 - 52 - 6 - 99 + 1 - 50) + 62) + 71 + 24 + 76 - 6) - 0 - 7 - 41 - 46 + 35 + 35 - 6 - 11 - 83 - 78 - 98 + 45) + 85 - 37 - 25 - 32 + 21 + 98 - 52 - 2) - 4 - 69) - 11 - 77) + 59 - 42 + 6 - 19 + 76 - 21 + 18 + 62 + 35 + 29 + 55 - 51) + 17 + 56 - 44 - 32 - 59 + 81 - 54 - 89 - 72 + 25 + 41 - 69 - 23 - 55 - 79 - 55 + 25 + 63 - 0 + 12 - 79) - 84 + 88 - 7 - 17 + 51 - 34 + 94 + 92 - 94 - 11 + 60 - 56 - 30 - 61 + 20 + 19 - 38 + 64 - 48) - 25 - 36 - 53 + 18 + 98 + 80 - 16 + 62 + 8 - 77 - 98) + 21 + 83 - 66 + 76) - 87 - 43 - 45 + 64 + 69 - 47 + 49 + 66 + 55 - 31) - 67 - 92 - 54 - 46 + 90 + 15 + 32) - 95) - 94 - 32 + 83) - 23 - 83 - 54 - 87 - 54) - 3 - 41 - 61 - 98 - 52 - 69 - 36 - 26 - 65 - 18 + 75 + 27 - 98 + 48 + 62 + 36 - 66 + 27 + 43 - 65 - 6 + 96 - 86 + 25 - 39 + 70 + 78 - 46 + 55 - 66 + 72) - 88 - 62 + 21 + 67 + 99 - 6 - 6 + 70) + 81 + 51 - 38 + 78 - 53 + 90 + 27 - 13 + 89 - 27) + 2 + 37 + 89 + 76 - 10 - 60 + 30 + 33 - 58) + 10 + 51 + 50 - 20 - 37 - 20 - 43 - 37 + 80 + 76 + 72 - 24 - 92 + 7 - 39 + 32 - 82 + 55 - 66 - 100 + 73 + 82 - 62 - 53 - 47 + 15 - 45 + 1) + 69 + 64 - 76 - 57) - 23 + 46 - 76 + 63 + 71) + 77 - 43 - 75 - 41 - 91 + 52 - 46 - 6 + 33 - 64 - 35 + 15 + 98 + 52 - 41 - 69 + 69 + 42 + 72 - 19 + 16 - 98 - 75 - 89) - 23 - 46 - 28 - 95) - 14 - 6 - 50 - 40 + 84) - 13 - 86 + 9 + 41 + 63 + 53 - 34 + 25 + 95) - 74 + 73 - 24 - 59 + 52 + 29 + 53 - 3 - 31 + 66) + 86 + 6 - 10 + 41 - 14 + 76 + 67 - 50 + 99 - 80 + 48 - 8 + 49) + 53 - 46 - 4 - 11 + 16 - 7 + 38 + 4 + 48 - 7 + 5 + 69 + 1 - 82 + 90 + 14 + 95 + 95 + 73 - 36 - 20 - 21 + 30 + 11 + 83 + 90 - 30 + 50) + 82 - 15 - 25) - 19) + 27 - 99 + 50) - 35 + 90) + 82 + 35 - 51 + 4 + 90 - 85 - 86 + 36 + 71 - 85 + 18 - 32 + 25) + 36 + 45 - 90 + 18 + 14 + 16 + 35 - 97) - 74 + 33 + 17 + 2 - 92 + 34 + 23 - 13 + 65 + 97 - 49 - 61) - 18 + 89 + 76 + 46 + 63 - 58) + 74 - 69 - 49 + 43 + 34 + 52 + 23 - 64 + 100) - 64 - 69 + 7 - 75 + 48 + 56) - 70 + 35) - 79 + 54 + 41 + 40 - 26 + 23 - 52 + 55 - 43 - 25 - 44 + 21 - 42) - 53 + 7 + 28) - 68 + 3 + 6 - 30 - 13 - 76 - 98 + 12 + 37 - 60 - 78 - 94 + 44 - 6 - 10 + 66 + 27 - 77) - 68 + 14 - 3 + 91 - 95 - 51 + 89 + 38 - 8 - 65 - 48 + 80 - 2 - 51 - 95 - 89 - 53 + 19 + 98 + 29 - 7 - 30 - 32 - 29 - 80 + 82 + 67 + 95 - 41 + 74 + 75 - 15 - 42 + 92 - 34) + 84 - 84 + 56 - 42 + 19 + 46 - 71 + 62 + 55 + 8 - 82 - 3 + 5 + 47 + 21) + 55 + 88 + 74 - 33 + 52 - 53 - 94 + 85 - 46 + 72 - 55) + 33 + 80 - 21 - 5 + 51 + 49 - 64 + 49 + 97) - 77 + 85 - 99) + 59 + 32 + 27 - 70 + 26 - 81 + 16 - 4 + 20 + 94 - 67 + 41 - 33) - 1 + 15 - 100 + 91 - 35 - 69 - 30 + 22 - 54 - 0 + 18 + 63 - 53 - 40 - 74 - 10 - 55 - 6 - 48 + 3 - 7 - 78 - 92 - 59 - 17 + 40 - 29 - 38 - 6 + 98 + 94 + 80 - 47 - 44 - 95 + 89 - 15 - 47 + 84 - 93 - 76 - 95 - 95) + 39 - 80 + 80 + 68 + 14 + 80 + 17 + 9 + 67 - 49 + 53 + 25 - 78 - 78 - 3 + 62 - 62 - 18 - 52) - 43 - 38 - 86 + 33 + 30 + 39) + 94 - 53 + 31 - 17 + 40 - 1 + 50 + 98 - 59 + 69 + 14) + 23 + 95 + 12 + 14 - 24 - 4 - 5 - 20 + 59 + 83 - 16 - 56 + 95 + 31 - 23 - 97 + 70 - 57 - 79 + 57 + 74 + 100 - 31 - 93 + 4 + 74 - 26 - 73 - 42 + 33 - 99 + 68) + 75 - 69 + 43 + 100 - 50 + 59) + 6 - 51 + 64 - 23 + 26 + 91 - 39) + 66 + 53 - 9 + 8 + 89 - 69 - 19 + 86 - 71 - 39 - 46 - 34 - 14 + 32 + 71) + 62 - 97 - 48) + 27 - 100) - 55 + 3 + 59 - 73 + 85) + 20) - 81 + 6 + 25 - 70 - 51 + 98) - 10 + 77 + 81 - 5 - 77 - 69 + 63) + 43 - 29 - 95 + 92 - 72 - 80) + 82 + 75 + 70 + 92 - 48 - 55) + 15 + 77 + 0 + 45 - 63 - 40 + 94 + 31) + 97 + 97 - 7 + 45 - 83) - 82 + 81 - 20 - 5 - 25 + 23) - 28 + 10 + 85 + 22 - 22 - 15 - 29) - 62 + 90) + 54 - 96 + 65) + 27 - 94) - 3 - 72 + 54 + 29 - 62 + 91 - 6 + 7 - 21 + 65 - 94 + 74 + 26) + 43 + 75 - 15 + 32 + 84) - 100 - 85 - 56) - 14 + 14 + 95 + 57 - 22 + 45 - 11 - 66 - 45 + 71) + 87 + 8 + 48 - 74 + 20) + 90 + 7 - 73 + 22 - 94 - 4 + 54 + 24 + 78 - 38 + 99 + 19 + 79 - 26 + 66 - 17 - 47) + 30 + 25 + 7 + 65 + 99 + 3 - 37 + 99 - 81 + 88 + 8 - 47 - 88 + 88 + 7) + 75 - 25 + 24 + 0 - 94 + 50 + 98 - 68 + 57 - 92 + 19 - 82 - 96 - 59 - 36 - 93 + 31 - 2) - 92 - 100 + 88 - 1 - 34 + 77 - 45 - 73 + 28) + 41 - 83 - 60 + 85 + 72 - 37 - 36 - 11 - 75) - 62 - 79 + 70 + 69 + 20 + 99 - 32 + 55) + 62 + 99 + 66 + 58 - 3 - 78 - 24 - 82 - 12 - 53 - 12 + 58 + 69 + 63 - 12 - 44 - 57 - 30 + 84 - 17 + 51 - 38 - 62 + 79) - 20 - 37 + 57 + 42 + 23 + 86 - 68 + 60 - 39 - 92 - 66 - 46 + 60) - 37 - 31 - 86 - 89 - 34 - 30 - 98 - 74 - 6) + 16 - 47 - 4 + 2 + 8 - 23 + 28 + 42 - 77 - 100 - 57 + 66 + 80 + 97 + 56 + 32 + 54 - 83 - 11 + 39 - 38) - 2 - 98 - 62 - 95 + 79 + 65 - 74 + 85 + 34 + 10 + 33 + 16 + 29 - 72 - 33 - 65 - 58 + 92 - 68 - 40) - 44 - 80 + 18) - 60 + 35 + 6 + 81 + 89 - 64 + 38 + 46 + 30 - 72 - 66 + 1 - 11 + 89 - 84 - 60) - 43 - 87 + 72 + 4 - 80 + 3 - 42 + 6 + 46 - 13 + 56 + 98 - 57 - 60 + 82 - 64 - 34) - 68 + 15 - 17) + 99 - 70 + 0) + 83 - 9 + 81 + 87 + 92 + 27 + 97 - 45 + 2 + 72) + 52 + 42 - 37 - 80 - 15 + 50 + 64 - 11 + 31) - 52 + 40 - 28 - 41 + 46 - 32 + 45 + 87 + 14 + 4 + 67 - 63 - 80 + 32 - 99 + 60 + 55 + 77 + 69 + 54) + 3 + 23 + 45) + 62 + 5 - 36 + 61 + 22 - 100 - 57 + 77) - 3 + 6 - 14 + 88 + 38) - 64 - 33 + 65 + 3 + 65 - 50) + 85 - 63) + 8 - 100 - 32 - 5 + 42 - 69 - 88 - 93 - 30 + 42 + 47 - 20 - 81 + 57 - 55 - 10 - 9 - 73 + 59 + 43 - 40) - 3 + 93 - 35 + 15) + 8 - 84 + 48 + 94 - 48 - 15 - 69 + 45 + 88 + 80 - 66) - 46 - 64 - 84 + 54 + 58) + 92 - 16 + 5 + 13 - 56 + 85 + 43 + 41 - 8 + 79 - 41 + 78) - 57 + 34 + 88 + 31 - 27 + 42) + 39 - 46 + 78 - 65 - 24 - 73 - 66 + 85 + 48 + 83 - 15 + 16 - 12 - 56 + 98 - 52 - 14) + 90 - 86 + 83 + 0 - 71) + 92 - 13 + 78 - 30 - 20 + 60 + 47 + 40 - 11 - 60 + 6 - 77 + 27 - 88 - 37 - 58 - 78 + 14 - 35 + 22 + 36 - 25 + 68 + 41 - 58 + 66 - 24 + 35 - 81 - 38 - 15 - 17) - 10 - 67 + 20 + 99 + 48 - 21 - 78) - 21 - 18 - 92 + 1 + 20 + 49) - 1 - 72 + 22 - 40 + 98 + 79) - 4 + 97 + 74 - 41 - 36 - 77 - 85 + 79 - 55 + 57 - 75 - 72 + 74 + 62 - 22 - 32 + 36 - 53 + 87) - 45 + 56 - 22 - 16 - 34 + 7 - 96) + 60 + 32 - 43 + 89 + 96 - 13 - 58 - 89 + 91 - 31 + 58 + 61) + 59 - 88 - 6 + 66 - 65 + 96 + 88 - 88) - 14 + 96 - 21 - 56 + 28) - 26 + 16 - 86 - 86 + 11 + 62 + 42 - 10 - 68 - 72 - 42 - 99 + 91 - 31 - 83 + 85 + 2 + 27 - 46 - 5 + 29 - 100 - 88 + 51) - 50 + 15 - 8 + 63 + 85) - 11 - 47 - 78 - 37 - 45 - 2 + 34 + 26 + 20 + 35 + 30 - 61 - 63 - 43 + 23 - 22 - 73 - 15 - 43 - 72 + 79 + 16 + 61 - 64 + 29 + 50 + 31 - 19 + 56 + 36 - 40 - 52) - 49 - 76 - 85 - 15 - 77 - 76 - 37 - 33 + 77 - 87 + 34 - 44 + 69 + 53 - 32 - 19 + 57 + 40 + 97 + 97 - 50 + 98 - 17 - 68 - 100 - 8 + 56 + 42) - 77 + 57 + 0 + 54 + 38 - 71 + 85 - 44 - 33 - 98 + 44) + 12 - 34 + 41 + 33 - 79 - 5 + 58 - 13 - 98 - 55 - 37 + 72 + 43 + 100 + 24 + 29 - 24) + 42) + 35 - 89 - 80 + 6) + 49 + 3 - 31) + 55) + 44 - 91 + 19 - 88 + 24 + 74) + 38 - 62 + 68 + 68 + 27 + 93 + 13 - 67 - 32 + 80 + 72 - 20 - 72) + 42) + 7 + 95) - 17) + 2 + 74 - 61 + 2) + 41 + 97 - 25 - 2 - 60 - 87 + 99 + 81 + 38 - 17 - 33 - 76 + 73 - 24) + 29 + 89 - 53 - 1 + 73 - 89 + 70 - 84 - 90 + 75 + 35 + 13 - 8 + 25 + 90 - 22 - 98 - 49 - 20 + 42 - 22 + 10 + 47 + 27 - 48 - 18 - 69 + 62 - 99 + 23) + 36 + 99 - 62 + 42 + 66 - 83 + 94 - 11 + 73 + 50 + 71 - 43 + 86 + 55 + 34 + 66 + 69 + 8 + 53 - 100) + 32 + 10 + 90 - 89 - 40 + 94 - 100 + 87 + 7 + 73 - 81 + 51 + 96 + 49 + 89 + 78 - 70 + 63 - 56 + 67 + 54 - 69 + 63 + 7 - 22 - 45 - 1 + 11 - 99 - 34 + 38 + 59 + 73 - 20 - 6 + 70 + 94 + 77 + 75 - 35 - 20 + 86 + 3 - 68 - 49 - 3) - 27 + 0 - 57 - 53 - 60 - 93 + 38 + 45 - 31 - 60 - 47 + 83 + 78 - 23 - 63 + 47 + 41 + 41) - 44 - 19 + 38 + 36 - 44 - 43 - 50 + 25 + 10 + 60 - 88 + 54) + 55 + 31 - 39 - 7) - 91 + 59 + 9 - 55 + 28) - 75 + 100 + 81 - 15 + 48 - 71 + 14 + 64 + 23 + 14 - 15 + 5 + 15 + 22 + 72 - 74 - 42 + 76 + 11 - 16 + 100 + 9 + 59 + 40 - 81 + 68 + 46 - 51 - 79) + 79 + 70 + 15 + 20 + 14) + 95 + 83 - 15 - 30 - 82 + 52 - 97 + 43 + 88 - 51 - 14 - 26 - 4 + 68 - 93 + 97 - 76 - 7 + 31) + 13 - 56 - 72) + 93 + 53) + 11 - 49 + 54 + 52 - 7 + 70 - 7 - 22 - 63 - 93 + 19 + 10 + 34 + 36 - 40 + 68 - 61 - 81 + 19) + 42 + 22 - 65) - 11 + 45 + 84 - 21 + 46 - 63 + 41 - 8 - 43 + 48 - 21 + 75 - 17) - 88 + 82) + 39 + 17 + 8 - 52 + 13 - 92 + 88 - 60 - 93 - 53 + 96 + 74 + 57 + 79 - 6 + 45 - 25 - 70 + 22 - 14 - 52 - 67 - 36 + 47 - 54 - 44 + 72 + 55 - 42 - 27 - 99 - 61 - 43) + 91 - 78 - 7 + 99 - 69 + 93 - 94) + 100) - 38 - 48 + 43 + 42 + 33 + 35 + 21 + 60 - 14) + 89 + 73 + 62) - 24 + 4 + 85 + 42 + 73 - 23 + 76 + 15 + 87 + 79 + 62 + 51 + 32 - 12 + 37 + 73 - 57) + 71 - 5 + 49 + 15 - 9 + 45 + 27 - 1 + 31 + 67 - 20 + 99 + 78 + 40 - 33 + 83 - 48) - 77 + 27 + 16 - 31 - 11 + 49 + 83) - 95) + 92 - 65 + 16 + 50 - 96 + 41 + 21 - 30 + 91 - 15 + 31 - 26 - 36 + 83 + 44 + 0 + 69 - 10 + 85 + 22 - 13 + 20 - 28 + 23 - 88 + 25 - 39 - 32 + 11) + 60 + 24 + 75 - 55 - 4 - 52 + 49 + 32 + 82 - 59 - 93 - 16 + 10 - 7) + 82 + 4 + 2 + 6) + 75 - 13 - 81 + 53 + 13 + 82 + 85 - 31 + 68 + 27 + 41 + 89) + 36 - 52 + 27 + 28 + 3 + 8 - 76 - 95 - 9 + 82 + 55 + 80 + 21 + 39 - 83 + 86 + 29 - 50 + 90 - 2 + 77 - 29 + 71 - 18 - 74 - 40 - 11 + 92 - 31 - 54 - 99 - 36 + 39 - 29 - 65 + 47 - 65 + 42) + 37 + 63 + 54 - 12 - 40 + 70 - 16 + 83 - 83 - 78 - 63 + 63) + 30 - 23) + 31 + 17 + 55) + 0 + 14 + 77 + 64 - 77 - 64 + 68) + 74) + 49 - 86 - 83) - 84) + 7 + 11 - 94 + 27) + 74 + 70 - 39 + 38 + 22 - 73 + 30) - 84 - 73 - 61) + 19) + 11 + 41 + 32 - 83 - 54 + 70 - 12 - 32 - 70 + 21 + 32 + 11 - 5 - 31 + 32 + 36 + 69) - 92 + 75 - 55 + 48 - 9 + 67 - 7 + 42 - 69 - 70 - 11 + 93 - 3 + 57) + 4 + 16 - 54 + 85 + 62 + 100 - 83) - 30 - 71 + 90 + 71) - 87 + 73 + 35) + 50 + 51) + 39 - 24 - 61 + 65 - 69 - 75 + 61 + 2 - 66 + 94 - 26 + 58 + 69 + 64 - 4 + 77) - 18 - 94 + 60 - 68 + 0 + 19 - 43 - 64 - 27 + 95 + 24 - 30 + 29 + 67) - 17) - 76 - 88 + 87 - 85 - 7 + 49 + 97) - 44 - 99) - 53 - 49 + 99 - 49 + 61 - 87 + 19 + 36 + 51 + 61 + 71 - 12 + 39) + 5) - 33 - 59 + 3 - 82) - 59 + 19 + 100 - 70 - 71 - 62 - 4 - 41 + 37 + 75 + 95 + 76 + 79 - 51) - 50 + 46 - 78 - 98 + 2 + 63 - 88 + 10 + 87 - 26 - 3 + 53 - 1) + 50 - 46 - 47 + 45 + 58 - 93 - 62 + 35 + 34 - 23 - 97 - 27 - 93 - 51 + 17 + 74 - 42 + 3 + 13 + 24 + 70 - 92) - 27 - 51 + 92 + 99 + 49 - 72 + 30 + 61 + 10 + 86 + 48 + 99 + 35 + 71 + 90) + 25 - 14 + 79 + 7 + 49 - 32 - 52 - 9 - 95 - 70 + 94 + 24 - 99 - 74) - 90 - 41 + 48 + 33 + 31 + 86 - 86 - 28 - 43 + 26 + 6 + 22 + 68 - 45 + 71 - 55 + 10) + 73 - 55 - 63 - 12 - 84 - 17 - 48 - 63 - 19 - 28 + 68 - 22 + 48 + 73 + 34 + 55 - 26 - 41 - 86 - 31 - 24 + 32 - 47 + 1 - 81 - 28 - 11 + 65 - 83 + 75) - 31 - 87 - 39 - 20 + 16 - 94 - 5 + 48 - 35) - 46 + 55 + 99 + 10) - 15 + 0 - 29 - 49) - 71 + 57 - 84 - 80 - 73 - 43 + 7 - 51 - 71 + 48) - 62 + 7 - 73 - 96 - 4 + 10 + 71 - 55 + 72) - 24 + 13 + 4 + 88 + 36 + 11 - 91 - 37 - 91 + 78 + 40 + 49 - 29 - 55) + 31 - 98 - 95 + 62 - 38 + 75 + 24 + 50 - 15 - 94 - 48 + 2 - 52 + 2 - 54 + 70 + 9 - 30 - 19 + 6 - 20 + 74 + 81 + 55) + 3) + 40) + 26 - 84 + 49) + 10 + 47 + 36 + 72 - 23 - 73 - 96 + 49 - 40 + 41 - 71 + 5 - 12 + 41 + 61 - 0 + 60 + 98) - 13 - 62 + 7 - 18 - 84) + 40 - 11 - 50 - 7 - 70 - 83 + 85 - 4 + 99 - 23 - 0 - 19 + 29 - 76) + 41 - 48 + 70 + 50 + 17 - 77 + 91 - 14 + 46 - 78 - 42 + 61 + 54 + 61) - 38 + 63 + 99 + 91) + 72 - 9 + 84 - 89 + 2 + 35 - 54 - 63 - 14 - 71 - 67 + 12) - 57 + 100 - 9 - 60 - 14 + 31 + 17 - 10 - 59) + 13 + 7 - 86 + 36 + 21 + 94 - 71 - 58 + 16 - 36 + 3 - 4) + 49 - 40 + 81 - 62 + 76 + 7 + 91 + 89 + 47 + 74 + 8 + 43 - 98 + 91 + 90 + 37 + 23 + 71 + 51 - 8 - 6 - 42 + 75 - 94 + 40 + 49 - 48 + 79 - 31 + 20 - 59 - 40 + 34 + 44 - 26 - 44 - 0 - 57) - 82 - 13 - 99 + 21 + 85 + 30 - 28 - 3 + 3 + 93 + 62 - 48 + 74 + 30 + 99 - 97 - 62 - 15 - 54 - 65 - 32 - 71 - 97 - 93 - 40) - 12 - 57 - 6 - 24 + 34 + 85 + 66 - 50 + 47 - 2 - 26 + 27 + 54 + 91 + 72 - 85 + 39 + 34 - 31 + 27 - 24 + 99 + 4 + 60 + 35 - 8 - 22 + 56 + 73 - 45 + 77 - 68 + 5 + 14 - 5 + 26 + 92 + 86 + 84 + 69 + 37 - 81 + 67 - 42 + 58 + 73) + 39 - 88 + 84 + 12 - 93 + 5 + 50 + 40 - 59 - 19 - 43 + 77) - 77 - 34 + 18 + 55 - 64 - 78) - 95 + 92) - 39 - 37 - 25 - 10 - 62 + 1 + 3 + 9 + 35 + 29 - 99 + 85 + 25) + 0 - 79 - 22 + 92) + 58 - 81 - 45 + 37 + 67 - 71 - 78 - 36 + 47 - 40 - 63 + 86 - 98 + 21 - 88 - 26 + 34 - 6 + 23 - 56 + 79 + 80 + 13 + 26 - 2 - 26 - 56 - 34 - 13 + 71 + 61 - 51 - 74 + 48 - 22 - 10 + 70 + 11 - 94) + 47 - 28 - 72 - 100 + 6 + 99 + 82 - 22 + 99) + 13 + 40 + 51 - 10) - 73) + 3 - 98) + 79 + 63 + 84 - 80 - 39 + 50 + 45 + 47 - 96 + 87) - 22 - 91 - 32 - 27 - 82) - 28 + 19) + 16 + 8 + 47 - 28 - 83 + 28) - 39 - 38 - 5 - 16 + 59 - 39 + 78 + 74 + 15 + 88 - 84 + 49 + 79 + 50 + 84 + 93 + 11 - 81 - 28) - 23 - 83 + 94 + 57 - 89 + 87 - 99 + 45 - 79 + 82) + 29) - 23 + 95 - 38) + 34 + 81 - 63 - 1 - 96 + 65 - 17) + 41 - 22 - 11 + 74) + 74 - 35 - 60 - 45) - 45 + 41) + 17 - 23 + 61 - 77 + 81 + 88 - 68 - 94 + 34 - 4 - 54 - 1 + 43 + 85 + 18 - 77 - 15 - 92 + 97 - 99) + 60 + 29) - 0 + 37 + 97 + 45 + 25 - 89 - 78 - 40 - 31 - 96 + 88 + 11 - 7) + 26 - 7 + 52 + 94 + 17 + 34) - 23 - 50 - 69 + 20 - 81) - 41 - 9 + 64 - 44 + 37 - 11 - 59 - 96 - 33 - 4 + 97 - 48 + 84 + 12 + 22 - 53 + 55 + 95 + 88) + 40 + 15 - 1 + 100 + 68 + 98 - 14 + 78) + 24 - 23 - 12 + 64 + 34 - 84 + 91 + 97 - 75 + 33 - 73 + 19 + 26 - 78 - 16 - 64 + 13 + 56 - 2 - 38) + 17 + 86 - 22 + 50 - 100 - 40 + 20 - 4 - 63 - 26 + 5 - 27) - 78 - 92 - 44 - 44 + 38 + 22 + 35 + 93 + 51 + 10 + 45 + 49 + 36 + 40 - 43 + 53 - 90 + 68 + 57) + 25 - 94 - 59 + 66 - 0 - 53 - 97 + 87 - 2 + 62 + 56 + 52 - 44 - 71 + 76 - 56 + 15 - 5 + 96 + 5) + 29) + 35 + 18 + 78 + 17 - 48 - 7 + 34 + 49 - 40) + 21 + 64 - 58 - 27 - 54 + 46 + 59 - 92 - 57 - 24 - 88 - 15 - 68 + 99 - 96 - 93 + 89 + 86 + 24 + 66 + 25 + 36 + 31 + 20 + 29 - 42 + 50 - 26 - 53 - 69 + 75) - 7 + 35 + 68 - 93 - 100 + 29 + 64 - 55 + 78 + 40 + 64 + 21 + 43 + 26 + 22 + 4 - 9 - 53 - 20 - 1 + 38 + 53 + 47) + 16 + 68 - 62 + 18) + 13 + 2 + 94 + 65 + 12 + 40) - 47 + 97 - 8 + 46 - 0 + 65 + 58 + 62 + 66) - 72 + 65 - 61 - 2 - 2 + 51 - 45 - 66 + 1 - 8 - 60 - 96 + 15 + 87 - 76 - 76 + 48 + 82 - 53 - 68 - 97 + 11 + 63 - 82 + 45 + 34 + 25 - 7 + 91 - 40 + 32 + 5 - 3) + 86 + 39 + 8 + 77 + 81 + 40 + 90 - 81) + 96 + 91 - 50 + 53 + 22 - 27 + 79 + 8 - 45) + 3 + 88 - 30 - 46 + 7 + 36 - 72 - 90 + 75 + 98 - 66) + 55 + 51 + 97) + 15 - 0 + 80 + 85 - 56 - 34 - 50) - 83 + 67 - 58 - 4 + 94 + 50) + 24 + 42 + 94 - 92 - 9 + 52 - 87 - 95) - 41 + 23 + 50 + 52 - 29 + 58 - 97) + 58) - 55 - 82 - 72 - 65 + 51 - 45 - 80 + 28 + 4 + 6 + 10 + 12) - 60 + 58) - 97 - 12 - 94 + 47 + 44 + 45 - 83 + 31) + 60 + 56 + 9 + 90 + 4 + 67) - 76 + 61) - 97 - 12 + 98 - 76 - 60 - 54 - 97 - 70 + 66 + 8 - 13) + 62 - 38 - 43 + 11 - 72 + 82 + 88 - 50 - 62 - 25 + 72 + 69 + 54 + 28 - 59 - 17 - 75 + 78 - 84 + 74 - 88 - 49) - 31 - 72 + 53 - 23 + 36) - 32) + 75 + 54) + 45 + 50 - 0 + 92 - 46 - 40 - 61 + 12 - 68) + 4 + 7 + 44 - 63 + 27 + 27 - 15 - 10 + 19 - 20 + 57 - 97 - 85 - 32 + 39 - 34 + 100 - 56 - 78 - 40 + 80 - 50 - 28 + 72 + 92 + 38 - 76 + 49 + 40 - 50 - 71 - 61 - 39 - 93 - 69 + 82 - 2 + 82 + 94 + 18 + 84 + 18 - 29) + 64 + 1 - 1 - 35 + 15 - 96) + 5 - 35 - 62 - 77 + 90 + 54 - 91 + 4 + 0 - 90 + 45 + 29 - 89 + 12 + 46 - 63 - 9 + 6 - 3 + 28 - 29 - 7 - 56 + 94 - 54 + 29) + 84 - 42 + 66 - 30) + 37 - 26 + 27 - 43 - 4 - 90 - 47 + 26 - 42 + 71 + 28) + 6) - 66 + 31 - 58) + 73 + 89 + 27 - 23 - 98) - 61 - 13 + 25 + 73 - 80) - 77 - 18 + 39 + 75 - 65 - 8 + 60 + 19 - 84 - 71 - 21 - 11 - 77) + 45 + 53 + 99 + 11 + 96 + 83 + 73 + 49 + 50 + 81 + 94) + 94) - 100) - 29) - 24 + 63 - 37 + 78 - 96 - 95 - 97) - 27 + 100 + 5 + 0 + 77 + 11 + 88 + 39 - 26 + 17 + 35 + 42 + 46 - 21 + 58 + 40 - 65 - 59 - 63 - 53 - 87 + 80 - 44) + 56 + 83 - 12 + 6 + 41 - 27 + 73 + 53 - 7 + 24 - 72) + 31) + 8 + 70 + 42 - 81 - 84 - 28 - 91 + 69 + 60 + 96 - 50 + 70 - 43 + 67 + 21 - 87 - 9 + 67 - 62 + 93 - 19 - 6 + 60 + 91 + 12 - 30) + 84 + 62 - 8 - 76 - 37 + 39 - 86 + 73 + 47 + 45 - 75 + 44 - 37 - 22 - 91) + 89 - 14 - 51 - 95 + 95 - 42 - 90 - 57) - 76 + 85 + 2 - 75 + 24 - 88 + 3 - 32 - 30 + 88 - 71 - 99) + 2 - 97 + 1 - 20 + 75 - 20 + 69 + 0 + 58 - 40 - 59 - 34 + 95 + 71 + 5 + 32 + 87 + 41) + 5 + 23 - 22 - 41 - 2 + 85 - 2 - 84 + 30 - 6 + 78 - 45 + 88 - 7 - 78 - 39 - 72) - 81 - 79 + 39 - 45 - 100 + 94 - 7 + 20 + 91 - 92 + 4 - 56 - 57 - 49 + 88 - 28 + 25 + 77 + 3 + 67 + 98) - 31 + 24 + 86 - 35 - 34 + 11 + 97) + 54 + 100 - 99) + 26 + 84 + 73) + 46 - 83) - 14 - 29 - 24 - 54 - 98 - 30 + 22 + 24 - 14) - 67) + 28 + 77 - 47 - 49 - 43 + 61) - 41 - 24 + 81 + 70 + 35 - 29 + 39 - 64 + 93 - 53 + 71 - 99 - 32) - 71 - 55 - 85 + 64 + 81 - 44 - 12 + 24 + 11 + 81) + 69 - 71 - 10 - 90 + 45 - 78 + 33) - 81 + 4 - 32 + 75 + 35 - 2 - 13 + 10 + 9) + 4 - 10 + 37 - 30 + 34 + 21 + 58 - 78 + 68 + 32 - 93 - 17) + 0) - 61 - 68) + 9) + 81 + 81 - 33 + 80 - 2 + 15) - 14 + 100 - 58 - 64 + 47 - 22 - 93 - 20 + 98 + 98 - 9 + 38 - 81 + 86 - 35 + 16 + 83 - 45 - 1 - 79 + 57 + 84 - 28 + 5 - 46 - 90) - 27 - 79 + 42 - 16 - 72 + 97 + 6 - 33 + 89 - 73) + 8) - 95) + 33 + 9 - 6 + 48 + 88 + 11 + 54 + 77 - 57 - 76 + 61 + 94 + 21 - 64 - 6 + 74 + 61 - 100 + 58 - 76) + 97 + 88 + 31 - 31 + 92 - 33 - 77 + 31 + 88 - 70 - 6 + 50 + 87 + 61 + 56 - 76 + 31) - 49 + 27 + 19 - 35) + 62 + 88 + 12) + 82 + 6 - 99) + 99) - 83 - 47 + 18 - 84 - 17 - 64 + 2 + 49) - 74 + 56) - 49 + 24) + 100 + 79 + 49) - 81 + 35 + 44 - 73 - 45) + 100 - 25) + 79 + 98) - 29 + 98 + 13 - 90 + 23 - 26 + 5 + 61 + 55 - 82 - 80 + 98 - 13 - 92 + 10 + 0 - 34 - 17 - 42 + 14 + 96 - 87) + 13 + 6 - 28 + 40 - 28 + 28 + 91 + 68 - 11 + 24) - 79 + 0 - 8 - 22 - 67 + 79 - 81 + 13 - 22 - 22 - 5 + 41 + 9 + 55 - 12 - 55 - 63 + 26 + 82 - 32 - 11 + 86 - 36 - 57 - 54 - 72 + 53 - 25 + 57 - 89 + 18) + 50) - 26 + 63 + 17 - 63 + 53) + 45 - 35 - 66 + 59 - 65 + 30 + 91 - 87 - 30 - 79 - 11 - 25 + 49 + 30 - 81 - 40 + 50 + 7 - 48 + 71 - 21 + 17 + 60 + 85) - 54 - 90 - 3) - 25 + 44 - 82 + 98 + 20 + 36 - 86 + 32 - 60 - 12 + 75 + 43 + 57 + 65 - 95 + 1 - 74 - 3 + 54 - 22 + 32 + 44) + 27 - 75 - 9 - 55 - 63 - 72 - 20 - 29 + 75 - 9 - 31 + 75 - 6 - 86 + 20) + 81) + 18 - 15 - 25 - 87 + 13 - 83 + 96 + 8 - 21 + 50) + 16 + 0 - 56 + 73 + 43 + 21 + 2) - 7 + 10 + 93 + 4 + 69 - 72 + 54 + 78 + 93 - 5 + 6 + 43 - 55 + 97 - 94 - 91 - 96 - 89 - 74 + 57 + 63 - 55 + 71 + 67 + 10 + 36 + 65 - 90 - 97 + 53 - 48 - 95 + 29 + 52 - 49 - 3 - 13 - 28 + 37 + 42 + 98 + 14 - 64 - 9 + 70 + 24 - 64 - 62 + 49 + 31 + 92 - 81 + 20 + 50 - 97 + 48 + 39 + 50 - 16 - 8) + 26 - 33 + 99 - 90 + 83 + 12 - 52) - 87 - 93 - 54 + 16 + 92 + 52 + 82 - 42 + 55 + 36 - 59 + 42 - 11 + 62 - 50 + 77 - 2 + 99 - 15 + 47 + 75) - 25 + 90) - 19 + 26 - 28 + 12) + 24 + 88 - 60 - 81 + 87 - 8 - 37 + 55 - 2 - 28 - 1 + 1 - 40 + 54 - 83 + 12 + 43) + 54 - 64 + 19 + 27 + 30 + 88 + 92 - 28 + 8 - 72 + 78 - 8 + 78 - 36 + 51 + 55) + 79 + 27 + 91 + 44 - 70 + 39 + 51 - 3 - 28 - 89 + 33 - 8 + 17 - 49 - 7 - 68 - 97 - 76 - 62 - 0 - 5 - 33) - 4) - 96 - 68 - 6 + 32 + 40 - 96 + 27) + 35 - 2 + 12 - 78 + 90 + 37) - 81 + 58 + 9) - 0 - 68 - 67 + 44 - 62) + 99 - 0 - 84 + 20 + 71 - 42 + 49 + 7) + 50) - 42 + 90 + 87 - 100 - 75) - 36 - 22 + 54) - 37 + 30 + 63 + 61 - 46 - 87 + 63 + 40) + 70 - 94 - 83 - 73) + 82 - 53) - 29 + 1 - 37 + 77 + 88 - 40 + 53 - 12 - 8 - 45 - 66 + 18 + 43 - 42 + 98 + 32 + 35 + 57 - 84 - 96 - 37) - 99 + 44 + 63 + 46 + 41 - 48 + 95 + 15 - 89 + 65 + 66 - 84 - 92) + 43 + 23 - 7 - 82 + 34 + 76 - 92) + 72 - 42 - 21 + 11 - 77 - 12 - 9 + 98 + 83 - 47 - 75 + 66 - 17 + 60 - 54 - 49 - 62 + 43 + 55 + 93 - 89 - 91 - 39 - 99 + 83 + 27 + 18 - 22 - 72 - 48 - 10 - 34 - 25 + 69 + 0 - 51 + 100) - 15 - 57) - 59 - 69 + 80 - 2 + 92 - 76 - 8 - 70 - 63 + 35 - 98 - 22 - 24 + 13 + 45 - 11) - 26 + 68 - 44 - 19 + 69) - 67 - 27 + 4 - 65 + 69 + 31 - 100 - 58) - 56 - 88 + 67 + 55 - 4 + 22 - 8 + 0 + 35 + 79 - 20) - 0 - 69 - 27 + 51 + 21 - 99 + 23 + 10 + 70 - 95 + 57 - 34 + 47 - 88 - 24 + 57 + 65) + 22 + 31 - 31) - 94 - 67 - 79 + 94 + 7 - 62) - 30) - 100 + 68 - 21 + 19 - 34 - 17 - 50 - 66 - 60 - 59 + 90 + 5 + 46 - 66 + 82 - 65 + 85 - 60 - 25 + 13 - 82 - 50 + 59 - 52 - 81 - 74 + 67 - 83 - 61 + 53 + 49 - 94 + 27 + 58 - 21 + 68 - 47 + 82 - 27 + 93 + 86) - 5 - 52 + 62 + 50 + 9 + 29 - 91 + 16 + 36 - 38 + 27 + 67 - 78 + 26 + 93 - 12 + 25 - 22 + 2 - 45 + 38 - 18) + 81 - 24 - 12) + 63 + 9 - 27 + 7 + 2 - 71 - 11 - 91 - 22 + 40 + 22 - 88 + 70 + 10 - 47 + 45 - 9 + 77 - 49 - 27 - 4 - 36 + 100 + 57 + 85 - 7 + 47 + 8 + 83 + 10 + 44 + 85 + 14 + 41 - 9 - 67 - 3 + 0 + 50 - 92 + 70 - 50 + 75 + 54 - 37 - 14 - 40 - 40 + 5 - 60) + 96 + 57 - 83 + 55 - 19 + 63 - 9 + 24 - 71 + 56 - 85 - 63 - 82 + 8 + 10 + 28) + 9) - 3 - 83 + 77 - 96 - 78 - 55 - 2 - 99 - 94 - 4 - 55 + 56 + 41 - 76 + 57) - 7 - 6 - 61 - 35 - 86) - 78) - 20 + 38 - 90 - 80 - 63 + 48 + 96 + 86) - 29 - 1 + 88 - 63 + 36 + 98 + 51 + 87 + 49 - 32 - 58 - 18 + 3 + 38 - 9 - 87 - 57 + 14 + 18 - 91 - 82 - 62 + 10 + 24) - 37 - 21 + 89 + 73 - 15 + 65 - 49 + 40 + 63 + 59 + 29 + 69 + 89 - 30 + 92 + 10 - 51 + 75 - 43 + 97 - 65) + 83 + 73 + 40 + 72) - 81 + 34 - 33 + 19 + 9 + 65) - 86 + 22 - 68 + 16 + 45 - 19 - 25 + 61 - 29) - 65 - 41 + 50 + 69 + 10 + 30 - 71 - 76 + 15 - 48 + 19 - 0 + 53 - 82 - 24 + 65 + 17) - 94 - 85 - 19 - 96 - 56 - 17 - 45 + 41 - 74) - 23 + 96 - 27 - 11 - 28 - 11 + 68) + 86 + 92 - 4 - 88 + 50 - 90 + 94 + 97 + 54 + 20 + 47) - 4) - 81 - 79 + 94 + 60) - 54) - 47 + 96 - 90 + 27 + 53 - 1) - 59 - 28 - 2 + 22 + 14 - 93 + 96 - 84 + 33 - 75 - 44 + 76 + 8 - 26 + 99) + 49 + 73 - 54 - 47 - 19 + 48 - 82 + 33) + 10 + 70 - 50 - 70 + 50 - 86 - 69 - 96) - 15 - 34 + 31 + 57 - 66 - 65 + 70 + 43 - 73 - 13 - 89 - 19 - 26 - 31 - 97 + 49 - 38) - 20 + 53) + 94 + 70 - 27 + 56 + 86 + 33) - 26) + 2 - 87 + 16 - 78 - 2) - 5 + 12 - 31 + 7 + 30 - 54 + 100 - 82 + 67 + 92 - 90 - 72 - 22 - 20 + 73 + 31) + 52 + 56 - 51 + 12 + 11 + 27 + 51 + 11 - 76 - 88 + 1 + 37 + 98 - 62 - 44 - 74 - 40 + 30 - 47 - 62 + 59 + 84) + 18 + 95 + 100 - 74 - 97) + 10 - 50 - 56 - 16 + 59 - 45 - 59 + 48 - 33 - 18 + 18 + 71) + 15 - 32 + 34 - 36 - 54 + 34 + 22 + 34 - 63 - 63 + 78 + 54 + 12 + 39 - 54) - 49 - 24 + 41 - 64 + 26 - 97 - 65 + 13 + 59 + 97 + 26 - 61 - 71 + 26 + 65 + 100 + 6 + 12 - 19 + 61 + 58 - 10 - 82 - 51 - 48) + 73 - 86 - 25 + 99 - 6 - 22) - 83 + 82 + 92) - 63 - 55 - 28 + 15 + 10 - 67 - 47 - 76 + 63 + 12 + 49 - 95 - 33) + 10 - 46 - 14 + 84 + 48 + 100 + 2 - 46 + 65 + 17 - 72 + 50 + 24 + 15 - 60 + 19 - 14 - 8) + 21 + 67 + 34 + 17 - 19 - 2 - 81 + 88 - 66 + 23 + 15 + 85 + 5 - 24 + 33 + 30 + 5 + 62 - 19 + 19 - 2 - 33 + 46 - 39 - 78 + 30) + 67 + 47 - 33 + 95) + 1 + 61 - 37 + 66 - 45 - 98 - 98 + 90 - 5 + 6 + 80 - 77 + 24 - 41 + 72 + 15 - 17 + 3 + 56 - 8) + 22 - 85 - 93 + 50 + 28 - 38 + 75 + 68 - 73 - 58 + 93 - 41 + 54) + 44 - 24 + 26 - 51 + 74 - 39 - 24 + 48 + 35 + 38 - 31 + 66 + 41 + 85 - 28) + 73 - 64 + 46 - 20 + 26 - 23 + 68 - 82 + 8 - 68 + 54 + 31 - 64 + 39) + 10 - 17 - 40 - 35 - 53 - 13 + 77 - 6 + 51 - 38 - 65 + 14 + 46) + 67 + 3 + 38 - 26 + 1 + 54 - 31 - 67) + 32 + 76 - 34 + 87 - 8 - 54 + 39 - 88) + 42 - 98 + 95 - 0 + 74) + 21 + 67 + 71 + 3 - 81 + 93 - 15 - 40 + 43 + 67 + 87 - 51 + 34 + 98 - 6 + 46 + 0) + 61 - 24 + 67 + 61 + 12 + 2 - 97 + 30) + 0 - 30 - 4 - 68 - 22 - 97 - 62) + 16 + 85 - 23 - 21 - 79 - 29 + 67) + 93 - 80 - 66 + 83 + 44 + 50) + 55 - 71 - 13 + 49 + 63) - 88 + 47 + 39 - 44) - 63 - 6 + 60) + 4 + 75 + 69 - 76 + 24 + 10 + 23 + 79 + 24 + 88 + 84 - 54 - 67) + 56 + 46 - 59 + 77) - 82 - 100 + 13 + 21 + 93 + 9 + 33) - 32) + 36 + 4 + 2 + 25 - 61 + 91 + 92 + 78 + 69 - 10 + 99 - 99 - 21 - 100 + 0 - 17 + 45 - 29) - 82 + 13 - 75 + 7 - 92 + 70 - 49 - 26) + 38 + 74) - 1) - 16) + 90 + 26 - 62 + 92 + 52 - 16 + 16 - 8 + 46 + 16 - 72 + 18 + 70 - 23 + 67 - 98 + 13 + 48 + 46 - 27 + 35 + 100 + 71 + 8) - 97 + 39 - 96 - 1 + 13 - 31) + 69 - 98 + 17 - 97 - 46 + 51 - 26 + 65 + 51) - 3 + 4 + 45 - 79 + 30 + 89 + 32 + 50 + 5 + 81 - 17 - 45 + 98 - 65 - 15 - 1 + 37) - 29 + 90 - 25 + 29 - 45 - 95 + 10 - 79 + 68 - 11 - 48 + 11 + 88 - 10 + 58 - 41 - 8) + 50 + 47 + 34 + 19 + 32 + 20 - 78 - 19) - 1 - 93 + 10 - 36 + 7) - 1 - 72 - 82 + 94 - 54 - 82 + 94 + 12 - 1 + 79 + 36 - 94 + 32 + 7 + 0) + 38 + 19 - 66 + 69 - 86 - 73 - 19 + 23 + 25 + 79 + 46 + 60 - 27 - 84 + 31 + 65 + 53) + 80 + 99 + 82 - 26 + 36 - 31 - 76 + 28 + 96 - 43 + 61 + 21 + 62 + 68 + 92 + 100 - 90 - 47 + 56 + 55 - 60 + 3 + 93 - 90 - 5) - 11 - 66 + 2 - 37 + 48) + 25 - 94) + 88 + 35 - 1 - 39 - 22 + 8 - 83 - 98) - 66 + 98 + 36 - 39) + 83 - 53 - 65 + 40 + 82 - 51 + 25 + 56 + 50 + 46 - 12 - 91 - 7 + 52 - 78 + 10 + 42 - 68 + 52 + 57 - 100 - 45 + 21) - 66 - 45 + 2 + 49 + 41 - 48 - 12 + 38 + 88 - 100 - 1 + 11 + 82 + 19 - 61 - 88 - 44 + 92) + 97 - 51 - 10 - 82 - 70 + 74 - 5 + 2 - 46 + 88 - 55 + 6 - 73 + 39 + 71) + 74 - 43 + 48 - 61 - 81 + 4 - 50 + 46 + 96 + 48 + 0 + 29 + 40) + 100 - 14 + 66) + 11 + 79 - 32 + 15 + 74 - 4 + 71 + 68 + 93 - 15 - 49 + 8 + 71 - 15 - 9 + 58 - 73 + 46 + 53 - 17 - 29 - 59 + 10 - 93 - 8 + 71 + 8 - 32 - 54 - 9 + 22) + 58 - 8 + 91 + 7 - 81 - 20 + 98 - 98 - 7 + 40 - 75 - 63 + 46 + 99) + 42 + 96 - 90 + 6 - 78 + 77 - 89 - 57 + 40 + 47 + 79 + 22 + 29 + 88 + 71 - 87 - 53 - 62 + 43 - 29 - 84 + 66 - 39 - 81 + 91 + 73) - 26 - 14 - 29 - 91 - 24 - 72 + 8) - 86 + 87 + 61 - 63 - 51 - 30 + 87 - 54 - 60 - 4 + 91 + 89 + 84 + 67 - 74 + 23) - 85 + 32 + 73 - 64 + 42 - 15 + 83 - 71 - 57 + 77 + 20 - 81 + 8 + 10 + 91 + 66 + 3 - 75 + 46 + 31 + 97 - 70 - 6 + 9 - 98 - 52 + 45 + 68 + 19 + 77 - 42 + 72) + 25 + 95) - 95 - 30 + 44 + 34 - 91 - 99 + 66 + 62 - 17 + 8 + 8 - 4 - 61 - 17 - 46 + 8 - 38 + 43 - 58 + 15 - 85 - 100 + 12 + 48 - 43 - 96 - 92 + 45) - 4 + 52 - 92 + 32 - 12 + 40 + 28 + 87 - 92 - 54 - 11 - 70 + 5 + 77 + 23 + 34 + 21 + 6 - 23 + 10 - 83 + 91 - 77 + 0) + 67 + 97 + 63 + 83 - 93 + 26 + 62 - 35 + 32 - 4 + 14 - 86 + 30 - 35) + 81 - 100 - 24) + 69 + 48 - 84 + 25 + 39 + 81 - 82 - 12) - 85 - 20 - 28 + 78 + 6 + 3 - 4 + 93 - 53) - 65 - 40 + 2) + 41 + 10 + 0 + 67 - 81) - 5 - 68) - 63 - 73 - 95 + 49 - 87 - 12 + 62 - 32 - 21 + 85 - 90 + 16) + 5 + 95 + 64 - 42 + 54 - 29 - 42 - 86 + 50 - 54 - 14) - 74 - 31 + 41 - 27 + 88 - 74 + 9) - 25 - 8 + 69 - 12 + 47) - 56 + 44 - 82 + 18 + 24 - 47) - 46 + 95 + 85) - 69 - 97) + 49 + 65 + 18 - 77 + 33 + 20 - 36 + 43 + 84 + 63 + 10 + 1 + 31 - 51 - 99 - 31 + 22 - 45 - 61 + 23 + 35 - 55 + 7 + 78 + 5 + 39 + 42 + 67 + 29 + 7 - 23 - 30 - 22 + 30 + 46 + 20 + 26 + 41 - 42 - 82 - 85 + 55 + 92 - 41 - 33 + 21 + 7 - 93 + 100 + 81) - 21 - 83 - 100 + 62 - 43 - 96 - 100 + 90 + 8 + 26 + 23 - 55 + 87 - 60 - 84 + 34 + 99 + 90 - 19 + 61 - 12 + 83 - 65 - 1 + 6 + 76 + 56 - 81 - 65 - 4 - 90 - 83 - 100 + 93 + 85 - 37 - 34 + 68 - 52 + 39 + 100 - 54 + 86 - 55 - 7 - 59 + 6 - 41 - 22 - 10 - 78) + 84 - 27 - 99 - 98 + 54) - 12 + 9 + 89 + 91 - 0 - 20 + 64 - 11 - 47 + 22 - 35 - 95 - 71 - 76 + 27) + 10 + 78 + 81 + 13 + 67 - 71 + 5 + 40) + 55 - 6) + 10 - 76 - 72 - 31 - 53 - 75 - 29 + 20 + 37 + 56 + 2 - 76 + 38 + 44 + 31 - 66 - 85 + 3 - 73 - 44) - 65 - 28) - 21 - 14 + 40 + 55 - 84 - 44 - 48 + 78 + 58 - 88 - 16 - 16 + 31 + 44 + 91 - 11 + 59 + 4 + 1 + 64 + 85 - 33 + 72) + 5 + 56 - 65 + 67 + 98 - 98 + 76 + 55 - 93 + 52 - 49 - 69 + 43 - 76 - 27 + 35 - 21 - 50 - 93 - 72 + 16 - 56 - 3 + 45 + 85 - 63 - 81 - 71 - 14) + 32 - 29 - 42 - 80) - 40 - 81 - 42 - 12 + 69 + 27 - 85 - 90 + 41 - 20 + 40 + 57 - 27 - 99 + 32 + 0 + 29 - 75 - 77) + 77 + 67 - 60 + 80 + 30 - 58 + 100 + 49 - 71) - 21 - 44 - 48) + 90 - 49 - 15 - 66 - 18 + 22 - 13 - 16 + 36 - 63 - 6 - 66 - 1 - 40 - 51 + 20) + 100) + 89 - 67 + 3) - 96) - 47 - 97 + 92) - 44 - 100 + 25 - 21 + 48 + 37 - 44 + 48 + 77) - 29 - 54 + 40 - 47 - 15 - 19 - 87 - 90) + 7 - 41 - 81 + 65 - 83 + 14 + 40 - 63 + 79 + 98 + 47 - 4 + 67 - 81 - 48 - 40 - 78 + 93 + 44 - 58 - 2 - 21 + 82 - 84 - 62 - 40 - 41 + 9 + 70 - 27 + 16 + 22 - 87) - 0) + 86) + 12 - 37 + 43 - 97 - 54 - 83 - 56 + 92 - 34 - 5 + 9 - 36 + 62 - 24 - 16 - 90 - 83 - 20 - 13 + 23 - 77 - 68 - 25 - 13 + 31 + 32 - 59 + 73 - 84 + 58) + 33 + 37 + 79 + 79 + 88 + 17 + 55) - 44 + 82) - 21 - 39 - 44 + 91 + 77 + 58 - 68 - 79 + 11 - 47 + 26 - 49 - 69 - 70 + 59 + 47 + 58 - 48 + 65 - 95 - 46 - 53 + 19 - 5 + 83 - 79 - 2 + 67) - 16 + 18 - 98 + 39 - 34 + 50 - 1) - 95 + 67 - 40 - 82 - 47 + 79 + 36 - 86 - 5 + 20 - 16 - 55 - 7 + 20 + 34 + 40) + 85 + 93 - 56 + 11 + 91 + 100 + 8 - 79 + 86 + 28 - 95 - 28 - 93) - 75 - 6 + 90 - 25 - 97 + 82 + 2 + 76 + 10) - 61 - 37 + 88 - 72 + 38 + 85 - 8 - 89 + 95 + 41 + 100 - 62 - 58 - 92 + 80 - 79 + 63 - 97 - 68 - 28 + 12 + 34 + 3 + 46 - 8 + 61 - 79 - 15 - 94 - 63 + 53 + 73 - 65 - 82 - 40 + 27) + 88 + 28 - 3 + 29 - 31 + 63 + 20 - 27 - 40 - 58 - 50 - 25 - 90 - 1 + 56 + 56 + 19 + 60 - 63 + 71 + 10 - 68) - 56 + 29 - 100 + 30 - 91 - 29 + 16 - 92 - 59 + 15 + 57 + 69 - 52 + 79 - 19) + 97 + 46 - 31 + 95 - 34 + 60 - 98 + 74 - 47 + 73 + 41 + 8 + 75 - 91 + 39 + 37 - 28 - 24 + 74) + 4 - 5 + 47 - 4) + 65 - 38) - 84 - 79 - 36) - 83 + 78) - 54 + 98 + 69 - 68 + 70 + 23 - 40 - 84 - 24 - 66 - 84 - 91 - 87 + 2 - 5 + 47 + 86 + 42) - 73 + 52 - 6 - 98 - 44 + 65 + 0 + 99 - 66 - 53 + 69 + 17 - 14) + 97 - 35) - 79 + 1 + 41) + 54 - 69 - 15 - 23 - 36 + 16 + 8 - 78 + 35 + 28 - 1 - 99 - 81 - 100 - 7 + 46) - 37 - 24 + 94 - 83 - 57 + 61 - 38 + 73) - 96 - 72 - 5 + 40 + 99 + 38 - 9 + 29 + 23 + 9 - 34 - 23 - 14) + 50 - 0 + 99 + 92) - 20 - 52 - 98 + 20 - 80 - 60) + 28 - 27 - 56 + 85 - 17 - 91 + 77 - 53 - 49) + 33 + 66 - 28 + 75 - 97 + 11 + 20 - 73 + 3 + 74 + 58 - 97 + 58 + 95 - 52 - 16 - 60 - 38 - 8 + 38 + 15 + 58 + 2 - 44 + 76 + 16 + 5) + 41 - 31 - 18) + 26 + 28 + 72 - 78 + 3 - 17 + 42 - 45 + 4 - 1 - 84 + 7 - 84 + 56 + 60 - 12 + 72 + 41) - 21) - 44) - 53 + 60 + 45 - 75 - 34 + 48 + 63 + 69 - 63 - 2 - 81) - 59 - 70 + 29 - 35 - 71) - 23 + 88 - 58 - 8 - 97 + 29) + 87 + 84 + 63 + 52 + 83 + 56 - 66 - 50 + 33 + 100 - 94 - 76 - 10) - 89 + 64 + 74) + 91 + 19 - 12 - 85 + 28) + 3 + 12 + 70 - 60) + 15 - 52 + 34 - 72 - 20) - 92 - 83 + 5 - 20 + 42 + 45 + 89 - 8 - 93 + 85 - 11 + 89) + 5 - 89 + 39 + 12 + 29 + 99 - 46 + 15) - 100 + 2 + 21 + 41 + 11 + 52 - 98 + 23 + 1 + 71 + 28 - 17) + 55 - 92 - 10 - 91 - 85 + 54 - 69 - 98 + 31 + 33 + 52) - 58 + 64 - 88 + 15 - 19 - 34 + 86 - 69 + 26 + 100) + 0 - 1 + 89 - 15) + 11) - 28 - 44 + 36 + 32 + 14 - 77) - 93 + 87 + 78 + 21 + 25 + 96 - 0 - 11 - 63 + 90 - 77 - 59 + 55 + 44 - 42) + 71 + 22 - 35 + 90 + 33 - 65 - 72 + 7 + 68 - 12 + 42 + 27 + 36 - 69 + 42 + 89) + 29 + 41 - 38 + 56 - 65 - 45 + 67 + 48 + 82 - 73 + 24 - 71 - 55 - 97 + 14 + 37 - 84 + 79 + 85 - 19 + 40 - 30 + 30 + 21 - 43 - 49 + 68 - 10 + 10 - 69 + 12) - 93 - 45 - 67 - 63 - 33 + 64) - 84 + 73 + 3 - 40 + 98 + 6 + 41 + 51 - 71 - 83 + 32 - 27 + 8 + 1 + 92 - 24 - 73 + 11 - 26 + 20 + 78 + 98 + 86 + 70 - 82 - 5 + 12 + 47 + 14 + 70 - 40 + 60 + 70 + 100 - 71 + 54 - 61 + 65 + 9 - 38 + 56 + 44 + 67) + 62 - 41 + 70) - 45 + 12 - 99) + 38 - 43 + 74 - 13) + 50 - 79 + 10 + 62 + 23 + 72 - 10) - 87) - 30 + 59 - 67 + 24) - 80) + 76 - 59) + 11 + 61) + 75 - 54 + 42 + 3 - 19 + 79 + 77 + 75 + 71 + 49 + 98 + 0 + 93 + 65 - 29 - 96 - 16 - 59 + 88 - 61 + 21 + 40 - 80 - 63 - 92) + 67 + 31 + 88 - 60 + 67 + 58 + 95 + 1 + 58 + 72 - 70 + 36 - 82 + 58 - 13 - 62 - 8 - 28 + 83 + 10 + 0) + 97 - 98 - 43 + 42 + 30 + 92 - 79 + 92 + 70) - 15 + 89 + 28 - 30 + 66 + 88 + 1 - 93 - 12 - 69) + 83 + 33 + 1 - 85 - 37 - 13 - 100 + 0 - 92) + 80 + 16 + 45 - 81 - 32 + 78 + 42 + 67 + 48 - 16 - 21) - 70) + 26 + 60 - 75 - 81) - 66 + 12 - 0 - 8 - 6 + 54 - 6 + 99 - 83 + 6 - 87 - 90 - 21 + 2) + 2 + 30) + 100 - 75 - 72 + 98 + 1 + 45 - 99) + 76 - 73 - 6 + 10 - 17 + 16 - 29 - 19 - 52 - 76 + 79 + 39 - 37 - 21 - 72 + 97 + 99) - 2 - 38 + 17 - 32 + 24 + 50 + 27 + 64 - 53 + 82 + 9 - 94 + 96 + 54 - 42 - 25 + 47 + 25 - 81 + 26 - 99 + 53) + 55 - 67 - 91 + 80 - 33 - 87) - 62 - 50 + 43 + 59 + 90 + 31 + 37 - 10 - 82 + 73 + 61 - 87 + 90) + 3 - 72 - 33) - 55 + 84 + 77 + 63 + 46 - 7 + 74 + 27 + 72 - 64 + 66) - 79 + 88 - 48 + 77 + 59 - 83 - 6 + 4 - 19 + 39 + 7 + 98 + 82 + 78 + 15 - 38 - 35 + 69 + 94 - 13 - 78 - 22 - 11 - 35 + 11 - 7) - 16 - 24 + 89 + 74 - 57 - 9 + 97 + 66 + 58 - 85 + 62 - 6 + 16 + 98 + 91) + 18 - 13 - 78 + 90 + 25 + 16 + 15) - 86 - 35 + 33 - 13 - 47) + 94 + 86 + 92 + 26 + 45 + 4 + 55 - 95 + 69 - 29 - 25 - 20 + 57 - 54 + 23 - 21 - 79 - 0 - 42 + 55) - 5 + 65 + 34 - 27) - 25 - 12 - 3 + 24) - 24 + 77 - 47 - 46) + 57 + 18 + 81 + 95 + 41 + 31 + 89 + 20 - 47 - 17 + 19 - 9 - 14 + 31) + 9 + 69) + 86 - 69 + 48 + 95 - 37 + 6 + 13 + 50 + 84 - 3 - 98 + 99) - 43 + 24 + 14 + 22 - 45 - 37 + 12 - 13 - 6 - 4 - 28 - 66 - 82 + 65) - 98 + 16 - 21 + 66) - 13 - 83 + 18 + 64 + 7 - 5) + 65 + 31 - 95) - 33 - 39 - 19 + 15 - 82 - 81 + 39 - 16 - 35 + 44 - 27 - 35 + 55 + 69 + 37 + 35 + 19 + 95 - 71 - 14 + 19 + 19 + 58) + 16 - 19 + 69 - 16 + 19 - 93 + 36 - 3 - 83 + 51 - 2 + 70 + 90 - 2 + 90 + 73 + 27 - 77 + 3 + 39 + 44 - 87 - 80 - 64 + 19 + 81 - 53 - 62 - 26) + 62 - 28 - 11 - 12 - 49 + 8 - 90 - 21 + 86 - 14 - 79 - 15 + 64 - 75 - 74 - 86 + 2 + 45 + 56 + 11 + 56 - 76 - 94 - 42 - 72 + 17 + 92 + 30 + 0 - 28 - 47 - 17 + 22 + 19 + 16 + 89 + 55) + 39 - 32 + 50 - 49 + 47 - 9 - 61 - 92) + 82) - 24 - 4 - 98 + 21 - 5 - 15 - 53 - 80 + 84 + 59 - 3) + 72 - 88 - 91 + 37 + 29 - 43 - 48 - 41 + 21 - 47 + 77 - 28 + 99 - 77 + 7 - 57 + 76 - 66 - 40 + 70 - 81 - 49 - 14 - 79 - 83 + 41 - 96 - 47) - 8 + 82 + 78 - 31) - 98 + 84 + 28 + 74 + 32 + 74 + 47 - 9 - 17 - 41 + 100 - 4 + 50 - 70 - 30 - 22 + 7 + 80 - 21 + 70 + 73 - 49 + 100 - 98 + 10 - 78 - 23 + 76) + 19 + 55 + 85) + 49 - 19 - 31 - 95 - 25 - 96) - 88 - 14 + 41 - 42 + 48 + 62 + 34) - 40 + 42 - 100) + 8 + 91 - 83 - 0 - 37 + 61 + 47 - 86 - 55 - 89 - 78 - 15 + 87 - 39 + 48 - 44 + 79 - 67 + 53 + 33 + 17 - 84 - 29 - 90 - 4 - 23 + 60 + 29 + 76 + 1 - 68) + 76 - 46) + 97 - 35 + 83 + 40 + 73 - 92 - 97) - 41 + 56 - 61 + 11 + 51 + 12 - 33 + 13 - 27 - 91) + 40) + 95 + 66 + 96 - 26 + 26 + 45 + 21) - 66 - 57 - 34 + 90 - 31 - 63 + 100 + 68 + 45 + 47 - 35 + 8 - 53 - 45 - 14 - 92 - 40 - 3 - 1 - 46 + 66) - 10) + 30 - 2 + 56 - 73 + 22 - 97 + 49 + 19 - 29 + 78 + 87 + 45 + 20 + 50 - 96) + 41 - 10) + 74 + 64 - 95 - 56 + 33 + 25 - 61 - 11 - 67 - 22 - 90 + 82 - 65 - 6 - 48 - 38 + 19 + 32 - 52 + 58 - 5 - 30 + 27 - 26 - 94 + 53 + 71 + 43 + 14 + 89 - 42) + 75 - 22 + 88 + 14 - 92 + 82) + 57 - 94 - 35 + 40 - 43 - 2 - 18 - 6 + 11 + 5 + 64) + 78 - 35 - 98 + 84 - 56 + 75 + 27 + 60 - 26 - 25 - 75 - 25 - 92 + 83 + 64 - 4 + 76 - 93 + 100 + 41 - 63 - 76 + 85 + 26 - 32 + 6 + 12 + 67 - 91 + 52 + 11 - 84 - 45 + 75 - 15 - 86 + 35 + 100 + 48 - 99 - 15) + 69 + 46 - 62 - 11) + 34 - 66 + 31 - 18 - 36 + 31 - 26 + 59 + 58 - 21 + 33 + 98 + 37 + 46 + 59 - 42 + 5 + 43 + 8 + 74 - 69 + 28 + 86 + 29 + 100 + 13 - 20 - 8 + 15 + 84 - 83 + 73 + 51 + 39 - 0 + 18 - 0 - 41) - 41 + 24 + 77 - 46) - 34 + 32 - 2 + 68) + 89 - 9 + 25 + 50 - 46 - 47 + 95 + 96 + 41) - 35 + 62 + 51 + 81 - 21 + 100 + 71 + 94) - 6) - 23 - 68 + 24 + 82 - 93 + 6 + 49 - 93 - 38 + 39 - 51 + 95 + 84 - 88 - 78 - 32 - 31 + 25 - 96 + 79 - 25 - 85 - 59 - 9 + 74 + 16 - 97 + 42 - 39 - 29 - 13 + 30) - 16 - 69 + 83 - 14 + 73 - 86 + 71 - 21 + 1 - 53 + 54 - 90 + 77 + 49) + 42) + 70 - 80) - 66 + 44 - 79 - 28) - 82 + 55 - 6 + 65 + 2 + 45 - 73 + 3 + 48 - 34 + 82) - 1 + 54 - 5 - 2 + 12 - 27 - 15 - 93) - 37 + 21 + 63 - 66 + 20 - 17 - 96 + 82 - 52 - 12 + 28 - 63 - 73 - 68 - 56 - 51 - 15 + 20 + 27 + 76 - 1 - 11 + 8 - 83 + 89 + 41 + 24 + 46 + 19) - 89 + 49 - 1 + 38 + 4 - 51) - 48 + 45 + 2 + 51 + 17 + 72 - 53 - 42 - 44 + 44 + 86 - 27 + 13) - 93 + 16 - 68 + 35) - 41 + 35 - 69 - 13 - 99 + 64) - 7 - 97 - 4 - 97 - 36 - 91 + 96 + 79 + 67 - 87) - 95 + 56) - 90) - 15 - 84 - 34 + 96 + 16 + 73 - 29 - 76 + 99 + 56 - 57 - 7 - 72 - 51) - 32 + 11 - 0 + 91 + 16 + 94 + 89 - 44 + 95 - 67 - 42 + 4 - 18 + 27 + 18 - 7 - 52 + 83) + 94 + 20 - 54 + 1 + 58 + 95 - 17 + 49 - 92 + 90 + 2 + 34 - 5 - 37 + 89 - 95 - 40 - 62 - 67 + 79 - 2 - 55 - 68 + 78 + 75 + 86 + 68 - 2 - 35 + 22 + 8 - 89) - 59 - 13 - 36 - 98) - 47 + 16) - 76 + 69 + 44 + 14 + 31 - 83 - 31 - 94 - 71 - 9 + 52 - 90 - 44 - 20 + 31 + 53 + 45 + 53 + 30 + 32) - 74 - 13 - 37 - 26 + 23 - 18 + 31 - 28) + 73 - 4 - 80 - 2 + 5 + 17) - 88 - 51 + 44 + 40 + 12 - 18 - 66 - 91 - 89 - 45) + 72 - 25 - 22 + 58 - 1 - 13 + 49 + 53 + 66 - 89 + 43 - 81 + 61) - 31) - 33 - 70) - 69 - 99 + 68 + 67 - 67 - 72 + 74 - 31 - 72 + 64 - 6 + 46 - 49 - 60) - 32 + 23 + 68 - 55 + 80 - 13 + 41 - 8 + 98 - 14 + 90 - 64 - 82 - 72 - 37 - 20 + 64 + 20 - 63 - 53 - 6 - 34 - 52 - 73 + 100 - 5 - 27 - 30 - 76 + 72 + 27 - 69 + 95 - 82 - 2 - 43 - 83 - 78) - 80 - 100 - 94) - 53 - 38 + 3 - 68 - 82 - 98 + 90 - 14 + 93 - 80 - 23 + 96 - 86 + 81 - 67 - 48 + 62 - 20 + 50 + 91 - 73) + 95 + 25) + 75 - 35 + 65 + 61 + 37 + 78 - 59 + 20) - 89) + 15 - 29 + 98 + 39 + 1 - 13 - 18 + 10 - 65 - 40 - 64) + 75 + 63) - 22 + 73 + 92 + 80 + 85 - 86 - 49 + 82 - 64 + 42 + 39 + 16 - 48 - 26 - 58 - 11 - 40 - 36 - 85 + 75 - 46 + 26 - 6 - 36 - 70 - 45 - 94 - 90 + 17 - 32 - 71 - 13 + 69 - 13 + 3 + 69 + 22 - 22 - 76 - 35 + 23 - 69) + 58 + 27 - 45 - 78 + 59 - 43 + 66 + 97 + 76 - 81 - 6 + 88 - 75 + 95 + 34 + 30) + 39 - 67 - 31 - 12 + 24 - 74 - 18 + 32 + 68 + 28 + 24 + 87 + 33 - 59 - 22 - 35 + 95 - 23 - 13) + 25 - 42 + 91) + 60) - 80 + 22 + 13 + 42 + 75 + 69 + 47 - 40 - 65 + 85 - 36) - 64 + 6 - 44) - 46) - 53 - 13 - 91 + 37 + 85 - 17 - 81) + 33 - 11 + 91 - 10 + 46 - 22 - 99 - 53 + 17 + 10 - 75 - 33 - 72) - 36 + 96 + 6 + 87 - 65 + 68 + 74 + 22 - 2 - 66 - 82 - 11 + 31 + 7 + 19 + 67 - 49 + 80) - 42) - 13 - 14 - 76 - 88 + 23 + 54 - 41 + 51 + 51 + 31 + 8 + 28 + 52 - 26 - 83 - 81 + 84 - 25 - 73 + 33 - 61 - 23 - 79 - 63 + 54 + 35 + 84 + 27 - 55 + 89) + 13 + 85 - 56 - 40 + 68 + 67 - 47 + 34 + 77 + 79 + 14 + 35 + 16 - 53 + 44 - 79 - 2 - 98 - 70 + 35 + 1 + 83 + 10 - 19 - 10 + 18 - 74 - 93 + 29 + 82) - 12 + 44) + 67 - 44 - 96 - 43 - 66 - 31 - 81 + 81 + 58 - 43 + 43 - 86 + 86 - 3 - 46 - 70 - 85 - 81 - 17 - 16 + 33 + 38 + 59 - 20) + 22 + 98 - 1) + 10 + 35) - 44 + 62 + 60) - 85 + 19) - 36 - 5 + 35 - 23 + 4 - 10) - 28 - 86 + 27 - 6 + 63) - 24 + 99) + 48) + 80) - 7 - 62 + 32 + 53 - 49 - 86 + 1 - 10 + 49 + 41 - 73 + 67 - 42 + 66 - 75 - 97 - 31 + 8 - 59 - 15 + 1 + 1 - 76 - 19 + 13 + 84 - 83) + 28 - 94) + 55 + 56 - 65 + 14 + 24 + 65 + 64 - 70 - 20 + 51 - 2 - 3 - 13 + 19 + 39 + 58 - 89 - 32 - 92 - 28 - 20 + 78 + 59 - 76 - 86 - 46 + 31) - 27 - 62 - 17 - 89 + 30 - 77 - 91 - 71 + 18 - 0 + 53 - 94 + 74 - 80 + 6 + 72 + 18 - 64 - 33 + 11 + 60 - 58 - 11 - 93 - 29 - 66 - 1 + 26 + 67 + 98 + 95 + 84 - 23 + 45 + 85 + 63 + 22 + 63 + 17 + 93) + 6 + 2 - 12 - 21 + 51 + 81 - 5 + 97 + 70 - 3 - 16) + 10 - 3 + 96 - 55 + 71 - 9) + 4 - 96 - 29 + 28 + 30) + 42 - 74 + 69) + 49 - 48 - 44 + 87 + 58 + 51 - 55 - 90) + 45 - 87 - 71 + 43 - 36 + 34 + 32) - 87 + 17 + 30 + 41 - 69 - 90 - 10) - 90 + 62 + 3 + 96 + 49 - 10 + 32 - 59 - 84 - 57 - 57 + 63 - 12) + 56 - 46 - 8 + 69 - 43 + 46 - 39 - 29 - 5 + 96 - 92 - 10 - 74 - 38 + 53 - 97 + 1 + 42 - 98 + 63 + 42 - 80 + 19 + 60 - 46) + 64 + 22 + 18 + 81 + 87 + 7 + 9 + 8 + 77 - 13) - 72 + 37 - 72) + 53 + 58 - 43 - 33 - 67 + 79 + 43 - 12 + 60 - 1 + 38) + 6 - 2 - 98 - 67 - 89) - 84 - 44 + 99 + 58 - 82) - 73 + 37 + 83 - 5 - 24 - 88 - 37 + 32 + 23 - 59 - 41 - 86 - 81 + 13 + 1 + 87 + 98 + 12 - 61 - 75 - 74 + 91 - 44 - 90 - 99 + 3 + 56 + 57 + 78 - 27 + 100 + 35 - 10 + 20 + 76 + 79 + 19 + 60 - 99 - 39 - 57 + 46 - 45 + 33 - 34 + 97 + 42 + 56 - 54 - 24 - 0 + 42 - 11 - 37 - 79 + 80 + 53) - 13 + 21 + 67 - 0 + 2 - 5 - 22 + 93 - 23 + 43 + 26 - 50 + 16 + 82 + 87 + 24 + 57 + 33 - 96 - 79 + 56 - 8 + 45 - 86 - 41 - 37 - 59) + 71 - 82) - 68 - 44) + 48 - 13 - 61 - 55 + 21 - 56 + 75 + 23 - 31) - 94 + 89 + 65 + 83) + 63 + 22 + 35 - 59 + 61 - 95 + 12 - 3 + 45 + 80 - 25 - 10 - 65 + 48 + 80 + 8 + 37) - 69 + 70 - 38 - 81 + 78 - 70 - 64 + 32 - 19 - 32 - 14 + 57 - 85 + 86 - 89 + 72 + 30 - 47) - 17) + 11 + 98 + 99 + 75 + 16 - 39 + 22 - 62 - 70 - 97 + 58 - 1 - 32 + 44 - 21 - 94 + 24 - 2 - 11) + 62 - 28 - 32 + 75 + 48 + 74 + 30 - 67 - 94 + 25 + 93 - 34 - 35 - 27 - 1 - 46 + 65) + 69 + 73 + 81 + 52 - 99 + 79 - 55) + 74 + 46 - 86 - 18 - 73 + 92 + 9 - 69 + 14 + 6 - 31 - 4 - 14 - 17 + 73) - 5 - 13 - 76 + 76 + 24 - 51) + 59 + 39 - 0 + 26 + 63 - 44 + 70) - 14 + 10 - 71 + 16) + 25 - 50 + 69 + 60 + 58 - 73 + 52) + 95 + 66 - 38 + 89 + 8 - 88 + 0 - 22 - 12 - 52 - 78 + 35 - 77 - 38 + 47 + 77 + 89 - 44 + 26 - 96 - 47 + 84 + 1 - 61 - 80 + 84 + 83 + 79 - 21 - 85 + 63 - 29) + 61) - 3 - 27 - 96 + 80 - 97 - 79) + 0 + 83 + 47 + 52) + 94 - 17 + 27 + 9 + 13 + 81 + 37 + 21) + 48) - 10 - 54 + 69 - 26 - 93 - 62) - 99 - 2 + 75 + 24 - 98 + 29 - 27 - 69 + 83 - 20 + 73 - 70 + 56 - 15 - 80 - 35 - 18 + 85 + 52 - 36 + 73 - 72 + 16 + 60 - 34 - 34 + 42 + 78 - 6 - 69 + 99 - 59 + 10 - 25 - 42 - 87 + 91 + 82 + 6) - 47 + 17 + 65 + 54 + 46 - 46 + 77 + 85 - 69 + 62 + 58) + 2) + 22) - 50 - 5 + 97 + 86 - 7) + 37 + 36 + 27 + 88 + 81) + 23 + 60 - 55 - 38) - 7 + 12 + 57 - 49 - 89 + 57 + 92) + 87) + 43 - 6 - 48 + 12 + 26 + 10 + 51) + 38 - 19 - 29 - 26 + 6 - 46 - 49 - 56 + 98 - 97 + 61 - 77 + 48 + 13 - 84 + 94) + 97 - 85) - 67) + 19 + 22 + 23 - 25 - 75 - 82 - 64 + 10) - 35 - 62 + 85 + 32 - 39 + 65 + 27 - 30 - 25 - 64 + 69 + 13 + 8 + 58 - 40 - 52 - 26 - 71 - 56 - 85 - 44 + 98 - 85 - 67 - 16 - 91 + 27) - 24 - 1 + 97 + 43 - 7 - 4 - 62 + 39 + 89 + 19 - 31) + 47 - 39 + 7 + 31 + 42 + 99) - 48 - 3 - 72 - 42 - 99 - 33 - 54 + 55 + 8) - 13 - 37) + 64) - 77 - 62 + 53 - 42 - 62 - 98 + 15 - 41 + 7 + 21 + 88 + 57 + 86) + 28 - 50 - 72 - 6) - 83 + 74) + 52 + 4 - 58) + 76) - 7 + 46 - 45 - 43 + 60 + 55 + 74 + 76 - 41 - 8 - 50 - 9 - 7 + 44 + 3 - 36 + 8) - 75 - 71 - 61 + 71 + 42 - 13 - 40 - 71 + 92) + 54 + 44 - 6 + 28 + 98 - 52 + 80 - 73 - 20 - 43) - 37 - 8 + 61 + 4 + 39 - 98 + 62 + 70 - 18 - 27 + 41 - 25 + 76) - 15 - 26 - 95 + 3 - 22 - 47 - 91 - 84 - 31 - 59 + 29 + 91 - 7 + 14 - 4) - 35 - 94) - 64 + 30 + 0 - 96 + 29 - 79 + 21 + 22 - 100 - 3 - 56 + 62 + 48 - 57 + 9 + 70) - 7 - 99 + 74 + 1 - 5 - 35 - 23 - 88 - 70 - 37 + 28) + 17 + 97 - 62 + 81 - 97 + 10 - 44 + 86) + 27 + 91 + 8 - 55 - 83 - 81 + 49 - 1 - 70 - 44 + 21 + 73 + 5) - 13 + 79 - 35 - 75) + 12 + 77 + 73 + 44 + 53 - 80 + 99 + 10 - 53) - 58 - 67 + 58 + 36 + 20 - 85 + 15 + 38 - 3 + 91 - 84 - 14 + 87 + 71 + 67 - 0 - 51 + 91 + 75 - 47 + 74 + 83 - 29 - 49 - 45 - 26 - 14 - 66 - 91 + 14 + 86 + 72 - 64 - 40 - 88) + 48 - 21 - 77 - 10 - 94) + 16 - 86 + 55) + 44 + 62 - 69 - 54 + 72 + 82 - 58 - 43 + 76 + 72 - 52 + 4 + 81 + 65 + 57 + 20 + 76 + 52 + 33 - 11 - 3 - 24 + 74 - 62 - 20 - 94 - 35 + 78 + 14 - 14 + 7 + 4) - 18 - 39 - 36 + 73 + 35 - 82 - 52) - 28 + 80) - 89 - 68 - 95 + 2 - 71 - 30 - 99 - 78 - 65 + 45 + 4 + 43 + 53 + 34 + 5 + 39 - 0 - 11 - 83) - 5 + 50 + 17 + 28 + 20 + 40 - 70 + 47 - 44) - 74) + 9 + 18 + 54 - 45) + 22 - 40 - 69 + 3) - 71 - 96 - 63 - 73 + 23 - 94 + 29 - 16 - 70 + 40 - 26 + 81 + 43 + 24 - 70) + 27 + 17 - 12 + 96 - 10 + 13 - 88 - 23 - 41) - 31 + 6 + 91 + 72 + 87) + 19 + 9 + 82) - 49 - 67) - 66 + 48 + 94 - 93) + 67 + 77 + 46 + 1 - 90 - 1 - 28 - 10 - 65) + 91 + 17 + 61 - 16 - 15 + 6) + 68 + 61 - 89 - 79 - 97 + 97 - 75 - 19 + 52 - 58 + 46 + 91 - 69 - 5 - 77 - 24 - 56 + 44 - 19 + 91 + 27 + 56 + 73 + 48 - 52 - 54) + 92) + 36 + 26 + 13 + 98) - 35 + 69 - 13 + 47 - 23 + 0 - 76) + 82 - 88 - 36 + 67 + 12 - 99 + 20 - 11) + 1 + 87 + 34 + 82 + 90 + 79 - 74) + 91 + 85 + 79 - 57 + 83 - 44 + 86 - 75 - 16 + 51 + 64 + 86 + 34 - 68 - 98) + 87 + 71 + 25) - 50 - 8 - 75 - 20 - 32 + 84 + 40 - 31 + 47 - 13 + 78 - 45 + 5 - 89 - 23 - 40 - 84 + 25 + 82 - 65 + 69 - 68 - 57) - 62 - 95 + 43 + 7) + 91 + 30 + 49 + 22 - 83 + 2 + 97) - 40 + 22 + 71 + 1 + 8 - 98 - 67 - 35 - 39 - 25 - 24 - 77 - 75 - 69 + 70 - 1 - 95 + 23 + 54 + 45 + 93 + 94 + 90 - 58 + 49 - 72 - 24) - 82 - 70 - 23 - 26 + 47 - 51 + 29 + 92 - 55 + 52 + 8 - 45 - 1 + 53 - 86 - 49 - 11 - 32) - 87 + 27 - 5) + 18 + 2 - 74 + 13 + 100 - 14 + 86 + 39) - 10 - 43 - 77 - 84 - 2 + 34 + 65 - 21 + 58 - 5 + 6 - 18 + 73 + 29 - 77 + 15 + 71) - 38 - 53 - 25 - 80 + 58 + 75 - 47 - 84 + 59 - 83 + 96 - 86) + 48 + 7 + 89 - 58 + 37 + 35 + 76) + 25 + 84 - 22 + 75) + 59 - 16 - 7 + 61 + 74 - 6 + 16 + 35 - 54 + 84 + 50 - 46 - 3 + 70 - 24 - 44 + 67 - 55 - 6 - 18 - 56) + 1 - 47 - 24 - 43 - 86 - 59 - 71 + 84 - 19 - 55 - 47) + 60 - 38 + 4 + 65 + 27 + 50 - 30 - 18 - 63 - 4 - 21 + 100 - 66) - 1 - 32 - 81 - 56 - 32 + 100 - 45 + 99 - 74 + 20 - 50 - 75) - 95 + 72 + 2 + 98 + 97 + 78 + 66 - 15 - 70 + 21 + 87 + 70 + 74) - 80 + 75 + 87 - 62 - 81 + 43 - 76 + 0 + 5 + 85) - 34 + 86 + 32) + 70 + 2 - 30 + 14 + 25 - 15 - 5 - 25 + 40 - 80 - 6 - 75 - 93 + 29 - 85 + 44 - 96 + 60 + 5 + 23 + 98 + 25 - 52 + 3 + 4 + 50 - 83 + 73 - 16 + 33 - 49 - 80 + 39 - 77 + 11 - 2 - 43 - 34 - 74 - 16 + 24 - 69 - 79 - 63 - 22) + 74) - 86 - 25 - 41 + 24 + 32 + 59 + 17 - 67 + 12 + 44 - 98) + 70 + 89 - 96 + 89 - 62 + 72 + 29 + 85) - 55 - 49 + 39 - 53 + 74 - 76 + 51) + 33 + 0 - 76 - 87 + 96 - 71 - 30 + 1 - 74 + 90 - 55 - 46 - 66 - 72 + 71 + 60 - 93 + 78 - 0 + 67 - 89 + 50 - 65 + 43 - 76 - 56 - 22 - 94 - 32 + 12 + 90 + 89 - 1 + 0 + 36 - 99 - 95 + 53 + 47) + 6) + 63 - 50 + 21 + 20 + 1 + 1 - 46 - 84 - 4 - 89 - 49 + 50 + 63 + 9 - 38 - 50 - 30) - 73 + 98 - 18 - 10 + 72) + 80 - 97) - 97 + 64 - 57 - 62 + 2 + 98 + 82 - 22 + 85 + 91 + 55) + 30 - 0 + 30 - 57 - 2 - 36 + 21 + 60 + 36 + 64 + 53 - 2 - 45 - 59 + 40 - 41 - 51) + 63 - 16 + 62 + 45 - 75 + 76 - 8 + 33 - 51 - 27 + 78 - 29 + 60 - 47 + 55 - 60 + 85 + 0 + 37 + 86) - 43 + 63 + 73 + 27 + 11 + 5) - 50 - 9) - 31 + 64 + 33 - 12 - 36 + 23 - 5 - 80 + 23) - 58 + 98 + 79 - 13 + 77) - 73) + 67) + 66) - 44 + 5 - 59) + 35 + 65 + 19 - 69 + 37 - 98 + 57 - 7 + 43 + 52 + 71 + 56 + 54 + 16 + 1 + 31 + 54 + 76 + 61 - 14 - 23 - 11 - 72 + 38 + 31 - 37) - 85 - 22 - 61) - 54 - 40 + 40 - 26) - 50 - 33 - 84 + 77 + 85 - 60 - 49 + 88 - 98 + 27 - 89 - 12 - 96 - 15 - 72 + 77 + 54 - 83) - 66 + 91 - 96 - 42 + 22 + 48 + 13 - 87 - 35) - 14 - 11 - 5 + 9 - 83 + 34) - 39 + 87 + 14 + 16 - 59 - 30 - 8 - 61 + 67 - 34 - 13 - 98 + 72 + 90 - 42 - 80 + 26 - 15 - 93) + 6 - 95 + 51 + 6 + 71 - 18) - 70 - 70 - 3 + 65 + 44 + 73 + 71 - 27 + 69 + 97 + 92 - 33 - 95) - 86 + 65 - 43 - 38 - 39 - 84 - 15 - 34 + 50) - 77 - 89 - 74 - 49 + 30 + 28 - 88 + 70 + 50 + 69) + 85 + 79 + 37 - 99 - 6 + 41 - 74 + 13 - 42) + 18 - 99 - 73 + 58 - 61 + 94 + 44) - 38) + 93 + 25 - 24 - 68 + 35 - 81 - 65 - 3 - 21 - 91 + 52 + 14 + 23 - 41 - 12 + 8 - 74 - 79 + 47 - 46 + 83) - 54) + 49 + 22 + 64 - 69 - 43 - 66 + 30 - 84 - 8 + 78) - 26 - 63) + 65 + 39 + 86 + 55 + 67 - 93 - 13 - 31 + 81 + 29 + 62 - 13 + 4 - 55) + 54 + 28 - 1 + 33 + 81 - 89 + 13) - 48) - 7 - 41 - 64 + 42 + 39 - 88 + 93 + 56 + 2 - 39 - 27 + 25 + 3 - 44 + 91 - 38 - 16 - 73 - 75 - 18 + 12 + 9 - 93 + 36 + 92 + 9) + 66 - 29 + 77 + 79 - 9 - 30 - 85 + 73 + 83 - 15 - 89 + 92 - 57 + 21 + 35 + 59 - 87 - 83 + 51 - 69 + 3 + 80 + 47 - 65) + 5 - 52 + 48 + 59 - 19 + 55) + 51 - 51 - 28 + 37 + 98 - 37 + 97 - 44 - 90 + 41 + 65 + 18 + 58 - 34 - 100 - 80 + 21 + 47 + 68 + 80) + 59 + 20 + 74 - 20 + 68 - 48 + 97 - 6 - 16 + 46 + 26 + 12 + 94 + 39 - 83 - 3 + 20 + 52 - 33 + 60 + 95 + 97) + 2 - 51 + 48) - 36 - 67) - 72 + 19 - 33 - 76 - 45 - 84 - 36 - 15 + 37) - 70 - 55 + 56 + 47 - 85 + 75) + 46) + 22 + 75 + 36) + 7 - 1 + 54 - 76 + 72) + 39 - 82 - 83 + 81 - 57 - 55 - 41 + 38 - 7 + 38 - 12 + 30 - 21 + 15 - 84 + 60 - 38 + 16 + 33 - 92 + 45 - 92 - 88 - 39 - 14 + 64 + 30 + 41 + 53 + 4 + 50 + 11 - 73 + 4 + 53 + 12 - 16 + 37 - 3 - 16 - 13 - 2) - 85 + 68 - 55 - 41) - 90 + 51 - 21 - 99 + 13 + 96 + 74 - 52) - 96) - 79 - 83 - 53 - 84 - 78 + 94 - 19) + 60 - 51) + 53 - 23 - 35 + 73 + 92 + 41 + 43 - 18 + 97 + 25 + 16 + 29) + 86 + 34 + 7 - 54 - 35) + 38 - 66 - 23 + 58 + 82 - 29 - 80 - 74) - 34 + 20 - 95 - 3 + 99 + 35 + 77 - 64 - 43 - 60 - 81 + 90 + 36 - 21 - 39 - 99 - 68 + 7 - 18 - 3 + 60 - 17 + 98 + 84 - 45 + 70 + 89 - 22 - 13 + 30 + 81 - 7 + 99 - 9 + 15 - 80 + 27 - 14 - 78) + 33 + 17) + 34 - 22 - 70 - 61 + 47 + 6 - 90 - 85 - 46 - 60) + 23) + 66 + 93 - 32 + 74 - 58 + 8 + 47 + 58 + 56 - 23 + 59 + 31 + 52) + 54 - 43) - 73 - 90 - 87 + 43 + 48 + 19 + 17 - 36 - 52 + 42 - 60 - 19 - 18) + 0) + 62 + 34 + 87) - 88 - 74 + 4 + 53 - 56 - 72 + 73 + 96) + 40 - 29) + 32 - 54 + 15 - 5 - 50 - 92 - 49) - 87 + 27 - 32 + 11 + 38 + 16 + 48 + 90 + 10 - 92 + 15 - 49 + 86 + 16 - 77 - 86 - 87 - 92 - 37 - 26 - 11 + 83 + 64 + 34 - 49 - 61 - 99 + 74) + 87 + 99 + 2 + 51 - 60 + 38) + 5 - 53 - 88 - 52 + 33 - 25 - 91 - 16 - 92 + 89) - 52 - 66 + 66 - 35) - 98 + 31 - 19 - 57 - 60 + 41 + 36 + 32 + 76 + 37 + 27 + 39 + 68 + 64 + 10 - 58 + 64 + 99 - 43 - 76 - 8) - 55 + 47 + 32 + 92 + 74 - 1 - 7 + 10 - 15 - 63 + 23 - 67) + 19 + 5 + 59) - 46 - 26 + 72 - 37 - 75 + 21 - 91 - 63 - 49 + 60 + 19 - 78 - 48 - 97 - 61 + 57 + 12 - 92 - 82 + 2 - 87 + 77 - 65 + 91 - 89 + 32 - 84) - 81 - 27 + 74 + 82 + 5 - 19 - 67 - 75 - 58) + 39 + 7 - 66 - 66 - 99 - 36) + 59 - 8 - 67 + 0 - 85 - 44 + 91 + 10 - 31 + 43) - 0 - 84 - 48 + 97 + 91 - 26 - 53 - 51 + 42 - 70 - 74 + 28 - 16 + 2 + 36) + 62 + 29 - 14 + 42 + 96 + 48 - 39 - 12 - 24 - 70 - 66 + 38 + 1 + 39 - 26) - 92 - 84 - 53 - 81 - 41 - 85 + 52 + 66 - 97 + 83 - 23 + 18 - 97 - 33 - 56 - 14 - 26 - 64) + 7 + 51 + 41 + 39 + 72 - 90 + 47 + 46) - 71) - 81 + 38 - 81 + 39 - 78 - 16 - 12 - 0 - 81 - 29 - 65) - 47 - 7 + 76 - 82 + 85) + 23 + 12 + 56 + 32 + 92 - 11 - 4 + 28 + 50 + 74 + 16 + 35 + 67 - 37 + 100 + 2 - 79 - 10 + 10 + 1 + 62 - 92 - 92 + 78 + 72 + 89 + 52 + 72 + 97 - 48 + 62 - 96 + 18 + 52 - 35 + 52 + 91 + 1 - 87 - 3 + 33) + 97 + 12 + 58 + 36 - 87 - 52 - 35 - 41) + 85 - 74 - 71 + 18 + 70 - 60 - 47 - 51 - 71 - 58 - 32 + 20 + 80 - 95 - 7) - 48 + 10 - 57 - 52 - 82 + 55 - 80 + 35 - 99 + 8 - 1 - 88 + 83 + 50 + 91 - 70 - 32 - 8 - 14) - 28 - 3 + 41 + 59 + 82 + 92 + 25 - 55 + 60 + 28) - 16 - 19 - 4 + 39 + 39 - 42 + 71 + 7 + 53 + 87 - 13 - 81 - 79) - 68 + 89 - 44 - 86 - 24 - 7) + 70 + 0) + 51 + 40 + 83 - 19) + 90 + 74 - 16 + 71 + 64 + 31 - 69 + 98 + 72 + 17 - 46 + 50 + 85 - 33) + 17 + 1 + 33 + 71 - 67 + 53 - 6 + 71 + 55 + 52) + 76) + 25 + 13 - 81 + 88 - 82 - 86 + 6 - 84) + 44 + 43 + 69 + 60 + 25 - 10 - 4 - 72 + 3 + 11 + 35 + 9 + 67) + 44 + 65 - 77 - 22 - 40 - 82 + 23) + 5 + 86 + 41 - 79 + 69 - 78 + 93 - 16 + 30 - 56 - 90 - 55 + 37 + 17 + 100 - 79 - 27 - 0 - 15 + 66) + 77 - 53 + 98 - 34 - 47 - 26 - 41) - 95 - 0 + 67 + 17 + 15 + 57 + 55 + 15 - 91 - 4 + 57 - 42 + 46 - 64 + 59 + 83 + 90) + 28 + 97 - 92 - 13 - 71 - 31 + 64 + 74 - 89 + 81 + 62 + 58 - 5 - 64 + 77 - 5 + 67 - 52 + 6) - 37 + 6 - 91 + 78 - 58 + 47 - 91 + 4 + 57 - 25 + 36 + 46 + 62 + 40 - 57 - 99 - 62 + 11 + 47 - 54) - 16 + 9 + 23 - 65) - 70 + 49 - 89 - 40 - 71 - 22 + 58 + 10 - 24) - 60 - 9 - 18 + 76 - 40 - 88 + 30 + 11 - 30 - 9 - 18) + 86 - 74 + 97) - 65 - 100 - 78 - 96 - 80 - 13 + 52 - 30 + 75 - 15 - 97 - 22 + 100 - 95) - 74 - 94 + 4 - 82 + 47 + 93 - 95 - 100 + 10 - 80 + 23) + 29 + 91 + 62 + 97 - 20 + 48 + 8) - 25 + 4 - 59 - 0 - 41 + 35 + 24 - 40 - 22 - 36 - 94 + 36 + 64 + 11 + 48 - 40 - 79 + 87) + 84 - 5 + 7 - 4 - 35 + 9 - 17 + 34) - 32 + 63 - 59) - 28 - 42 + 17 + 63 - 19 + 85) - 32 + 68 - 74 - 28 - 91 - 41 + 72 - 90 + 34 - 63 + 68 - 70 + 52 + 61 + 18 - 91 - 65 - 88 - 75 + 16 + 45 - 39 + 63 + 36 + 91) + 80) - 56) + 52 + 17 + 52 + 92 - 38 + 1 + 11 - 36 - 50) - 55 + 93 - 99 + 74 + 92 + 83 - 3 - 77 - 27 + 11 - 27 + 9 - 19 + 71 + 37 - 22 - 51) - 29 + 80 + 91 + 28 + 35 - 66 + 51) + 45 + 93 + 88) - 38) - 19 + 80 + 37) + 62 - 82 - 57 + 16 - 45 + 57 + 69 - 83 - 57 + 72 + 4 - 15 + 22 + 60 + 42 + 60 + 10 - 18 + 5 + 5 - 11 + 29 + 9 + 69 - 55 - 94 + 61 - 84 - 61 + 64 + 69 + 27 + 20 - 36 + 48 + 58 - 21 - 96) - 41) + 1 + 15 - 26 + 74 + 42 + 44 - 91) + 41 - 41 - 59) + 55 + 60 - 21 - 90 + 3 - 26 - 89 + 26) - 55 - 36 - 35) + 74 - 77 + 85 + 31) - 64 - 17) - 28 + 4 + 86 + 76 + 81 + 83) + 82 - 10 + 67 + 45 + 61 + 92 - 33 - 3 - 34 - 46) - 95 + 53 + 9 + 34 - 80 - 79 + 2 - 15 + 15 + 31 - 12) - 20 + 36 + 92 + 81 - 50 - 56 - 39) + 45 + 23 + 11 - 100 + 1 - 91 + 50 - 1 + 24 - 60 + 34 + 11 - 78 - 21 - 64 + 20 - 90 + 96 - 66 + 78 - 84 - 63 + 19 + 32 - 26 + 11 + 42 + 52 - 22 - 6 - 57 + 16) - 53 + 5) + 32 - 43 - 31 - 13 - 97 + 35 - 12 - 66 + 81 - 59 - 93 + 54 - 23 - 36 + 94 - 39 + 77 - 17 - 42) + 56 - 67 - 13 + 28 - 10) - 65 - 65 - 91 + 86) + 22 + 12 + 83 + 16 + 88 - 77 - 41 + 41 - 94 - 4 + 85) + 6 + 55 - 87 - 39 + 12 + 22) + 45 - 24 - 28 + 65 + 85 - 66) - 50 + 81 - 76 + 27 + 76 - 61 - 18 - 25 + 83 - 25 + 83 - 79 + 78) + 30 - 93 + 78) + 62 - 10 - 49 + 3 - 55 - 100 - 40) - 38 - 38 + 92 + 48 - 8) + 90 - 6 + 52 - 27 - 45 - 73) + 66 + 80 + 31) + 44 - 91 - 77 + 5 + 19 + 47 + 86 + 17 - 24 + 23 + 87 - 69 - 54 + 45) - 47 + 81 - 68 + 38) - 76) + 56 + 11 + 28 - 26 + 49 + 57 + 89 + 85 + 44 - 57 - 39 - 51 + 75 + 16 - 57 + 58 - 82 + 54 - 6 + 49 - 98 + 95 - 90 - 99) + 90 - 9 - 35 - 80 + 64 - 3 - 64 - 4 + 93 + 91 - 1 + 61 - 4 + 96 + 9 + 42 + 25 + 1 + 34) - 39 + 8 - 39 - 54 - 0) + 24 + 15 - 60 - 68 - 20 - 51 + 67 + 97 + 38 + 74 - 34) - 29 - 74 - 53 + 55 - 70) - 43 + 3 + 13 - 1 + 76) + 34 + 19) + 14 + 94 - 92 - 32 - 7 + 48 + 51 + 81 + 40 + 86) - 30 + 8 + 60 - 32 - 52 + 14 + 48 + 67 - 33 - 19 - 90 - 47 + 33 - 47 - 4 - 78 + 61 - 98 - 80 + 64 - 23 - 54 - 21 + 28 + 42 + 77 - 26 - 18 + 81 - 29 - 98 + 66 - 78 - 5 - 38 - 66 - 38 + 26 + 95 + 37 + 59 + 37 + 83 + 0 + 22 - 99 + 82 + 42 - 33 - 87 - 16 + 42 + 15 - 17 + 19 - 63 + 98) + 9 + 65 - 55) + 41 + 6 + 94 + 94 - 96 + 85 + 12 - 32 + 54 + 14 + 73 + 24 - 12 + 7 + 75) - 1 - 24 + 41 - 66 - 91 + 23 - 25 - 17 - 29 - 84 - 53 + 85) + 96 - 82 - 12 - 37 - 29 + 57 + 54 - 2 - 73 + 20 - 79 + 59 - 81 - 51 + 80 - 62 + 45 - 42 + 29 + 11 + 66 + 75 - 69 + 44 + 94 + 6 + 30 + 2 - 95 + 70 + 64 - 90 - 35 + 93 - 87 - 22 - 6 - 19 - 24) + 91 + 81 - 1 - 33 - 6 - 55 - 21 - 40 - 5 - 81 - 44 - 47 + 17 - 99 + 58 - 10 + 58) + 70) - 81 + 94 - 66 - 96 + 37 + 96 - 0 + 0 + 32 + 53 + 61 + 91) - 84 - 75 + 51 + 82 + 42 - 32 + 59 + 79 + 58 + 68 + 0 + 99 + 33 - 84 + 22 + 79) + 91 + 97 + 59 - 68 + 12 - 49) + 49 - 83 + 55 + 73 - 66 + 72 + 25 - 72 + 35 - 55 - 39 + 2 - 66 - 92 + 55 + 47 + 21) - 22 + 94 - 56 + 15 - 37 - 39 - 27 + 50 - 63 + 8 + 99 + 69 + 88 + 29 - 51) - 39) - 28 + 88 + 78 + 92 + 62 + 34 + 63 - 62) - 43 + 5) - 86 + 87 - 71 + 89) + 27 - 73 + 89) + 16 - 58 + 25 + 69 + 16 - 36 - 17 - 67 + 17 - 56 - 9 - 81 + 19 + 47 + 45 - 38 - 28 - 27) - 62 - 3 + 64 - 8 + 53 + 32 - 28) + 96 - 91 - 37 - 68 + 26 - 37 + 74 + 35 + 20 - 40 + 51) + 76 + 32 + 58 - 97 + 62 + 88 + 30 - 65 - 53 - 100 - 1 + 100 - 65 - 96 + 77 - 10 - 2 + 45 + 18 - 13 - 64 - 43 + 81) + 7 - 41 + 46 + 87 - 84 - 86 - 15 + 59 + 90 + 49 + 29 - 40 + 83 + 67 + 0 + 17 - 88 - 35 - 43 + 72) - 38 + 21 - 40 - 86 + 51 + 31 + 6 + 91 - 82 - 40 + 99 - 84) - 65 - 80 + 40 - 56 - 31 - 15 + 99 + 22 + 6 + 48 - 78 + 67 - 38 - 3 - 74 - 44 + 66) - 78) - 33 - 44) + 91 - 63 + 61 - 86 - 86 + 97 + 9 + 81 - 58 + 7 + 69 + 8) - 21 + 46 + 31 - 1) - 35 - 67 + 54) + 49 - 77 + 33 - 86) - 100 - 24 - 5 - 81 - 34) - 66 + 40 + 25 + 57 + 61 - 53 + 82 + 26 - 71 - 46 - 3 + 65 + 29 - 11 + 22 + 67 - 84 - 56 + 76 + 77 - 100 + 29 - 53 - 28 - 100 - 26 - 77 - 68 - 25 - 13 - 35 - 40 - 14 - 84 - 93 + 75 + 100 + 29) - 35) - 11 + 68 + 58) + 2 + 77 + 44 - 29 + 84) - 92) - 31 - 14 + 79) - 17 - 90 + 57 - 2 - 49 + 24) + 84 - 64 + 96 - 26 + 95 + 79 + 60 - 51 + 86 - 46 + 41) - 52 - 56 - 2 - 63 - 65 - 88 - 3 - 80 + 12 + 2 - 48 - 61 + 27) - 16) - 68) + 94 - 63 - 69 - 94 + 26 + 64) + 89 + 19) - 98 - 54 + 39 - 72 - 7 + 49 + 79 - 79 + 70 + 7 - 61 + 72 - 51 + 37) - 16 - 46) + 29 - 7) - 9 + 48) + 96 - 88 + 52 + 51 + 42 - 15 + 100 + 69 - 14 + 57 + 57 + 53 - 10) + 34 + 67 + 57 + 79 + 87) - 42 + 46 + 35 + 71) + 5 - 23 - 33 - 41) - 79 - 12 - 92 + 85 + 16 + 34 - 19 - 1 - 14 + 93 + 78 - 3 + 62 - 28 - 17 - 38) - 49 - 16 + 65 - 93 - 92 - 100 + 63) - 74) - 86 + 9 - 83 - 5 - 93 + 95 - 67 - 95 + 32 + 35 + 43 - 97 - 81 - 95 - 93 + 60 - 82 - 21 - 71 + 14 + 19 - 49 - 21 + 28 - 58 + 80 - 73 - 33 + 20 + 16 + 82 - 3 + 9 - 30 - 6 + 53 - 33 - 11 + 21 - 18 + 31 - 64 - 62 + 34 - 36) + 58 + 56 - 62 - 36 + 16 - 81 + 73 - 17 + 73 - 10 + 1 - 46 - 89 - 76 + 76 + 87 - 23 + 49 + 37 - 44 + 57 + 45 + 95) + 66 - 34 + 54 + 86 - 66 + 79 + 45 + 72 + 41) - 38 - 35 - 73 - 13 - 68 + 74 + 67 - 91 - 7 + 5 - 52 + 16 + 9) + 50 - 92 - 46 - 5 - 91 - 68 - 81) + 8 - 30) - 46 - 80 + 71 + 46 - 56 + 63 - 100 + 89 - 10 - 33) + 96 - 12 + 20 - 33 - 4 - 18 - 28 - 9 + 51 + 68 + 84 - 31 + 52 - 4 - 62) + 80 + 66 + 31) + 40 - 66 - 16 - 64 + 17 - 42 + 34 + 43 + 30 + 49) + 51) - 71 - 17) + 22) - 27 - 27 - 13) - 19 - 51 - 29 - 51 - 29 - 18 - 14 - 74 - 16 + 10 + 87 + 47 + 42) - 61 + 64 - 47 + 62 + 13 - 24 + 29 + 52 + 5 + 69 + 1 - 49 + 96 - 8 - 18 + 10 - 14 + 25) - 71 - 72 + 38 + 78 - 13 + 68) - 74 + 95 - 86) - 40 + 41 - 62 - 10 - 83 + 43 + 45 + 16 - 60 + 67 + 69 + 38 + 98 - 3 - 81 + 41 - 33 - 100 + 50 - 65 + 20 + 40 + 38 - 99 + 12 + 53 - 42) - 61 + 54 - 100 - 10 - 4 + 22 - 38 - 25 + 60 - 61 + 19 - 20 + 41) + 42) - 40 + 56 - 86 + 0 + 90 + 59 + 2 - 92 + 17 + 100 - 57 + 60 + 4 + 53 + 94 + 33 - 0 + 6 - 25 + 78 - 75 + 60 - 49 - 78 + 95) - 66 + 39 + 78 + 56 - 84 + 3 + 42 - 44 + 97 + 22) - 27 + 26 - 29 - 85 - 35 - 69) - 49 - 44 + 72 + 71 + 85 + 15 - 2 - 40) - 96) + 77 + 72 + 59) - 76 - 74 + 69 - 25 - 55 + 51 + 13 - 60 - 42 + 73 - 41 - 32 - 26 + 90 + 95 - 59 - 8 + 15 - 3 - 58) + 30 + 64 + 52 - 19 + 88 - 1 + 82 - 89 - 14 - 21 - 98 + 97 - 60 - 41 + 16) - 55 + 59 - 74 + 95 - 0) - 39 + 74 - 41 + 83 + 97 - 8 + 48 - 76 + 50 - 79 + 37 + 83 - 77 - 2) - 29 - 17 - 3 - 68 + 81 + 42) + 89 - 64 + 41) - 99 + 83 + 51 + 10 - 42 + 45 + 74) - 40 + 30 - 36 - 13 - 56 - 58 - 20 - 44 + 55) - 15 - 9 + 75 + 57 + 39 - 12 - 56 + 38 + 47 - 91 - 45 + 10 - 26 - 3 + 72 - 24 + 81) + 57 + 90 + 91) + 55 + 19 + 77 + 11 + 54 + 2 + 38 + 68) + 80 - 51 - 80) - 64 - 99 + 19) + 67 + 52 - 88 - 49 - 33 + 24 - 65 + 34 + 28) - 38 + 36 - 59 - 17 - 10 - 81 + 58 + 55 - 90 - 46 + 29 - 78 - 46 - 21 - 5 - 24 - 0) - 99 + 77 + 81 + 14 - 6 - 11 - 32 - 12 - 58) - 2 + 69 + 63 + 48 + 55 - 73 + 95 - 76 - 19 + 41 + 65 - 34 + 92 - 93 - 37 + 55 - 37 - 36 - 4 - 47 - 75 + 42 - 13 - 72 - 51 - 51 + 35) + 1 + 63 + 44 - 26 - 8) + 92 - 46 - 52 - 10 + 56 + 79) + 93 + 80 + 1 - 29 - 28 + 16 + 11 + 10 + 49 - 45 - 27 + 54 + 74) + 12 + 18 + 81) + 46 + 8 + 24 + 14 + 7 - 55) - 40 - 42 + 19 + 89 - 55 - 77 + 70 - 18 - 20 - 84 - 16 - 29 - 50 + 78 - 96 + 4 + 88 + 69 + 71 - 58 + 64 + 64 - 71 - 43 - 55 - 43 - 0 - 0 + 43 + 68 - 27 + 90 - 56 + 88 + 45 - 93 + 28 + 67 - 69 - 88 - 7 + 32 - 27 + 52 + 78 - 10 - 93 + 55 + 74 - 76 - 63 - 72) + 93 - 84 - 61 - 83 - 57) + 36) + 80 + 77 + 44 - 33 - 1 - 36 - 24 - 96 + 70 - 64 + 35) - 24 - 20) - 69 + 81 - 57 - 92 + 52 + 51 - 93 - 61 + 19 + 81 - 20 - 29 + 19 + 69 - 56 + 43 - 47 + 3 - 29) + 53 + 42) - 69 + 38 + 58 + 54 - 98) + 69 + 38) - 80 - 88 + 84) - 33 + 97 - 63 - 54 + 25 - 5 - 30 + 30 + 23 - 24 + 95 + 36 - 0 + 12 + 96 + 0 - 8 - 82 - 52 + 16 + 52 - 20 + 1 - 23) + 80 + 72 + 83 + 58 - 41 - 98 + 75 + 62 + 98 - 87 + 57 + 76 - 73) + 20 + 32) + 29 - 9 - 88) - 69 + 92 + 78) + 83) - 60 - 80 - 87 - 44) + 43 + 72 + 37 - 78 + 41 - 5 + 36 - 58 - 39 + 56 - 34 + 28 + 6) - 57 - 16 + 61 + 50 - 90 - 54 - 98 + 67 + 34 + 58 - 23 + 95 - 56) - 18 + 67) - 80 + 62) + 66 - 70 + 30 + 35) - 48 + 67 + 71 - 60 - 83 - 100 + 9 - 78) - 81 + 71 - 90 - 94) + 10 - 14 + 64 + 53 - 17 - 66 + 19 + 27 - 41) + 21 - 21 + 53 - 67 + 97 - 17 - 35 - 59 + 35 + 77 - 26 + 98 - 34 - 66 + 78 - 92 + 92 + 45 - 72 - 72 - 60 + 85 + 92 + 51 + 63 - 76 + 38 + 55 - 89 - 84 + 93 - 34 + 64 - 41 - 85 + 70) + 48 + 8) + 22 - 55 - 94 - 55 + 5 + 43 + 61 - 35 + 37 + 63 - 63 - 100) + 87 - 23 - 98 + 35 - 71 - 70 - 33 + 2 + 76 - 3 - 97 + 28 + 64 + 81 + 91 + 100 - 89 + 27 - 28 - 20) - 76 - 30 - 41 + 13 - 2 + 98 - 49 + 8 + 56 - 49 + 57 + 92 + 88) + 96 + 79 - 76 + 76 + 65 + 20 + 18) - 84 + 30 - 48 - 0 - 66 - 35 + 69) + 1 - 85 - 82 - 14 - 28 - 43 + 21 + 18 + 60 + 40 + 72 - 61 - 60 + 17 + 2 - 80 - 10 + 45 + 87 + 87 - 48) - 41 - 52 + 59 + 49 - 97 + 83) - 39 + 58 - 69 + 81 - 23 + 53 - 24 + 74 + 13 - 40 + 26 + 70 - 79) + 61 + 19 - 87 + 66 + 31 + 28 + 17 - 23 - 50 - 95 + 52 + 78 - 85 + 98 + 59 - 59 - 12 - 73 + 71 + 83) + 12 - 49 - 3 - 13 - 64 - 92) + 7 + 85) - 8 - 49 - 30 - 73 + 51 + 97 + 69 - 49 - 67 + 6 - 54 + 89 - 57 - 15 + 21 + 70 + 5 + 80 - 79 + 9) - 9 - 89 + 37 - 76 + 51 - 35 - 79 + 89 + 13 + 41 + 41 - 39 + 3 - 33) - 58 - 39 + 28) - 89 + 33 + 4 - 77 - 22 - 67 - 89) - 38 - 82) + 13 - 57 + 69 + 81 - 84 - 39 - 21 - 89 + 19 + 64 - 27 - 6 - 83 - 5 - 33 - 72 + 63) - 11 + 16 - 40 - 72 + 25 - 93 - 63) - 33 - 29 + 79 + 100 + 64 + 13 + 33 - 72 + 80 + 61 + 96 + 74 + 35 - 97 + 53 - 91 + 17 + 98 - 1 + 25 - 44 + 17) + 67 - 53 + 28 + 11 - 33 + 46 - 74 + 28 + 17 - 11) - 12 - 57 - 45 - 59 - 11 + 40 + 24 - 16 - 46 - 85) - 56 - 64 + 40 + 20 + 50 - 90 + 80 - 64 + 24 + 96 - 30) + 93 - 87 - 0 - 6 - 4 + 48 + 91 - 89 + 49 - 60 + 4 + 70 - 49 + 55) + 80 - 84 + 59 - 21 - 97 - 56 - 33) - 16 + 100 + 61 + 91 + 38 - 76 + 29) + 70 - 84 - 90 - 25 - 69 + 61 - 58 - 24 + 21 + 94 - 77 + 92 + 22 + 15 + 78 + 21 - 29 - 94) - 38) + 71 - 34 - 64) + 51 + 33 - 16 - 92 - 40 + 46 - 93 + 11 + 32 + 93 - 78 + 26 - 36 - 54 - 20) + 31 + 33 - 82 + 26 + 65 + 5 + 97 - 1 - 97) + 43 - 87 - 12 - 10 - 71 + 68 + 41) - 50 - 55 - 92 - 93 - 58 + 79 - 0 + 65 - 64 - 47 - 64 + 91 - 48 - 51 - 44 + 50 - 96 - 26 + 81 - 92) - 63 + 99 - 63 - 43 - 48 - 84 - 43 + 92 - 30 - 18 - 63) - 34 - 60 - 95 + 85) + 15) + 86 + 65 - 31 + 42 - 9) - 15 - 82 - 59 - 37 + 56 - 24 + 63 - 78 + 41 - 2) - 45 - 49 - 48 - 31 - 74 - 96 + 67 + 19 + 68 - 44 + 30 + 73) - 47 + 80 - 24 - 11 + 64 + 20) - 53 + 44 - 73 + 92 - 46 + 80 + 19 + 44 + 73 - 38) + 76 - 86 - 67 + 15 + 24 - 74 + 17 + 94 + 11) - 5 + 7 - 10 - 53 - 89 - 4 + 36 - 79 - 22 - 86 + 46 - 4 - 14 - 63 - 4 - 39 - 50 - 37 + 64 + 0 + 51 + 22 - 57 + 22 + 12 - 64 + 69 - 41 - 31 + 62 - 33) - 60 + 56 + 77 + 41 + 82 + 77 + 71 + 48 + 15 - 54 + 30 + 74 - 25 - 99 - 83 - 48 + 28 - 29 + 93 + 46 - 7 + 70) + 53 - 84 + 79 - 73 + 57 - 96) - 26 - 14 - 25 - 100) + 73 - 19 - 39 - 20 - 83 - 30 + 17 + 38 + 95 + 22 - 94 + 49 - 5) + 94 + 98 - 64 + 22 + 25 + 0 + 10 - 24) - 18 + 59 - 55 + 40 - 88) - 55 + 68 + 27) - 55 + 5 + 28 - 70 - 100 - 76 - 77 - 2 - 35 + 60 + 83 - 94 - 91 - 83 - 63 - 37 - 35 - 83 + 22 - 22 - 38 + 91 - 44 + 90 - 52) - 46) + 96 - 15) - 59 + 35 + 47 - 91 - 28 + 21 + 2 - 39 + 18 + 33 - 6 - 100 - 9 + 31 + 45 + 50 - 90 - 45 + 62 + 98 + 17 - 27 + 12 - 70) + 2 - 80 + 71 - 15 - 88 + 98 - 37 + 64 + 20 - 44 + 15 - 7 - 93 + 80 + 72 - 50 - 72 - 7 + 82 - 53 - 75 + 23 - 25 - 24 - 45 - 48 + 65 + 9 - 96) + 6 - 85) + 59 - 34 + 13) - 2 - 96 + 72 - 91 - 87 - 93 + 97) + 9 - 64 - 77 + 31 - 63 - 76 - 76 + 98 - 19 - 92 - 38 + 92 - 67 + 31 + 23 - 18 + 73 + 62 - 63 - 21 + 32 + 67) + 0 + 6 - 76) - 65) - 19 + 86 - 88 - 50 + 99 + 58 - 2 - 93 - 6 - 29 + 62 + 55 - 12 - 0 - 99 + 37) + 73 + 35 + 70 + 33 - 38) - 31 + 30 + 60 + 32 + 21) - 55 + 46 - 9 + 91 - 42 - 70 - 17 - 36 + 9 - 20 + 79 + 31 - 44 + 33 + 0 + 75 - 53 - 7 + 55 + 16 + 69 - 17 + 55 + 54 - 8 + 19 - 99 - 95 - 15 - 20 - 84 + 49 - 76 - 77 + 94 - 41 - 31 + 66 + 71 - 81 - 11 - 29 + 91 - 82 - 55 + 49 + 5 + 71 - 71) - 35 - 69 - 73 + 59 - 65 + 28) - 67 + 52 - 76 - 56 + 85 - 80 - 91 + 41 - 62 + 74) + 88 + 75 + 77) - 54 + 94 - 67 + 48 + 87 + 60 + 42 + 11 + 49 - 59 + 64 + 87 - 39 - 88 - 81 - 74 - 71) - 4 + 18 - 90) - 87 - 57 - 77 + 27 + 80 - 3 + 47 - 65) - 35 - 3 + 100 - 63) - 49 + 54 + 47 - 23 + 0 + 68 - 14 - 100 - 45 - 57 + 85 + 54 - 14 + 8 - 11 - 30) + 95 + 13 - 0 - 7 + 38 + 62 + 15 + 81 + 24 + 41 - 12 + 42 + 19) + 31 + 69 + 15 + 67 + 76 + 2 - 21 - 89 - 17 + 88 + 73 - 70 + 1 - 95 - 1 - 67 + 93) - 4 - 88 - 76 - 57 - 31 + 72 - 90 - 31 + 27 + 44) - 78 + 21 - 80 - 26 + 99 + 1 + 81 - 84 + 96 + 55 - 35 + 51) - 77 - 45 - 55 - 73 - 71 - 59 + 49 + 79 - 94 + 96 - 100 + 71 - 7 + 12 + 23 + 95) - 1 + 35 + 13 - 24 + 53 + 64 + 27 + 8 - 77 - 49) + 66 + 69 - 24 + 12 - 92 - 18) - 91 - 0 + 58) + 67) - 3 + 83 + 62 + 69 + 2 + 85 + 15 + 64 + 59 + 58 - 82 + 52 - 81 - 11 - 45 - 51 + 3 - 92 + 100 + 69 + 82 + 13 - 100) - 91 + 40 + 95 - 44 - 47 - 17 + 72 - 19 - 0 - 28 - 13 + 56 - 57 + 79 + 51) + 34 - 48 + 20 + 29 - 43 + 49 - 2 - 67 + 96 - 87) + 74) + 1 - 92 + 100 + 99 - 71 - 62 + 17 + 6 + 99 - 38 - 79 - 95 - 59 - 41) - 51 + 49 + 60 + 57 + 91 + 43) - 14 + 75) - 43 + 90 + 27 + 63 - 77 + 0 - 81 - 65 + 100 - 81 + 34 + 35 - 65 - 64 + 72 - 70 + 100) + 31 - 98 - 29 - 51 + 91 + 45 + 57 + 83 + 81 + 83 + 6 + 84) - 65 - 30 - 49 - 48 + 36) + 97 - 2 + 84 - 98 - 72 - 92 - 42 - 80 - 46 + 40 + 18 + 100 + 47 - 79 - 86 - 88 + 84 + 60) - 43 + 19 + 86 - 36 - 81 - 13 + 54 - 96 + 14 + 78 - 16 + 3 - 41 + 63 + 82 + 2 + 33) - 42 + 10 - 1 + 77 + 13 + 41 - 49 - 25 - 56 - 26 - 71 - 27 + 77 + 13 + 44) + 72) - 11 - 65 + 79) - 4 - 15 + 26 - 97 - 74 + 39 - 56 - 11 - 37 - 21 - 10 + 70 - 22 + 67 - 41 - 51 + 85 + 74 + 78 - 46 - 86 + 23 - 72 + 61 + 30 - 78 + 58 + 63 - 15 - 7 - 42 + 16 - 0 - 78 + 76 - 49 + 34 + 34 + 36 - 41 + 82 + 12 + 72 + 48 + 63 - 17 + 71 + 64 + 13 - 65 + 22) + 8 + 7 + 47) - 52 - 90 - 43 + 99 + 52) + 12 - 53 + 72) + 73 - 21 - 91 + 57 - 26) + 34 - 86 - 8 - 93 + 74 + 31 - 100 - 17 - 36 - 23 + 28 + 6 + 60 - 85) + 48) + 100 - 1 - 29 - 9 + 31 + 39 + 48 - 69 + 55) + 8) - 89 + 42) + 23 - 100 - 59 + 13 + 13 + 10 - 92 - 25 + 45 - 60 + 64 + 42 + 70 + 71 + 94 + 18) + 86 - 24) + 44 - 36) - 34) - 88 - 51 + 35 - 80 - 6) - 97 + 52 + 24 + 52 + 12 + 50 + 93 + 93 + 38 + 53) + 49 - 4 + 61 - 25 - 47 - 98 - 26 + 97 + 74 + 71 + 54 - 72 + 25 - 43 + 62) + 62 + 59 - 67 - 98 - 20 + 13 + 92 - 38 + 56 + 29 - 92 - 31 + 2 + 21 - 76 + 90) + 41) + 24 - 86 - 97 + 26 - 68 - 52 - 91 - 39 - 53 + 55 + 37 + 56 - 69 + 91 + 47 + 66) + 76 + 79 + 6) + 25 + 53 + 59 + 9 + 68 + 49 - 70 - 62 + 79 + 80 - 37 - 3 + 81 - 16 - 20 + 69 + 13 + 19 - 93 + 26 + 55 - 19 + 49 + 64 + 98 + 22 + 93 + 58 - 89 - 1 + 84 - 3 - 92 - 0 + 20) - 20 - 89 - 67 + 49 + 27 - 80 + 73 + 17 + 88 + 83 - 67 - 54 - 14) + 88 - 73 + 55 - 6 - 93 - 47 + 85 + 53 - 18 + 66 + 66 - 82 + 49 - 70) + 12 + 60 + 41 + 39 - 39) - 81 + 17 + 83 + 34 - 52 - 64 - 16 + 88 - 0 + 22 + 18) + 29 + 22 - 75 - 58 + 14 - 2 + 87 + 3 + 36 - 52 - 21 - 82) + 43 + 82 + 53 - 27 - 91 + 93 + 63 + 78 - 80 + 23 - 76 - 6 - 67 - 73 + 28 - 73 + 0 + 20 - 15 + 35 + 100 - 69 + 6 + 25 - 22) - 16 + 69 + 84 - 19) + 49 + 69 + 29 + 67 - 56) - 82 - 52 + 72 + 1 + 76 - 39 - 56 + 85 + 89 + 3) - 32) + 17 + 26 - 100 - 90 + 37 + 93 - 73 - 9 + 11 + 72 + 61) - 65 - 75 - 9 + 96 - 36 + 23 + 20) - 83 - 6 + 0 - 14 + 27 - 82 + 95 + 75 - 84 + 97 - 10 + 64 - 39) - 98) - 70 + 52) + 49 - 48 + 35 - 63 - 40 - 37 - 41 - 18 - 80 - 12 + 62 + 73) + 60 + 38) - 39 + 15 - 98 - 90 + 85 + 4 + 89 + 55 + 75 - 94 + 35 - 29 + 99 - 71 - 70 - 46 + 43 - 74 + 52) + 76 + 77 - 68 + 21 + 44 - 84 + 48 + 80 + 21 + 12 + 37 + 25 - 63 - 18 + 5 - 42 - 77 + 33 + 36 + 96 + 17 + 53 + 72 - 46 - 41 - 47 - 1 + 6 + 9 + 68 - 5 - 69 + 15 + 20 + 15 - 38 - 71 - 96 + 46 + 69 + 99) - 34 - 23 + 39 - 18) + 1 + 7 - 22 + 93 + 33 + 67 + 45 + 97 + 19 - 83 - 18 + 88 - 76) - 80 - 5 + 100 - 58 - 35) + 92 - 49 - 88 - 21 - 4 + 57 - 28 + 37 - 46 - 18 + 23 + 97 - 7 + 17 + 36 - 31 + 9 + 21) + 1 + 16 + 95 - 26 - 15 - 33 + 10 - 59 + 20 + 23 - 81 + 47 - 5 + 95 - 72 + 93 - 95 + 45 + 57 - 2 + 90 + 43 - 26 - 40 + 99 - 15 + 98 + 81 - 3 + 33 + 98 - 17 - 49 - 83 + 91 - 93 + 24 - 63 + 76 + 16 - 12 - 4 + 1 + 31) + 27 + 17 - 91 - 52 - 43 + 31 - 62 + 22 - 74 + 93 + 47 + 32 + 88) - 91 - 39 - 65 + 69) + 31 - 98 - 45 - 50 + 8) + 48 - 25 - 42 + 1) - 95 + 77 - 66 - 100 + 29 - 100 + 97 - 29 - 12 + 39 - 25 - 19 + 52 + 42 + 9 + 34 - 24 + 90 - 62 - 66 + 2 - 4 + 0 + 76 - 92 - 53) + 77 + 19 - 18 + 28 + 15 + 57 - 91 - 90 + 0 + 24 - 96 - 26 - 82 - 48 + 78 + 45 + 29 + 42 - 31 - 63 - 91 - 40 + 10 - 77 - 30 + 96 + 80 + 13) - 86 + 91 - 49 - 55 - 95 - 98 - 43 + 18 - 73 - 90 + 38 + 48) + 48 + 0 - 22) - 58 + 73 + 13 - 0 + 31 - 85 - 19 + 19) - 75) - 29 + 44) - 58 - 67 - 41 - 75 - 20 - 51 + 57 + 33 - 93 + 17 - 85 - 61 - 83 - 11 + 100 - 10 + 88 + 36 - 29 - 64 + 89 - 53 + 73 - 30 - 25) + 28 + 50 - 33 - 7) - 37 - 84) - 45 - 20 + 29 + 85) - 67 - 30 - 11 - 17 + 39 + 30 + 32 - 23 - 47 + 94 - 91 + 93) + 64 - 34 - 100 + 99) - 45 + 41 + 52 + 80 + 68 + 74) - 42 + 83 - 51 - 84) + 83 + 95) + 61 + 96 + 79 - 53 - 47 - 7 + 28 + 94 - 73 + 88 + 97 - 60) - 77 + 17) + 48 - 41 - 22 + 66) + 6 + 20 + 55 + 21 - 47 + 29 - 21 + 51 - 90 - 12 + 65) - 81 - 81 - 46 - 83 - 80 - 53 + 25 - 26 - 83 + 25 - 23 - 20 + 36 - 69 - 13 + 93 + 16 + 6) - 34 + 60) - 72 - 87 + 82 + 62 - 14 + 41 + 66 - 43 - 4 + 65 - 78) + 9 + 47) - 94 + 24 + 9 + 59 + 83 - 72 + 99 + 3 - 50 - 58 - 10) + 16 + 79 - 31 - 69 + 10 + 34 - 54 - 88 + 21 + 73 - 91 - 53 - 25 - 32 + 68 - 17) + 12 + 22 + 0 + 13 + 11 - 6 - 69 + 87 + 15 + 16 - 31 + 79 - 12 + 35 - 57 + 28) + 21 + 43 - 69 - 89 + 92 + 25 + 87 - 46 - 8 + 27 - 39 + 84 - 12 + 86 - 22 - 0 - 1 + 95 + 9 + 14 + 95 - 20 - 47 + 82 - 13 + 59 - 95 + 43 + 61 - 16 + 10 + 46 + 67 + 81 + 100 + 5) - 15 + 37 - 10 + 9 - 3) - 78 - 2 - 27 + 19 + 18 - 58 + 90) + 0 + 67) - 2 + 63 + 48 - 15) + 90 - 4 + 64 - 35 + 65 - 59 - 54 + 34 - 77 - 96 + 62 + 80 - 5 + 43) + 0 + 78) - 69 - 18 - 56 + 11 - 85 + 95 - 63 + 53 - 18 - 87 - 17 + 26 + 37) - 87 - 97 - 7 + 48 + 26 + 0 + 9) - 68 + 89) - 27 - 78 + 91 - 67 + 74 - 68 + 44 + 19 + 93 + 75 + 18 - 9 + 91 + 60 + 83 + 80 - 12 + 44 - 94 + 30) - 13 - 99 - 4 - 79 - 15 + 5 + 69 + 41 - 68 - 99 - 6 + 8 - 0 - 81 - 83 - 3 + 47 - 78 + 93 + 1) + 49 + 43 + 35 - 4) + 69 - 54 - 95) - 46 + 72 - 98 + 70 - 0) + 53 - 90 - 20 - 26 + 76 + 88) - 77 + 23 + 34 - 85) - 66 - 26) - 86) + 94 + 91 - 12 + 34 + 75) - 98) + 30 - 81 + 64 - 36 + 76 + 20 + 86 + 1 + 87 + 79 + 59 - 48 + 100 - 77 - 67 + 47 - 96 - 71 - 34 + 82 + 45 - 70 + 48) - 90 + 63 + 47 + 56) - 81 - 4 + 25 - 64 + 49 + 21 + 59 + 40) + 46 - 80 - 24 - 100 + 33 - 72 - 32 + 49) - 56) - 72 + 75 + 84 + 35 - 83 + 26 - 44) - 10 + 1 - 27 + 46 - 70 - 18 + 68 + 63 + 5 - 75 - 25 - 89 + 22) + 98 + 59 - 81 - 25 - 89 + 95 - 66 + 13 - 86 - 11 - 99 + 94 + 99 + 100 + 35 + 11 + 18 + 59 - 11 - 84) - 72 + 53 - 100 - 1) - 0 + 43 + 68 - 69 + 55) + 81 + 63 + 62 - 65 - 36 - 61 - 27) + 5 + 12 - 56 - 82 + 25 + 42 + 50 + 58 - 20 + 23 - 1 + 75 - 81 + 96 - 29 - 37 - 81 - 73 + 32 + 84 + 100 - 87 + 58 + 0 - 12 - 91 + 93 + 85 + 43 - 82 - 74 - 59 - 79 - 5 - 47 - 98 + 20 - 10 - 31 - 29 - 19 - 27 + 3 - 2 + 73 + 12 - 33 + 14 - 19 + 18 - 13) + 76 - 73 - 30 + 29 + 89 - 99 - 80 + 68 + 64 - 82 + 100 - 94 - 8 + 41 + 35) + 29 - 53 - 50) - 5 + 17 + 72 + 61 + 94 + 41 - 87 + 41 + 68) - 66) + 51 + 61 - 35 + 58 + 99 + 56 - 48 - 34 + 10 + 25 - 23 + 92 - 51 + 13 - 80 - 16 - 0 + 12 - 80 - 91 - 1 + 76 - 100 - 42 + 87 + 61 - 40 + 61 + 17 + 4 + 10) + 72) + 0 + 78) + 24 - 59 - 69 + 83 + 3 - 66 - 78 + 88 - 67 - 84) + 94 - 24) + 1 + 29 - 64 - 90 + 9 + 78 + 18 + 57 - 79 - 62 + 40 - 34 - 46 - 87 - 34 + 90 - 58 - 44 + 17 + 72) - 59 + 68 - 18 + 49 + 13 + 49 + 37 + 25 - 82 + 25 + 7 - 28 + 89 + 16 - 72 + 72 - 22 - 17 + 25 - 76 + 8) - 27 - 49 + 71) + 47) - 54 + 26 + 68 - 64 + 46 + 22 - 70 + 25 - 2 - 44 - 59 - 39 - 5 + 73 - 35 + 11 - 93 + 100) - 66) + 18 - 9 + 75 + 91 + 57 - 56 - 41 + 64 - 49) - 46) + 74 - 93 - 80 - 54 - 50 + 44 + 71 + 91 - 79 - 42 - 94) - 0 + 60 + 65 - 54 + 75 - 88 - 6 - 48 + 34 - 60 - 45 + 76 + 76 + 4 - 13 + 53 - 15 - 52 - 2 + 98 + 71 + 18 + 71 + 31 + 9 + 22 - 82 + 67 - 59 + 31 + 25 + 3 + 28 + 80 + 58 - 65 - 25 + 62) + 68 + 50 + 83 + 69 - 30 + 4 - 10 + 79 + 67 - 70 - 15 - 97 - 5 - 4 - 25 + 90 + 43 - 83 + 39 + 37 + 70 + 58 + 9 + 30 + 92 + 48 - 32 + 26 - 58 + 4 - 20 - 42 - 72) - 38 + 51 + 36 + 100 + 7 - 99 + 22 - 60 - 35 + 0 + 97 - 100 - 62 + 0 - 73 + 32 + 48) - 90 - 59 - 43 - 62 + 35 + 15 + 53 + 17 + 86 - 14 + 76) + 29 + 80 + 99 - 62 + 68) - 28 - 95 - 51 + 43 + 55 + 53 + 30 - 93 - 3 + 39 - 73 - 54 + 76 - 76 + 9 + 7 - 29 - 82 + 87 - 79) + 46 + 31 - 16 + 16 - 73 + 70) - 52 - 75 + 74 - 75 + 80 + 87 + 98) - 10 + 41 + 19) - 56 - 40 + 50 + 44 + 99 - 49 - 98) + 27 + 7 - 75 - 92 - 72 + 85 + 55 + 89 + 78 + 53 + 33 - 1 + 88) + 24 + 4 + 64 + 89 + 59 + 66 - 82 - 64 - 64 + 47 - 68 + 17 + 77) + 99 + 55) - 35 - 43 - 28 + 4 - 67 - 47 - 55 - 27 + 83 - 56 + 2) + 53 + 22 - 66 + 59) - 32 - 16 - 99 + 99 + 20 + 0 + 100 + 70 + 48) - 9 - 91 - 7 + 56 + 14 - 69 + 7 - 24 - 72 - 80 + 64) + 12 - 1 - 71 - 79 + 17) + 29 - 95) + 52 + 61 + 45 + 99) + 7 + 68) - 36 - 47 + 8) - 20 + 89 - 81 - 20) - 65 - 38 + 79 + 32 - 14 + 99 + 71 - 98 + 18 + 40 - 100 + 81 - 70 - 33 - 70 - 7 + 69 + 21 - 96 + 22 + 39 - 28 + 39 + 54 + 32 + 5 - 19 + 31 - 15 - 84 + 7 - 1 + 77 - 74 + 48 + 62 - 14 + 89 + 8 - 16 - 87 + 25 - 36 - 82 - 25 - 93 + 17 + 52 - 64 + 76 + 21) - 20 + 60 - 3 - 21 - 18 - 97 + 89 + 11 - 11 - 50 - 37 - 48 - 46 - 53 - 93 + 88 - 18 - 5 + 60 + 75 - 69 + 28 - 49 + 57 - 43) - 86 - 78 - 60) + 61 - 81 - 36 - 96) + 38 - 49 - 48 + 89 - 70 - 4 - 49 + 53 + 76 + 0 + 63) - 1 + 56 + 8 + 0 + 10 - 70 + 69 - 78 - 63 + 15 + 61 + 58 - 27 + 99 + 55 + 18 + 98 + 28 - 91 + 76 + 41 + 94 + 74 - 64 - 45 + 13 + 38 - 40 + 31 - 67 - 24 + 5 - 28 + 28 + 32 - 17 + 73 - 54 + 44 - 80 - 21 - 9) + 62 - 38 - 58 + 1 + 12 + 60 + 40 - 37 + 54 + 39 - 41 - 30 + 77 - 57 - 20 - 38 - 32 - 46 + 4 + 86 + 99 - 69 + 30 + 79 - 15 + 3 + 96 - 89) + 30 - 79 - 36 + 32 - 25 - 50 - 27 - 14 + 21 + 28 - 23 - 4 + 11 - 59 - 95 + 30 + 35 - 96 - 66 + 76 + 86 + 4 + 41 - 91 - 97 - 91 + 82 - 42 - 84 - 1 - 100 + 18 - 95 - 64) - 27 + 40 + 19 + 99 - 1 - 96 - 97 - 18 - 42 + 21) - 1 + 21 + 14 - 100 - 97 + 30 - 3) + 25) + 93 + 62 + 96) - 73 - 33 + 12) + 91 - 51 + 14 + 83 + 4 - 48 + 55 + 97 - 69 - 85 + 23) - 23 - 60 - 21 - 54 - 79 + 36 - 63 + 21 - 61 + 42 - 2 + 14 - 45 + 21 + 23 - 58 - 99 - 95 + 21 - 13 + 73 + 1 - 22) - 49 - 87 - 63 + 16 - 37 + 37 - 27) - 92 - 37) - 64 - 6) + 17 + 49) + 19 - 19 + 50 - 9 + 43 - 100 - 28 - 45 + 12 - 51 - 23 - 1 + 75 - 80 + 72) + 20 - 64 - 96 + 60 - 91 + 17 + 10 + 18 + 76 - 39 - 1 + 7) - 7 - 64 + 20 - 61 + 30 - 4) + 90 - 38 - 38) + 32 + 15 + 52 + 13 + 41 + 10 + 16 - 53 + 48) + 58 - 33 + 67 - 77 + 3) - 64 - 43 - 98 + 75) + 77 - 90 - 65 + 86 - 100 - 90 + 43 + 75 + 12 + 44 - 32 - 95 + 90 - 0 + 70 + 80 + 90 - 37 - 83 - 63 + 27 - 59 - 82 + 76 + 14 + 53 + 51 + 69) - 89 + 16 - 16 + 8 - 78 - 78 + 70 - 34 - 67 + 17 + 69 - 36 + 31 + 31 - 56 + 91 + 21 + 97 + 52 + 48 + 23 - 5 - 43 - 71 - 39 + 66) + 80 + 30 + 22) + 99 + 69 + 63 - 89 - 73 + 15 + 78 - 65 + 51 + 4 + 4) - 80 - 58 - 24 - 46 + 80 - 8) + 56 - 64 - 66 - 72) - 4 - 8 - 65 + 66 + 34 + 24 + 76 - 72 - 1 + 10 + 45) + 6) - 12 + 44 + 77 - 30 - 53 + 28 - 40 - 16 + 58 + 39 + 97) - 22 - 33 - 78 + 25 + 98 - 24 - 37 + 24 - 97 + 68 + 92 + 22 - 47 + 73 + 42 - 98 - 21 - 79 - 10 + 80 - 83) - 20 - 12 + 41 + 44 + 58 - 12 - 91 + 86 + 91 - 44 - 54 + 95 + 84 - 59 + 77 - 91) + 11 + 95 - 89 + 63 + 14 - 17 + 55 + 62 - 63 - 37 + 76 + 12) - 14 + 30 - 16) + 17 + 75) + 8 - 88 + 4 - 46 + 26 - 53 + 70 - 9 - 92 + 61 + 22 - 78) - 8 + 91) + 82 + 69 - 72 + 88 - 50 + 84 - 64 + 45 + 50 + 95 - 94 - 34 + 91 + 58 - 86) + 11 - 16) - 24 + 89 + 60) + 31) + 10 - 10 - 18 - 64 - 74 + 45 + 22 - 29 - 25 - 79 - 37 - 87 - 70 - 73 - 48 - 89) - 15 + 18) - 6 + 59 + 65 + 59 - 2) - 56 + 62) + 65 + 2 - 59 - 71 - 92 + 65 + 10 - 75 + 46 - 14 + 69 + 49 - 96 + 89 + 9 + 91 - 71 - 18 + 71 + 35 + 40 - 8 + 20 - 45 + 6 - 17 + 91 - 2 - 59 - 45) + 98 - 82 - 75 + 39 + 48) - 25 - 7 - 12 - 50 - 56 - 41 - 99 + 90 - 52 - 99 + 94 - 37 - 9 + 45 + 64 - 81 - 60 + 97) - 51 - 72 + 64 - 92 + 32 + 20 + 25 + 3 - 90 - 49 - 67 - 61 + 71 - 79 + 88 + 27 + 47 - 100 + 27 + 76 + 86 + 73 - 81 + 57 + 70 - 52 + 77 + 20 + 42 + 61 - 72 - 41 - 57 + 80 + 99 - 46 + 45 + 93 - 91 + 100) + 6 - 25 - 92 - 86 - 56 - 9 - 78 - 95 + 86 + 84 + 32 - 59) - 91 - 22 - 83 - 41 + 10) + 83 - 60 - 66) - 39 + 80 + 16 - 89 - 61 + 98) + 61 - 36 - 79 + 71 + 31 + 34 - 4 + 70 + 7) - 26 + 48 + 5 + 43) - 98 + 84) + 33 + 51 - 90 - 3 + 35 + 3 - 25 - 35 + 79 + 65) - 10) - 23 - 71 + 89 + 49 + 12 - 13 - 7 + 10 - 14 - 16 + 10 - 32 - 57 - 45 - 9 + 17 - 78 - 25 - 32 - 26) + 27 + 71 - 32 + 91 - 47 - 73 - 47 - 64 + 51 - 85 + 24 - 46 - 46) - 2 + 46 + 89 + 11 - 100 - 0 - 61 + 9 + 71 - 44 + 54 + 86 - 68) - 55 + 54 - 78 - 72 - 60 - 57 - 94) - 73 + 64 - 36) + 48 - 68 + 24 - 3 + 37 - 79 - 61 - 26 + 41) + 2 + 54 - 13 - 89 - 73 + 61 - 16 - 77 - 93 + 20 - 76 + 0 - 43 - 78 + 5 + 25 - 4 + 72) + 27 - 56 + 57 + 60 - 16 - 38 + 11 - 36 + 50 - 26 - 85 - 72 - 64 - 82 + 82 - 30 + 49 + 98) + 57 - 15 - 58 + 42 - 16 + 24 + 49 + 85 - 84 - 100 + 30 + 82 - 55 - 100 + 17 + 85 + 54 + 98 - 22 - 19 - 100 + 90 + 86) + 55 - 48 - 68 - 88 - 75) + 84 + 6 + 69 - 52 + 71) - 68 + 5 + 69 + 76 - 60 - 27 - 72 - 91) + 19 + 74 - 65 - 69 + 100) - 51 + 84 + 5 - 76 + 49 + 2 + 61 + 93 + 85) - 80 + 1 + 72 - 13) - 10 + 49 + 27 + 28 + 81 - 64 - 18 + 97 + 69 + 78 - 47 - 22 - 96 + 63 - 25) - 100 - 73) + 58 + 15 + 58 + 36 + 53 - 75 + 80 + 44 + 9 - 26 + 73 + 3) - 71) + 48 - 53 + 22 - 17 - 94) - 63 + 2 - 87 - 14 + 69 + 3) - 74 - 32 - 51) - 76 + 51 - 46 - 4 - 50 + 25 - 39 + 53 + 29) + 41 - 19 + 31 + 33 - 47) - 53 - 94 + 3 + 2 + 17 - 96 - 44 - 19 + 37 - 3 + 29 - 20 + 34 - 88 + 87 - 3 + 63 - 53 + 43) + 14 + 89 - 20) + 61 - 58 + 83 + 2 - 32) - 61 - 71 + 87 - 13 + 18 - 66 + 27 + 24 - 45 + 3 - 15 - 62 + 2 - 84 - 97 + 66 + 37 + 38 - 8 - 87 + 86 + 45 + 2 - 27 - 80 - 82 + 26 + 24 + 90 - 39 + 90 - 48 - 86 + 14) - 93 + 41 + 6 + 7 - 62 - 0 + 51 + 27 + 53 - 86 - 77 - 33 - 79 + 61 - 75 + 77 - 69 + 26 + 84 - 97 - 5 + 22 + 94) - 31 - 30 + 84 + 26 - 34 + 0 + 71 - 100 + 21 - 27) + 91 + 28 - 97 + 56 - 40 + 97 + 34 - 85 - 37 - 11 + 25 + 65 + 24 - 95 + 53 - 16 + 32 - 99 - 92 + 69 + 19 - 85 - 66) - 23 - 69 - 69 - 37 + 96 - 74 + 13 - 32 + 94 - 0 - 83 - 67) + 98 - 84 + 92 + 88 - 51 - 32 - 62 + 100 + 39 + 8 + 2 + 36 + 67 + 61 + 82 + 67 + 93 + 47) - 35 + 51 + 66 + 94 + 57 - 39 + 94 + 41 - 14 - 37 + 99 + 56 - 50 - 65 - 15 + 76 - 90 + 32 + 33 - 58) + 82 - 55 + 97 + 34 - 22 - 45 + 29 - 69 + 23 + 9 + 49 + 80) - 64 - 72 - 11 + 9 + 68 - 99) - 61 - 29 - 65) - 52 + 25 - 57 + 84 + 70 + 2 - 6 + 63 - 33 + 3) + 90 - 31 - 67 - 51 - 33 + 18 - 75 + 72 + 0 + 17 + 73 - 12 + 23 - 65 - 41 + 92 + 62 - 41 - 40 + 13 + 99 + 25 - 5 + 38 - 19 - 67 + 89 + 39 - 94) - 92 + 79 - 96 - 93 + 47 - 88 + 28 + 23 + 24 - 87 - 26 + 57 - 19) - 33 - 27) - 4 + 96 - 28 + 98 - 39) - 78 + 0 - 87 - 4 - 70) + 73 + 61 + 77 - 47 - 84 + 49 - 42 - 59 - 35 - 89 + 30 - 45 + 67 + 71 - 73 - 44 + 51 + 28 + 99 - 97 - 12 + 68 - 12 - 3 + 60 - 100 + 53 - 58 - 78 + 19) - 28 + 8 - 57) - 32 + 45 + 71 - 67 - 33 - 36 - 35 - 93 - 62 + 34) + 76 + 86 - 70 - 52 - 79 - 99 + 27 - 43 - 61 + 6 - 88 - 45 + 62) - 91 - 54 - 14 - 47 - 73 + 75 + 99 + 51 + 42 + 84 - 92 - 34 - 37 - 38) - 33 - 17 - 83 + 95 - 30) + 48 - 19 + 19 - 29) - 0 - 32 - 97 - 42 - 40 + 39 - 32 + 58) + 42 - 85 + 72 - 12 + 35 + 48) - 70 - 55 + 67 - 31) + 96 + 43 + 47 + 17) + 86 - 25 + 30 - 53 + 35 + 66 + 49 - 75 + 85 - 25 + 90 - 3 - 65 - 26 + 89 + 42 + 47 + 13 - 75) + 44 + 3 - 88) - 75 - 40 + 4 + 95 - 33 - 7 + 18 - 51 + 47 - 57 + 17 - 16 + 62 - 54 + 70) - 3 + 55 - 31) + 10 + 14 + 63 - 33 + 30 - 97 - 64 + 5 - 39 + 56 + 72 + 97 - 57 + 76 + 23 + 59 - 67 - 33 + 57 - 70 + 0 + 75 + 84 + 73) + 23 - 75 - 0 + 42 - 37 - 46 - 69 - 79 - 9 + 2 - 57 + 39 - 7 + 43 - 14 + 27 + 7 - 10 - 63) - 51 + 49 - 2 + 40 - 40 + 59 - 33 - 18 - 98 + 65 + 41 + 85 + 55 + 75 + 52 + 81 + 84 + 28 + 82 - 38 - 65 - 8 - 58 + 69 + 97 - 96) + 37 + 80 + 53 - 42 + 13 - 93 - 94 - 6 + 43 - 89 - 81 - 77 - 76 - 32 - 84 + 79 + 99 - 94) + 60 + 5 + 36 + 15 + 49 + 23) + 41 + 89 + 100 - 20 - 44 - 86 + 75 - 48) - 62 + 90 - 49 - 51 - 83 + 24 - 97) + 45 + 56 + 29 - 63 - 91 + 3 + 22 + 88 + 75 - 98 - 48 - 68 + 62 + 24 - 10 + 68) - 89 - 100 - 43) - 95 + 62 - 62 - 55 + 54 - 23 + 35 - 44 - 17 + 53 - 37 + 74 - 95 - 74 - 83 + 62 + 82 - 55 - 35 - 42 - 38 + 32 - 54 + 90 - 60 - 80) + 9 + 59 + 27) - 90 + 48 - 37 + 0 - 83 + 94 - 85 - 45 - 43 - 37 + 53 + 47 - 66) - 79 + 61 - 100 - 94 + 76 - 16 + 43 - 28 + 60 + 1) + 15 - 24 + 34 + 39 + 64 - 10 + 26 - 14) - 99 - 68 + 20 - 2 - 25 - 0 - 23 - 73 + 53) + 32 - 24 - 8 - 7 + 99 + 58 - 16 + 4 + 72 - 58 - 72 + 34 + 100 + 19 + 58 - 15 + 44 - 24 - 22 + 23) - 44 + 97 + 64 + 37 - 2) - 12 - 95) - 2 + 53 - 14 - 40 - 18 + 23 + 69 - 2 + 1 + 53 - 12 - 36 + 11) - 49 - 8 - 2 - 7 + 46 + 79 + 91 + 93 + 63 + 98 - 19 - 73 - 10 - 53 - 47 + 79) + 72 + 25 - 49 + 96) - 2 - 4 - 43 + 23 + 71 + 43 + 59 + 48) - 62 + 59 - 93 - 91 - 52 - 46 - 61) + 5 + 19 + 57 + 48 + 81 + 13) + 29 + 65) - 1 - 58 + 67 + 43 + 2 - 3 - 60 + 54 - 98 + 91 + 61 - 4 - 68 + 80 + 8 + 48 - 54) - 81 - 87 + 23 + 85 - 83 + 75 - 52 - 53) - 25) + 87 + 80 + 86 - 30 + 4 - 98 + 83 - 14) - 19 + 55 - 63 + 16 + 35 - 50 - 93 - 18 + 44 - 94 - 16) - 82 + 80 + 38 - 76 - 6 + 40 + 27 + 9 + 23 + 95 + 93) + 74 + 74) - 78 + 20 - 52 + 99 + 91 + 63 - 13) - 20 + 39 - 8 + 40 - 79 + 62 + 32) + 18) - 12 + 58 - 56 - 55) + 79 - 42 + 3 - 23 + 72 + 33 - 65 - 27 - 78 - 27 + 12 + 39 + 55 + 40 + 37 + 19 - 8 - 66 - 28 - 82 - 19 - 59 - 2 + 100 - 97 + 11 - 9 - 85 + 12 + 93 - 35) + 85 - 66) - 12) + 30 - 86 - 86 + 26 + 72 + 45 - 29 - 89 + 58 + 47 + 4 + 74 + 53 - 18 + 71 - 18 - 74 - 24 - 85 + 69) - 19 - 24 + 98 - 0 - 82 + 63 - 56 + 83) - 84) + 30 - 22 - 55) + 15 + 44 + 68 - 52 + 84 + 89 + 38 - 31 + 39) + 65 + 74 - 46 + 81 - 2 + 100 + 48 + 75 + 60 - 78 + 81 + 26 + 80 + 35 - 48 + 15 + 35 - 57 - 62 - 98 + 15 + 51 - 78 + 88 - 99 - 82 - 89 - 79) + 91) - 8) - 73 - 2 - 0 + 56) - 24 - 4 - 25 - 73 - 99 + 76 - 58 - 5 + 1 + 82 + 84 + 69 - 28 - 93 - 89 - 71 - 58 - 55 + 30 + 2 + 44 + 47) + 86 - 33 - 9 - 17 - 90 - 59 - 82 + 17 + 4 - 44 - 10 + 94 - 68 + 49 + 37) - 26 + 73 + 5) + 0 - 57 - 76 - 30) + 54) + 39 - 65 - 29 - 18 + 64 + 12 - 68 + 34 - 80 - 5 + 7 - 18 - 57 - 3 - 67 + 40 - 55 - 53 + 5 - 72 + 31 - 37 - 35 - 4 + 15 + 21 - 23) - 12) - 91 + 60 - 76 + 83) + 47 - 4 + 96 + 80 + 50) + 85 + 75 + 49 - 7 + 20 + 61 - 60 - 81) - 21 - 61 + 25 - 67 - 71 + 73 - 25 + 62 - 28 - 33 + 16) - 47) - 42 + 66 - 31 - 0 + 91) + 43 + 77 + 16 + 95 - 10 + 51 + 15 + 60 - 100 - 19 - 86 - 72 + 0 - 31 - 97 - 48) + 93 + 43 + 75 - 61 + 29) + 74 - 98 - 63 + 85 + 37 + 26) - 19 + 46 + 51 - 7 - 53 - 2 - 91 - 23 - 60 - 99 - 15) - 29 - 57 + 3 - 70 - 81 - 69 - 23 + 50) - 27 - 31 + 16 + 71 - 38 - 42 - 51) - 27 - 22 + 72 + 30) + 66 - 3 + 30 + 9 - 94 - 96 - 39 - 64 - 59 - 25 + 43 - 34 - 80 + 67 - 69) + 58 + 64 + 93 + 16 - 64 + 30 - 77) - 0) - 63) - 17 + 98 + 21) + 53 + 86) - 28 + 41 - 32 - 51 - 32) - 85) - 2 + 77 - 13 + 76 + 17 - 19 + 11 + 7 - 71 - 15 - 86) + 97 + 57 - 93 + 41) + 86 - 7 + 36 - 60 - 52 - 58) + 62 + 46 - 95 - 87) - 11) + 40 - 75 - 18 + 9 + 76 - 40) - 93 - 49) - 51 + 5 - 16 + 48 + 78 + 70 - 83 - 10 + 74) + 65 + 61 - 35 - 23 - 68) - 95 + 92 - 42 + 23 + 98 + 64 - 88 - 90 + 50 - 17 + 95 + 61 - 59 - 60) + 17 + 86 + 70 - 97 - 34 - 5) + 80 + 46 + 87 - 24 - 18 - 14 + 12 + 16 - 31 - 44 + 69 - 69 - 55) + 51 + 81 - 18 + 55 + 67 + 2 + 7 + 54 - 29 + 56 + 95 + 26 + 55 - 41) - 78 - 87 + 36 + 23 - 27 + 40 + 73 - 41 - 84 + 51) + 29 - 25 - 72 + 88 - 1 + 84 - 98 - 65 + 46 + 66 + 10 + 21 + 70) - 23) + 62) - 62 + 80 - 84 + 95 - 44 + 85 - 49) + 38 - 6 - 62 - 89 + 38 + 58 + 37 + 10 + 88 - 11 - 99 + 100 - 77 + 85 + 70 + 42 + 2 + 78 - 44 + 31 - 37) - 42 - 65 - 36 - 15 - 28 - 41 + 86 - 48 + 89 - 25 + 3) + 19 - 79 - 61 - 42 + 2 - 67 - 94 - 37 + 36 + 54 + 98 - 82 + 19 + 79 + 71 + 68 + 11 + 13 + 76 - 35 + 25 - 25 + 100 + 93 - 75 + 50) + 92 - 73 - 94 - 24 + 51 + 81) - 17 + 87 + 86 + 23 - 32 - 62 - 98 + 4 - 42 - 93 - 81 - 91) + 55 - 51 - 39 + 94 + 30 + 29 - 43) - 51 + 76 - 96 + 84 + 13 + 83 + 14 + 37 + 18 - 75 + 79 - 30 - 96 - 50 - 3 + 28 + 67 - 80 - 57 - 75) - 100 - 22 - 72 + 45) + 71 + 17) - 49 - 46 + 10 - 70 - 10 + 29 - 75 + 9 - 3 + 22) - 82 + 88 - 96) - 56 - 76 + 34 - 98 - 72) + 77 + 16 - 43) - 70 - 40 - 31 + 66) + 79 - 68 + 50 - 65 - 94 + 94 - 68) + 52 + 60 - 21 + 55 + 88 - 58 - 68 - 22 + 60 - 65 + 36 + 18 + 83 + 54) - 37) + 71) + 31 + 45 - 51) + 8 - 87 - 3 - 36 + 90 - 65 + 25 + 12 + 75 - 50 - 99 - 5 - 78 + 32 + 85 - 31 - 14 - 4 - 85 - 39) - 1 - 6 - 43 + 3 - 92 + 25) + 73 + 19 + 74 + 89 + 10) + 54 + 8 - 22 - 89 + 15 - 13 - 36 - 30 - 54) + 41 + 23 + 11 - 72) + 85) - 32 - 0 - 26 + 96 + 47 + 80 + 38 - 98) + 22 + 86 + 92 + 79 - 35 - 73 + 20 - 19 + 88 - 28 + 44 - 87 + 2 - 43 + 37 - 64 + 83 - 80 + 46 - 19 + 73 - 3 + 51 + 0) + 85 - 4 - 81 + 64 - 43 + 54 + 87 + 99 - 41 - 87 + 33 - 39 - 54 - 13 + 40 - 70 - 23 + 22 - 57 + 15) + 57 - 56 - 0 - 71) + 78 - 42 + 39 + 25) - 6 + 18 - 75 + 25 - 80 + 73 - 0 - 38 + 24 + 40 + 77 - 3 + 72) - 51 + 71 - 53) - 92 - 45 + 41 + 59 + 13 + 47 + 98 + 17 - 27 + 100 - 18 + 26 - 94 + 74 - 91 - 41 - 6 - 100 - 56 - 36 + 10 + 37 + 30 + 58 - 35) - 15 + 43 - 57 - 92 + 72 + 63 - 10 - 25 + 13 - 9 - 26 + 36 + 91 - 84 + 75 - 17 + 49 - 79) + 9 - 76 + 99 + 47 + 88) + 56 - 25 - 14 + 13 + 28 + 40 - 82 - 11) + 65 - 28 - 13 - 11 - 2 + 74 + 46 - 80 + 34 + 79 - 13 - 66 - 58 + 64 - 97 - 86 - 47 + 18 + 49 - 34) - 63 + 51) - 98 + 29 - 19 - 99 - 72) + 35 - 85 - 25) + 85 - 54 - 67 - 35 - 86 - 53 + 51 - 48 + 74 - 67 + 22 - 54 + 49 - 91 - 84 - 78 - 100 + 52 + 37) - 88 + 74 + 67) - 27 + 65 - 82) - 38 + 20 - 0 - 74 + 48 - 32 - 97 - 10 + 94 + 62 - 95 + 29 + 64 + 48 + 17 - 76 + 100 + 77 - 30 + 11 - 54 - 41 + 36 + 4 - 41) - 9 + 56 - 95 - 30) + 79 + 88) + 73 + 31) - 51 - 90 + 10 + 89 - 4 - 8 + 18 + 0 - 83 - 64 + 86 + 9 + 7 + 14 + 62 - 42 + 23 - 85 + 53 + 54 + 76 + 80 + 41 - 33 + 37 + 23 + 94 + 28 - 70 - 96 - 90) + 74 + 43 - 11 + 38 - 21 + 68 - 77 + 47 - 84 - 76 + 8 + 33 + 5 - 91 + 1 + 56 - 4 + 90 + 78 + 9 + 53 - 61 + 44 - 27 + 23 - 92 - 30 + 57 - 13 - 45 + 91 + 11 - 69) + 69 + 97 + 26 - 57 + 61 + 69 - 28) + 31 + 74 - 89 - 36 - 32 + 15 - 44 - 86 + 80 - 90 - 12 + 74 - 29 + 66 + 97 + 44 - 21 - 63 + 71 - 28 + 26 + 48) + 83) + 35 + 80 + 83) - 45 - 39 - 97 - 90 - 35 + 14) - 46 + 21 + 45 + 19 + 49 + 26 - 9 + 35 - 42 + 95 + 15 + 60 - 28 - 90 - 83 - 72 - 46 + 100) + 82 - 55) - 70 - 34 + 52 - 9 - 76 - 43 - 61 + 6 + 24 - 4 - 69 - 55 + 13) - 52 + 43 + 29 - 1 + 51 + 67 - 15 + 20 - 72 - 87 + 59 + 76 - 11 + 69 + 5 + 41 - 52) + 54) - 17 - 46 - 47 - 4 + 15 - 80 + 52 - 98 + 11 + 86 + 71 + 82 + 83 - 13 + 43 + 26 - 7) + 65 + 3 + 23 - 99 - 89 - 8 + 47 - 92 + 40 - 26 - 74) + 9 - 2 + 7 + 2 - 71 - 99 + 69) + 81 - 17 - 67 - 82 + 38 + 82 + 92 - 40 - 4 - 82 - 11 + 8 + 1 + 14 + 63 + 28 - 26 + 72 + 22 + 8 - 90 - 21 + 39 - 61 - 48 - 69 - 76 + 9 + 44 - 72) - 89 + 45 - 67 - 32) - 77 + 80 - 100 - 39 - 37 + 100 + 48 - 25) - 70 + 47 - 43 + 74 - 68 - 60 - 73 - 93) + 88 - 74 - 53) - 0 + 76 + 16 + 90 - 77) + 83 - 51 - 14 - 63 - 1 - 40 + 47 + 2 - 46 + 86 - 7 + 39 + 54) + 83 - 90 - 39 + 82 - 45 + 66 + 83 - 21 - 15 + 24 + 37) + 7 + 29 + 40 + 7 + 13 - 33) - 42 + 12 - 21 - 58 + 90 - 87 + 75 - 9 - 59 - 81 + 96 - 25 - 97 - 26 - 6 + 98 - 85 + 78 - 53 - 42 + 52 - 98 - 90 + 61 - 14 + 44 + 27 - 86 + 68 + 70 - 51 + 23 + 50 + 4 - 13 + 27 + 85 + 85 - 18 + 17 + 54 - 45 + 49 + 46 + 14 - 100 + 24 + 46 + 85 - 15 - 62 - 42 - 60 - 30 - 62 - 3 + 87 - 83 + 57 + 12 + 34 - 26 - 90 + 28 + 62) + 27 - 55 - 38 - 27 + 64 + 9 - 19 + 11 + 71 + 60 - 73 + 19 + 77 + 70 + 8 - 44 - 44 + 22 - 24 - 33 + 67 + 54 + 16 - 48 - 85 - 45 + 49 - 58 + 19 - 5 + 100 + 19) + 90 - 78 + 11 - 100 + 15 + 66 + 18 + 52 + 2 - 46 - 24 - 88 + 17 - 27 - 7 - 84 - 47 - 1 - 92 - 68 - 45 - 26 + 11 + 43 + 77 + 86 + 1 + 60 + 86 - 3 + 26 - 4 + 23 - 33 - 46) - 77 - 20 + 100 + 45) - 33 + 43 - 52) + 82 - 38 + 22 - 53) + 34 + 99 - 83 - 78 - 67 + 82 - 64 - 98 + 72 + 68 - 94 - 6 - 1 - 67 + 60) - 56 - 81 + 63) - 48 - 10 + 55 - 33 + 94 + 51) - 42 - 88 + 65 - 60 + 92 + 72 + 96 + 79 + 100 + 12 + 32 + 12 + 15 - 66 + 92 - 27 + 25 + 29 - 21 - 22 - 83 - 31 + 58 - 6 + 11 + 66 - 83 - 16 - 19 + 89 + 66 + 92) + 4) + 10) + 40 - 65 + 33 + 54) - 93 + 60 - 35 - 22 - 37 + 84 - 1 - 79 + 70 + 3 - 13 - 32 - 66 - 2 - 69 + 63 - 52 - 26 - 15 + 37 + 53 - 34 + 8 + 32 + 64 + 33 - 97 + 73 - 52 + 95 - 4 - 0) - 91 - 29 + 66 - 21 + 51 + 14 - 15 - 80) - 16 - 32 - 98 - 86 + 87 - 83 - 28 - 10) - 76 - 92 + 35 - 10 + 25 + 100 - 12 + 37 + 83 + 12 + 87 + 24 - 55 + 92 + 10 + 72 - 66 - 69 + 20 - 51 - 78 - 78 + 6 - 20 - 61) - 20 + 40 - 90 - 62) - 45) + 68 + 23 - 15 - 72 + 58 - 52 - 90 + 5 + 81 - 45 + 82 - 96 + 78 - 46 + 13 + 87 - 39 - 2) - 83 - 45 - 20 - 51 - 56 + 16 - 51 - 46 + 70 + 53 - 66 + 86 - 26 - 44 - 17 + 0 + 94 - 27 + 5) - 55 - 51 + 82 - 40 - 36 - 44) + 46 + 12 - 76 + 79 + 93 + 48 + 7 + 68 - 27 - 4 - 79 + 80 - 71 + 40 - 89 + 54 + 35 + 24 - 40 + 4 + 6 + 65 - 37 - 47 - 82 + 84) + 5 - 35 - 12) + 50 - 22 + 57 - 30 + 68 - 79 - 0 - 35 - 55 - 74) + 65 + 92 - 18) - 32 + 5 - 10 - 58 - 19 + 28 - 20 + 42) + 87 + 60 - 53 - 12 - 0 + 49) + 49 - 1 - 32 + 39 + 62 - 80 - 50 - 13 + 32 + 18 + 13 - 68) - 66 - 77 - 57 + 14 - 22 - 99 + 28 - 91 - 79 - 80 - 21 + 62 + 80 + 10 - 72 - 39 + 96 + 63 + 22 - 39 + 17 - 11 + 9 - 94 + 93 - 47 - 69 + 18 - 3 - 7 + 51) - 98 - 10 + 48 - 47 - 3 + 40 - 5 + 37) + 25 - 22) - 71 + 59 + 61 + 43 + 11) + 59) - 4) - 23 - 77 - 12) + 13 - 88 - 67 + 17 - 26 - 40 - 53) + 13 + 98 + 96 + 51 - 25 + 60 - 8 - 70 - 2 + 13 - 92 - 35 + 18 + 94 + 16 + 28 + 92 - 68 + 71 + 24 + 78) + 74 + 62 - 2 - 5 + 5 - 95 + 5 - 68 - 38) + 33 + 11 + 72 + 48 + 65 + 24 - 48 + 7 + 83 - 46 + 84 + 7 - 26 - 39 - 79) - 36 - 0 + 70 + 92 - 43 - 62 + 13 - 50 + 81 + 68 - 70 + 9 - 84 + 4 - 98) - 21 - 25 + 43 + 7 - 44 - 80 + 65 - 42 + 34 - 36 - 0 - 11 + 68 + 9 + 61 - 61 - 28 + 9 - 70 - 15 - 76 + 26 + 65 - 96 - 34 + 88 - 90 + 36 - 24 + 9 + 9 + 81 + 68 + 36 - 35 + 37 + 84 - 81 + 26 - 28 + 19 + 0 + 79 - 80 - 49 - 46 - 7 - 69 - 83 + 95 + 22 - 0 + 2 + 49 - 95 + 74 - 96 - 28 - 21) - 22 - 79) - 93 - 61 - 10 + 26 - 51 + 14) - 25 + 70 + 68 + 22 + 99 - 8) - 29 - 24 + 46 - 54 - 80 + 86 - 79 + 76 - 77 - 30 - 59 - 35 + 95 - 3 + 89) - 72 - 71 - 39 - 55 + 28 + 0 + 70 + 24 - 4 + 62 - 51 - 86 - 4 - 61 + 45 + 55 + 19 - 67 + 37 - 3 - 31 - 53 - 24 - 6 + 91 - 57 - 59 + 86) + 38 - 81 + 66 - 16 - 65 - 27 - 72 + 62 - 50 - 0 + 17 + 8 + 13 + 44 - 42 + 50 - 33 - 10 + 48 - 74) + 35 + 66 - 50 - 82 + 44 - 85 + 60 + 76 + 56 - 50 - 48 + 89 + 56 + 3 - 89 + 93 - 50 + 16 - 25 - 85 - 27 + 6 + 53 - 55 - 73 - 83 + 93 + 15) + 56 - 13 - 49 - 37 + 52 + 1 + 85 - 7 + 70 + 26 + 34 - 61 + 82 - 23 - 40 - 8 + 61 + 32 - 71 + 65 - 50 - 99 + 43 + 28 + 0 + 66 + 24 - 74 - 22 - 89 + 84 - 70 + 27 - 22 + 80 + 67 - 3 + 98 + 19 + 2 - 21 - 95 + 72 + 12 - 10) + 95) - 32 + 68 - 63 + 43 + 37) + 49 - 90 + 95 - 55 + 57) + 68 - 19 - 60) + 87) + 21 - 19 - 33 + 64 - 90 - 39 - 10 + 35 + 5) - 31 + 57 - 96 - 13 - 61 + 13 - 95 - 78 + 76 + 48 + 63 - 87 - 16 - 4 + 9 - 93 + 93 - 32 + 81 - 9 + 2 + 40 - 17 + 70 - 98 + 100 + 67) - 81 + 88 + 64 - 26 + 62) - 66 + 13) - 46 - 85 - 99 - 91) - 45 - 25 - 79 - 30 + 32 + 5 + 54 - 100 - 40 - 37) + 99 - 52 + 20 + 60) - 9 + 24 - 88 + 25 + 83 + 49 - 10 - 29 - 19 - 5 + 69 - 99 - 3 - 60 - 6 - 14 + 55 - 47 - 83 + 36 + 52 - 80 + 41 - 69) + 71 - 95) - 15 + 98 + 42 - 66 + 7 - 44 + 78 - 1 - 23 - 95 - 23) - 38 - 49) + 57 + 41 + 36 + 54 + 61 - 11 - 16 - 70 - 39 - 45 - 54 + 45) - 22 - 24 - 47 - 50 + 59 + 78 + 25 + 37 - 59 - 81 + 6 - 100 - 13 + 34 - 50 - 49 + 31 - 35 + 3 + 34 + 84 + 46 + 98 + 11 + 11 + 92 + 91 + 30 + 11 - 24 - 62) + 8 + 41 + 26 + 92 - 90 + 4 - 12 + 12 + 84 - 67 - 0 + 75 + 88 - 6 + 17 - 49 + 90 - 77 - 18 - 4) - 61 + 74 + 78 + 39 + 80 - 60 + 33 + 17 + 17 + 63 + 57 + 62 - 45 + 80 - 89 + 93 - 11 + 30 - 77 + 68 + 93 + 53 + 85 - 33 + 89 + 41 + 76 - 19 - 38) + 44 - 0 - 49 - 12 - 41 - 33 + 50 - 17 + 14 - 53 + 29 - 17 - 34 - 92 + 62 - 18 + 59 - 64 - 37 - 25 + 67 - 29 + 67 + 98 - 53 + 58 + 57 + 15 - 82 + 1 - 75) - 47 - 43 - 40 - 90 - 25 + 16 - 62 - 96 - 65 + 48 + 85) + 98 + 6 - 11 + 51) - 90 + 69 - 56) + 42 - 85 + 90 + 82 + 17 - 87 - 60 - 40 - 7 + 49) - 81 + 65 - 100) - 46 - 81 + 14 - 23 - 94 - 37 - 77 + 50 - 46 - 79 + 87) - 76 - 85 - 64 - 13 - 69 + 50 - 49 + 97 + 67 + 7 - 57 - 17 + 47 - 89 - 35 - 99) + 45 + 65 - 78 - 60 + 40 - 69 - 58 - 55) - 16 - 54 - 73 - 58 + 79 - 11 + 96 + 59 + 14 - 45 - 38) - 32 - 47 + 53 + 27 - 9 - 62 - 67 + 9 + 57 - 50 + 36 + 82 - 34 - 56 + 60 + 90 - 71 - 39 - 62 + 31 - 19 - 12 + 94 + 77 - 82 - 97 - 28 + 78 - 36 + 93 + 68 - 33 + 75 + 41 + 20 - 21 - 7 + 44) + 50 + 42 - 80 - 53) - 86 + 8 - 33 + 41 - 3 - 83 - 39 - 35 - 22 - 65 - 13) + 92 + 73 + 15) - 82 - 25) + 41 - 46 - 10 - 98 + 69 + 73 + 47 - 58 + 32 - 21 - 49 + 1 - 40) + 93 - 6 + 44 - 0 - 6 + 3 + 4 - 38 - 80 - 35 - 89) - 64 - 22) + 72 - 41 + 83 + 47 - 39 + 10 + 87 + 18 + 17 - 2 - 96 + 65 + 100 + 62 - 88 - 94 + 88 - 70 - 20) + 39 - 57 - 98 - 35 - 79 + 43 - 28 + 18 + 14 + 54 + 70 - 61) + 43 - 77) - 87) + 15 - 96) + 39) - 73 - 69 + 81 + 18 - 70 + 2 - 63 + 94 - 9 - 97 + 56) - 17 + 3 - 50 - 35 - 52) - 1 - 0 + 30 - 57 - 48 + 42 - 7 + 17 - 71 + 80 - 25) - 45 - 98 + 43 + 0 + 99 - 85) - 99 + 74 - 73 + 77 + 89 + 11 + 36 + 18 - 13 - 60 - 40 + 6 + 41) - 95 + 36) - 41 - 41 + 93 + 8 - 52 - 56 - 55) - 47 - 32 + 51 - 30 + 44 - 73 + 22 - 97 - 99 - 74 + 15 - 78 + 16 + 12 + 51) - 75 - 11 - 15 + 9 - 57 - 89 + 54 + 65 + 98 - 60 - 98 - 89 + 31 - 51 - 24 + 85 + 83 + 55 + 66 - 89 - 23 + 79 + 70 + 9 + 42 + 44 - 71 + 98 + 60) + 16 + 9 - 99 + 52 - 44 + 3 - 23 - 10 + 29 - 77 - 77 - 69 + 31 + 42 + 67 - 77 - 84 + 93 - 36 - 5 + 60 - 67) + 65 - 76 + 13 + 91 - 26 - 43) - 47 + 50 - 62 - 69 - 94 - 94 - 19 + 45 - 25 - 27 + 8 - 4 - 25 - 19) - 49 + 32 - 53 - 10 + 31 - 23 + 66 - 76 - 36 + 28 - 58 + 94 - 60 + 79 + 32 + 12 + 50 + 83 - 5 + 27 + 68 + 56 - 27 + 94) - 0 + 72 + 65 - 57 - 23 - 62) - 59 - 74) + 9 - 45 + 38 - 35 + 6 + 20 - 84 + 96 + 29 - 61 + 4 + 49 + 29 - 60 - 82 - 8 - 79 - 50 - 2 - 90 + 52 - 74 - 76 + 67 - 91 - 46 + 83 - 28 - 50"
_calculate(s, 0, len(s)-1)

KeyboardInterrupt: 

In [ ]:


def calculate(self, s: str) -> int:
    """
    When we see a parenthesis, we need to call a recursion to calcualute result inside the (...)
    """
    striped_s = s.strip()
    result = 0
    cached_operator = "+"
    i = 0
    while i < len(striped_s):
        # If Number
        if striped_s[i] in self.num:
            # accumulate number
            temp_num = 0
            while i < len(striped_s) and striped_s[i] in self.num:
                print(f"{i}: {striped_s[i]}")
                temp_num = temp_num * 10 + int(striped_s[i])
                i += 1

        # If is symbol
        if i < len(striped_s) and striped_s[i] in self.symb:
            if cached_operator == "+":
                cached_operator = striped_s[i]
            else: # cached_operator == "-":
                if striped_s[i] == "-":
                    cached_operator = "+"

        # If is a parenthesis
        if i < len(striped_s) and striped_s[i] in self.parent:
        i += 1

        # do the arithmetic
        if cached_operator == "+":
            result += temp_num
        else:
            result -= temp_num

        # set it back to default
        cached_operator = "+"
    return result

In [ ]:
# 224. Basic Calculator

NUM = [f"{i}" for i in range(10)]
SYMB = ["+", "-"]
PAREN = ["(", ")"]

class Solution:

    def find_close_paren(self, i:int, s:str):
        """
        Find the corresponding closing paren of the open paren at s[i].
        """
        parent_count = 0
        i = i+1
        while i < len(s):
            if s[i] == "(":
                parent_count += 1
            if s[i] == ")":
                if parent_count <= 0:
                    return i
                else:
                    parent_count -= 1
            i += 1
        return 

    def operate(self, temp_result:int, num:int, operator_str:str):
        if operator_str == "+":
            temp_result += num
        else:
            temp_result -= num

        return temp_result

    def _calculate(self, s:str, l, r):
        """
        Comptue the value in the range s[l:r+1]
        """
        result = 0
        cached_operator = "+"
        i = l
        while i < r+1:
            # If Number
            if s[i] in NUM:
                # accumulate number
                temp_num = 0
                while i < len(s) and s[i] in NUM:
                    temp_num = temp_num * 10 + int(s[i])
                    i += 1
                # Compute
                result = self.operate(result, temp_num, cached_operator)
                # Set op back to default 
                cached_operator = "+"

            # If is symbol
            if i < len(s) and s[i] in SYMB:
                if cached_operator == "+":
                    cached_operator = s[i]
                else: # cached_operator == "-":
                    if s[i] == "-":
                        cached_operator = "+"
            
            # If is opening parenthesis
            if i < len(s) and s[i] == "(":
                # First find the index of the closing parenthesis
                close_idx = self.find_close_paren(i, s)
                # Compute the value inside the parenthesis
                inner_result = self._calculate(s, i+1, close_idx)
                # +/- to the current rensult
                result = self.operate(result, inner_result, cached_operator)
                # Set op back to default 
                cached_operator = "+"
                # Jump to the closing paren location
                i = close_idx

            i += 1
        
        return result

    def calculate(self, s: str) -> int:
        """
        When we see a parenthesis, we need to call a recursion to calcualute result inside the (...)
        """
        return self._calculate(s, 0, len(s)-1)
    